## Run model and collect predictions on BABILong

In [1]:
import os
os.chdir('..')
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets
from tqdm.auto import tqdm
import pandas as pd
import time
import json

from pathlib import Path

from babilong.prompts import DEFAULT_PROMPTS, DEFAULT_TEMPLATE, get_formatted_input
from babilong.babilong_utils import compare_answers

from modeling_amt.language_modeling import *

In [88]:
class Holder:
    def __init__(self) -> None:
        pass

args = Holder()
# args.task_dataset = "qa1_single-supporting-fact"
# args.max_n_facts = 50
args.vary_n_segments = False
args.max_n_segments = 128
args.segment_size = 1024
args.sample_size = 512
args.segment_alignment = 'right'

base_model_path = "/home/jovyan/kuratov/models/Llama-3.2-1B-Instruct/"
# base_model_path = "unsloth/Llama-3.2-1B-Instruct"
dtype = torch.bfloat16
device = 'auto'

In [186]:
model_name = "unsloth/Llama-3.2-1B-Instruct"
# model_cpt = '/home/jovyan/gkuzmin/rmt_it/data/pretrained_models/RMT-Llama-3.2-1B-Instruct-8x1024-mem16-lora-babilong-qa1-5_ct-v3.1/model.safetensors'
# model_cpt = checkpoints[0]
use_peft = 1
segment_size = 1056
max_n_segments = 32
layers_attr = "model.layers"
add_question_prompt = "Answer with a single word."
segment_alignment = "right"
lora_r = 8
lora_alpha = 32
lora_dropout = 0.1
mem_size = 16
d_mem = 64
wrap_pos = False
no_correction = False
attend_to_previous_input = 0
api_url = ''

In [187]:
from peft import get_peft_model, LoraConfig, TaskType
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True,
                                                device_map='cpu', torch_dtype=dtype,
                                                attn_implementation='flash_attention_2')
if use_peft:
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        r=lora_r, 
        lora_alpha=lora_alpha, 
        lora_dropout=lora_dropout,
        )
    model = get_peft_model(model, peft_config)
device = "cpu"
if d_mem is not None:
    mem_cell_args = dict(
        base_model=model,
        num_mem_tokens=mem_size,
    )
    # additional parameters for ARMT model
    mem_cell_args['d_mem'] = d_mem
    mem_cell_args['wrap_pos'] = wrap_pos
    mem_cell_args['correction'] = not(no_correction)
    mem_cell_args['use_lora'] = use_peft
    if layers_attr is not None:
        mem_cell_args['layers_attr'] = layers_attr
    # if attend_to_previous_input is not None:
    #     mem_cell_args['attend_to_previous_input'] = bool(attend_to_previous_input)
    cell = AssociativeMemoryCell(**mem_cell_args)
    model = AssociativeRecurrentWrapper(cell,
                                        segment_size=segment_size-2*mem_size,
                                        max_n_segments=max_n_segments,
                                        segment_alignment=segment_alignment,
                                        attend_to_previous_input=attend_to_previous_input,
    ).to(device)

In [188]:
model_cpt = checkpoints[-1]

In [189]:
cpt = torch.load(model_cpt, map_location=device)    
model.load_state_dict(cpt, strict=True)
# except:
#     # if the model saved in safetensors
#     from safetensors.torch import load_model
#     load_model(model, model_cpt, device=device)
#     #model.load_state_dict(model_cpt, map_location=device)

device = 'cuda:0'
model.to(device)
model.to(dtype)
model.name_or_path = "custom_rmt" # workaround
model.device = device

In [157]:
# class Holder:
#     def __init__(self) -> None:
#         pass

# args = Holder()
# # args.task_dataset = "qa1_single-supporting-fact"
# # args.max_n_facts = 50
# args.vary_n_segments = False
# args.max_n_segments = 128
# args.segment_size = 1024
# args.sample_size = 512
# args.segment_alignment = 'right'

# base_model_path = "/home/jovyan/kuratov/models/Llama-3.2-1B-Instruct/"
# # base_model_path = "unsloth/Llama-3.2-1B-Instruct"
# dtype = torch.bfloat16
# device = 'auto'

# tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(base_model_path, trust_remote_code=True,
#                                              device_map='cpu', 
#                                              torch_dtype=dtype,
#                                             #  use_cache=False,
#                                              attn_implementation='flash_attention_2')

# from peft import get_peft_model, LoraConfig, TaskType

# peft_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM, 
#     inference_mode=False, 
#     r=8, 
#     lora_alpha=32, 
#     lora_dropout=0.1
#     )
# model = get_peft_model(model, peft_config)

# mem_cell_args = dict(
#     base_model=model.cpu(),
#     num_mem_tokens=16,
# )
# mem_cell_args['d_mem'] = 64
# mem_cell_args['wrap_pos'] = False
# mem_cell_args['correction'] = False
# mem_cell_args['layers_attr'] = "base_model.base_model.layers"

# cell = AssociativeMemoryCell(**mem_cell_args)
# model = AssociativeRecurrentWrapper(cell, 
#                                 segment_size=args.segment_size,
#                                 max_n_segments=args.max_n_segments, 
#                                 vary_n_segments=args.vary_n_segments,
#                                 segment_alignment='right',
#                                 # k2=-1,
#                                 # return_all_logits=False,
#                                 attend_to_previous_input=False,
# )

In [190]:
model.rmt_config

{'segment_size': 1024,
 'max_n_segments': 32,
 'segment_alignment': 'right',
 'attend_to_previous_input': 0}

In [191]:
model = model.eval()

In [192]:
# ls /home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_4x1024_mem16_bs64_bptt--1_from_cpt_2-4_lora_ct-v3/run_1

In [193]:
# cpt_path = "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_1x1024_mem16_bs64_bptt--1_from_cpt_0-1_lora/run_1/checkpoint-7500/pytorch_model.bin"
# cpt_path = "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_2x1024_mem16_bs64_bptt--1_from_cpt_1-2_lora/run_1/checkpoint-8000/pytorch_model.bin"
# cpt_path = "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_4x1024_mem16_bs64_bptt--1_from_cpt_2-4_lora/run_1/checkpoint-8000/pytorch_model.bin"
# cpt_path = "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_4x1024_mem16_bs64_bptt--1_from_cpt_2-4_lora_ct/run_1/checkpoint-5000/pytorch_model.bin"
# cpt_path = "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_4x1024_mem16_bs64_bptt--1_from_cpt_2-4_lora_ct-v2/run_1/checkpoint-2500/pytorch_model.bin"
# cpt_path = "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_linear_adamw_wd1e-03_2x1024_mem16_bs64_bptt--1_from_cpt_1-2_lora_ct-v3/run_1/checkpoint-15500/pytorch_model.bin"

checkpoints = [
    "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_cosine_adamw_wd1e-03_1x1024_mem16_bs64_bptt--1_from_cpt_0-1_lora_ct-v3/run_1/checkpoint-6500/pytorch_model.bin",
    "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_cosine_adamw_wd1e-03_2x1024_mem16_bs64_bptt--1_from_cpt_1-2_lora_ct-v3/run_1/checkpoint-28000/pytorch_model.bin",
    "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_cosine_adamw_wd1e-03_4x1024_mem16_bs64_bptt--1_from_cpt_2-4_lora_ct-v3/run_1/checkpoint-24500/pytorch_model.bin",
    "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_cosine_adamw_wd1e-03_8x1024_mem16_bs64_bptt--1_from_cpt_4-8_lora_ct-v3/run_1/checkpoint-30000/pytorch_model.bin",
    # "/home/jovyan/rmt/runs/test/babilong_multitask/meta-llama/Llama-3.2-1B-Instruct/lr_3e-04_d64_cosine_adamw_wd1e-03_16x1024_mem16_bs64_bptt--1_from_cpt_8-16_lora_ct-v3/run_1/checkpoint-6000/pytorch_model.bin",
]

eval_model_template = 'test/armt-llama3.2-1b-{}-ct-v3-retrain-align_right-prompt-v2-test'

dataset_name = "RMT-team/babilong"
results_folder = "/home/jovyan/rmt/babilong/babilong_evals/"

In [194]:
weights = torch.load(checkpoints[-1])

model.load_state_dict(weights, strict=True)

<All keys matched successfully>

In [195]:
# model.memory_cell.model.base_model.model.model.embed_tokens.weight[0:1, 0:20]

In [196]:
# for n, p in model.memory_cell.model.base_model.named_parameters():
#     print(n, p[:10])

In [197]:
# for n, p in model.named_parameters():
#     print(n, p[0, :10])

In [198]:
generate_kwargs = {
    'max_new_tokens': 30,
    'max_length': None,
    'num_beams': 1,
    'do_sample': False,
    'temperature': None,
    'top_p': None,
    'top_k': None,
    'pad_token_id': tokenizer.pad_token_id
}

if generate_kwargs['pad_token_id'] is None:
    generate_kwargs['pad_token_id'] = tokenizer.eos_token_id

# print(f'prompt template:\n{DEFAULT_TEMPLATE}')

In [199]:

tokenizer.eos_token_id

128009

In [200]:
model.memory_cell.model.base_model.model.model.layers[0].layer.self_attn.q_proj.lora_A.default.weight[0:1, 0:20]

tensor([[ 0.0342, -0.0064, -0.0303, -0.0115, -0.0036,  0.0171, -0.0471, -0.0089,
         -0.0089,  0.0371, -0.0069, -0.0022, -0.0239,  0.0096,  0.0125, -0.0231,
         -0.0173,  0.0049, -0.0269, -0.0496]], device='cuda:0',
       dtype=torch.bfloat16, grad_fn=<SliceBackward0>)

In [201]:
model.memory_cell.model.base_model.model.model.layers[0].W_mq.weight[0:1, 0:20]

tensor([[-0.0320, -0.0195,  0.0037, -0.0053,  0.0206,  0.0063, -0.0074,  0.0386,
         -0.0120,  0.0060, -0.0016, -0.0038,  0.0225,  0.0352, -0.0155, -0.0014,
          0.0054, -0.0092,  0.0270,  0.0170]], device='cuda:0',
       dtype=torch.bfloat16, grad_fn=<SliceBackward0>)

In [202]:
# cpt_path

In [173]:
weights = torch.load(cpt_path)#, map_location=device)

model.load_state_dict(weights, strict=True)

<All keys matched successfully>

In [174]:
# weights

In [113]:
model

AssociativeRecurrentWrapper(
  (memory_cell): AssociativeMemoryCell(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(128256, 2048)
            (layers): ModuleList(
              (0-15): 16 x AssociativeLayerWrapper(
                (W_mq): Linear(in_features=2048, out_features=64, bias=False)
                (W_mk): Linear(in_features=2048, out_features=64, bias=False)
                (W_mv): Linear(in_features=2048, out_features=2048, bias=False)
                (W_mb): Linear(in_features=2048, out_features=1, bias=True)
                (layer): LlamaDecoderLayer(
                  (self_attn): LlamaFlashAttention2(
                    (q_proj): lora.Linear(
                      (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.1, inplace=False)
  

In [117]:
model.load_state_dict(weights, strict=True)


<All keys matched successfully>

In [203]:
for n, p in model.memory_cell.named_parameters():
    print(n, p[0, :10])

memory tensor([ 0.0070, -0.0124,  0.0410, -0.0322,  0.0082,  0.0262,  0.0776,  0.0442,
        -0.0332,  0.0835], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SliceBackward0>)
model.base_model.model.model.embed_tokens.weight tensor([ 0.0031,  0.0178,  0.0210, -0.0101,  0.0070,  0.0068,  0.0139,  0.0052,
         0.0094,  0.0062], device='cuda:0', dtype=torch.bfloat16)
model.base_model.model.model.layers.0.W_mq.weight tensor([-0.0320, -0.0195,  0.0037, -0.0053,  0.0206,  0.0063, -0.0074,  0.0386,
        -0.0120,  0.0060], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SliceBackward0>)
model.base_model.model.model.layers.0.W_mk.weight tensor([ 0.0117,  0.0054,  0.0014, -0.0046,  0.0020, -0.0098, -0.0021, -0.0262,
        -0.0089,  0.0078], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SliceBackward0>)
model.base_model.model.model.layers.0.W_mv.weight tensor([ 0.0135, -0.0090, -0.0017,  0.0042, -0.0049, -0.0041,  0.0140, -0.0074,
         0.0035, -0.0069], devi

IndexError: too many indices for tensor of dimension 1

In [176]:
model.memory_cell.model.base_model.model.model.embed_tokens.weight[0:1, 0:20]

tensor([[ 0.0031,  0.0178,  0.0210, -0.0101,  0.0070,  0.0068,  0.0139,  0.0052,
          0.0094,  0.0062,  0.0052,  0.0008, -0.0009,  0.0021, -0.0247,  0.0254,
         -0.0078,  0.0267,  0.0096, -0.0137]], dtype=torch.bfloat16)

In [177]:
model_inputs

{'input_ids': tensor([[128000, 128006,   9125,  ...,  78191, 128007,    271]],
        device='cuda:0')}

In [178]:
generate_kwargs

{'max_new_tokens': 30,
 'max_length': None,
 'num_beams': 1,
 'do_sample': False,
 'temperature': None,
 'top_p': None,
 'top_k': None,
 'pad_token_id': 128009}

In [181]:
model.to(device)
model.to(dtype)
1

1

In [204]:
with torch.no_grad():
        attn_mask = torch.ones_like(model_inputs['input_ids'].bool().to(device))
        # output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)
        output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)

In [205]:
output

tensor([[    65,  78932, 128009]], device='cuda:0')

In [ ]:
for n, p in model.memory_cell.named_parameters():
    print(n, p[0, :10])

memory tensor([ 0.0070, -0.0124,  0.0410, -0.0323,  0.0082,  0.0263,  0.0775,  0.0441,
        -0.0332,  0.0836], grad_fn=<SliceBackward0>)
model.base_model.model.model.embed_tokens.weight tensor([ 0.0031,  0.0178,  0.0210, -0.0101,  0.0070,  0.0068,  0.0139,  0.0052,
         0.0094,  0.0062], dtype=torch.bfloat16)
model.base_model.model.model.layers.0.W_mq.weight tensor([-0.0320, -0.0195,  0.0037, -0.0053,  0.0206,  0.0063, -0.0074,  0.0386,
        -0.0120,  0.0060], dtype=torch.bfloat16, grad_fn=<SliceBackward0>)
model.base_model.model.model.layers.0.W_mk.weight tensor([ 0.0117,  0.0054,  0.0014, -0.0046,  0.0020, -0.0098, -0.0021, -0.0262,
        -0.0089,  0.0078], dtype=torch.bfloat16, grad_fn=<SliceBackward0>)
model.base_model.model.model.layers.0.W_mv.weight tensor([ 0.0135, -0.0090, -0.0017,  0.0042, -0.0049, -0.0041,  0.0140, -0.0074,
         0.0035, -0.0069], dtype=torch.bfloat16, grad_fn=<SliceBackward0>)
model.base_model.model.model.layers.0.W_mb.weight tensor([ 0.0325, 

IndexError: too many indices for tensor of dimension 1

In [130]:
torch.save(model_inputs, 'tmp')

In [131]:
# model_inputs['input_ids'].tolist()

In [121]:
tasks = ['qa1', 'qa2', 'qa3', 'qa4', 'qa5']
# split_names = ['0k', '1k', '2k', '4k', '8k', '16k']
# split_names = ['32k', '64k']
split_names = ['16k']

use_chat_template = True
use_instruction = False
use_post_prompt = False
use_examples = False

device = 'cuda:0'


for cpt_path in checkpoints[-1:]:
    eval_model_name = eval_model_template.format(cpt_path.split('adamw_wd1e-03_')[1].split('_')[0])
    print('Evaluating ', eval_model_name)

    # with open(cpt_path, 'rb') as cpt:
    #     weights = torch.load(cpt)

    weights = torch.load(cpt_path)#, map_location=device)

    model.load_state_dict(weights, strict=True)
    # from safetensors.torch import load_model
    # model_cpt = '/home/jovyan/rmt/babilong/notebooks/tmp_data/model.safetensors'
    # load_model(model, model_cpt, device=device)
    model.to(device)
    model.to(dtype)

    model = model.eval()

    model.device = device

    for task in tqdm(tasks, desc='tasks'):
        # configure the prompt
        prompt_cfg = {
            'instruction': DEFAULT_PROMPTS[task]['instruction'] if use_instruction else '',
            'examples': DEFAULT_PROMPTS[task]['examples'] if use_examples else '',
            'post_prompt': DEFAULT_PROMPTS[task]['post_prompt'] if use_post_prompt else '',
            'template': DEFAULT_TEMPLATE,
            'chat_template': use_chat_template,
        }
        prompt_name = [f'{k}_yes' if prompt_cfg[k] else f'{k}_no' for k in prompt_cfg if k != 'template']
        prompt_name = '_'.join(prompt_name)

        for split_name in tqdm(split_names, desc='lengths'):
            # load dataset
            data = datasets.load_dataset(dataset_name, split_name)
            task_data = data[task]

            # Prepare files with predictions, prompt, and generation configurations
            outfile = Path(f'{results_folder}/{eval_model_name}/{task}_{split_name}_{prompt_name}.csv')
            outfile.parent.mkdir(parents=True, exist_ok=True)
            cfg_file = f'{results_folder}/{eval_model_name}/{task}_{split_name}_{prompt_name}.json'
            json.dump({'prompt': prompt_cfg, 'generate_kwargs': generate_kwargs}, open(cfg_file, 'w'), indent=4)

            df = pd.DataFrame({'target': [], 'output': [], 'question': []})

            for sample in tqdm(task_data, desc=f'task: {task} length: {split_name}'):
                target = sample['target']
                context = sample['input']
                question = sample['question']

                # format input text
                input_text = get_formatted_input(context, question + 'Answer with a single word.', prompt_cfg['examples'],
                                                    prompt_cfg['instruction'], prompt_cfg['post_prompt'],
                                                    template=prompt_cfg['template'])
                # print(input_text)

                # with open('tmp-1.txt', 'a') as f:
                #     f.write('\n' + input_text)

                if use_chat_template:
                    input_text = [{'role': 'user', 'content': input_text}]
                    model_inputs = tokenizer.apply_chat_template(input_text, add_generation_prompt=True,
                                                                    return_tensors='pt').to(device)
                    model_inputs = {'input_ids': model_inputs}
                else:
                    model_inputs = tokenizer(input_text, return_tensors='pt',
                                                add_special_tokens=True).to(device)

                sample_length = model_inputs['input_ids'].shape[1]
                # with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                # print(model_inputs['input_ids'][0][:1000])
                # with open('tmp-1.txt', 'w') as f:
                #     f.write(str(model_inputs['input_ids'][0].cpu().tolist()))
                # 1/0
                with torch.no_grad():
                    attn_mask = torch.ones_like(model_inputs['input_ids'].bool().to(device))
                    # output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)
                    output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)

                output = output[0]#[sample_length:]
                output = tokenizer.decode(output, skip_special_tokens=True).strip()


                df.loc[len(df)] = [target, output, question]
                df.to_csv(outfile)
                if df.shape[0] > 7:
                    1/0
            # write results to csv file
            # df.to_csv(outfile)

Evaluating  test/armt-llama3.2-1b-8x1024-ct-v3-retrain-align_right-prompt-v2-test


tasks:   0%|          | 0/5 [00:00<?, ?it/s]

lengths:   0%|          | 0/1 [00:00<?, ?it/s]

Using the latest cached version of the dataset since RMT-team/babilong couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '16k' at /home/jovyan/.cache/huggingface/datasets/RMT-team___babilong/16k/0.0.0/ee0d588794c7ac098062ee0d247c733d62e94fe2 (last modified on Thu Aug  1 18:01:42 2024).


task: qa1 length: 16k:   0%|          | 0/100 [00:00<?, ?it/s]

ZeroDivisionError: division by zero

In [123]:
df

,target,output,question
0,bathroom,bathroom,Where is Mary?
1,kitchen,kitchen,Where is Sandra?
2,kitchen,kitchen,Where is Mary?
3,kitchen,kitchen,Where is John?
4,bedroom,bedroom,Where is Sandra?
5,office,office,Where is John?
6,garden,office,Where is Mary?
7,bathroom,office,Where is Sandra?


In [103]:
target

'bathroom'

In [104]:
output

'office'

In [53]:
model_inputs

{'input_ids': tensor([[128000, 128006,   9125,  ...,  78191, 128007,    271]],
        device='cuda:0')}

In [62]:
output

'office'

In [54]:
x1 = [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 35628, 397, 8586, 304, 279, 27394, 568, 1436, 3412, 279, 6671, 369, 264, 4545, 13, 1442, 568, 198, 29177, 636, 76996, 1139, 279, 3838, 11, 279, 2800, 1053, 539, 5030, 13, 48600, 11, 279, 198, 13616, 444, 88, 574, 1633, 6453, 13, 2009, 77638, 818, 11, 323, 568, 8964, 889, 499, 527, 1633, 198, 9336, 11, 13118, 1210, 58072, 956, 279, 6825, 9567, 279, 8732, 10481, 82, 198, 1820, 1023, 1938, 30, 1628, 358, 4510, 279, 35120, 31202, 82, 527, 304, 23383, 315, 53474, 36399, 198, 39428, 264, 1938, 323, 24779, 9096, 323, 11594, 1555, 499, 11, 4491, 13, 1472, 2351, 279, 1455, 5526, 893, 389, 279, 6825, 1210, 17882, 374, 1618, 311, 1518, 198, 22479, 6691, 313, 1875, 1, 47439, 1359, 1071, 76996, 13, 17882, 374, 264, 1633, 2294, 1732, 11, 735, 2637, 779, 387, 198, 10097, 635, 311, 1461, 1210, 7639, 1047, 436, 37588, 520, 279, 5222, 315, 2009, 77638, 818, 596, 7899, 11, 539, 311, 38141, 345, 1073, 3388, 11, 719, 311, 5044, 13, 17586, 904, 10186, 2944, 11, 568, 574, 1203, 304, 198, 95509, 11, 490, 31557, 279, 3224, 19795, 11, 323, 30488, 1461, 27567, 61067, 14518, 1285, 3814, 9135, 330, 791, 4059, 596, 264, 22999, 3245, 11, 4536, 956, 433, 13, 763, 279, 198, 86258, 2533, 9083, 5874, 4024, 1555, 430, 3321, 358, 3077, 1027, 311, 20437, 323, 198, 1445, 0, 735, 13, 17948, 264, 2489, 449, 813, 24981, 6206, 13, 4800, 430, 279, 3245, 1047, 2586, 11, 568, 198, 16514, 16089, 311, 3663, 433, 13, 763, 279, 61363, 11, 813, 11594, 5643, 2840, 13111, 2403, 279, 198, 9471, 13, 330, 32576, 856, 7899, 3952, 499, 1203, 311, 20437, 1210, 5348, 2136, 1047, 58842, 389, 1124, 1578, 11, 3734, 198, 2000, 279, 41630, 37066, 315, 735, 13, 578, 19228, 1828, 6134, 617, 264, 3958, 14464, 315, 11961, 1120, 4871, 279, 198, 2407, 127324, 1210, 358, 3358, 3137, 311, 499, 11, 422, 499, 3358, 198, 47965, 2103, 13, 330, 40, 3077, 1027, 1618, 313, 258, 279, 3363, 11, 358, 3152, 313, 2000, 264, 1060, 13, 4418, 956, 198, 41119, 433, 313, 2356, 77638, 818, 13, 358, 3077, 2751, 264, 2361, 304, 279, 6962, 5274, 11, 57460, 950, 13, 358, 617, 2944, 311, 1781, 358, 2846, 2133, 311, 387, 7882, 198, 455, 13, 3011, 690, 387, 17510, 11, 7344, 17510, 38502, 1210, 17882, 75940, 11, 719, 568, 1766, 912, 26613, 4339, 13, 8442, 264, 961, 315, 1148, 735, 627, 84359, 2751, 311, 1461, 13, 1789, 264, 4545, 568, 574, 1203, 304, 264, 11495, 28913, 11, 323, 420, 198, 1543, 4028, 505, 1461, 313, 275, 574, 539, 92495, 0, 330, 2181, 596, 539, 2653, 990, 11, 323, 433, 596, 6220, 13, 1442, 358, 1304, 264, 16930, 1070, 596, 912, 2324, 198, 71, 8764, 389, 433, 13, 9843, 358, 1903, 264, 1529, 8154, 11, 264, 2305, 477, 1403, 4227, 13, 1102, 2853, 757, 2380, 11441, 704, 315, 856, 1866, 18301, 13, 17882, 596, 7899, 8710, 430, 568, 574, 810, 1109, 9850, 13850, 26, 568, 574, 198, 782, 15456, 398, 7882, 13, 3277, 264, 1060, 198, 42700, 555, 313, 1820, 89862, 1047, 8208, 1523, 11, 323, 19093, 7020, 719, 1148, 499, 1051, 389, 198, 275, 313, 906, 6688, 709, 13, 358, 313, 258, 5651, 584, 2231, 709, 264, 21354, 369, 499, 520, 279, 7926, 13, 358, 198, 42700, 1523, 369, 279, 313, 2000, 279, 3600, 1210, 330, 10267, 433, 4822, 1359, 1071, 735, 13, 30666, 13, 330, 40, 2846, 5710, 439, 3117, 439, 279, 7926, 5900, 345, 3852, 5269, 13, 1628, 11, 369, 32450, 596, 25491, 345, 15357, 956, 387, 14931, 369, 757, 13, 358, 2846, 810, 2262, 291, 1109, 358, 3077, 1027, 369, 264, 1317, 198, 1712, 1210, 578, 5895, 304, 17882, 596, 7899, 574, 7231, 1648, 311, 66510, 13, 8595, 11, 1695, 1283, 402, 729, 11, 893, 11, 358, 1550, 701, 198, 9446, 311, 11477, 11, 323, 358, 3077, 1027, 41442, 922, 433, 3596, 2533, 1210, 3277, 430, 198, 71, 680, 6901, 358, 6688, 709, 13, 2052, 264, 893, 304, 1057, 4913, 706, 374, 264, 3738, 1749, 345, 90153, 313, 6797, 433, 1148, 499, 1093, 11, 313, 438, 10082, 304, 5678, 13, 358, 5675, 856, 198, 726, 91980, 26, 430, 596, 682, 13, 1789, 922, 264, 1060, 358, 574, 198, 15770, 19212, 14931, 369, 7182, 13, 330, 2746, 1475, 46098, 6688, 709, 1606, 568, 5675, 5157, 313, 40, 3077, 1120, 3309, 499, 358, 198, 23770, 701, 5784, 311, 11477, 13, 2684, 574, 1120, 264, 6140, 369, 279, 893, 11, 323, 358, 198, 91453, 856, 25775, 304, 856, 6206, 323, 6818, 433, 13, 735, 13, 16392, 4856, 584, 6751, 323, 94972, 813, 13961, 927, 279, 44952, 13881, 13, 28905, 304, 1450, 11, 568, 14980, 37874, 704, 520, 279, 60243, 32329, 355, 5021, 449, 1202, 27631, 198, 1073, 9958, 13, 12361, 315, 279, 6825, 449, 1202, 11594, 15316, 11, 568, 5602, 279, 3026, 198, 383, 1047, 3967, 323, 6575, 449, 323, 15972, 11, 813, 4885, 889, 12570, 813, 198, 11789, 11, 889, 1047, 10456, 1461, 11, 1690, 315, 1124, 11, 20802, 922, 264, 40907, 198, 99793, 743, 304, 264, 7147, 315, 279, 2362, 7926, 26, 568, 5602, 872, 55349, 12580, 323, 198, 77731, 6548, 13, 1283, 6755, 27567, 1548, 6755, 279, 8579, 23941, 273, 315, 76996, 596, 8679, 439, 1364, 3782, 1139, 279, 2697, 198, 3039, 4920, 1124, 13, 96424, 96860, 1432, 32, 2478, 2919, 1306, 17882, 596, 18324, 315, 735, 2637, 1403, 1455, 13548, 2574, 198, 71, 680, 6901, 311, 76996, 13, 3861, 574, 430, 46993, 4691, 1077, 311, 387, 43165, 315, 16044, 198, 266, 1077, 13306, 13, 3005, 574, 11928, 11, 323, 198, 43068, 1077, 2107, 13, 9393, 1364, 1436, 539, 636, 2162, 430, 3814, 11, 323, 1606, 279, 2697, 3838, 198, 32345, 912, 21186, 11, 1364, 6267, 279, 3754, 311, 1077, 6691, 323, 3288, 264, 5296, 311, 2009, 198, 44, 2303, 818, 1473, 1170, 946, 735, 2637, 313, 40, 1097, 11928, 11, 323, 8871, 374, 389, 856, 2010, 520, 420, 9568, 13, 358, 1097, 439, 198, 64589, 315, 433, 439, 422, 433, 1051, 264, 71812, 11, 323, 439, 422, 358, 1047, 2884, 2555, 311, 198, 5919, 5976, 433, 11, 4619, 315, 1120, 16026, 430, 54108, 358, 4985, 13, 358, 1097, 4477, 198, 576, 389, 279, 44402, 11, 779, 430, 994, 358, 12157, 856, 6548, 358, 1253, 1518, 1102, 13, 358, 1097, 198, 2642, 14152, 1120, 1457, 358, 1097, 7422, 810, 315, 279, 2107, 1109, 315, 1148, 433, 3445, 13, 15668, 5246, 358, 4985, 21818, 1523, 323, 1501, 433, 311, 279, 26741, 13, 358, 4985, 733, 311, 279, 6134, 994, 279, 3814, 29198, 374, 13326, 15038, 11, 323, 198, 413, 682, 2212, 323, 1095, 1124, 1518, 433, 11, 2085, 5605, 264, 3492, 13, 1472, 617, 1027, 1633, 1695, 311, 757, 11, 25237, 735, 13, 1102, 374, 499, 889, 617, 1903, 3284, 198, 576, 23871, 315, 10705, 311, 45427, 13, 358, 1097, 26455, 7182, 311, 387, 1633, 1695, 345, 438, 539, 779, 46604, 11, 323, 311, 3021, 856, 14207, 95054, 8051, 358, 617, 7000, 1457, 13, 9083, 36627, 706, 1120, 40588, 7913, 757, 1455, 47626, 11, 323, 358, 1097, 2771, 8009, 198, 41444, 706, 2225, 73695, 323, 25565, 13, 735, 13, 1766, 279, 5296, 389, 279, 14321, 2007, 994, 568, 2751, 2162, 430, 3814, 11, 323, 198, 7063, 4588, 433, 50007, 311, 1373, 13, 33739, 38678, 3987, 568, 2643, 617, 1047, 430, 198, 1964, 12822, 1053, 5471, 1077, 26586, 568, 7020, 1457, 574, 927, 13, 3161, 279, 198, 9638, 304, 813, 1450, 11, 568, 7731, 555, 813, 2007, 323, 7111, 8469, 1139, 279, 4384, 198, 42820, 13, 2030, 810, 323, 810, 279, 2324, 315, 279, 8952, 1053, 2995, 2177, 1077, 13, 1283, 198, 20370, 76, 4147, 11, 2288, 11, 1633, 559, 4361, 67, 398, 11, 430, 11, 1047, 568, 3596, 1047, 264, 3987, 430, 1364, 198, 46068, 2586, 311, 2512, 369, 1461, 11, 813, 1633, 9546, 304, 279, 2697, 3838, 198, 76, 3748, 660, 2403, 1461, 13, 2684, 574, 7000, 315, 279, 41919, 315, 25768, 280, 383, 574, 2744, 1070, 11, 1093, 45563, 13, 3277, 1364, 9107, 279, 6134, 11, 1364, 2663, 198, 1, 59978, 1, 505, 279, 14321, 13, 1442, 24101, 1550, 539, 4320, 11, 1364, 2663, 1461, 11, 304, 1790, 198, 1820, 1890, 7899, 13, 1283, 1047, 5918, 264, 7147, 315, 19675, 430, 1047, 449, 52784, 1524, 17882, 596, 198, 2827, 31756, 323, 8835, 13, 2030, 52876, 19675, 4131, 1193, 449, 892, 280, 438, 568, 574, 3995, 13, 4800, 323, 1243, 682, 813, 46616, 1589, 26902, 1603, 264, 198, 6519, 290, 430, 11, 994, 568, 76035, 311, 3663, 433, 11, 36014, 1461, 555, 1202, 1633, 8333, 13, 1628, 430, 1938, 682, 813, 43132, 42914, 4024, 1523, 1603, 76996, 596, 6661, 13, 11699, 1633, 198, 1658, 1201, 2136, 323, 37805, 13194, 313, 1962, 430, 568, 1550, 539, 1390, 1077, 37805, 280, 8248, 568, 1589, 4234, 779, 1790, 810, 13, 1283, 22982, 5678, 3663, 1523, 389, 279, 4950, 11, 449, 198, 1820, 5684, 33745, 304, 813, 1450, 13, 76996, 596, 6661, 574, 539, 279, 1193, 832, 568, 4036, 430, 1938, 13, 3277, 11, 304, 198, 2376, 311, 45563, 596, 75400, 11, 568, 16392, 17345, 323, 10235, 369, 14177, 11, 568, 198, 15456, 459, 653, 57228, 35498, 389, 279, 2007, 13, 1102, 574, 505, 7639, 17882, 25, 27567, 1170, 946, 11396, 11672, 56, 4031, 11, 313, 40, 617, 1027, 2133, 2212, 304, 264, 3460, 315, 90409, 682, 1938, 13, 578, 198, 34210, 430, 358, 1193, 6755, 701, 7899, 323, 80987, 5602, 499, 1566, 3814, 706, 198, 28010, 279, 4459, 3245, 1524, 810, 50204, 13, 358, 617, 264, 8430, 315, 61353, 2826, 922, 4560, 311, 1518, 499, 1578, 779, 5246, 13, 358, 2846, 198, 11130, 311, 5201, 701, 513, 9134, 13, 2030, 1070, 527, 1063, 2574, 430, 617, 2751, 198, 998, 387, 14407, 13, 1472, 1071, 1566, 3814, 430, 2574, 1051, 330, 64750, 1, 449, 499, 13, 3234, 499, 1440, 904, 893, 304, 1057, 198, 97235, 889, 706, 539, 30, 1628, 11, 369, 8850, 499, 1781, 358, 656, 539, 1440, 1148, 358, 1097, 198, 83, 30855, 922, 11, 279, 3245, 574, 52844, 291, 704, 520, 279, 3314, 13581, 994, 279, 198, 7998, 315, 279, 21354, 3782, 709, 13, 10846, 44954, 2751, 709, 323, 1071, 25, 330, 38, 306, 42407, 345, 543, 315, 603, 3974, 810, 477, 2753, 304, 9168, 15316, 13, 6914, 1461, 889, 374, 2085, 198, 8890, 3036, 4315, 603, 2571, 279, 1176, 9998, 9135, 358, 3952, 856, 2697, 1841, 323, 23980, 2212, 279, 198, 11389, 19795, 11, 323, 279, 43726, 358, 4024, 279, 810, 55588, 701, 2361, 198, 17106, 373, 13, 358, 2846, 539, 2133, 311, 3350, 904, 5868, 922, 279, 1917, 33921, 3026, 1093, 198, 9514, 11, 8051, 433, 596, 837, 3403, 13, 1472, 3318, 304, 198, 64, 6962, 5274, 11, 1418, 2362, 507, 68070, 5169, 293, 2234, 645, 323, 60884, 11, 323, 358, 14993, 3235, 198, 263, 1148, 358, 9687, 505, 499, 0, 1102, 5097, 25775, 311, 3094, 1523, 505, 279, 99589, 499, 14980, 389, 13, 2100, 433, 596, 198, 1962, 66275, 560, 430, 706, 743, 499, 1523, 1618, 13, 578, 1176, 11, 323, 1888, 11, 374, 369, 499, 311, 733, 1203, 13, 2360, 832, 706, 4529, 701, 2035, 11, 1606, 912, 832, 1436, 656, 279, 990, 13, 2030, 422, 198, 9210, 596, 704, 315, 279, 3488, 11, 313, 438, 1193, 499, 1440, 430, 11, 369, 1193, 499, 1440, 198, 1820, 13363, 11, 313, 1820, 1828, 1888, 3245, 374, 420, 11, 323, 304, 682, 75372, 358, 1304, 279, 198, 82, 43216, 13, 12040, 279, 3314, 40786, 1234, 701, 3118, 836, 11, 323, 994, 499, 3077, 2751, 701, 198, 64686, 11, 2586, 304, 449, 757, 13, 358, 3358, 387, 3794, 264, 198, 89719, 14254, 810, 1109, 358, 3041, 13, 1102, 374, 264, 22999, 2144, 430, 264, 893, 889, 374, 11112, 653, 57755, 43629, 922, 198, 65762, 374, 3629, 279, 13836, 315, 16044, 311, 1023, 3026, 13, 578, 14992, 17882, 345, 89894, 813, 72974, 34504, 323, 539, 2288, 14572, 16252, 398, 11, 574, 3339, 459, 198, 26133, 430, 8967, 813, 17139, 56528, 11, 323, 3815, 433, 26085, 3725, 11, 449, 813, 198, 58331, 1825, 13, 1102, 574, 1093, 7639, 311, 1304, 1778, 459, 3085, 11, 1093, 1461, 311, 1304, 433, 198, 300, 422, 568, 1051, 10371, 264, 4799, 323, 539, 2389, 14782, 832, 13, 2030, 279, 3085, 2163, 198, 40617, 653, 4307, 291, 13, 1283, 1047, 47712, 5678, 304, 279, 8335, 11, 323, 1766, 5678, 198, 53757, 287, 13, 2360, 21354, 389, 279, 7926, 7147, 1436, 2349, 430, 13, 1628, 994, 345, 5185, 430, 3814, 11, 17882, 1766, 1461, 389, 279, 44952, 323, 3779, 14638, 311, 198, 14819, 11, 279, 6671, 14958, 35957, 13, 1283, 15393, 1202, 3987, 33028, 198, 9493, 735, 13, 326, 9381, 1139, 70205, 950, 28485, 13, 330, 40, 2846, 539, 11112, 29948, 1405, 358, 1097, 11, 499, 1440, 11, 7639, 1359, 568, 1071, 13, 330, 40, 3077, 198, 93613, 2380, 42120, 11012, 323, 264, 3070, 315, 83190, 420, 7474, 11, 9087, 198, 998, 3197, 264, 490, 46651, 2933, 11, 39061, 304, 27397, 7147, 77819, 369, 279, 3130, 1120, 198, 42450, 11, 313, 23770, 499, 5406, 433, 30, 313, 438, 8040, 264, 8334, 42070, 449, 264, 5041, 198, 9210, 62990, 2212, 279, 16120, 1093, 264, 4349, 645, 59904, 2212, 264, 12786, 396, 9135, 330, 2746, 499, 2351, 2133, 311, 387, 70946, 313, 1875, 46240, 25237, 12637, 1359, 1071, 735, 13, 30666, 11, 330, 333, 358, 1047, 912, 5647, 315, 28485, 11, 358, 1288, 198, 3427, 50007, 311, 45427, 11, 2543, 389, 279, 6962, 11, 323, 1304, 264, 357, 531, 20857, 20396, 198, 18614, 64482, 13, 3296, 279, 1648, 11, 430, 596, 2555, 358, 29695, 9135, 22395, 856, 1023, 7640, 11, 358, 53221, 279, 1370, 9761, 369, 198, 64465, 3177, 13, 578, 42447, 4791, 15502, 25283, 1063, 25396, 41476, 439, 13306, 198, 53430, 82, 11, 323, 313, 1875, 92493, 16392, 323, 1344, 2234, 813, 36213, 1139, 279, 16763, 13, 735, 13, 16392, 449, 1461, 11, 323, 682, 279, 56089, 8430, 315, 279, 7274, 574, 198, 52905, 9988, 1139, 813, 1566, 2478, 4339, 13, 330, 40, 2846, 539, 439, 653, 911, 21508, 439, 499, 1781, 11, 7639, 1359, 568, 1071, 13, 330, 40, 313, 9514, 3077, 9087, 198, 64, 2763, 13, 358, 2846, 439, 1664, 1022, 439, 358, 23528, 311, 387, 11, 323, 198, 58234, 13, 17882, 596, 16907, 8622, 16348, 488, 2231, 735, 13, 304, 264, 22999, 2361, 313, 2414, 1461, 345, 300, 433, 1051, 11, 449, 264, 18255, 68078, 13, 76996, 596, 26875, 4225, 266, 4090, 369, 198, 1820, 3995, 46098, 574, 7982, 13, 1628, 1405, 1603, 198, 383, 2643, 617, 6612, 35516, 304, 2133, 311, 279, 3160, 315, 10163, 1077, 11, 1457, 198, 26301, 6206, 1051, 17791, 13, 4497, 1109, 3131, 568, 1047, 198, 83261, 76996, 1203, 311, 279, 8952, 304, 813, 1841, 13, 2009, 77638, 818, 11, 48270, 5795, 520, 198, 30115, 2543, 11, 1766, 5678, 13176, 1403, 27548, 11, 832, 719, 2697, 2731, 198, 54895, 279, 1023, 13, 578, 31017, 2643, 1629, 264, 23583, 3388, 11, 13696, 304, 198, 5730, 8950, 313, 64, 1060, 315, 23871, 369, 1077, 11, 323, 1243, 1148, 11103, 449, 198, 6102, 11, 439, 568, 7020, 1461, 11, 1053, 40605, 3152, 25, 40320, 826, 3201, 11, 79157, 1285, 198, 4310, 311, 1077, 11, 4225, 27900, 1385, 11, 58701, 13, 2582, 11, 433, 2643, 387, 2753, 6129, 719, 198, 60301, 18813, 43251, 369, 1077, 13, 7639, 2643, 2571, 28930, 311, 279, 29592, 345, 79, 1759, 361, 1077, 369, 264, 892, 11, 313, 42, 13, 1047, 3970, 1461, 656, 420, 11, 313, 438, 1243, 11, 7982, 198, 83, 2757, 11, 2349, 311, 1063, 502, 33464, 13, 763, 3060, 1162, 11, 568, 1436, 1193, 3868, 198, 438, 3821, 11, 12459, 813, 4851, 704, 2391, 279, 1317, 59938, 994, 24101, 1373, 198, 1964, 330, 44653, 61399, 1, 50007, 323, 568, 7731, 7636, 449, 813, 13961, 389, 279, 198, 13616, 444, 88, 13, 76996, 4024, 389, 3814, 14523, 20193, 1306, 1077, 26586, 13, 2052, 315, 1077, 198, 1382, 398, 3995, 2324, 1047, 1027, 18255, 1139, 1403, 5596, 25, 1938, 11, 994, 832, 198, 51908, 477, 6575, 11, 323, 3814, 11, 994, 832, 46498, 13, 4800, 1364, 574, 48984, 311, 198, 64, 1373, 4345, 479, 25, 832, 6575, 304, 279, 3814, 323, 46498, 304, 279, 1938, 13, 2468, 279, 842, 315, 1077, 1176, 3814, 1934, 76996, 198, 17249, 1148, 1364, 1436, 6227, 315, 264, 2697, 33487, 315, 83048, 596, 13, 3005, 198, 17249, 433, 311, 279, 842, 315, 1077, 4689, 1934, 11, 902, 574, 311, 279, 2515, 430, 198, 73826, 1047, 1027, 11594, 2391, 279, 3814, 3734, 279, 12818, 13, 330, 3112, 1587, 433, 539, 2873, 2653, 311, 499, 345, 415, 3277, 682, 279, 13180, 374, 2867, 323, 6437, 345, 415, 1628, 358, 1288, 1093, 779, 1790, 311, 1514, 345, 415, 2057, 617, 311, 733, 311, 4950, 555, 1938, 7673, 578, 1938, 18328, 7077, 389, 279, 1934, 11, 323, 574, 5115, 26681, 1534, 13, 330, 2746, 279, 3814, 37465, 527, 311, 8493, 872, 892, 3339, 709, 32349, 1359, 1364, 198, 84359, 5425, 398, 11, 330, 906, 4265, 2731, 2349, 420, 8952, 1139, 264, 3995, 23628, 1270, 325, 1083, 661, 13, 1442, 1364, 6944, 311, 29011, 922, 279, 12248, 304, 279, 8761, 11, 1364, 198, 5562, 656, 779, 304, 6300, 1376, 1210, 330, 40, 1541, 956, 1781, 1364, 1903, 433, 709, 1359, 1071, 279, 11452, 11, 4560, 539, 311, 15648, 13, 330, 40, 3077, 6755, 2555, 1093, 433, 15038, 11, 323, 11, 1148, 449, 279, 8798, 323, 279, 198, 53318, 315, 9629, 11, 358, 1541, 956, 1518, 1268, 904, 315, 1124, 636, 904, 6212, 1210, 2030, 11, 1606, 26434, 2011, 387, 13468, 11, 1364, 6267, 389, 279, 21818, 279, 198, 78191, 11953, 2212, 25, 330, 5618, 9502, 3814, 6821, 304, 61801, 1210, 3005, 259, 26902, 1139, 1077, 3428, 4950, 520, 11888, 297, 63510, 198, 258, 279, 6693, 11, 1884, 2919, 11, 449, 1077, 70960, 7013, 63266, 20716, 4591, 1523, 198, 1964, 1203, 323, 1077, 34296, 1071, 11, 323, 7214, 1077, 4642, 3995, 4059, 198, 28611, 449, 5448, 313, 19875, 483, 596, 13306, 11, 2999, 13, 7639, 12579, 279, 6134, 315, 1077, 198, 820, 26741, 323, 1364, 539, 1070, 11, 13142, 313, 17206, 40788, 648, 11, 6832, 3446, 574, 1457, 279, 198, 82, 42725, 315, 279, 6825, 13, 362, 2478, 4038, 1603, 1364, 1053, 539, 617, 42777, 198, 998, 1781, 315, 40788, 648, 13, 3005, 1053, 617, 22311, 1077, 1139, 279, 4363, 315, 198, 58174, 19101, 1474, 592, 15548, 456, 13, 2030, 279, 6825, 596, 45593, 1051, 539, 10168, 198, 68753, 3520, 596, 11555, 1457, 13, 3005, 87420, 927, 40788, 648, 264, 2294, 3568, 11, 323, 927, 198, 87643, 323, 1077, 3169, 13, 1952, 1077, 1176, 3814, 389, 14523, 11, 264, 3828, 1047, 1027, 7263, 304, 505, 279, 17569, 13, 3005, 1047, 4529, 264, 21109, 313, 77, 43440, 7020, 1120, 1148, 13, 3277, 279, 2655, 288, 1047, 198, 376, 1142, 311, 1505, 704, 11, 1364, 1047, 1193, 1071, 25, 330, 3923, 596, 279, 1005, 7673, 1884, 58302, 994, 1364, 1436, 539, 636, 198, 998, 6212, 13, 9029, 1051, 3169, 313, 5794, 1051, 3169, 11, 2216, 11, 313, 438, 3686, 11, 369, 1063, 198, 20489, 477, 1023, 11, 1884, 2574, 1047, 311, 387, 13, 4740, 264, 892, 76996, 1053, 656, 3059, 5052, 3725, 13, 2030, 555, 2380, 297, 63510, 1364, 574, 198, 33222, 709, 323, 32623, 13, 68502, 315, 198, 26894, 6267, 42902, 82, 2212, 1077, 6548, 323, 7577, 1063, 315, 1077, 10107, 1933, 13, 28232, 2380, 323, 3116, 297, 63510, 304, 279, 6693, 1364, 574, 43206, 389, 198, 67, 20850, 555, 264, 4832, 52819, 315, 6212, 13, 2684, 574, 264, 16750, 369, 21811, 389, 198, 67, 20850, 13, 578, 2362, 3814, 3821, 1543, 1047, 264, 1648, 315, 64802, 709, 389, 832, 16387, 7113, 13, 578, 3814, 37465, 37287, 814, 2643, 5043, 268, 264, 29519, 389, 1461, 0, 48600, 11, 520, 3116, 3782, 4216, 1474, 80675, 20472, 26, 430, 436, 37588, 1077, 13, 1628, 198, 10924, 430, 3782, 279, 1206, 1683, 315, 4216, 14403, 2695, 351, 2439, 323, 279, 16392, 82757, 315, 198, 67, 6513, 927, 279, 77395, 13, 94666, 304, 279, 3814, 11, 3131, 520, 87117, 323, 1578, 9017, 198, 67, 6513, 11, 1364, 54244, 3831, 3776, 11033, 13, 2030, 1306, 264, 2046, 477, 1403, 1077, 50581, 198, 52898, 41398, 259, 2784, 439, 264, 925, 13, 6385, 8216, 574, 304, 264, 2678, 3130, 3345, 311, 1077, 2380, 78251, 13, 2030, 1364, 7731, 198, 1225, 2697, 11, 439, 264, 5030, 315, 2144, 13, 6385, 12014, 574, 8987, 389, 1077, 280, 32158, 1903, 21420, 20101, 13, 578, 3389, 7474, 22178, 1051, 5052, 1285, 11, 34653, 819, 280, 1820, 6453, 6901, 78251, 41398, 3201, 1093, 76304, 82, 505, 279, 5213, 3177, 3221, 198, 1820, 6134, 13, 1628, 505, 704, 315, 1521, 76304, 82, 3782, 6896, 69733, 24196, 11, 83372, 198, 31061, 3808, 11, 279, 87211, 315, 264, 10747, 389, 264, 96803, 11, 902, 574, 279, 8450, 315, 198, 339, 874, 13, 2057, 1124, 11, 8530, 1120, 198, 64, 2697, 76656, 449, 892, 323, 1790, 2532, 11, 279, 87211, 10747, 8967, 539, 779, 198, 59178, 50690, 439, 91904, 13, 330, 8161, 956, 7940, 1093, 430, 11, 1716, 26, 814, 2351, 539, 66302, 291, 11, 499, 1440, 1210, 330, 4071, 422, 499, 617, 264, 34653, 323, 527, 98208, 313, 1875, 1, 1016, 874, 88, 4400, 0, 330, 12487, 1359, 76996, 1053, 2019, 11, 16448, 594, 9887, 11, 330, 20670, 527, 2133, 311, 1518, 757, 1210, 22547, 1870, 279, 9191, 7724, 5602, 430, 1364, 1053, 539, 3665, 11937, 13, 2435, 198, 44336, 1077, 1633, 1790, 11, 323, 814, 11, 2288, 11, 1047, 3940, 304, 449, 10032, 7693, 198, 438, 28682, 6206, 26, 719, 279, 16579, 323, 832, 18651, 315, 872, 2532, 198, 32345, 61508, 1124, 9235, 13, 2435, 1051, 11297, 11, 7155, 62166, 11, 4062, 100051, 198, 55377, 1572, 11, 3815, 872, 1888, 11, 315, 3388, 11, 719, 61469, 505, 76996, 304, 430, 198, 50828, 2532, 574, 315, 279, 4059, 11, 1418, 11074, 574, 315, 279, 4851, 13, 2057, 1124, 345, 92724, 574, 264, 3245, 311, 387, 12715, 389, 264, 1934, 26, 311, 76996, 11, 433, 574, 5439, 198, 263, 279, 29679, 315, 1077, 13836, 13, 22770, 22983, 36627, 4024, 389, 3814, 14523, 520, 279, 1890, 892, 313, 1964, 1566, 3814, 198, 8095, 11, 439, 433, 574, 76996, 596, 1176, 13, 3005, 1047, 198, 14215, 315, 279, 2380, 78251, 389, 279, 6558, 1120, 3770, 76996, 11, 323, 315, 279, 198, 1637, 1139, 902, 682, 13147, 5157, 1051, 4529, 13, 1102, 574, 264, 5107, 198, 8095, 11, 8530, 279, 1455, 5107, 304, 279, 3838, 13, 2522, 8362, 989, 264, 3814, 4024, 198, 1729, 2085, 1202, 33622, 477, 51632, 1162, 13, 31137, 258, 6751, 11, 279, 13147, 26741, 198, 32345, 1202, 1866, 3814, 29198, 13, 7984, 660, 198, 54803, 811, 323, 17563, 1047, 79266, 279, 4967, 35789, 13, 22770, 22983, 11, 2728, 198, 4429, 14523, 11, 16632, 70756, 1077, 28004, 13, 330, 40, 3077, 2744, 1047, 2574, 5128, 2653, 1618, 1359, 1364, 29786, 27851, 13, 330, 4599, 358, 733, 704, 11, 358, 3358, 3060, 387, 40883, 3403, 311, 1629, 264, 4459, 8952, 198, 15698, 71, 6601, 11, 477, 358, 3358, 387, 11953, 704, 7693, 1176, 1210, 3005, 7020, 1077, 2731, 1109, 1364, 7020, 198, 1820, 1023, 37465, 13, 15344, 69074, 1051, 15320, 40986, 323, 9455, 198, 1964, 520, 264, 4814, 13, 9843, 520, 3325, 1475, 3814, 11, 9083, 36627, 1053, 6865, 264, 198, 3594, 108593, 505, 279, 1203, 62862, 430, 8599, 279, 1403, 27035, 11, 323, 345, 9738, 704, 11, 1053, 1518, 76996, 596, 74820, 3663, 323, 10284, 14425, 11059, 2107, 198, 65, 2518, 927, 279, 37649, 3880, 607, 13, 330, 40, 2846, 36075, 3725, 14931, 311, 20753, 499, 1359, 1364, 1053, 2019, 11, 330, 8248, 2100, 9976, 12, 4516, 198, 55767, 956, 617, 264, 34653, 9061, 5233, 477, 11, 330, 40, 3077, 264, 5333, 1618, 889, 41716, 1077, 198, 83543, 1210, 5112, 1053, 1833, 11295, 4860, 323, 18813, 11295, 11503, 13, 24191, 439, 22770, 22983, 99733, 323, 38569, 279, 3828, 32115, 11, 433, 2646, 198, 14310, 8293, 311, 1077, 311, 26122, 1077, 13291, 13, 19292, 279, 46968, 889, 2567, 198, 1820, 2294, 3335, 690, 2231, 430, 311, 1077, 6807, 13, 76996, 5602, 1077, 1176, 4648, 20193, 1306, 1364, 4024, 389, 3814, 14523, 13, 1102, 574, 198, 1820, 1455, 17936, 3217, 315, 682, 1077, 2324, 26, 323, 3686, 11, 439, 4648, 5900, 11, 433, 198, 16514, 11594, 3403, 13, 2100, 53722, 574, 433, 430, 76996, 11, 449, 735, 3238, 82, 2697, 3821, 198, 258, 1450, 11, 574, 539, 2771, 7041, 994, 433, 7077, 13, 578, 3177, 574, 1633, 5213, 198, 30998, 485, 279, 2697, 4264, 13, 3861, 4545, 279, 11071, 574, 934, 85040, 10284, 198, 8154, 279, 14993, 369, 11745, 11, 279, 1828, 433, 574, 2103, 13, 3011, 2324, 11, 779, 4754, 11, 779, 28040, 264, 198, 1626, 11, 1436, 842, 779, 5335, 8129, 13610, 11, 430, 279, 1317, 8209, 1288, 30754, 198, 33222, 304, 420, 61510, 2987, 313, 275, 9508, 311, 1077, 430, 1364, 1436, 539, 2559, 198, 275, 13, 18152, 311, 682, 1077, 1023, 502, 5435, 315, 5496, 574, 420, 832, 315, 23069, 13, 3005, 1903, 21294, 11, 315, 3388, 11, 902, 279, 47626, 37465, 29695, 311, 198, 11998, 313, 18275, 1354, 2163, 922, 11, 6103, 389, 1077, 7576, 13, 3005, 97139, 291, 1077, 198, 700, 76, 21037, 304, 4106, 3090, 832, 3814, 11, 323, 89815, 459, 958, 818, 555, 11889, 198, 40617, 3492, 430, 10455, 84164, 556, 596, 9499, 574, 264, 7895, 323, 5899, 12628, 13, 3005, 1095, 264, 1624, 404, 1245, 8893, 12731, 505, 279, 26741, 2500, 3814, 323, 733, 198, 1334, 1570, 1523, 279, 4027, 12, 13295, 1603, 1364, 11352, 1148, 1047, 7077, 0, 5112, 1364, 39575, 11937, 555, 16706, 1523, 279, 11245, 62862, 323, 198, 81088, 279, 91740, 1203, 3254, 37240, 13, 1789, 46993, 596, 13306, 279, 6825, 22982, 1022, 1202, 1377, 370, 69260, 323, 19655, 198, 64, 13306, 60718, 13, 763, 279, 7314, 433, 574, 9850, 13850, 922, 1063, 315, 279, 198, 15216, 13, 330, 2127, 1384, 1251, 505, 279, 3838, 6134, 311, 279, 44962, 11046, 11, 323, 264, 79680, 9135, 35120, 31202, 11, 889, 574, 9455, 24981, 14740, 520, 279, 52290, 89, 198, 7830, 13, 330, 3112, 2500, 1384, 1251, 520, 279, 8993, 11, 449, 264, 2579, 28954, 9135, 35120, 31202, 1047, 11721, 2162, 323, 574, 3339, 709, 802, 265, 1590, 315, 2800, 323, 198, 265, 38475, 13, 330, 10445, 656, 814, 2610, 364, 336, 422, 814, 1541, 956, 7095, 364, 336, 7673, 2030, 279, 6420, 315, 279, 86219, 1047, 1027, 34707, 13, 1102, 32230, 311, 198, 40617, 1690, 2574, 430, 1051, 2731, 25565, 13, 1283, 16392, 323, 1156, 363, 839, 520, 813, 198, 48841, 13, 330, 2675, 3371, 32980, 2555, 369, 757, 1359, 568, 4224, 277, 839, 13, 330, 2675, 3371, 1461, 994, 568, 198, 325, 288, 813, 7126, 11689, 1523, 8761, 11, 323, 568, 274, 1468, 258, 6, 709, 1070, 7636, 389, 198, 9210, 35528, 11, 358, 1390, 1461, 311, 3009, 323, 3820, 757, 709, 994, 358, 65350, 1461, 13, 2206, 198, 81749, 11, 1418, 856, 4538, 2064, 6572, 2212, 304, 264, 1841, 0, 330, 2675, 1095, 757, 6865, 315, 1461, 5754, 2902, 788, 258, 518, 323, 198, 40, 3358, 5622, 1461, 9135, 578, 13306, 574, 311, 387, 520, 4330, 297, 63510, 13, 1115, 11, 304, 5196, 11, 711, 1142, 682, 198, 48126, 6055, 315, 279, 6825, 11, 902, 574, 3060, 12502, 304, 279, 1633, 4216, 198, 57902, 1251, 520, 279, 16879, 8993, 477, 520, 8223, 297, 63510, 304, 279, 11714, 520, 198, 1820, 86464, 13, 2684, 574, 2555, 54317, 922, 4330, 297, 63510, 13, 1102, 1047, 264, 55641, 8430, 430, 8530, 1778, 264, 198, 5730, 8950, 574, 539, 5115, 5897, 13, 578, 3488, 315, 1148, 311, 10051, 6244, 11, 369, 279, 3026, 11, 459, 55349, 832, 13, 3279, 198, 417, 59100, 291, 459, 2362, 3776, 13091, 377, 23283, 266, 323, 1047, 264, 330, 53, 1, 315, 3776, 6730, 1347, 292, 743, 198, 258, 279, 28705, 13, 45236, 11, 279, 10707, 3913, 11, 49959, 264, 4018, 14075, 11, 323, 11021, 264, 198, 943, 49136, 311, 10051, 449, 433, 13, 578, 47842, 9976, 1773, 3635, 2363, 8479, 889, 84569, 520, 198, 26353, 42, 5633, 518, 323, 889, 11, 555, 2944, 315, 813, 95623, 290, 11, 574, 73170, 48080, 315, 198, 1820, 28361, 2212, 1461, 11, 28670, 264, 46664, 8679, 1355, 3159, 11, 323, 6646, 198, 40617, 726, 311, 279, 842, 315, 813, 2919, 279, 1193, 10489, 1651, 2757, 893, 304, 279, 198, 65294, 13, 578, 14992, 17882, 574, 311, 387, 832, 315, 279, 603, 71, 388, 13, 3277, 279, 32594, 3782, 198, 412, 449, 279, 4756, 1160, 323, 420, 574, 11352, 11, 439, 1664, 439, 430, 198, 68753, 3520, 574, 279, 43165, 315, 16044, 11, 1070, 574, 264, 12742, 934, 1553, 1555, 279, 198, 71127, 4967, 35789, 13, 362, 47027, 261, 574, 19144, 311, 1505, 704, 198, 4581, 24553, 82, 13, 1102, 574, 279, 1938, 315, 279, 13306, 1243, 11, 323, 76996, 11, 889, 1047, 198, 1962, 1027, 311, 4950, 520, 682, 11, 574, 11961, 304, 264, 40798, 3321, 304, 279, 98632, 10843, 198, 28192, 327, 11, 46479, 1077, 7013, 13, 330, 40, 313, 40, 1120, 5895, 1359, 1364, 1071, 11, 330, 333, 499, 1053, 1095, 1063, 315, 279, 7724, 2586, 198, 258, 311, 1518, 499, 994, 499, 2351, 26435, 7673, 330, 2181, 596, 1384, 3725, 57399, 11, 4536, 956, 433, 30, 330, 11787, 499, 2133, 311, 4321, 1523, 279, 60779, 449, 1461, 7673, 2435, 1047, 264, 89955, 1566, 3814, 11, 719, 315, 3388, 358, 574, 539, 198, 19041, 13, 578, 47027, 261, 1047, 1027, 42075, 311, 1505, 704, 1023, 2574, 26, 779, 1364, 743, 198, 998, 990, 449, 264, 8571, 520, 76996, 596, 7013, 13, 330, 1548, 596, 1384, 3725, 1695, 31348, 11, 4536, 956, 568, 7673, 3005, 574, 539, 48080, 315, 279, 5528, 315, 279, 2978, 13, 1442, 198, 576, 3828, 574, 53226, 1077, 27567, 7189, 3358, 617, 311, 1781, 430, 927, 1359, 1364, 1071, 11, 449, 264, 2840, 396, 315, 95046, 304, 198, 1964, 6548, 13, 330, 4599, 499, 1440, 264, 1732, 50136, 1664, 11, 499, 20781, 1440, 3508, 198, 383, 596, 1695, 31348, 477, 539, 1210, 330, 40, 23289, 1359, 1071, 279, 47027, 261, 11, 4401, 279, 1317, 69864, 315, 76996, 596, 198, 51729, 1555, 1077, 19779, 11, 330, 9210, 994, 499, 527, 520, 2162, 499, 1518, 1461, 3629, 1210, 76996, 2751, 1022, 279, 3321, 1355, 484, 11, 323, 11, 4737, 279, 47027, 261, 37163, 398, 555, 198, 1820, 28004, 11, 17011, 1077, 9017, 279, 6134, 13, 330, 2675, 733, 1203, 311, 279, 7724, 1359, 1364, 1071, 11, 330, 438, 3371, 1124, 311, 2586, 304, 323, 1518, 198, 2727, 994, 358, 1097, 26435, 11, 323, 3371, 1124, 420, 25, 358, 1541, 956, 1440, 3508, 358, 1097, 311, 198, 19599, 1523, 279, 60779, 449, 2999, 13, 3005, 83198, 279, 47027, 261, 704, 1139, 279, 14321, 323, 16447, 279, 6134, 4920, 198, 1964, 13, 3011, 1984, 304, 1202, 49017, 8813, 22770, 22983, 36627, 13, 3005, 11, 2288, 11, 1047, 539, 46498, 2391, 279, 1938, 13, 3277, 279, 47027, 261, 889, 198, 32345, 7263, 1077, 279, 1934, 1047, 8208, 704, 11, 1364, 11203, 304, 1077, 1317, 4251, 198, 9471, 2427, 785, 11, 6206, 57220, 7018, 1234, 1077, 2010, 11, 323, 45135, 520, 279, 35684, 12970, 198, 346, 8138, 315, 1077, 2697, 3130, 13, 3005, 5602, 1070, 76996, 304, 1077, 4251, 8679, 2133, 1523, 279, 60779, 315, 279, 198, 65294, 26, 1364, 5602, 279, 1912, 2212, 279, 51943, 26, 323, 11, 439, 22225, 439, 1364, 11203, 198, 19041, 11, 1364, 7020, 430, 7639, 17882, 596, 6548, 1053, 387, 11, 539, 389, 279, 42447, 11, 719, 198, 263, 279, 3828, 889, 14980, 30488, 1077, 13, 578, 22999, 3245, 574, 430, 22770, 22983, 6612, 430, 1364, 1436, 3009, 279, 13306, 198, 333, 1364, 4934, 311, 13, 3005, 4265, 7077, 389, 264, 2766, 315, 2038, 313, 35676, 264, 13306, 198, 32345, 1027, 10717, 369, 2753, 13, 1102, 4856, 44898, 1077, 311, 1781, 315, 23351, 198, 1820, 13306, 11, 779, 430, 76996, 323, 7639, 1053, 539, 4321, 1523, 279, 60779, 198, 998, 3522, 13, 2684, 3782, 11, 520, 1566, 11, 459, 6596, 1603, 279, 13306, 11, 264, 326, 620, 304, 279, 34653, 819, 198, 33078, 315, 279, 3766, 2305, 13, 763, 279, 52290, 89, 198, 74, 7606, 11, 59872, 315, 25485, 11, 366, 55319, 1227, 29, 3868, 388, 11, 10054, 12, 47100, 1949, 60719, 11, 323, 18083, 13, 763, 279, 74721, 11, 304, 279, 4219, 315, 264, 198, 15470, 11, 1603, 264, 27306, 17203, 902, 1047, 1027, 11953, 50007, 369, 1077, 198, 68244, 275, 11, 7731, 11, 389, 420, 1077, 1938, 315, 2919, 11, 279, 42447, 13, 2052, 279, 2132, 3446, 198, 32345, 1027, 10235, 369, 15051, 323, 18911, 13, 8956, 1705, 1051, 2103, 13326, 304, 279, 3130, 3770, 13, 426, 18746, 98250, 1051, 93858, 198, 263, 279, 2697, 62862, 11, 58218, 927, 520, 1855, 502, 10264, 315, 279, 29519, 323, 198, 74826, 6821, 311, 46993, 1555, 279, 8036, 6134, 25, 27567, 1, 14364, 23162, 3830, 11, 46993, 13, 3639, 690, 499, 198, 2099, 656, 449, 1124, 682, 7673, 5810, 596, 2500, 315, 279, 19228, 889, 6944, 311, 1518, 1268, 499, 198, 7349, 13, 3234, 2019, 499, 649, 956, 617, 904, 15613, 1457, 1210, 46993, 7731, 7636, 304, 279, 4219, 315, 1077, 11071, 13, 578, 61365, 98250, 1047, 1027, 198, 267, 944, 398, 37913, 311, 2586, 1139, 1077, 3130, 13, 330, 40, 9167, 956, 1047, 264, 6140, 311, 1781, 369, 264, 2305, 1359, 1364, 1071, 13, 330, 3112, 358, 3077, 2751, 198, 15031, 2574, 358, 3077, 2751, 311, 1781, 704, 1210, 2030, 11, 994, 76996, 3782, 11, 1364, 3288, 369, 1077, 13, 76996, 1766, 1077, 11961, 389, 264, 198, 267, 3168, 10716, 11, 304, 1077, 13306, 65510, 11, 449, 1077, 59355, 9041, 704, 389, 264, 2678, 198, 2752, 13, 1628, 11, 1306, 76996, 1047, 59334, 1077, 25, 27567, 7189, 3077, 264, 1695, 4059, 539, 311, 656, 433, 1210, 330, 2675, 2351, 19781, 323, 23418, 11, 430, 596, 682, 1210, 2030, 430, 4536, 956, 1148, 596, 5076, 449, 757, 13, 32445, 430, 59355, 198, 15031, 2035, 323, 2503, 1523, 1210, 46993, 574, 38048, 29465, 3640, 11, 264, 1633, 36301, 5916, 13, 76996, 3463, 198, 1347, 3422, 1288, 387, 4856, 28639, 13, 2030, 1234, 1077, 6548, 1051, 5238, 430, 76996, 198, 32345, 2646, 3970, 1070, 1603, 13, 330, 40, 2846, 539, 2133, 311, 387, 46337, 11, 76996, 13, 358, 3358, 733, 1555, 449, 433, 11, 315, 198, 12120, 13, 1102, 1053, 2231, 90411, 304, 1077, 25165, 422, 358, 1903, 264, 6237, 1457, 1210, 330, 20577, 1195, 6688, 813, 49683, 14177, 520, 279, 14438, 10349, 1566, 3814, 13, 94192, 2663, 7126, 709, 311, 11477, 323, 198, 84359, 430, 42216, 1047, 94972, 264, 16893, 315, 13378, 1139, 279, 27374, 13, 1283, 12775, 956, 198, 82850, 1618, 311, 11477, 1210, 1628, 439, 369, 279, 1023, 313, 66372, 433, 5828, 956, 42216, 889, 1550, 198, 275, 1210, 14853, 4038, 1603, 11, 8530, 11, 76996, 1436, 539, 617, 6981, 291, 1077, 26, 719, 198, 28956, 4038, 1047, 1903, 264, 2349, 304, 76996, 13, 578, 4684, 18680, 19219, 380, 4108, 198, 1073, 1077, 3828, 5812, 912, 5129, 19089, 369, 8206, 13, 3005, 2231, 1077, 11977, 2212, 198, 19875, 483, 596, 28004, 13, 330, 32, 893, 889, 21662, 374, 264, 11102, 312, 291, 1359, 1071, 46993, 13, 330, 4897, 596, 1148, 358, 2846, 198, 9738, 311, 32918, 323, 16025, 389, 279, 2800, 315, 856, 2324, 313, 64, 11102, 312, 291, 13, 3005, 2751, 709, 6288, 11, 323, 11, 28848, 1077, 1317, 78184, 5542, 4028, 279, 6558, 345, 53533, 291, 279, 6134, 13, 5112, 505, 4871, 1077, 45011, 425, 1364, 7263, 704, 323, 5762, 198, 998, 76996, 264, 6661, 13, 1102, 574, 1633, 2875, 26, 76996, 1373, 433, 520, 264, 34522, 25, 27567, 27264, 701, 3938, 10177, 422, 568, 8964, 264, 3828, 520, 220, 11702, 11556, 17569, 13, 14853, 4038, 1603, 11, 279, 17569, 1053, 617, 8967, 4400, 311, 76996, 13, 58795, 15187, 76996, 7020, 889, 279, 3828, 520, 220, 11702, 11556, 17569, 574, 13, 578, 198, 19716, 1364, 5762, 304, 1077, 1450, 574, 8952, 5684, 449, 279, 14836, 21830, 1022, 13, 578, 4459, 274, 541, 307, 3446, 11203, 1603, 1077, 25, 32171, 56310, 11, 449, 1077, 15792, 3663, 198, 438, 61243, 7013, 11, 323, 279, 17222, 389, 279, 6558, 315, 279, 26741, 30488, 1077, 0, 3861, 315, 279, 61365, 98250, 270, 61292, 65300, 389, 279, 6134, 4994, 13, 330, 14364, 9249, 29062, 1359, 1364, 2663, 12304, 398, 1555, 279, 6134, 13, 330, 2675, 1518, 1359, 46993, 1071, 33201, 6751, 13, 330, 40, 617, 4036, 2500, 9249, 198, 95857, 11, 323, 42216, 374, 63891, 0, 358, 3077, 2751, 311, 733, 1555, 449, 433, 11, 358, 198, 13066, 2972, 13, 578, 1193, 6811, 1990, 757, 323, 1023, 61365, 374, 430, 358, 1440, 198, 12840, 358, 2846, 3794, 13, 330, 2181, 596, 2288, 3389, 311, 656, 4205, 775, 13, 358, 1097, 539, 2133, 311, 3041, 420, 198, 37569, 5812, 4205, 311, 3137, 922, 1210, 3005, 13061, 709, 1077, 59355, 323, 743, 279, 22760, 295, 389, 1077, 2010, 13, 76996, 14980, 198, 4291, 279, 6661, 304, 1077, 6206, 3238, 82, 11503, 311, 1077, 4106, 3488, 198, 32345, 1027, 420, 25, 27567, 25938, 374, 912, 5647, 304, 3411, 1203, 7389, 433, 8779, 603, 311, 1427, 8469, 13, 3639, 701, 2697, 3828, 315, 279, 26741, 706, 1027, 374, 539, 779, 3062, 439, 1148, 198, 32158, 374, 2133, 311, 387, 1210, 330, 13461, 49780, 420, 311, 387, 837, 1359, 1364, 1071, 311, 46993, 14297, 11, 313, 1, 438, 433, 198, 18864, 1193, 387, 39270, 1306, 682, 11, 46993, 11, 313, 275, 596, 22225, 927, 323, 2884, 198, 4291, 13, 1102, 596, 539, 42216, 596, 3347, 430, 10742, 499, 1457, 26, 433, 596, 813, 3938, 449, 198, 9514, 11, 4536, 956, 433, 7673, 362, 7200, 315, 294, 1412, 24201, 46793, 16392, 198, 4908, 264, 22760, 295, 505, 1077, 8579, 7013, 11, 323, 505, 433, 11, 42071, 311, 279, 842, 315, 198, 1964, 5542, 11, 11299, 11816, 1306, 11816, 315, 8579, 259, 41154, 13, 3005, 28902, 279, 22760, 295, 198, 10727, 3725, 449, 2678, 70723, 2442, 18033, 28042, 13, 5112, 1364, 16392, 323, 2231, 1077, 6206, 198, 263, 76996, 596, 28004, 13, 330, 791, 4382, 8206, 374, 1359, 1364, 1071, 30666, 11, 330, 9210, 358, 2643, 3412, 42216, 422, 198, 40, 42777, 313, 466, 37852, 13, 1102, 596, 856, 22519, 198, 9210, 596, 13194, 11, 4400, 775, 1210, 1628, 8617, 1550, 46993, 52290, 89, 733, 1523, 311, 1077, 13306, 13, 76996, 14980, 369, 264, 4545, 11, 1077, 6548, 389, 279, 6661, 1364, 5762, 13, 40612, 11, 304, 198, 1964, 502, 19675, 11, 1364, 1047, 9687, 1690, 15234, 2574, 13, 3861, 315, 1124, 574, 198, 576, 25, 430, 3278, 1093, 32171, 56310, 1550, 539, 34568, 872, 33218, 26, 430, 279, 198, 1889, 315, 279, 100225, 574, 330, 63007, 311, 279, 32232, 13412, 5233, 430, 832, 6476, 279, 198, 5924, 11, 323, 2834, 477, 5675, 11, 323, 422, 568, 5675, 11, 3952, 813, 16088, 13, 94192, 775, 304, 279, 8952, 889, 7020, 1077, 3446, 11, 315, 3388, 13, 13538, 2133, 63891, 11, 76996, 9277, 279, 6661, 304, 264, 829, 60496, 323, 743, 198, 11029, 311, 433, 449, 264, 2489, 13, 4427, 315, 279, 12164, 685, 1047, 8636, 704, 315, 1077, 6548, 13, 578, 54050, 11, 4869, 11, 574, 198, 74303, 22568, 555, 3738, 2574, 13, 1789, 2937, 11, 433, 27458, 279, 1384, 1251, 198, 300, 16168, 369, 279, 52547, 15051, 11, 323, 8710, 264, 31954, 311, 37085, 198, 258, 1234, 279, 3185, 994, 912, 832, 574, 3411, 13, 35120, 31202, 11112, 198, 1116, 2656, 311, 1935, 279, 64260, 596, 6916, 902, 574, 9076, 1077, 11, 323, 1071, 1364, 198, 8890, 22456, 1364, 574, 3025, 311, 4321, 709, 7636, 13, 32980, 35120, 31202, 3782, 11, 439, 58046, 3847, 813, 2361, 11, 304, 264, 4686, 198, 6583, 1386, 12844, 596, 28403, 315, 18012, 2107, 323, 91441, 11, 449, 279, 20858, 430, 574, 198, 26301, 3314, 5842, 48809, 927, 813, 4851, 13, 578, 6825, 3782, 10799, 7162, 11, 43169, 449, 264, 8547, 315, 27924, 439, 311, 198, 13066, 716, 13, 12540, 814, 2231, 2555, 389, 279, 45115, 1603, 814, 2163, 11, 304, 1162, 198, 3323, 10054, 12932, 323, 19692, 1051, 10434, 520, 279, 3838, 30, 2582, 574, 433, 1120, 439, 1664, 198, 998, 7095, 311, 15369, 11, 323, 11, 422, 279, 52290, 89, 87117, 19168, 46579, 11, 311, 2503, 198, 2996, 311, 264, 9439, 40459, 994, 814, 2751, 2162, 30, 2057, 735, 2637, 11961, 304, 279, 1203, 315, 279, 8993, 1990, 5340, 65670, 323, 24101, 11, 279, 198, 68467, 7113, 574, 76996, 313, 68753, 3520, 1193, 13, 1283, 15746, 1077, 1176, 7504, 1523, 279, 198, 2852, 273, 11, 5602, 1077, 46175, 733, 709, 439, 1364, 18661, 3273, 1082, 323, 12410, 11, 15746, 198, 1820, 55060, 315, 1077, 3995, 7216, 304, 1202, 342, 2933, 4341, 4251, 439, 1364, 5946, 1461, 198, 438, 4024, 4741, 3347, 279, 1317, 7123, 315, 1589, 44159, 13272, 82, 13, 4740, 1637, 568, 1436, 198, 1962, 6227, 279, 13306, 4717, 520, 682, 13, 578, 2532, 369, 1461, 574, 76996, 345, 74303, 1384, 291, 323, 1633, 6129, 11, 30488, 279, 51943, 13, 1102, 574, 76996, 889, 3782, 198, 2996, 279, 60779, 311, 279, 92906, 28022, 42400, 315, 279, 13306, 15572, 11, 76996, 198, 4291, 7639, 30488, 1077, 0, 1952, 813, 1314, 7731, 5340, 65670, 11, 3515, 8813, 279, 1176, 99589, 315, 1077, 198, 943, 7076, 13, 2435, 1051, 810, 1109, 198, 9210, 313, 20670, 1051, 92906, 28022, 13, 84854, 1070, 11, 1364, 6445, 16195, 6548, 198, 2017, 279, 8993, 11, 10409, 449, 4754, 61365, 13, 2057, 5340, 65670, 11, 1243, 11, 430, 6664, 13658, 574, 264, 3938, 315, 26762, 46793, 198, 438, 84064, 263, 11, 279, 16267, 315, 9886, 11, 38586, 93534, 287, 38586, 13, 2030, 311, 198, 57856, 11, 10307, 279, 22260, 449, 73500, 6548, 323, 19661, 1740, 940, 1529, 51220, 198, 75, 3153, 11, 574, 5108, 1077, 6596, 13, 84854, 1203, 304, 279, 62516, 11, 449, 1077, 6206, 198, 20557, 291, 927, 1077, 23788, 34740, 11, 1364, 1071, 264, 2697, 23788, 369, 1077, 7833, 198, 82279, 10003, 11, 13176, 704, 505, 279, 51943, 449, 2867, 11, 653, 2642, 14152, 6548, 13, 1666, 76996, 323, 7639, 24465, 3221, 279, 6134, 11, 13142, 46506, 12669, 11, 889, 1047, 1027, 198, 10276, 520, 279, 1203, 315, 279, 8993, 11, 6656, 6288, 323, 4024, 704, 13, 1283, 198, 267, 26902, 11, 4856, 11, 439, 422, 568, 1436, 539, 1518, 13, 96424, 94515, 1432, 791, 87117, 520, 279, 5929, 30304, 14894, 1047, 539, 1027, 279, 1566, 87117, 198, 77264, 22983, 36627, 323, 7639, 17882, 1047, 4529, 3871, 13, 22770, 22983, 1047, 198, 4626, 369, 1077, 20769, 264, 2678, 6424, 2949, 4228, 3937, 5620, 6138, 315, 198, 1820, 3363, 11, 323, 1403, 477, 2380, 3115, 2391, 1077, 1403, 5672, 1022, 14523, 17882, 198, 32345, 8208, 704, 311, 1518, 1077, 13, 1789, 3131, 430, 568, 1436, 1518, 76996, 11, 568, 5602, 22770, 22983, 11157, 13, 3005, 7020, 5115, 1664, 279, 3169, 315, 893, 449, 8884, 1364, 574, 14892, 313, 9210, 568, 198, 41450, 2343, 439, 2697, 439, 3284, 13, 2030, 1364, 7020, 11, 2288, 11, 430, 11, 1095, 1461, 1390, 264, 198, 1626, 3403, 11, 568, 1053, 2343, 904, 3430, 369, 433, 11, 1524, 11103, 13, 578, 1633, 85009, 269, 304, 1077, 3663, 574, 304, 1077, 4799, 13, 3005, 1053, 2231, 279, 3245, 198, 20322, 11, 323, 1501, 1884, 7893, 287, 37465, 11, 449, 872, 281, 1245, 6548, 323, 11714, 198, 652, 5184, 11, 264, 3245, 477, 1403, 13, 12220, 430, 4553, 20769, 568, 2646, 5602, 1077, 304, 4205, 810, 37067, 198, 54895, 279, 45648, 315, 4251, 37862, 27946, 398, 1825, 520, 279, 28691, 11, 43475, 198, 21621, 709, 311, 1501, 1077, 7731, 6577, 11977, 13, 2684, 1051, 912, 1023, 4580, 388, 520, 279, 198, 56492, 83408, 13, 3005, 7731, 369, 4207, 304, 279, 7474, 59938, 304, 279, 9518, 198, 17884, 10409, 449, 24149, 12690, 430, 77317, 279, 27834, 11, 15884, 198, 3950, 927, 264, 2363, 11, 719, 449, 1077, 27989, 6548, 2744, 389, 279, 5754, 13, 3005, 1373, 198, 33, 51520, 11, 80554, 11, 4593, 258, 22464, 68, 13, 9843, 568, 1766, 1077, 449, 264, 2363, 430, 1364, 198, 71, 561, 1570, 44133, 13, 1283, 29676, 389, 9298, 433, 11, 323, 27528, 433, 13, 1221, 7096, 291, 449, 433, 11, 1364, 1529, 53078, 323, 12504, 1077, 198, 58331, 13, 5414, 35072, 52671, 1766, 304, 433, 279, 1455, 1672, 80227, 315, 72739, 11, 439, 198, 32158, 1047, 10825, 13, 330, 40, 2733, 1778, 459, 43358, 994, 358, 1097, 449, 499, 1359, 1364, 1071, 13, 330, 40, 4934, 311, 1440, 264, 198, 56492, 810, 922, 279, 2574, 499, 656, 1210, 3011, 2231, 872, 5133, 389, 264, 502, 323, 11084, 8197, 13, 2684, 10924, 198, 383, 23781, 15243, 15173, 4619, 315, 27065, 13, 5414, 990, 11, 264, 19584, 2363, 311, 813, 3278, 1603, 345, 6789, 1825, 311, 1077, 13, 4800, 323, 1243, 872, 6721, 20954, 9670, 304, 2555, 198, 64750, 13, 358, 649, 3137, 198, 8845, 449, 499, 2085, 3060, 34734, 477, 49394, 1113, 499, 13, 1472, 527, 279, 198, 3646, 25530, 5333, 358, 1440, 313, 438, 832, 315, 279, 4509, 10462, 478, 1210, 1283, 1047, 10717, 279, 5780, 389, 279, 73024, 315, 264, 24898, 369, 279, 37996, 37864, 198, 77763, 315, 14933, 6322, 279, 1684, 13, 330, 2170, 1317, 439, 499, 3137, 8221, 1359, 1364, 1071, 11, 330, 40, 2733, 430, 1070, 374, 4400, 198, 35970, 304, 1057, 1694, 3871, 26, 719, 994, 499, 2019, 279, 1023, 3245, 313, 1875, 1, 3957, 433, 5076, 311, 3371, 264, 5128, 5333, 499, 50863, 1077, 7673, 1283, 38817, 5678, 2212, 304, 279, 10954, 323, 7731, 3411, 520, 1077, 13, 330, 791, 781, 899, 13744, 11013, 304, 279, 1917, 9135, 3005, 1047, 3685, 433, 369, 520, 3325, 264, 2046, 11, 719, 1077, 13051, 574, 1664, 2884, 13, 8489, 2884, 1101, 574, 1077, 21847, 2391, 279, 5105, 24998, 12141, 13, 2360, 11, 1364, 574, 539, 19021, 11, 1364, 1071, 13, 1102, 574, 1193, 430, 568, 1047, 743, 1077, 198, 83360, 13, 3277, 1364, 2751, 704, 315, 279, 1841, 11, 1364, 293, 1037, 1461, 1695, 45427, 323, 198, 19045, 1481, 9188, 13, 4740, 430, 4400, 1436, 617, 8774, 1461, 3201, 11, 323, 1364, 7020, 433, 13, 330, 1692, 18651, 2225, 8137, 323, 1514, 26, 9093, 568, 50243, 5333, 439, 279, 198, 3646, 11660, 315, 23939, 1210, 362, 42786, 315, 8137, 1047, 10862, 1139, 872, 198, 37195, 13, 1283, 3937, 3093, 704, 311, 279, 8961, 279, 1828, 1938, 11, 311, 387, 3309, 430, 9083, 36627, 198, 32345, 8208, 369, 264, 1317, 4321, 323, 1047, 539, 1071, 994, 1364, 1053, 387, 1203, 13, 7357, 893, 13452, 311, 1781, 430, 198, 383, 374, 264, 2766, 315, 264, 38736, 13, 7639, 23183, 813, 18623, 11, 323, 11, 9564, 813, 198, 7063, 4994, 279, 34799, 365, 13883, 25675, 11, 56696, 1529, 411, 989, 389, 4579, 304, 279, 198, 19612, 22770, 22983, 1047, 4529, 13, 1283, 1766, 1077, 11, 3663, 1523, 11, 1234, 264, 5021, 345, 20986, 28639, 323, 24634, 323, 18534, 682, 279, 6029, 315, 264, 15748, 10723, 198, 496, 50428, 13, 3005, 16392, 304, 22047, 994, 1364, 6755, 813, 3094, 11, 323, 92037, 264, 198, 5447, 477, 1403, 11, 449, 1077, 6206, 704, 1603, 1077, 13, 358, 313, 40, 617, 2751, 311, 198, 19553, 264, 2697, 892, 7636, 13, 1283, 7020, 433, 574, 1514, 12, 36022, 11, 719, 4856, 15262, 433, 26, 323, 11, 1606, 568, 574, 198, 84270, 439, 10151, 1285, 439, 1364, 574, 11, 568, 17948, 264, 2489, 389, 279, 38411, 315, 279, 5021, 198, 438, 3177, 291, 264, 36213, 1603, 568, 19089, 13, 330, 40, 574, 16984, 315, 420, 1359, 568, 1071, 11, 5737, 709, 13, 358, 1097, 539, 2216, 264, 40148, 11, 22770, 22983, 1210, 1102, 574, 279, 1176, 892, 568, 1047, 1511, 1077, 836, 13, 330, 61935, 1523, 323, 1095, 603, 3137, 2574, 927, 1210, 3005, 7731, 1523, 520, 264, 6220, 6138, 11, 323, 7111, 4028, 279, 2697, 33850, 198, 998, 1461, 449, 279, 1794, 655, 6548, 430, 1051, 1077, 2294, 9513, 13, 330, 2675, 649, 10150, 311, 387, 1633, 19858, 1359, 1364, 1071, 11, 330, 28753, 420, 374, 1193, 1514, 198, 998, 499, 26, 358, 1440, 433, 13, 358, 2846, 264, 1695, 11700, 323, 198, 1962, 313, 359, 266, 70178, 13, 2030, 1148, 374, 1514, 369, 499, 374, 539, 14647, 1514, 369, 198, 2727, 13, 1789, 279, 1176, 892, 11, 568, 1766, 5678, 35090, 304, 1077, 100226, 13, 1442, 1364, 39169, 313, 383, 574, 520, 198, 1820, 39324, 315, 904, 5333, 889, 39169, 13, 1115, 3460, 315, 3245, 4250, 733, 389, 11, 2999, 13, 3005, 1550, 16106, 1243, 313, 8110, 24014, 26, 323, 568, 4024, 927, 30488, 1077, 323, 3952, 1077, 198, 258, 813, 11977, 13, 1472, 1304, 757, 2733, 1093, 198, 64, 1156, 801, 3833, 11, 323, 358, 3077, 1193, 1027, 4737, 264, 2697, 2766, 315, 23871, 13, 7639, 11, 323, 59334, 1077, 1578, 389, 279, 23726, 13, 578, 832, 2449, 22770, 22983, 1047, 2163, 704, 315, 1077, 29217, 574, 11937, 13, 3005, 1047, 3967, 279, 893, 11, 1047, 4529, 279, 6671, 520, 1202, 6300, 907, 13, 2030, 198, 32158, 1047, 2163, 704, 420, 3062, 8331, 304, 279, 24524, 11, 313, 9210, 8331, 198, 8370, 304, 1475, 5133, 1990, 893, 323, 5333, 27667, 279, 198, 26378, 367, 11, 313, 1820, 5333, 13, 32745, 1077, 38714, 45715, 1047, 2586, 264, 502, 323, 33812, 2449, 13, 3005, 198, 14965, 11, 1093, 735, 13, 304, 813, 2697, 3130, 389, 279, 6825, 11, 1047, 2231, 16038, 3021, 323, 198, 1820, 2574, 34366, 11, 1766, 430, 433, 1053, 539, 387, 2231, 16038, 13, 3296, 279, 842, 315, 198, 1964, 2875, 20769, 22770, 22983, 36627, 574, 48295, 304, 3021, 449, 279, 14992, 198, 92493, 13, 2435, 8738, 311, 3449, 11, 539, 439, 3629, 439, 1603, 11, 719, 3131, 264, 2046, 345, 66372, 13, 578, 16659, 1051, 2539, 315, 8137, 1457, 26, 323, 422, 369, 279, 3828, 814, 198, 55437, 555, 420, 4367, 11, 814, 18661, 33464, 369, 279, 893, 13, 3005, 574, 559, 4361, 67, 198, 268, 1409, 311, 13383, 1077, 1866, 6671, 13, 3005, 198, 66, 1636, 11, 323, 568, 1550, 539, 13, 1102, 574, 682, 264, 1847, 1457, 11, 539, 11074, 13, 2052, 3278, 527, 42779, 26, 3278, 304, 3021, 527, 59594, 779, 13, 1666, 1664, 439, 198, 32158, 7020, 430, 813, 11939, 369, 1077, 574, 539, 279, 1972, 3245, 11, 779, 1101, 1364, 198, 8110, 1534, 430, 1070, 574, 7982, 709, 304, 813, 4851, 2555, 53007, 311, 279, 198, 8110, 3245, 369, 76996, 5874, 13, 53306, 14222, 6244, 44743, 1306, 264, 3137, 198, 20670, 1047, 927, 279, 87117, 2007, 520, 264, 3224, 5754, 37002, 279, 1938, 1306, 198, 19875, 483, 596, 13306, 13, 330, 4438, 574, 279, 13306, 313, 83, 3946, 638, 7673, 2684, 596, 2744, 2555, 57399, 311, 757, 304, 264, 893, 63143, 198, 40617, 726, 709, 369, 2324, 311, 832, 5333, 13, 330, 4897, 596, 539, 7041, 279, 7658, 323, 279, 1322, 764, 1441, 11, 374, 433, 7673, 2057, 1781, 315, 27397, 704, 315, 682, 279, 1917, 832, 5333, 345, 438, 4135, 287, 311, 8493, 279, 2800, 315, 832, 596, 2919, 449, 1077, 0, 10541, 313, 1875, 16366, 6548, 7111, 3347, 22770, 22983, 1139, 6138, 13, 330, 68753, 3520, 5874, 574, 832, 315, 279, 61365, 98250, 1359, 568, 1071, 40815, 398, 13, 330, 8100, 574, 198, 385, 899, 1291, 1109, 279, 42447, 1210, 330, 53040, 11, 719, 18754, 1359, 1071, 22770, 22983, 13, 358, 3077, 2216, 6818, 311, 198, 668, 613, 1077, 2574, 11, 719, 313, 9514, 1440, 313, 1, 3005, 70756, 1077, 28004, 13, 1442, 1070, 574, 264, 4483, 36244, 304, 813, 8071, 11, 568, 198, 588, 2230, 433, 62504, 398, 13, 2030, 11, 3131, 1578, 304, 279, 5780, 11, 568, 30280, 927, 323, 198, 631, 813, 41803, 2403, 11074, 13, 1472, 2351, 41690, 1359, 568, 1071, 506, 495, 18007, 13, 35053, 11, 8051, 568, 2643, 15648, 11, 279, 2217, 315, 76996, 11203, 1633, 198, 5669, 311, 813, 4851, 1884, 42774, 2919, 13, 76996, 3782, 1022, 3814, 14523, 279, 6278, 315, 6841, 13, 578, 3814, 14523, 1047, 198, 82850, 264, 892, 315, 55394, 9096, 311, 22770, 22983, 13, 7639, 1436, 4546, 76996, 1203, 311, 279, 8952, 304, 813, 1841, 13, 76996, 596, 4376, 96314, 520, 2162, 1051, 25975, 369, 57770, 552, 315, 80822, 389, 198, 77264, 22983, 596, 961, 13, 1952, 1778, 459, 13402, 11, 264, 2305, 1306, 279, 13306, 11, 1364, 198, 29177, 539, 6782, 11937, 13, 3005, 40001, 1077, 2362, 28391, 315, 47846, 11, 323, 198, 91453, 279, 259, 75143, 311, 264, 1486, 3221, 279, 842, 315, 279, 6825, 13, 4740, 93722, 198, 67643, 11, 1364, 14297, 15203, 279, 3160, 315, 279, 6825, 13, 46993, 323, 42216, 198, 32345, 539, 6052, 505, 872, 13306, 11879, 13, 578, 6841, 11714, 574, 198, 1962, 9439, 11, 323, 389, 279, 2697, 44952, 7731, 76996, 323, 2999, 13, 735, 13, 574, 198, 19041, 11, 2288, 11, 1047, 1364, 1193, 3967, 433, 11, 11961, 1203, 304, 279, 12737, 323, 5605, 198, 56492, 11, 813, 24981, 6548, 389, 76996, 596, 5643, 13, 3005, 4024, 389, 1523, 279, 6825, 304, 264, 74941, 198, 1073, 41690, 19788, 13, 4740, 430, 1403, 6848, 10837, 35135, 304, 22770, 22983, 596, 4059, 25, 832, 574, 311, 636, 198, 68753, 3520, 704, 315, 279, 1648, 11, 279, 1023, 574, 311, 1304, 17882, 30714, 311, 1077, 13, 763, 198, 1964, 4851, 1364, 7020, 430, 389, 279, 1176, 79101, 279, 2132, 13, 362, 2046, 3010, 1364, 1903, 279, 1890, 89706, 95440, 11, 719, 449, 264, 2204, 198, 1407, 13, 2030, 11509, 389, 279, 23162, 198, 11020, 9710, 315, 279, 2697, 3838, 574, 2009, 77638, 818, 13, 578, 60243, 32329, 355, 12690, 1051, 198, 55202, 520, 430, 892, 11, 21939, 13819, 3935, 11977, 40872, 311, 279, 6841, 13180, 13, 578, 198, 28451, 2922, 1141, 11, 902, 304, 279, 7474, 2163, 279, 76977, 304, 279, 12737, 11, 1457, 198, 939, 606, 1555, 279, 23962, 323, 22982, 1139, 3831, 16337, 2009, 77638, 818, 596, 16615, 198, 18257, 323, 743, 3663, 13, 3005, 4024, 389, 11, 89815, 11, 1077, 13326, 8271, 20225, 287, 93392, 13, 1102, 574, 279, 1176, 892, 198, 32158, 1047, 3967, 430, 735, 13, 12439, 304, 279, 5874, 3838, 13, 1102, 6688, 1077, 264, 5647, 315, 198, 1371, 81246, 323, 25114, 8850, 13, 3005, 1903, 1077, 1176, 11919, 927, 5081, 315, 1690, 2919, 311, 76996, 279, 198, 44018, 1938, 13, 2435, 2322, 304, 279, 35779, 48840, 304, 279, 31741, 1405, 279, 198, 28451, 17895, 369, 279, 26741, 6978, 574, 8774, 13, 5810, 11, 23255, 304, 49707, 198, 438, 11989, 291, 11, 3185, 555, 3185, 11203, 279, 98882, 67556, 304, 902, 198, 1820, 6978, 1047, 2322, 11677, 477, 17563, 13, 432, 2076, 323, 13413, 1918, 11, 1488, 339, 323, 198, 18883, 49958, 11, 11203, 4661, 31687, 13, 13759, 3201, 389, 279, 1023, 3185, 315, 279, 4251, 2695, 13883, 31741, 11, 3026, 1051, 198, 359, 10853, 30103, 6605, 43732, 315, 14403, 13, 57227, 82, 315, 40120, 3782, 1523, 279, 198, 5997, 277, 27896, 11, 31687, 872, 4251, 56737, 323, 13353, 279, 43732, 311, 15310, 13, 98325, 574, 279, 13901, 315, 279, 8952, 11, 902, 374, 2015, 13, 76996, 11, 305, 34552, 1203, 505, 3293, 1776, 2866, 311, 279, 62862, 10652, 198, 1073, 1077, 3814, 14523, 11, 31645, 520, 22770, 22983, 26085, 3725, 13, 330, 87643, 56310, 374, 2133, 704, 311, 11477, 13, 3277, 832, 43457, 1268, 5986, 1364, 574, 323, 1268, 584, 3463, 1364, 1436, 539, 198, 25859, 11, 433, 596, 4856, 264, 38586, 11, 4536, 956, 433, 7673, 76996, 25078, 449, 1063, 73082, 279, 37067, 36156, 2176, 67556, 304, 1077, 198, 10888, 13, 330, 8100, 649, 956, 733, 704, 304, 1884, 26, 358, 4985, 617, 311, 39580, 1077, 2555, 1210, 362, 198, 56492, 315, 279, 3177, 8636, 704, 315, 1077, 3663, 13, 330, 8100, 596, 1047, 264, 2653, 4465, 11, 323, 198, 32158, 706, 2834, 1359, 1364, 1071, 13, 330, 4071, 994, 358, 1781, 315, 1148, 1364, 596, 4762, 2133, 198, 1445, 311, 313, 1875, 77264, 22983, 70756, 1077, 28004, 13, 330, 2181, 596, 682, 304, 279, 1938, 596, 990, 1359, 1364, 13468, 1280, 14657, 4501, 13, 330, 2675, 649, 1935, 198, 49818, 709, 1139, 279, 9979, 323, 3041, 1124, 24981, 990, 1370, 287, 35267, 11, 477, 198, 631, 1124, 304, 279, 35189, 11245, 287, 13, 3005, 24465, 264, 6462, 505, 279, 35779, 323, 7111, 520, 433, 56949, 3725, 13, 330, 11649, 11, 1148, 656, 499, 1440, 922, 420, 30, 5810, 596, 264, 5333, 889, 3782, 304, 304, 264, 198, 9471, 70, 785, 323, 6857, 315, 1776, 32997, 13, 1628, 1457, 1364, 6944, 311, 733, 704, 304, 4376, 459, 198, 25704, 9135, 3005, 6656, 11, 389, 1077, 1648, 704, 315, 279, 35779, 48840, 11, 323, 6689, 264, 4062, 34522, 198, 266, 76996, 13, 330, 40, 7077, 311, 387, 389, 701, 8761, 279, 1023, 3814, 1359, 1364, 1071, 13, 330, 2675, 3974, 198, 582, 2177, 279, 8761, 505, 17882, 82, 518, 1541, 956, 499, 7673, 330, 40, 3463, 779, 26, 358, 1047, 6755, 499, 6604, 315, 279, 3838, 13, 4718, 313, 22479, 10868, 198, 16514, 11509, 389, 279, 7504, 1210, 1102, 4536, 956, 2216, 198, 1315, 311, 1650, 1461, 264, 3130, 261, 26, 568, 596, 832, 315, 279, 3070, 1457, 35277, 82, 836, 1047, 17948, 459, 2744, 27078, 44321, 304, 76996, 13, 578, 1403, 7724, 198, 42700, 9017, 279, 39736, 3871, 13, 3161, 264, 1633, 2697, 51475, 345, 68753, 3520, 15243, 315, 735, 13, 3005, 574, 18949, 520, 9083, 36627, 596, 11919, 16630, 345, 6200, 329, 430, 2574, 1051, 682, 1314, 1990, 1124, 1578, 13, 2468, 1077, 6558, 11, 1364, 198, 631, 264, 99149, 1450, 389, 279, 3828, 596, 6916, 13, 330, 40, 574, 16984, 358, 1047, 54221, 499, 477, 85334, 1503, 499, 1359, 1364, 1071, 13, 330, 40, 2846, 779, 198, 6200, 329, 433, 4536, 956, 779, 1210, 20695, 1051, 539, 2133, 904, 2288, 1664, 449, 735, 13, 3082, 11, 568, 1047, 4036, 813, 198, 75242, 520, 279, 5274, 11, 323, 449, 420, 3118, 3611, 10036, 768, 315, 17510, 38502, 198, 78289, 1590, 264, 2046, 568, 574, 3025, 311, 656, 3892, 2574, 13, 35120, 31202, 1457, 198, 86, 13883, 323, 11245, 291, 832, 1938, 264, 2046, 520, 279, 2697, 3838, 11, 779, 430, 45563, 198, 46068, 617, 810, 892, 311, 1427, 1306, 24101, 13, 1283, 1047, 7319, 1101, 279, 198, 6173, 315, 3300, 430, 568, 48582, 3288, 6460, 13, 578, 3245, 430, 7222, 839, 323, 10409, 1461, 449, 264, 5647, 198, 1073, 8060, 574, 7639, 17882, 596, 19451, 13, 1102, 574, 539, 9662, 18260, 26, 433, 574, 345, 28074, 291, 11, 21356, 49150, 11, 4661, 36237, 2335, 13, 2030, 568, 9539, 198, 25742, 291, 2009, 77638, 818, 596, 2361, 32677, 13, 2684, 574, 912, 837, 67128, 2200, 1990, 279, 1403, 3026, 26, 719, 1070, 574, 198, 7413, 1251, 311, 387, 6926, 15360, 11, 323, 31445, 264, 3738, 3392, 315, 198, 1658, 2538, 13, 17882, 6137, 311, 4546, 682, 813, 5435, 311, 2009, 77638, 818, 13, 2684, 1051, 1317, 198, 49512, 811, 304, 430, 2678, 8582, 3130, 13, 19292, 810, 1109, 832, 893, 477, 198, 22803, 889, 1550, 539, 1440, 315, 735, 3238, 82, 14209, 49009, 813, 2324, 311, 1461, 430, 4498, 13, 47124, 6137, 311, 2586, 304, 311, 1461, 198, 1527, 279, 14932, 25861, 13, 2057, 813, 1866, 59772, 574, 3779, 264, 502, 323, 198, 38548, 481, 15105, 13, 2030, 2009, 77638, 818, 11, 889, 1047, 1766, 44734, 422, 539, 198, 1834, 11, 574, 3131, 1578, 304, 5916, 449, 279, 990, 568, 10456, 13, 2684, 1051, 198, 15487, 994, 11, 3515, 8983, 13883, 264, 1162, 704, 3871, 323, 33740, 279, 1828, 198, 1316, 596, 990, 369, 7639, 11, 568, 1053, 4321, 369, 4207, 1139, 279, 3814, 704, 927, 279, 198, 71, 3385, 11, 11039, 813, 8209, 13, 578, 79422, 574, 389, 1461, 311, 387, 304, 279, 12314, 198, 1073, 2574, 1578, 13, 578, 3463, 315, 279, 6962, 5274, 323, 1202, 25114, 4883, 198, 82, 875, 6901, 1461, 13, 1102, 574, 389, 832, 315, 813, 1317, 23291, 430, 735, 13, 1766, 40788, 648, 13, 1102, 574, 6790, 1243, 11, 18004, 323, 7257, 11, 449, 264, 14739, 12056, 430, 5614, 311, 198, 30193, 439, 433, 11299, 13, 578, 3224, 19795, 1051, 39448, 6953, 752, 449, 27275, 11, 279, 5627, 579, 198, 22354, 12314, 449, 24905, 5294, 11141, 13, 578, 33201, 277, 1918, 315, 279, 47967, 430, 198, 34774, 13658, 32599, 813, 20247, 13, 1283, 1047, 93327, 311, 279, 842, 315, 279, 198, 28451, 24948, 1584, 11, 323, 3940, 813, 4321, 505, 1070, 13, 1666, 574, 813, 2587, 11, 568, 198, 86, 461, 912, 927, 68512, 11, 719, 264, 2875, 61221, 1234, 813, 23724, 13, 18024, 61752, 3235, 198, 1820, 5754, 568, 1047, 13061, 709, 264, 75642, 3833, 5679, 11, 323, 11, 439, 422, 304, 33638, 12876, 369, 198, 26380, 8396, 11, 433, 57677, 6702, 22489, 2915, 520, 813, 34460, 13, 31048, 8931, 505, 279, 842, 315, 279, 1841, 1584, 568, 1766, 264, 5754, 37002, 11, 323, 198, 90963, 304, 369, 264, 9168, 315, 93050, 13, 578, 5679, 198, 42700, 304, 449, 1461, 11, 323, 14980, 3411, 709, 1139, 813, 3663, 13, 1102, 574, 439, 422, 568, 198, 73563, 11, 719, 31156, 3249, 420, 55154, 11, 449, 279, 1156, 812, 315, 279, 5754, 198, 43887, 323, 279, 33965, 315, 70244, 927, 279, 5151, 13, 578, 3838, 574, 743, 304, 264, 34681, 520, 279, 4579, 315, 1403, 35231, 13, 17331, 279, 8770, 198, 1073, 279, 6790, 13658, 11, 433, 1047, 781, 25111, 82374, 1603, 1461, 13, 578, 6134, 198, 16514, 264, 19571, 11, 323, 568, 25319, 1139, 264, 2697, 14321, 9960, 449, 6892, 30193, 28954, 13, 2057, 279, 1314, 574, 279, 18397, 48840, 11, 279, 2007, 9960, 449, 264, 4251, 28392, 345, 438, 304, 1202, 4839, 4219, 459, 66637, 442, 3876, 15860, 315, 32720, 19837, 13, 2057, 279, 198, 2414, 11, 279, 14595, 1370, 9761, 315, 1778, 7634, 13, 1102, 2643, 617, 1027, 279, 1370, 9761, 198, 1073, 279, 5929, 30304, 14894, 304, 23329, 11, 72779, 659, 53034, 15512, 323, 682, 13, 6193, 198, 73826, 574, 21847, 323, 264, 47274, 2277, 22843, 315, 7878, 767, 97939, 13, 578, 3838, 198, 16514, 19738, 449, 502, 6308, 313, 1820, 30811, 3252, 2362, 27035, 559, 606, 449, 433, 11, 279, 198, 28404, 30103, 3690, 13, 2663, 735, 382, 3947, 1051, 6435, 60802, 50007, 11, 279, 15676, 315, 264, 44402, 27580, 345, 1820, 23941, 273, 315, 264, 5333, 596, 8679, 5108, 1523, 279, 32249, 13, 735, 2637, 11509, 198, 1371, 7770, 398, 389, 264, 28954, 86810, 430, 574, 279, 4219, 315, 279, 1370, 9761, 767, 97939, 345, 30972, 7069, 1022, 813, 61221, 13, 568, 1071, 311, 279, 64233, 8954, 389, 279, 198, 267, 1334, 5756, 13, 3005, 2231, 264, 1450, 2403, 279, 198, 11020, 6906, 311, 24981, 11937, 13, 40788, 648, 22225, 11, 719, 264, 502, 40788, 648, 0, 3161, 1077, 198, 51729, 46050, 6901, 2212, 1077, 3663, 11, 264, 7878, 6437, 523, 396, 89, 8679, 1825, 520, 279, 198, 87164, 11, 264, 3776, 72426, 15631, 389, 1077, 17659, 11, 1618, 574, 264, 40788, 648, 88018, 345, 13885, 7627, 810, 19411, 11, 1109, 568, 1047, 27569, 1077, 13, 2030, 1364, 1550, 539, 198, 3647, 458, 520, 1461, 13, 2684, 574, 2555, 922, 1077, 6548, 539, 20426, 279, 5679, 596, 198, 29199, 11, 97340, 11, 719, 34685, 13, 330, 11649, 11, 499, 3077, 1766, 757, 11, 4491, 13, 1628, 11, 994, 568, 5762, 704, 813, 1450, 345, 3647, 8138, 25, 330, 40, 1120, 1047, 311, 656, 433, 11, 4491, 13, 1472, 1427, 42727, 7060, 323, 313, 57621, 11, 40788, 648, 1210, 4946, 499, 617, 264, 10747, 315, 15600, 11, 477, 690, 499, 617, 2555, 775, 7673, 578, 31655, 315, 279, 6825, 574, 2103, 3831, 304, 40788, 648, 13, 578, 6825, 1550, 198, 1962, 29251, 315, 330, 34431, 775, 1210, 330, 3407, 94707, 9976, 1355, 14320, 1359, 1071, 2009, 77638, 818, 13, 330, 3112, 4985, 358, 3780, 264, 11989, 369, 499, 311, 198, 79, 3265, 7673, 1283, 574, 14931, 568, 1047, 1903, 279, 1529, 8154, 13, 469, 1325, 4501, 279, 6825, 323, 682, 430, 18713, 2692, 574, 264, 36366, 3917, 13, 1102, 574, 369, 420, 430, 1364, 1047, 48501, 198, 1820, 9043, 70, 992, 17025, 315, 1077, 2324, 520, 18083, 13, 26718, 2176, 596, 313, 2000, 420, 10160, 1355, 906, 418, 198, 56492, 3838, 11, 13413, 1570, 28360, 11, 72600, 40666, 13, 2684, 1051, 1403, 90016, 263, 198, 268, 27995, 3808, 927, 279, 26976, 301, 13, 578, 198, 1605, 574, 279, 5684, 1773, 980, 7555, 13, 735, 13, 31156, 1148, 22999, 31655, 315, 198, 726, 39130, 29875, 367, 1047, 9057, 40788, 648, 311, 5387, 279, 7555, 1070, 2073, 380, 75325, 13, 6984, 315, 1202, 2361, 315, 16044, 568, 5602, 279, 3828, 596, 49803, 315, 1077, 1866, 198, 47965, 4090, 13, 1952, 264, 23162, 28745, 11, 7041, 1990, 279, 1403, 9364, 11, 574, 198, 43063, 93484, 315, 32720, 19837, 13, 40788, 648, 7263, 279, 93050, 11, 2736, 9709, 11, 304, 264, 16615, 9168, 13, 735, 13, 1053, 198, 19553, 15236, 311, 6651, 433, 5678, 11, 719, 279, 93050, 574, 1695, 13, 330, 2675, 6688, 757, 264, 2543, 520, 1176, 1359, 1071, 40788, 648, 13, 330, 4071, 358, 1097, 1314, 16089, 311, 1518, 198, 9514, 11, 4491, 13, 4800, 430, 279, 19795, 527, 3958, 11, 19093, 4131, 1633, 1790, 13, 30070, 1457, 11, 735, 13, 323, 40788, 648, 11, 994, 814, 2322, 11, 1047, 2322, 10652, 750, 389, 279, 198, 5581, 5015, 315, 3691, 13, 2435, 912, 5129, 1047, 430, 11, 323, 1990, 1124, 2225, 198, 6789, 1093, 264, 22881, 872, 1566, 10652, 13, 4497, 19411, 433, 7995, 574, 345, 8248, 6380, 30, 2684, 574, 264, 289, 380, 31514, 922, 40788, 648, 596, 11013, 430, 743, 1461, 198, 86, 14518, 287, 13, 330, 1548, 596, 922, 279, 1888, 893, 389, 9578, 13, 1283, 596, 2646, 1071, 264, 5425, 3492, 311, 198, 2727, 313, 17206, 520, 1176, 11, 994, 358, 574, 7363, 19501, 323, 27207, 520, 1475, 5222, 1210, 330, 40, 27724, 264, 2763, 315, 5310, 940, 82, 994, 358, 1176, 3782, 11, 4401, 1022, 323, 26092, 198, 9493, 358, 6755, 1274, 2212, 279, 2035, 13, 1102, 1511, 311, 2873, 311, 757, 430, 1148, 198, 40, 4265, 2884, 574, 5439, 389, 856, 3663, 13, 40788, 648, 65657, 709, 520, 279, 1403, 9364, 927, 279, 26976, 301, 13, 330, 32148, 433, 374, 313, 9493, 568, 4131, 304, 19781, 11, 323, 358, 3077, 264, 16553, 5644, 477, 198, 15031, 41951, 13824, 323, 19335, 369, 813, 87117, 11, 323, 358, 1518, 1461, 3240, 311, 1427, 198, 4014, 291, 13, 1283, 13001, 813, 13961, 11, 323, 1690, 459, 11714, 568, 8779, 757, 449, 279, 198, 67, 21168, 13, 330, 40, 8434, 956, 733, 1203, 311, 1405, 358, 574, 11, 719, 358, 1097, 539, 6380, 11, 4491, 13, 1115, 2035, 374, 813, 11, 323, 568, 4265, 1093, 264, 8334, 311, 2586, 1139, 433, 198, 9493, 568, 596, 8208, 13, 422, 358, 1550, 617, 832, 26, 1148, 1053, 433, 387, 7673, 6, 82, 6548, 8272, 11074, 311, 279, 6945, 323, 279, 3596, 4354, 826, 30456, 13, 330, 3112, 1364, 313, 19041, 4536, 956, 904, 22199, 315, 1077, 313, 7673, 2684, 574, 912, 6425, 311, 40788, 648, 596, 3575, 13, 2009, 77638, 818, 11, 11509, 389, 279, 198, 383, 47601, 323, 3411, 1523, 520, 1077, 11, 15393, 430, 11, 1306, 682, 11, 40788, 648, 2011, 198, 1816, 704, 1077, 1866, 46035, 13, 2435, 15243, 3117, 1139, 279, 7982, 93722, 315, 279, 13658, 13, 40788, 648, 574, 198, 57932, 894, 369, 3754, 315, 279, 6825, 25, 2011, 1440, 315, 46993, 596, 13306, 11, 315, 198, 27588, 65670, 11, 315, 76996, 304, 1077, 8952, 13, 1628, 994, 568, 1047, 3309, 1077, 682, 11, 1364, 198, 37568, 21737, 11, 20700, 1077, 1450, 7197, 62626, 304, 1077, 19779, 13, 5112, 25, 27567, 1, 18293, 279, 3116, 315, 603, 1359, 1364, 1071, 15187, 11, 313, 1, 19875, 483, 52290, 89, 323, 76996, 198, 2732, 323, 9083, 5340, 65670, 323, 757, 11, 313, 438, 902, 832, 1053, 499, 617, 13061, 311, 198, 3427, 5076, 1093, 420, 30, 358, 8101, 11, 505, 279, 5992, 315, 2574, 11, 1455, 15687, 1053, 198, 19553, 3463, 433, 1053, 387, 279, 52290, 89, 3828, 13, 2435, 4265, 617, 13061, 5340, 65670, 198, 89734, 9497, 369, 279, 8952, 11, 323, 757, 369, 279, 8679, 28936, 11, 323, 433, 1053, 617, 198, 82850, 76996, 5874, 430, 2751, 12502, 323, 1047, 459, 35528, 13, 3005, 7111, 709, 520, 735, 13, 559, 4361, 67, 398, 13, 2435, 3287, 956, 1440, 757, 11, 323, 358, 198, 55603, 1124, 7556, 13, 2435, 1071, 76996, 5874, 574, 2133, 311, 32918, 2999, 13, 1666, 1364, 198, 52784, 1603, 1461, 1364, 7111, 709, 1139, 813, 3663, 13, 330, 2746, 499, 1093, 1077, 439, 1664, 439, 358, 1781, 499, 656, 11, 4491, 13, 2009, 77638, 818, 11, 499, 2834, 956, 1095, 198, 40617, 636, 1077, 1210, 330, 40, 1097, 16984, 430, 596, 539, 709, 311, 757, 11, 374, 433, 30, 3639, 1053, 358, 656, 449, 264, 7555, 345, 51, 484, 648, 7673, 358, 8101, 11, 304, 279, 198, 4930, 1629, 11, 430, 1053, 1797, 810, 1109, 3300, 1210, 3011, 574, 1148, 735, 13, 3952, 2162, 449, 1461, 1306, 813, 13123, 449, 40788, 648, 13, 1283, 198, 20949, 291, 433, 389, 813, 1648, 1203, 311, 279, 8761, 24948, 11, 439, 568, 28214, 2403, 198, 1820, 10160, 13, 468, 6241, 10398, 7977, 3235, 279, 5754, 1051, 198, 28611, 449, 3090, 323, 1047, 22088, 311, 31030, 13, 578, 11422, 1047, 6656, 311, 264, 198, 3696, 2299, 274, 19510, 430, 4018, 813, 3663, 13, 26924, 3195, 311, 279, 259, 75143, 1584, 11, 279, 5679, 198, 42286, 1022, 1139, 264, 555, 66815, 13, 578, 5679, 45135, 1306, 198, 40617, 11, 832, 4579, 9408, 13, 9843, 1578, 813, 6548, 1051, 1093, 40788, 648, 596, 11, 439, 1364, 1047, 198, 86, 4234, 1695, 1481, 9188, 505, 279, 45747, 13, 5414, 2010, 68265, 389, 813, 17659, 11, 735, 13, 9960, 8931, 315, 5754, 449, 813, 1317, 345, 2332, 24992, 18338, 11, 323, 21095, 813, 8209, 13, 15148, 40788, 648, 1314, 11, 1306, 682, 11, 323, 198, 32345, 568, 1027, 5076, 30, 8595, 1288, 568, 3369, 580, 5678, 11, 422, 433, 8967, 76996, 596, 198, 359, 71, 67391, 30, 8595, 539, 4287, 17882, 596, 3085, 323, 1212, 927, 1578, 30, 5112, 198, 333, 2574, 4024, 1664, 313, 1820, 62461, 574, 3831, 430, 13766, 88, 13658, 13, 1283, 198, 631, 433, 505, 1461, 520, 1566, 11, 1606, 315, 279, 29191, 430, 8996, 568, 1550, 198, 41450, 1304, 912, 2349, 304, 76996, 596, 17139, 5597, 13, 1442, 1364, 42777, 3403, 198, 2000, 17882, 11, 1364, 1053, 32918, 1461, 13, 96424, 53694, 1432, 20577, 1195, 323, 46993, 6052, 505, 872, 13306, 8577, 279, 1938, 735, 627, 4338, 21468, 40788, 648, 13, 24101, 5874, 1903, 1790, 315, 279, 19163, 11, 29676, 389, 198, 67, 4481, 369, 1124, 430, 3814, 520, 279, 2697, 3838, 11, 2011, 1520, 46993, 198, 81874, 1077, 490, 15655, 323, 31993, 1077, 13306, 21258, 922, 279, 13455, 13, 3005, 198, 16514, 53657, 1109, 1364, 1047, 1027, 369, 2919, 11, 810, 8173, 13, 578, 40164, 315, 198, 1820, 490, 46651, 2933, 10409, 1077, 449, 66939, 323, 264, 3460, 315, 41690, 66533, 369, 198, 68753, 3520, 11, 889, 1436, 617, 7000, 315, 1521, 2574, 13, 763, 264, 67163, 3460, 315, 1648, 345, 32158, 6691, 291, 46993, 304, 39381, 315, 1077, 1866, 10003, 13, 1628, 433, 574, 1077, 4062, 8071, 430, 42645, 291, 2555, 5076, 13, 9636, 1077, 28361, 574, 459, 1234, 3311, 315, 21137, 13, 24101, 11, 9257, 304, 71271, 422, 304, 4400, 775, 11, 6612, 433, 11, 323, 304, 10052, 311, 198, 15031, 8982, 315, 46993, 596, 430, 17948, 1077, 439, 2653, 11, 539, 5115, 27442, 345, 32158, 6688, 1077, 264, 22443, 83085, 11218, 13, 330, 12331, 4588, 2324, 5097, 264, 2697, 43468, 11, 856, 25237, 1359, 1364, 1071, 13, 330, 6153, 584, 198, 19553, 12439, 311, 13520, 369, 264, 1396, 315, 1667, 11, 433, 374, 539, 4228, 311, 3974, 198, 2000, 1063, 832, 775, 1210, 46993, 7833, 6901, 505, 279, 15600, 17203, 1364, 574, 77218, 13, 2030, 3249, 1288, 279, 5333, 656, 682, 279, 43468, 7673, 330, 29819, 527, 810, 743, 1359, 1071, 8009, 24101, 11, 889, 1047, 2646, 1027, 743, 304, 4205, 198, 258, 1077, 2324, 13, 330, 2181, 374, 16127, 369, 1124, 311, 3041, 304, 13, 1628, 11, 315, 3388, 11, 42216, 198, 285, 9191, 11, 323, 813, 26870, 313, 1875, 10227, 2753, 1071, 922, 42216, 596, 26870, 279, 2731, 1359, 70939, 46993, 13, 330, 40, 198, 41866, 311, 617, 12502, 264, 15860, 315, 26870, 1210, 3005, 6688, 927, 1077, 32165, 287, 11, 323, 7731, 1523, 1160, 16117, 555, 279, 4027, 11, 1418, 198, 57856, 7882, 922, 11, 13326, 449, 279, 2678, 7640, 430, 35072, 1077, 13, 19198, 5672, 315, 42216, 596, 8396, 304, 27862, 15055, 1047, 34386, 46993, 198, 998, 54716, 13, 3005, 7731, 449, 48135, 6206, 323, 7111, 1139, 264, 3938, 430, 198, 325, 16809, 311, 2997, 4400, 719, 42216, 25, 42216, 33173, 449, 813, 11013, 1825, 280, 20577, 1195, 81109, 1603, 17954, 11, 323, 6348, 23349, 3156, 568, 1047, 1047, 813, 198, 79217, 26, 42216, 46772, 1251, 927, 279, 17222, 13, 1628, 1070, 574, 264, 40130, 3185, 311, 279, 6945, 1109, 430, 13, 2684, 574, 264, 11376, 198, 1073, 42216, 64802, 30666, 1139, 813, 3130, 323, 16054, 1139, 279, 8987, 198, 26894, 11, 539, 315, 63463, 2136, 8530, 11, 719, 315, 7172, 13, 3005, 7020, 1457, 430, 433, 1053, 3621, 1578, 323, 1578, 11, 439, 1317, 439, 568, 198, 75, 2270, 13, 578, 6661, 1364, 1047, 4036, 389, 198, 1964, 13306, 1938, 574, 27724, 1139, 1077, 8271, 13, 2684, 1053, 387, 430, 304, 279, 198, 21733, 2288, 11, 4762, 13, 3005, 574, 3339, 264, 34300, 43789, 198, 266, 23871, 13, 2030, 430, 13658, 315, 279, 1176, 1938, 520, 2162, 1364, 574, 198, 69769, 1908, 13, 3005, 574, 16089, 994, 24101, 4024, 323, 2163, 1077, 7636, 555, 1077, 4027, 13, 2030, 994, 1364, 6755, 264, 3094, 304, 279, 14321, 11, 1364, 9107, 279, 6134, 11937, 13, 3005, 198, 32345, 11075, 311, 3449, 42216, 449, 264, 15648, 13, 92014, 7263, 4400, 280, 32158, 1047, 9687, 430, 2736, 13, 330, 31516, 27603, 315, 16267, 1359, 814, 2663, 7724, 1093, 279, 832, 389, 279, 17569, 13, 3005, 30315, 1418, 11, 449, 813, 1203, 311, 1077, 11, 568, 198, 939, 1982, 5678, 1093, 264, 2294, 5679, 13, 1283, 31645, 1523, 520, 1077, 11, 813, 47626, 6548, 18186, 13, 330, 2181, 596, 1695, 311, 387, 2162, 323, 311, 1518, 499, 1578, 13, 43549, 956, 499, 2586, 304, 311, 856, 198, 11029, 7673, 330, 2460, 279, 810, 2944, 3249, 499, 1288, 2586, 1359, 1364, 39169, 8485, 398, 11, 323, 5762, 279, 198, 11020, 7029, 13, 578, 2697, 1370, 9761, 574, 71414, 449, 4027, 323, 8579, 50552, 11, 10107, 449, 198, 83014, 348, 2315, 2539, 315, 19837, 13, 735, 13, 25319, 4871, 323, 3952, 264, 9200, 198, 48554, 315, 279, 3130, 13, 330, 26556, 603, 584, 617, 1903, 264, 5128, 1695, 2683, 315, 420, 11, 358, 198, 4291, 279, 5684, 323, 279, 19358, 11, 323, 499, 449, 701, 5128, 73271, 323, 198, 22479, 5128, 659, 1210, 46993, 5602, 813, 14765, 11, 323, 574, 198, 71, 680, 1291, 1109, 1364, 1047, 1027, 369, 5672, 13, 3005, 2231, 389, 279, 16579, 2697, 79991, 198, 438, 1099, 2492, 430, 1051, 264, 961, 315, 1077, 313, 28708, 1077, 46175, 1579, 11, 7111, 709, 520, 198, 40617, 449, 279, 2697, 33352, 2840, 3095, 430, 1364, 1047, 1766, 1051, 39928, 389, 198, 20577, 1195, 13, 3005, 3177, 291, 279, 9090, 2922, 1141, 311, 1304, 15600, 11, 24465, 704, 279, 1888, 10716, 198, 2000, 1461, 11, 323, 281, 12400, 264, 45738, 449, 1077, 1664, 1824, 1636, 15548, 6206, 13, 330, 3112, 1518, 11, 1618, 596, 264, 4579, 267, 1786, 1210, 330, 40, 1097, 73011, 21901, 315, 1694, 16993, 1142, 1359, 1071, 735, 2637, 323, 5115, 293, 1091, 291, 304, 198, 26301, 502, 16975, 315, 1664, 33851, 13, 1115, 574, 2731, 1109, 813, 4384, 3130, 198, 455, 31234, 11, 1109, 490, 31557, 3235, 3224, 19795, 11, 1109, 813, 1866, 11555, 13, 330, 5519, 198, 73457, 757, 682, 279, 26681, 315, 279, 6825, 1210, 330, 3947, 706, 1027, 912, 26681, 2533, 499, 4024, 3201, 1359, 1071, 735, 13, 1628, 11, 1606, 198, 9739, 574, 16089, 539, 311, 387, 2163, 311, 813, 1866, 11555, 11, 814, 32627, 520, 420, 198, 4590, 315, 40711, 28485, 13, 330, 89441, 1359, 1071, 2009, 77638, 818, 11, 330, 906, 617, 1027, 1633, 11594, 13, 358, 617, 1047, 856, 198, 52710, 9408, 323, 1097, 1457, 48041, 10332, 304, 17510, 38502, 11441, 264, 2046, 13, 4702, 994, 358, 1047, 682, 856, 6848, 8521, 369, 198, 69, 333, 15247, 11, 358, 636, 17510, 38502, 323, 617, 311, 312, 70208, 1124, 13, 330, 2181, 374, 1633, 29395, 481, 994, 832, 596, 8070, 9221, 264, 23104, 1359, 1071, 198, 19875, 483, 29059, 989, 13, 3005, 574, 9455, 304, 2009, 77638, 818, 2555, 430, 1364, 4460, 1120, 1243, 313, 64, 198, 32127, 488, 11, 264, 3460, 315, 6904, 2968, 11, 430, 1047, 4400, 311, 656, 449, 198, 383, 402, 1918, 13, 3005, 6612, 430, 1618, 574, 264, 893, 1364, 1436, 7095, 11, 4661, 2389, 579, 198, 258, 13, 3005, 15262, 813, 1317, 6206, 11, 813, 559, 79736, 719, 1664, 43321, 15447, 11, 813, 7060, 198, 5478, 449, 1202, 3831, 46175, 13, 3005, 2163, 1022, 1077, 2697, 7958, 811, 11, 313, 64, 198, 2248, 311, 813, 1866, 6996, 315, 1124, 11, 313, 438, 7731, 1203, 304, 1077, 10716, 11, 10307, 198, 1820, 4027, 13, 3277, 735, 13, 14896, 11, 568, 1436, 3137, 1664, 13, 578, 2650, 288, 1047, 1027, 311, 91366, 389, 198, 50828, 13306, 8577, 13, 1283, 7020, 91366, 26, 430, 6688, 1124, 264, 4279, 5015, 13, 1666, 369, 735, 2637, 568, 42762, 14333, 198, 1820, 2697, 4034, 313, 67, 4361, 5678, 520, 1566, 449, 23023, 704, 315, 813, 10716, 13, 330, 2675, 3077, 1027, 1633, 6555, 311, 2610, 757, 304, 11, 18083, 13, 330, 40, 3987, 499, 198, 14724, 2187, 757, 311, 2586, 1578, 13, 2030, 11, 315, 3388, 11, 499, 527, 2133, 311, 387, 1633, 198, 75593, 1210, 1102, 9508, 311, 46993, 1364, 1053, 2646, 387, 8485, 1578, 13, 3005, 1550, 539, 198, 53757, 1461, 311, 733, 3201, 13, 578, 5222, 315, 813, 5655, 7899, 6688, 1077, 264, 5647, 315, 198, 17476, 13, 3005, 15262, 279, 1206, 13671, 315, 279, 1450, 568, 5762, 704, 311, 1077, 11, 994, 520, 198, 4354, 568, 1903, 264, 3351, 9017, 279, 6134, 13, 86631, 358, 1097, 14931, 568, 13942, 1057, 2697, 4717, 1359, 1071, 2009, 77638, 818, 13, 1666, 568, 8036, 279, 6134, 4920, 1461, 11, 1070, 574, 264, 502, 3177, 304, 46993, 596, 198, 58331, 13, 20695, 1051, 539, 1314, 11, 719, 11, 1306, 682, 11, 814, 1051, 539, 75323, 13, 3861, 198, 46068, 2103, 617, 4885, 11, 2466, 323, 3831, 11, 24981, 315, 8071, 323, 7899, 13, 3277, 198, 20577, 1195, 3782, 2162, 11, 279, 15648, 1364, 6688, 1461, 574, 539, 9770, 13, 578, 1938, 596, 43844, 290, 1047, 1027, 3958, 369, 24101, 13, 2009, 77638, 818, 1766, 1077, 389, 279, 198, 66, 3102, 304, 279, 24411, 52319, 48840, 11, 323, 6688, 1077, 264, 4062, 34522, 315, 198, 680, 11310, 2711, 13, 3005, 574, 463, 7069, 709, 1579, 449, 67026, 11, 449, 264, 16893, 315, 198, 277, 13795, 92003, 30488, 1077, 13, 330, 10156, 313, 8846, 315, 11745, 1359, 1364, 281, 7719, 13, 76996, 313, 285, 198, 5065, 2162, 313, 998, 87117, 26, 323, 313, 1820, 3885, 313, 20577, 1195, 323, 313, 1875, 4897, 574, 439, 3117, 439, 1364, 2751, 13, 735, 2637, 3821, 304, 1450, 11, 1766, 1077, 28334, 15792, 345, 928, 88, 11, 42408, 13, 1283, 1047, 1027, 10235, 369, 1063, 1778, 13147, 11, 323, 568, 198, 104402, 4588, 1139, 813, 3130, 369, 1097, 4010, 5392, 275, 7853, 13, 3277, 568, 3782, 1203, 1364, 574, 4661, 198, 359, 64589, 13, 1283, 14760, 279, 48739, 198, 258, 264, 43713, 11, 323, 5762, 433, 927, 1077, 3663, 13, 4740, 264, 892, 279, 993, 10753, 31467, 345, 8248, 1077, 3044, 14958, 57376, 13, 5340, 65670, 11, 889, 1047, 2586, 2162, 555, 430, 892, 11, 7731, 555, 279, 27240, 323, 5762, 1077, 198, 82, 1601, 596, 1450, 13, 8442, 3131, 304, 279, 1828, 6596, 477, 779, 1550, 1364, 6604, 13, 5340, 65670, 574, 2288, 289, 62837, 311, 198, 24467, 279, 6721, 11827, 304, 902, 735, 13, 743, 311, 990, 927, 24101, 13, 330, 40, 3077, 1027, 264, 1633, 2653, 13219, 311, 1077, 1359, 1364, 1071, 13, 330, 2746, 499, 649, 6958, 1077, 198, 20322, 11, 358, 3358, 1456, 311, 1304, 709, 369, 433, 1210, 46993, 7731, 389, 279, 32249, 4994, 11, 60178, 323, 63628, 13, 2435, 1047, 198, 25526, 369, 76996, 26, 719, 279, 2697, 3838, 1047, 912, 21186, 11, 323, 279, 1984, 198, 16514, 6435, 304, 3794, 1022, 13, 3279, 3782, 26346, 287, 709, 279, 32249, 323, 1139, 279, 3130, 13, 330, 11649, 11, 420, 374, 12703, 11, 5340, 65670, 1359, 1071, 2999, 13, 330, 10445, 304, 279, 836, 315, 32450, 345, 9493, 358, 5828, 956, 2212, 11, 3287, 956, 499, 636, 2500, 10896, 13, 1442, 1364, 1047, 1047, 1063, 198, 309, 4010, 5392, 275, 7853, 313, 1875, 7189, 6688, 1077, 1063, 25719, 7853, 315, 1097, 4010, 1359, 1071, 735, 13, 30666, 13, 330, 3947, 574, 2216, 912, 198, 1712, 311, 3708, 369, 21739, 13, 3005, 4661, 4024, 1234, 520, 4376, 2320, 561, 4330, 1210, 7639, 1047, 8774, 813, 3492, 11, 323, 1524, 2999, 13, 1283, 198, 70, 525, 264, 4062, 34522, 520, 420, 16615, 3995, 893, 889, 12570, 779, 30666, 315, 1148, 198, 383, 1047, 2884, 369, 279, 14363, 5333, 11, 323, 4024, 389, 449, 813, 990, 13, 76996, 11721, 264, 2697, 1306, 4848, 11, 323, 505, 430, 4545, 279, 22047, 304, 198, 1820, 14363, 48840, 574, 520, 459, 842, 13, 3005, 7882, 46993, 505, 279, 32249, 345, 2940, 45563, 389, 1077, 12387, 1886, 2914, 2011, 46764, 927, 1077, 26, 743, 5340, 65670, 311, 198, 86, 34002, 1077, 6691, 596, 4950, 323, 3794, 433, 5644, 26, 9107, 11276, 11, 7263, 198, 1382, 323, 11594, 13, 1628, 1243, 11, 449, 4648, 304, 1077, 6548, 11, 1364, 3952, 709, 1077, 198, 3571, 30488, 1077, 6691, 13, 1115, 574, 912, 892, 369, 584, 55323, 26, 430, 1053, 198, 2063, 3010, 13, 9843, 1364, 6656, 311, 735, 2637, 11509, 3821, 3725, 30488, 1077, 13, 330, 40, 1781, 499, 617, 3967, 420, 369, 264, 1317, 892, 1359, 1364, 1071, 13, 1628, 11, 994, 568, 198, 23770, 539, 4320, 25, 330, 10445, 1550, 499, 1095, 757, 4822, 3201, 505, 1077, 30, 1102, 1053, 617, 198, 82850, 1778, 264, 2697, 892, 9135, 330, 1687, 1051, 4560, 311, 656, 1057, 1888, 369, 2225, 315, 499, 1359, 568, 20592, 13, 1102, 3782, 439, 264, 16106, 505, 279, 43957, 315, 279, 198, 29652, 596, 502, 3217, 13, 330, 8100, 706, 1047, 779, 2697, 315, 2324, 1359, 1364, 1071, 11, 927, 323, 927, 13, 330, 6153, 682, 11, 76996, 1359, 568, 1071, 11, 330, 1820, 6825, 3507, 2324, 25, 279, 1917, 374, 1193, 198, 35676, 14708, 13, 3005, 1047, 3021, 323, 2262, 11, 323, 1364, 198, 32345, 499, 1210, 24101, 8636, 264, 2697, 1306, 33433, 11, 264, 11594, 12579, 11, 779, 430, 1193, 76996, 198, 438, 279, 1403, 3026, 7020, 994, 1364, 4024, 3201, 13, 3842, 46368, 311, 279, 51902, 13, 12220, 682, 430, 1317, 11714, 1364, 1047, 7731, 3411, 1203, 927, 1667, 315, 198, 9181, 653, 15674, 2136, 288, 13, 578, 270, 1540, 315, 24101, 596, 90325, 10603, 1047, 2744, 7222, 839, 198, 258, 1077, 26000, 13, 3005, 1047, 1027, 2653, 11, 66637, 442, 3876, 11, 64788, 291, 13, 9843, 568, 3463, 1364, 574, 38678, 287, 11, 323, 198, 42700, 311, 1077, 13, 3234, 499, 1781, 499, 1436, 636, 1124, 682, 704, 315, 279, 3130, 323, 198, 1169, 757, 617, 1077, 7636, 369, 1120, 264, 2478, 4520, 7673, 1283, 23803, 279, 3130, 11, 323, 3952, 709, 813, 38494, 4994, 279, 6134, 13, 1628, 11, 439, 568, 198, 52784, 1070, 11, 568, 3463, 315, 1148, 568, 1047, 1071, 311, 76996, 922, 279, 6825, 13, 5810, 304, 420, 1633, 3838, 1051, 4648, 323, 198, 325, 36235, 26, 5340, 65670, 596, 6917, 2111, 2324, 26, 46993, 323, 42216, 7314, 264, 198, 4930, 323, 75699, 3938, 3871, 26, 5678, 11, 264, 8060, 11, 323, 459, 82509, 269, 13, 3277, 568, 9107, 279, 6134, 1578, 11, 76996, 574, 11509, 555, 1077, 6691, 596, 4950, 13, 10455, 11879, 291, 311, 279, 15197, 13, 1283, 4024, 311, 1077, 11, 323, 1364, 6656, 323, 2231, 1077, 2010, 2403, 813, 17308, 198, 4908, 264, 19781, 1716, 13, 330, 18293, 757, 3201, 11, 735, 13, 1359, 1364, 1071, 20428, 5092, 398, 13, 1628, 11, 449, 813, 6916, 2212, 1077, 11, 568, 6197, 1077, 704, 315, 279, 3130, 13, 42251, 315, 1077, 2678, 14247, 12960, 24101, 596, 4648, 574, 20781, 6612, 13, 5340, 65670, 11953, 1203, 311, 1077, 198, 27243, 264, 13710, 1918, 315, 9090, 430, 1903, 433, 5107, 311, 11984, 449, 198, 1820, 2678, 30118, 811, 315, 1077, 1938, 13, 19292, 24101, 596, 40968, 4107, 11, 902, 1047, 198, 33222, 57130, 1077, 11, 1047, 1027, 7106, 13, 3005, 2011, 617, 1047, 1077, 12544, 264, 198, 4930, 1712, 13, 3005, 27569, 1023, 3278, 315, 279, 6825, 889, 1047, 1922, 418, 1555, 198, 483, 544, 5499, 2919, 11, 323, 1047, 520, 1566, 17551, 1523, 872, 64074, 323, 8036, 198, 50828, 23900, 6548, 11, 311, 279, 29869, 37473, 16409, 315, 872, 8689, 13, 3639, 1550, 198, 20670, 1781, 922, 11, 1521, 3278, 11, 439, 814, 3419, 34259, 922, 30, 14910, 814, 47540, 198, 1820, 3242, 266, 1873, 430, 2322, 872, 22171, 3252, 19567, 11, 279, 85273, 198, 9210, 1053, 539, 1518, 1268, 814, 1051, 22109, 30, 8166, 24014, 11299, 389, 5340, 65670, 596, 198, 69, 8958, 34740, 439, 433, 11203, 389, 1077, 22095, 13, 2876, 1193, 369, 24101, 313, 2000, 24101, 596, 198, 4490, 22583, 17277, 13, 1952, 76996, 313, 438, 304, 2753, 6767, 11, 315, 3388, 11, 389, 735, 33501, 67643, 279, 1972, 1437, 3935, 315, 198, 1820, 21426, 13, 76996, 8774, 709, 1664, 3156, 1306, 279, 32079, 11, 719, 4024, 1523, 198, 1820, 1828, 1938, 449, 264, 3428, 34653, 13, 3279, 1071, 11, 323, 58644, 398, 55844, 1037, 279, 8952, 198, 33871, 3156, 10280, 13, 29084, 323, 11714, 735, 13, 10717, 520, 1077, 6134, 323, 198, 258, 3004, 369, 1077, 11, 323, 6693, 323, 11714, 3782, 76996, 596, 10052, 25, 27567, 1, 43182, 2731, 13, 2030, 279, 2919, 38247, 389, 323, 1364, 1550, 539, 636, 922, 13, 6419, 31234, 11, 46993, 323, 42216, 1047, 10862, 389, 279, 4883, 315, 5209, 84028, 198, 75593, 26640, 13, 42216, 596, 330, 52905, 67, 1, 574, 264, 49277, 832, 13, 2684, 1051, 74413, 198, 438, 73806, 11, 2046, 13368, 3521, 76339, 311, 3224, 2902, 20340, 13, 578, 6825, 14264, 198, 4575, 50786, 311, 9298, 79648, 3009, 1603, 279, 2697, 3838, 520, 682, 198, 31482, 315, 279, 3814, 13, 32980, 35120, 31202, 11, 10043, 42216, 596, 1841, 11, 3952, 311, 198, 13772, 287, 33173, 520, 279, 13587, 304, 7353, 53121, 11, 323, 52468, 813, 91222, 198, 998, 813, 6691, 13, 330, 2675, 2646, 1440, 1405, 499, 527, 449, 1124, 7752, 1359, 568, 1071, 27851, 13, 330, 1687, 198, 2527, 704, 369, 4376, 459, 6596, 596, 1629, 304, 279, 11714, 11, 323, 636, 2162, 449, 279, 198, 76, 34263, 2695, 351, 2439, 13, 1628, 279, 810, 1063, 315, 1124, 617, 1047, 311, 7172, 11, 279, 810, 814, 198, 53757, 311, 6678, 279, 5780, 13, 1442, 358, 636, 264, 6140, 11, 358, 2846, 2133, 311, 9567, 433, 1418, 198, 1820, 10160, 596, 856, 1648, 1210, 2030, 11, 3137, 439, 568, 2643, 11, 304, 32980, 35120, 31202, 596, 29947, 4851, 1070, 574, 912, 198, 61665, 315, 24521, 290, 13, 42216, 1047, 2728, 1461, 264, 893, 596, 2683, 11, 323, 568, 1053, 198, 30133, 555, 433, 11, 912, 5030, 1148, 3782, 13, 2684, 1051, 1063, 2574, 430, 32980, 35120, 31202, 1550, 539, 3371, 813, 6691, 13, 2684, 1051, 59938, 994, 279, 86631, 1841, 574, 10409, 11, 539, 449, 46993, 198, 438, 1077, 4885, 11, 719, 449, 3278, 315, 264, 2204, 1917, 26, 59938, 994, 279, 198, 18481, 574, 539, 264, 3224, 12675, 11, 719, 264, 5754, 37002, 26, 59938, 994, 198, 88960, 35120, 31202, 11, 98555, 505, 279, 5696, 596, 10954, 555, 1063, 63463, 12822, 345, 41450, 3412, 10508, 311, 279, 55060, 1841, 323, 2019, 1778, 35603, 315, 34296, 198, 300, 568, 1436, 6227, 13, 32980, 35120, 31202, 11, 889, 1047, 3940, 2324, 449, 2478, 198, 484, 36807, 11, 574, 304, 8137, 315, 13490, 1778, 439, 568, 1047, 13, 3861, 1778, 3814, 46993, 2231, 304, 11, 21078, 15508, 3725, 304, 1077, 4950, 11, 1418, 279, 198, 21321, 389, 279, 26976, 301, 39674, 839, 6596, 1306, 6596, 1139, 279, 3814, 13, 1283, 3288, 264, 5296, 505, 279, 5274, 304, 279, 6693, 1473, 7189, 3987, 499, 527, 539, 18290, 11, 76745, 13, 578, 1841, 14760, 1523, 3221, 279, 198, 16813, 10349, 1566, 3814, 11, 323, 1070, 574, 4400, 311, 656, 719, 311, 8493, 279, 198, 9471, 1070, 13, 358, 1053, 617, 3288, 499, 3492, 11, 719, 358, 1550, 539, 1390, 311, 436, 1559, 198, 9514, 13, 3639, 656, 499, 2019, 311, 279, 27803, 311, 45427, 323, 87117, 49043, 7673, 3005, 8122, 764, 20262, 279, 14438, 10349, 430, 6693, 345, 438, 1766, 430, 42216, 1047, 539, 1027, 1070, 13, 2030, 11, 8051, 1364, 7020, 1457, 198, 9210, 568, 574, 31952, 2299, 1077, 11, 439, 568, 2744, 1047, 98884, 1077, 11, 439, 4762, 198, 383, 2744, 1053, 11, 1364, 20365, 33337, 311, 17302, 1461, 449, 1148, 1364, 7020, 13, 3005, 198, 939, 13430, 11, 439, 1690, 264, 5333, 706, 14362, 3200, 1603, 11, 505, 75867, 1461, 449, 813, 198, 11828, 13, 2030, 279, 2132, 892, 433, 7077, 11, 1364, 574, 436, 37588, 13, 1102, 574, 4661, 10280, 198, 3473, 11, 323, 76996, 574, 1664, 389, 279, 1648, 311, 13654, 11, 65355, 323, 1633, 198, 5902, 11, 719, 2133, 14297, 709, 323, 1523, 279, 62862, 389, 735, 3238, 82, 6916, 11, 323, 198, 82, 15154, 449, 5340, 65670, 323, 735, 13, 520, 279, 14177, 2007, 13, 3005, 574, 60206, 311, 387, 198, 1445, 389, 14523, 369, 10280, 11, 323, 735, 13, 6612, 430, 568, 1053, 617, 311, 3041, 1077, 198, 455, 5246, 13, 2468, 2380, 297, 63510, 832, 6693, 76996, 436, 37588, 505, 264, 3177, 6212, 311, 6865, 264, 198, 81, 3713, 389, 1077, 6134, 13, 3005, 11953, 264, 198, 66, 3397, 11, 323, 1603, 1364, 12570, 1364, 7111, 520, 76996, 596, 3821, 389, 279, 96803, 198, 2048, 13, 330, 40, 26253, 856, 9042, 574, 5076, 1359, 1364, 1071, 13, 330, 40, 1097, 14931, 311, 289, 3448, 499, 345, 68753, 3520, 11, 719, 358, 1541, 956, 1440, 1148, 311, 656, 1210, 76996, 1047, 3177, 291, 279, 6962, 323, 574, 21939, 389, 1077, 32623, 2427, 785, 13, 330, 4599, 568, 4024, 704, 1550, 568, 2019, 313, 1875, 27423, 1071, 4400, 13, 76996, 11, 358, 1097, 2133, 2162, 304, 279, 198, 57902, 1251, 1210, 330, 2675, 1541, 956, 3152, 430, 11, 656, 499, 7673, 330, 8161, 956, 358, 1427, 439, 422, 358, 3152, 433, 30, 2650, 1790, 315, 420, 3460, 315, 3245, 374, 264, 5333, 198, 13066, 3950, 311, 46753, 7673, 4314, 2574, 2744, 2873, 17936, 304, 279, 198, 20231, 315, 279, 3814, 11, 719, 555, 6693, 313, 1875, 19875, 483, 421, 404, 839, 389, 1077, 13, 1472, 6227, 279, 6661, 358, 2751, 389, 856, 13306, 198, 1316, 7673, 330, 22404, 19704, 433, 477, 539, 1359, 1071, 46993, 5679, 3640, 398, 11, 330, 9210, 596, 7041, 1148, 706, 198, 71, 680, 6901, 13, 358, 2751, 2555, 704, 315, 430, 2697, 11494, 315, 264, 35120, 31202, 8334, 11, 323, 198, 1820, 2800, 358, 1440, 1606, 358, 1440, 42216, 13, 578, 8952, 1047, 15972, 76996, 832, 3245, 25, 430, 433, 3952, 1690, 1274, 311, 198, 7072, 264, 1917, 11, 323, 430, 704, 315, 1521, 1063, 1051, 40605, 43510, 13, 2030, 198, 1805, 1047, 14958, 369, 1077, 264, 2867, 59851, 13, 2684, 1051, 1778, 1274, 345, 438, 1606, 832, 574, 304, 279, 1917, 369, 2532, 832, 42777, 369, 1124, 13, 7570, 198, 1820, 328, 9073, 1047, 1027, 3169, 311, 279, 5333, 315, 279, 14708, 13, 2030, 1618, 60845, 76996, 1766, 279, 2294, 51215, 315, 279, 1917, 313, 9210, 198, 28753, 315, 420, 17192, 279, 1695, 7831, 810, 1109, 279, 45077, 13, 330, 2181, 3727, 757, 12491, 682, 279, 3026, 304, 279, 1917, 13, 42216, 34672, 369, 499, 11, 323, 3686, 568, 649, 656, 264, 3245, 1093, 420, 9135, 46993, 574, 83240, 20233, 7162, 709, 323, 1523, 279, 3130, 13, 386, 486, 41957, 2200, 198, 32345, 779, 8942, 291, 1077, 13, 3005, 574, 1457, 11, 389, 279, 7479, 520, 3325, 11, 2753, 12304, 1109, 198, 68753, 3520, 13, 330, 7009, 527, 539, 682, 1093, 42216, 11, 9901, 32450, 1359, 1364, 1071, 13, 3092, 7126, 374, 832, 11, 323, 701, 735, 2637, 1618, 304, 279, 3838, 11, 374, 198, 43063, 1210, 2468, 3116, 297, 63510, 304, 279, 6693, 42216, 86631, 3782, 2162, 13, 3005, 198, 444, 7096, 291, 1461, 304, 1077, 7833, 4251, 65510, 323, 30315, 369, 1461, 311, 6604, 13, 330, 40, 1097, 14931, 311, 387, 779, 3389, 11, 11517, 1359, 568, 1071, 13, 330, 791, 2144, 374, 11, 358, 1097, 682, 304, 13, 358, 198, 16514, 10043, 279, 1841, 704, 31048, 39697, 6588, 13, 1226, 42423, 704, 264, 28387, 323, 279, 3245, 198, 42286, 927, 1210, 46993, 14000, 1243, 430, 813, 1314, 6916, 574, 21363, 81073, 555, 813, 3185, 13, 96424, 94937, 1432, 41672, 86631, 1047, 1027, 32620, 20250, 311, 3041, 709, 682, 813, 49683, 26870, 198, 4291, 813, 13306, 1938, 13, 763, 813, 1280, 337, 306, 11, 4856, 45743, 1648, 11, 568, 574, 1790, 198, 258, 3021, 449, 813, 7555, 13, 2030, 449, 279, 31352, 90700, 826, 315, 279, 1176, 4038, 315, 198, 5730, 8950, 1047, 2586, 264, 12876, 311, 387, 26893, 3131, 1578, 520, 813, 3663, 198, 970, 13, 32171, 1047, 4529, 1461, 11, 539, 369, 1148, 568, 574, 11, 719, 369, 1148, 568, 9508, 198, 998, 387, 13, 3005, 7020, 1461, 1457, 313, 543, 813, 2678, 198, 485, 337, 2436, 11, 813, 7958, 811, 11, 813, 44667, 13, 25929, 389, 11, 1093, 1023, 198, 65762, 2533, 279, 1917, 6137, 11, 1364, 1053, 4048, 311, 834, 16167, 11, 311, 7958, 311, 198, 9978, 19704, 1461, 1148, 568, 574, 539, 13, 1628, 779, 11, 1203, 311, 32171, 4848, 5672, 1306, 813, 13306, 1938, 3782, 42216, 198, 4438, 68, 11, 539, 449, 264, 24710, 311, 14195, 279, 2362, 5133, 11, 719, 369, 198, 884, 60635, 2200, 13, 46993, 274, 22537, 291, 313, 383, 4934, 1695, 26085, 26, 46993, 574, 72729, 519, 313, 383, 198, 87692, 25065, 26, 1364, 834, 35012, 315, 1461, 323, 8710, 1077, 834, 55539, 313, 383, 198, 87692, 14765, 13, 1283, 4934, 2324, 311, 387, 10882, 323, 71414, 11, 2085, 198, 2827, 6417, 12922, 11, 264, 2697, 990, 323, 1790, 1514, 11, 264, 7172, 994, 832, 574, 198, 339, 874, 88, 13, 28704, 13909, 3582, 433, 574, 11, 323, 18538, 389, 264, 5076, 8197, 11, 8530, 345, 33980, 304, 813, 4851, 42216, 596, 1193, 79422, 574, 369, 23871, 26, 719, 420, 198, 71, 67391, 2011, 387, 315, 459, 4642, 3460, 313, 1962, 2262, 11, 902, 374, 28979, 11, 719, 198, 268, 4215, 479, 13, 2360, 33605, 3318, 1202, 2010, 198, 1885, 369, 603, 13, 4702, 264, 2697, 1629, 927, 279, 3224, 19795, 11, 36346, 7673, 1102, 574, 279, 13658, 315, 279, 1938, 1603, 46993, 596, 3814, 4034, 311, 198, 68753, 3520, 13, 578, 5274, 1047, 1027, 8036, 11, 56612, 311, 264, 4648, 11, 323, 42216, 574, 304, 198, 981, 6045, 315, 264, 13560, 13, 330, 1687, 3358, 733, 704, 311, 279, 62930, 7278, 16344, 323, 617, 198, 13066, 716, 1210, 330, 4897, 596, 539, 837, 11, 32171, 11, 323, 499, 1440, 433, 1210, 578, 19795, 527, 20268, 2653, 26, 459, 6596, 596, 1629, 198, 18614, 279, 3224, 690, 4546, 701, 1933, 1203, 1210, 6122, 323, 12141, 449, 701, 7555, 1359, 1071, 279, 3828, 345, 438, 1344, 2234, 3201, 505, 1461, 13, 578, 1566, 2478, 5672, 1047, 10409, 704, 1077, 15792, 7216, 11, 719, 1364, 2103, 39786, 198, 376, 2492, 315, 1077, 17563, 13, 3005, 198, 7349, 291, 2917, 13610, 8334, 819, 11, 4661, 1877, 1752, 13, 9393, 1364, 5602, 1461, 289, 1736, 994, 1364, 9932, 46993, 11, 1077, 5986, 6940, 198, 43842, 1503, 13, 330, 2675, 636, 704, 315, 1618, 1359, 1364, 1071, 15187, 13, 330, 40, 3287, 956, 2610, 499, 311, 2586, 198, 1445, 13, 1472, 2744, 7020, 358, 1053, 617, 311, 32918, 1063, 1938, 1210, 358, 3287, 956, 6865, 904, 6821, 315, 499, 21363, 198, 20019, 279, 8952, 311, 4048, 1268, 358, 574, 3794, 3235, 1210, 31909, 11, 832, 315, 313, 1, 1283, 20365, 33337, 927, 813, 7555, 596, 836, 13, 330, 32, 198, 29652, 358, 1440, 1633, 1664, 574, 304, 279, 4967, 35789, 13, 2684, 1053, 617, 1027, 198, 1820, 38736, 311, 2343, 422, 358, 4265, 439, 1790, 439, 2663, 709, 1210, 330, 2675, 2646, 3309, 757, 499, 1051, 2133, 311, 636, 12502, 1210, 39348, 291, 11, 568, 43578, 459, 6916, 2212, 1077, 13, 330, 40, 8967, 311, 3371, 499, 11, 26828, 26, 719, 499, 2751, 14363, 13, 5884, 5269, 11, 358, 313, 40, 38674, 311, 198, 73457, 499, 11, 26828, 1210, 2684, 574, 264, 10882, 8430, 315, 198, 5065, 2162, 922, 2133, 1070, 1578, 13, 1628, 11, 1457, 430, 279, 12047, 9568, 315, 198, 50828, 6574, 574, 927, 11, 568, 574, 74106, 44467, 13, 2030, 32171, 8738, 311, 198, 2752, 8071, 287, 1461, 1794, 655, 398, 13, 330, 40, 3077, 2751, 2555, 311, 3371, 499, 1359, 1364, 1071, 13, 330, 8161, 956, 617, 264, 5052, 11, 323, 1541, 956, 198, 4355, 7595, 13, 1442, 499, 656, 11, 358, 3358, 313, 40, 3358, 7940, 704, 315, 279, 3321, 13, 358, 3077, 2751, 264, 2035, 304, 198, 64, 3637, 13, 3005, 574, 264, 6555, 3828, 323, 568, 574, 21901, 315, 1077, 13, 1628, 568, 574, 539, 653, 726, 819, 922, 433, 13, 1283, 1550, 539, 1390, 1077, 311, 9352, 311, 904, 832, 775, 13, 330, 4054, 315, 279, 37465, 304, 279, 8952, 11, 264, 9083, 5874, 11, 706, 2751, 757, 2555, 311, 198, 3055, 520, 445, 11696, 263, 323, 473, 2925, 5673, 596, 13, 358, 1097, 2133, 389, 369, 279, 6186, 4251, 6412, 13, 1442, 198, 40, 1304, 1695, 814, 690, 2567, 757, 1210, 1283, 1047, 2231, 1077, 16038, 2085, 264, 5965, 76, 26, 323, 1457, 568, 2322, 1077, 17480, 198, 4291, 14765, 13, 2435, 1053, 617, 264, 13560, 198, 998, 3522, 11, 323, 1243, 814, 1053, 2019, 1695, 1481, 9188, 13, 1283, 574, 3794, 1022, 1664, 11, 682, 2574, 6646, 13, 2030, 430, 4536, 956, 904, 198, 20489, 3249, 584, 13434, 956, 387, 4885, 11, 374, 433, 30, 358, 1053, 1093, 311, 2733, 430, 358, 649, 3009, 304, 1457, 323, 1243, 323, 2019, 1268, 656, 499, 198, 3055, 1210, 578, 6420, 315, 76996, 596, 836, 7263, 709, 304, 813, 4059, 46993, 439, 568, 1047, 198, 2414, 1077, 430, 6693, 13, 3005, 1511, 311, 387, 264, 1695, 10775, 345, 8248, 1364, 1047, 2646, 1027, 279, 1890, 2533, 279, 1938, 315, 279, 13306, 13, 1283, 3463, 198, 1964, 19451, 9017, 1461, 574, 832, 315, 38141, 13, 2030, 904, 4879, 389, 813, 961, 311, 282, 73555, 433, 1193, 2322, 449, 9439, 21847, 13, 330, 40, 3358, 3371, 499, 1148, 584, 3358, 656, 1359, 568, 1071, 13, 330, 1687, 2834, 956, 733, 311, 904, 315, 279, 2362, 198, 27170, 13, 358, 3077, 1766, 264, 502, 5754, 7830, 304, 279, 3224, 430, 596, 62486, 198, 268, 1409, 311, 7937, 21739, 13, 1226, 3358, 733, 704, 311, 5124, 3886, 596, 323, 636, 1063, 14177, 13, 1628, 389, 279, 1648, 704, 568, 12439, 709, 311, 279, 6661, 315, 198, 50828, 9306, 13, 578, 6671, 91826, 660, 1461, 25, 32171, 449, 1077, 502, 3805, 198, 1073, 35460, 11, 1077, 502, 54266, 1073, 2136, 26, 813, 10882, 1841, 26, 32980, 35120, 31202, 596, 198, 4338, 846, 295, 1203, 323, 5225, 25212, 13, 578, 18427, 1047, 682, 279, 54904, 315, 264, 502, 62366, 304, 433, 13, 1283, 12020, 279, 198, 29652, 449, 18440, 768, 11, 1550, 539, 32262, 994, 1364, 16436, 264, 36213, 11, 6612, 198, 6200, 363, 11559, 13053, 9373, 323, 506, 495, 519, 520, 279, 1890, 892, 13, 3277, 279, 1841, 24465, 709, 1603, 279, 5124, 3886, 2035, 11, 568, 43578, 264, 198, 53770, 54135, 4121, 1139, 32980, 35120, 31202, 596, 539, 927, 92900, 1450, 13, 330, 40, 1541, 956, 4059, 279, 25212, 1359, 568, 1071, 13, 1628, 198, 88960, 68941, 813, 4817, 304, 33638, 13051, 13, 330, 3947, 596, 1120, 3403, 315, 279, 8117, 304, 757, 1359, 1071, 32980, 11, 330, 998, 1440, 1268, 311, 198, 36341, 264, 2763, 323, 2019, 4400, 11, 4491, 13, 1283, 93270, 31161, 398, 704, 315, 279, 1841, 323, 10235, 311, 51358, 433, 13, 330, 40, 3358, 1120, 3041, 1077, 279, 364, 13486, 927, 6, 1457, 323, 1243, 1359, 568, 1071, 13, 330, 8100, 3358, 198, 52738, 6573, 422, 358, 1095, 1077, 2559, 1210, 32171, 1047, 8208, 709, 279, 15376, 1853, 311, 279, 3838, 13, 3005, 1047, 279, 8352, 315, 198, 20986, 1664, 304, 1077, 15447, 11, 323, 1077, 2678, 9072, 449, 1202, 1317, 934, 484, 198, 438, 1077, 9048, 23283, 266, 1051, 47994, 323, 10671, 13, 3005, 2646, 40552, 14715, 11, 439, 198, 19875, 483, 574, 43131, 311, 656, 13, 42536, 369, 42216, 11, 40788, 648, 1550, 539, 1518, 1461, 13, 362, 8987, 6063, 43165, 198, 11748, 291, 520, 279, 2007, 304, 279, 18397, 48840, 11, 1418, 40788, 648, 41778, 289, 81635, 304, 198, 1820, 9979, 13, 32980, 35120, 31202, 11, 2133, 2212, 279, 3185, 1853, 311, 279, 9979, 6134, 449, 198, 41115, 315, 4106, 11033, 323, 264, 3224, 87117, 369, 813, 20268, 23152, 11, 5602, 198, 1964, 1555, 279, 3321, 58218, 74820, 927, 279, 45115, 11, 323, 20365, 33337, 13, 5112, 11, 2085, 264, 3492, 11, 568, 9165, 59091, 291, 1203, 311, 279, 1841, 1578, 11, 323, 11, 72179, 198, 18614, 279, 8941, 83353, 11, 9960, 5678, 449, 73671, 13, 763, 813, 13365, 332, 3093, 4059, 1051, 198, 66, 7770, 2294, 29600, 11, 323, 32883, 311, 813, 19683, 574, 832, 13, 578, 4330, 198, 78289, 1590, 304, 813, 18301, 1047, 4400, 8996, 311, 656, 449, 433, 13, 2468, 63134, 568, 1047, 8040, 264, 19675, 315, 3116, 4339, 13, 1102, 3952, 279, 198, 2050, 315, 279, 18288, 18592, 11, 279, 18165, 7498, 1392, 11, 323, 279, 356, 349, 331, 2191, 13, 1102, 198, 16514, 25, 330, 70738, 701, 1866, 2626, 1210, 578, 18841, 315, 40788, 648, 596, 26092, 42761, 8173, 719, 1550, 539, 54904, 198, 40617, 13, 1442, 1364, 4934, 311, 656, 279, 3460, 315, 3245, 1364, 198, 16514, 3815, 11, 430, 574, 1077, 31017, 13, 40788, 648, 323, 1077, 6278, 57859, 31657, 11, 42216, 198, 4438, 68, 323, 32171, 313, 1820, 54050, 574, 539, 50383, 449, 1778, 12135, 13, 1102, 198, 1068, 291, 1124, 449, 25065, 3156, 814, 1051, 1766, 704, 11, 994, 433, 9408, 1202, 198, 89112, 13, 3082, 311, 813, 11471, 11, 42216, 289, 3448, 291, 279, 21811, 8334, 1603, 11888, 198, 78, 63510, 13, 32171, 1047, 35661, 2697, 323, 29850, 4400, 26, 719, 86631, 574, 10284, 198, 54754, 7913, 13, 330, 36227, 1077, 279, 364, 13486, 927, 49982, 568, 3309, 32980, 11, 330, 438, 1243, 733, 1203, 323, 46764, 198, 18614, 279, 73671, 1578, 13, 11205, 5208, 574, 6435, 323, 11413, 927, 279, 198, 11389, 19795, 11, 719, 994, 814, 8813, 279, 3314, 5754, 86631, 22982, 1825, 279, 198, 339, 28635, 13, 1283, 3952, 17393, 198, 438, 2751, 3201, 449, 1124, 11, 32681, 520, 279, 3828, 596, 6962, 1725, 315, 73082, 13, 330, 14524, 3156, 358, 636, 7953, 4567, 11966, 8078, 1359, 568, 1071, 11, 330, 438, 358, 3358, 1095, 1077, 704, 13, 578, 3828, 7731, 30488, 1461, 449, 1077, 6548, 8521, 8469, 13, 1283, 1047, 1027, 16558, 345, 438, 279, 41463, 315, 279, 45304, 574, 304, 813, 7899, 13, 3005, 574, 2133, 311, 1304, 1461, 3974, 709, 311, 279, 6661, 315, 813, 11471, 311, 198, 3427, 3201, 520, 279, 3838, 6134, 26, 323, 810, 323, 810, 1364, 15393, 430, 433, 1053, 198, 1395, 5107, 13, 12361, 315, 32681, 994, 198, 32158, 24465, 1203, 505, 264, 463, 3018, 291, 2211, 676, 11, 568, 6656, 1765, 398, 13, 95944, 3357, 5238, 198, 9210, 1364, 27569, 9922, 505, 813, 57426, 8839, 311, 279, 24359, 315, 813, 198, 33780, 13, 17830, 1364, 4295, 389, 264, 3197, 311, 1304, 1461, 3009, 15038, 304, 1077, 12818, 198, 438, 1095, 1077, 636, 704, 315, 279, 1841, 13, 4800, 433, 5946, 1124, 11, 323, 439, 198, 61917, 814, 5946, 433, 13, 42216, 596, 1841, 5675, 389, 198, 1820, 35231, 11, 719, 18661, 389, 279, 1317, 2237, 50699, 11, 902, 30103, 3690, 449, 264, 198, 1030, 1113, 315, 15792, 10054, 13, 330, 40, 6562, 499, 4265, 1095, 1124, 636, 8469, 11, 42216, 13, 330, 40, 3309, 499, 584, 4265, 5944, 311, 45427, 1210, 3639, 279, 409, 10743, 574, 279, 5030, 449, 198, 65762, 11, 90064, 30, 5810, 574, 32171, 439, 198, 708, 655, 439, 46993, 13, 5414, 3177, 1841, 1940, 1810, 291, 323, 17948, 279, 2466, 1841, 17345, 13, 1952, 264, 11113, 5754, 198, 66372, 4400, 810, 6129, 1109, 11102, 27275, 79838, 1053, 617, 1027, 279, 198, 1407, 13, 2030, 389, 279, 10054, 279, 2678, 1841, 60132, 291, 2212, 323, 61014, 927, 279, 198, 7334, 315, 279, 6201, 13, 2468, 279, 5740, 315, 279, 4051, 1968, 433, 6656, 927, 13, 86631, 28634, 5678, 323, 14980, 198, 261, 440, 11, 449, 832, 6916, 21363, 520, 813, 3185, 13, 2684, 574, 912, 5222, 520, 682, 505, 198, 1820, 8334, 1234, 279, 8941, 83353, 13, 6419, 279, 6201, 75803, 264, 8987, 11, 46298, 6374, 12970, 198, 18257, 11, 1317, 11977, 17919, 16038, 279, 20268, 23962, 315, 12690, 13, 3277, 568, 198, 265, 3939, 279, 1841, 11, 507, 68070, 5169, 1766, 32171, 11961, 31175, 5757, 389, 279, 5015, 13, 763, 279, 198, 86, 25662, 315, 279, 1841, 279, 50552, 1047, 539, 1027, 56807, 3384, 11, 323, 555, 872, 3177, 198, 383, 1903, 704, 86631, 11, 2064, 17718, 294, 8934, 1570, 13, 578, 1023, 3697, 315, 507, 68070, 5169, 596, 4717, 1047, 93270, 1523, 279, 6201, 555, 430, 198, 1712, 13, 3161, 279, 12576, 315, 264, 26128, 11, 814, 2751, 279, 1841, 709, 13, 32980, 35120, 31202, 11203, 198, 67, 5599, 839, 389, 813, 3663, 30456, 13, 3277, 568, 3782, 311, 323, 9107, 813, 6548, 345, 87643, 4661, 97946, 1247, 291, 449, 16337, 13, 330, 40, 2846, 682, 1314, 1359, 1071, 32980, 35120, 31202, 13, 1628, 11, 994, 814, 9076, 1461, 198, 1336, 47190, 25, 330, 79384, 449, 279, 4027, 55051, 13, 358, 313, 40, 313, 1, 362, 993, 10753, 315, 198, 92724, 38817, 813, 3663, 13, 3161, 813, 11977, 568, 30831, 198, 40617, 726, 311, 264, 11961, 2361, 11, 323, 11299, 1203, 1578, 13, 96424, 53694, 5660, 1432, 1383, 10280, 6187, 76996, 574, 1203, 304, 279, 8952, 11, 264, 2697, 78853, 11, 719, 198, 838, 5485, 398, 11075, 311, 2567, 1077, 2324, 311, 1202, 1906, 315, 2532, 13, 3005, 1047, 264, 198, 36341, 449, 735, 13, 279, 3814, 1603, 1364, 2163, 13, 45563, 574, 704, 11, 323, 76996, 1047, 2231, 279, 18397, 48840, 304, 2015, 13, 735, 13, 7731, 555, 198, 1820, 2007, 323, 15746, 1077, 439, 1364, 7882, 922, 279, 3130, 13, 578, 3347, 2478, 5672, 1047, 1027, 1633, 11364, 311, 1461, 25, 311, 1520, 1077, 709, 323, 198, 2996, 279, 32249, 11, 311, 1373, 311, 1077, 304, 279, 59938, 439, 1364, 11203, 389, 279, 27240, 198, 258, 279, 52319, 48840, 26, 3010, 11, 439, 1364, 13241, 11, 311, 4546, 2678, 294, 62124, 2162, 198, 2000, 1077, 35788, 11, 323, 11, 3515, 14980, 927, 45563, 1418, 1364, 30205, 1124, 11, 311, 198, 68760, 1124, 304, 38586, 311, 430, 8582, 3130, 313, 383, 1047, 539, 1027, 779, 6380, 304, 198, 42820, 13, 330, 40, 3987, 499, 1541, 956, 2733, 439, 422, 499, 2011, 4822, 389, 1359, 1364, 1071, 74884, 13610, 13, 330, 2688, 198, 9210, 584, 1541, 956, 1390, 499, 313, 9514, 1440, 2731, 1109, 430, 1210, 330, 3947, 374, 912, 2035, 775, 304, 279, 4459, 1917, 430, 358, 1390, 311, 733, 311, 1359, 568, 198, 84359, 5042, 13, 330, 40, 2873, 311, 387, 2744, 39661, 389, 18570, 596, 45972, 311, 313, 998, 2567, 2574, 198, 998, 3522, 13, 5629, 11, 369, 1667, 323, 1667, 11, 433, 574, 71322, 5340, 65670, 26, 1457, 433, 374, 198, 9514, 1210, 330, 8161, 956, 499, 13383, 430, 11, 4619, 315, 701, 1694, 26259, 311, 757, 11, 433, 374, 198, 40, 889, 1097, 2073, 34229, 2915, 26259, 311, 499, 30, 358, 617, 12439, 198, 20019, 313, 258, 2204, 7634, 323, 304, 2204, 5627, 13, 358, 1053, 4856, 387, 198, 6881, 1109, 12660, 775, 304, 279, 1917, 1210, 2684, 574, 779, 1790, 430, 574, 75323, 304, 813, 198, 58331, 430, 568, 1550, 539, 1390, 1077, 311, 1518, 13, 3005, 1053, 387, 5115, 13171, 11, 568, 198, 83, 820, 5678, 11427, 351, 989, 11, 315, 78849, 1461, 704, 315, 33638, 58043, 422, 1364, 3596, 198, 8890, 22456, 13, 1628, 568, 574, 16984, 313, 2642, 14152, 11, 2533, 568, 4934, 1077, 779, 1790, 313, 9210, 568, 198, 41450, 387, 24931, 323, 7621, 2785, 3403, 311, 1935, 1077, 1524, 389, 1884, 3878, 13, 20696, 574, 5644, 369, 1077, 471, 311, 279, 8952, 13, 3005, 1047, 1027, 704, 198, 9210, 1938, 311, 2231, 19837, 389, 279, 11594, 25165, 1405, 24101, 11203, 449, 48135, 198, 89112, 26, 1364, 1047, 1903, 1077, 4883, 315, 2697, 21728, 389, 279, 6825, 26, 323, 1457, 198, 1964, 7937, 39585, 11, 19937, 11, 574, 304, 279, 14321, 13, 330, 644, 832, 1648, 11, 433, 690, 387, 264, 2697, 2731, 369, 499, 1109, 422, 46993, 323, 198, 20577, 1195, 1051, 539, 304, 279, 3838, 13, 330, 8100, 13452, 499, 11, 735, 13, 3005, 14117, 389, 499, 11, 2288, 11, 5423, 2533, 430, 3814, 198, 9493, 499, 3952, 2512, 315, 42216, 596, 6916, 1603, 584, 2751, 2999, 13, 358, 3629, 1781, 345, 42, 2637, 1148, 264, 1695, 10896, 499, 1053, 617, 1027, 13, 1472, 7020, 779, 1664, 1148, 311, 656, 198, 2000, 6691, 1210, 3005, 2103, 1436, 539, 7095, 1077, 7899, 922, 1077, 6691, 13, 330, 20577, 1195, 596, 6916, 374, 2133, 311, 387, 5115, 7833, 13, 3279, 374, 779, 12691, 315, 7639, 198, 2017, 433, 13, 9843, 520, 3325, 11, 15716, 814, 1051, 198, 998, 3522, 11, 1364, 7263, 7639, 1139, 279, 10652, 13, 1283, 374, 198, 88657, 11, 1541, 956, 499, 1781, 7673, 330, 26840, 1359, 1071, 735, 382, 1271, 3665, 813, 2324, 11, 568, 1436, 539, 2231, 904, 41463, 1139, 813, 7899, 13, 1102, 574, 539, 304, 3823, 7138, 311, 1755, 810, 315, 1461, 13, 330, 23025, 1317, 13739, 499, 617, 11, 9495, 304, 701, 3130, 313, 12840, 304, 279, 1917, 656, 499, 198, 36341, 922, 30, 3005, 574, 264, 2697, 41690, 315, 1884, 59938, 11, 994, 1364, 7731, 7636, 11, 477, 198, 9493, 5340, 65670, 11, 11961, 449, 1077, 11, 1903, 68782, 1234, 279, 29062, 311, 279, 198, 582, 45349, 3904, 315, 264, 24981, 2854, 315, 61165, 24196, 505, 4028, 279, 14321, 13, 7639, 3782, 304, 2744, 11, 1603, 568, 4024, 345, 438, 11, 49448, 927, 279, 1203, 315, 264, 10716, 11, 1053, 6179, 1077, 315, 279, 10973, 198, 10399, 2136, 315, 2324, 304, 279, 8952, 2085, 1077, 13, 330, 40, 733, 1475, 1938, 1606, 358, 2011, 1359, 568, 1053, 36015, 1077, 8485, 398, 26, 330, 8248, 11, 358, 3371, 198, 9514, 11, 279, 10885, 374, 8208, 704, 315, 433, 13, 3277, 1070, 574, 264, 6140, 430, 1475, 2107, 198, 16514, 21592, 2107, 11, 279, 17983, 5208, 3235, 264, 46979, 6244, 57399, 1210, 1283, 198, 32345, 264, 7362, 14397, 315, 21939, 704, 813, 6206, 11, 449, 264, 2697, 14362, 773, 315, 198, 1820, 28004, 13, 568, 1071, 313, 8370, 11, 1694, 25548, 11, 3445, 512, 31437, 279, 38736, 596, 279, 1005, 9135, 1628, 735, 13, 1053, 2559, 304, 279, 70458, 11, 30666, 20149, 11, 477, 733, 1203, 311, 813, 198, 3039, 323, 5409, 3201, 304, 813, 38411, 279, 2294, 6063, 6603, 389, 15173, 449, 198, 8370, 568, 323, 7639, 1047, 1027, 3318, 704, 264, 1162, 13, 2100, 735, 13, 7731, 555, 279, 18397, 48840, 2007, 323, 34793, 311, 1077, 3137, 315, 7639, 430, 198, 4354, 11714, 3871, 13, 35120, 31202, 311, 11477, 539, 311, 387, 2288, 1790, 64770, 922, 198, 88960, 13, 4800, 430, 499, 527, 198, 21470, 4885, 1359, 313, 32158, 91772, 1461, 289, 380, 3725, 11, 313, 1, 66372, 1063, 1938, 499, 690, 2586, 198, 998, 832, 315, 813, 7677, 13, 7570, 422, 499, 3287, 956, 3619, 7041, 11, 358, 1440, 198, 275, 1053, 54904, 499, 13, 1628, 313, 40, 4265, 1093, 499, 311, 1518, 757, 304, 856, 14113, 11, 735, 13, 1472, 198, 37593, 617, 1210, 3005, 14264, 264, 2697, 12703, 439, 279, 11714, 4024, 389, 13, 3005, 574, 2133, 311, 3194, 735, 627, 1225, 1790, 13, 6104, 1364, 574, 5986, 1364, 1047, 15746, 279, 9042, 369, 279, 892, 311, 198, 37437, 369, 1461, 13, 3005, 7020, 279, 1648, 568, 50365, 279, 4156, 6134, 13, 3005, 7020, 2288, 430, 11, 1120, 1306, 264, 26518, 430, 21699, 198, 1820, 1633, 9168, 304, 279, 1380, 316, 11, 735, 13, 1053, 2586, 311, 279, 4579, 315, 279, 32249, 198, 438, 1650, 25, 27567, 1, 25797, 2303, 11, 1070, 9135, 330, 32, 9188, 11, 264, 9188, 1359, 1364, 1053, 4320, 313, 8370, 574, 11, 568, 28478, 1077, 11, 279, 6300, 198, 2376, 13, 13440, 568, 3782, 709, 279, 32249, 520, 3131, 477, 3952, 813, 1648, 1203, 311, 45563, 1047, 198, 15237, 2954, 389, 3508, 813, 35491, 369, 279, 1938, 574, 14098, 477, 10437, 65, 31458, 13, 1283, 1053, 3194, 1077, 345, 37227, 26, 719, 568, 1053, 617, 5340, 65670, 323, 46993, 323, 313, 6102, 13, 6984, 304, 264, 12960, 198, 998, 7639, 11, 315, 3388, 13, 3005, 29676, 11, 430, 1566, 11714, 11, 389, 11961, 709, 449, 1461, 3156, 33433, 198, 1136, 12616, 304, 10280, 6187, 13, 46993, 323, 42216, 1051, 704, 26, 5340, 65670, 11, 3515, 198, 29844, 291, 76996, 449, 264, 97045, 430, 1047, 1027, 2163, 927, 304, 279, 8221, 505, 198, 1820, 42774, 596, 2626, 11, 1047, 379, 6513, 291, 11937, 311, 4950, 13, 3277, 279, 61794, 7376, 33433, 11, 76996, 436, 37588, 449, 264, 1212, 13, 3005, 198, 8110, 1534, 430, 14188, 315, 1124, 1047, 22066, 11, 323, 430, 735, 13, 578, 2697, 9042, 389, 279, 28745, 3952, 709, 279, 23104, 315, 279, 198, 65294, 288, 11, 323, 17948, 279, 6596, 304, 4062, 357, 4575, 4428, 8554, 13, 76996, 16392, 323, 4024, 927, 311, 735, 2637, 1077, 3776, 8679, 304, 8579, 61607, 922, 198, 1964, 13, 76996, 2163, 279, 2697, 3838, 520, 198, 51464, 11, 449, 279, 8761, 3177, 2103, 20252, 1555, 264, 8770, 315, 16054, 12056, 13, 578, 8952, 78251, 323, 75752, 1051, 2103, 3177, 291, 994, 1364, 4024, 389, 198, 67, 20850, 520, 8254, 297, 63510, 13, 3005, 1047, 1027, 12893, 311, 279, 3026, 596, 34933, 26741, 345, 438, 4024, 1070, 520, 3131, 13, 3005, 1047, 539, 3970, 22770, 22983, 36627, 2533, 1077, 198, 27510, 596, 4648, 26, 719, 1364, 1766, 1077, 389, 14523, 304, 279, 34933, 26741, 13, 1789, 279, 198, 5686, 892, 304, 3116, 4038, 11, 279, 1403, 7724, 1051, 3318, 3185, 555, 3185, 13, 76996, 596, 1421, 1947, 315, 1077, 3766, 2532, 1234, 22770, 22983, 1903, 1077, 198, 77, 651, 788, 13, 330, 1687, 1051, 682, 14931, 311, 6865, 315, 701, 12544, 1359, 1364, 1071, 13, 330, 40, 3987, 584, 4985, 198, 456, 389, 29921, 1210, 2468, 279, 3117, 842, 1403, 272, 2469, 198, 32345, 1027, 9277, 13, 330, 791, 26741, 374, 8987, 11, 4536, 956, 433, 7673, 2684, 527, 2380, 315, 198, 355, 313, 9514, 11, 7182, 11, 323, 264, 47027, 261, 1210, 578, 1176, 3177, 315, 279, 10280, 6693, 574, 5108, 1555, 279, 11276, 13, 22770, 22983, 2231, 704, 279, 13001, 323, 6656, 304, 264, 2626, 12970, 1648, 311, 1077, 198, 27295, 13, 330, 791, 47027, 261, 596, 836, 374, 27738, 9336, 1359, 1364, 1071, 13, 330, 32576, 499, 4265, 2731, 198, 8823, 1077, 449, 279, 17954, 82, 13, 1442, 1070, 596, 904, 1648, 311, 1304, 264, 16930, 11, 1364, 198, 80785, 433, 1210, 1102, 574, 1306, 8223, 994, 76996, 1766, 32980, 35120, 31202, 13, 5414, 6453, 11, 17345, 1448, 57658, 6548, 198, 7349, 291, 520, 1077, 505, 264, 28639, 3663, 13, 330, 40, 574, 304, 264, 879, 3130, 26, 719, 433, 2853, 27219, 628, 15655, 264, 2046, 11, 779, 358, 7882, 13, 3005, 1047, 37287, 311, 733, 11, 719, 735, 627, 32345, 28932, 2403, 433, 13, 3005, 574, 539, 3831, 11, 323, 1364, 1047, 2736, 16654, 198, 59178, 13, 1628, 1457, 279, 990, 315, 279, 26741, 17770, 2653, 13, 3005, 14980, 30488, 1461, 323, 31511, 48259, 813, 1450, 13, 1283, 92840, 311, 1781, 430, 1077, 47084, 574, 369, 813, 4498, 505, 279, 12675, 198, 1073, 264, 879, 8893, 311, 279, 1949, 26741, 13, 330, 12174, 11, 358, 2846, 682, 1314, 11, 9083, 76996, 1359, 568, 1071, 13, 86631, 374, 12798, 4848, 198, 78289, 1590, 264, 2046, 369, 757, 13, 578, 6811, 1990, 757, 323, 279, 1023, 87819, 198, 20019, 1618, 374, 430, 358, 636, 264, 26944, 8148, 389, 856, 35788, 323, 814, 1541, 956, 1210, 330, 42560, 11441, 264, 2046, 369, 264, 26944, 8148, 374, 2133, 1063, 13, 358, 2846, 912, 14545, 660, 198, 277, 380, 73632, 26, 358, 1541, 956, 617, 311, 617, 264, 26944, 8148, 1210, 330, 12389, 814, 3309, 499, 1148, 279, 12544, 374, 7673, 7639, 17882, 374, 2133, 311, 198, 86299, 389, 757, 13, 3639, 264, 3245, 433, 574, 198, 998, 387, 3025, 311, 1935, 420, 2324, 3502, 97586, 315, 32980, 35120, 31202, 596, 323, 1304, 433, 198, 14789, 1578, 0, 2052, 21522, 315, 3026, 1903, 709, 76996, 596, 1917, 25, 279, 2761, 301, 31095, 889, 82294, 198, 20322, 279, 26741, 304, 1344, 3713, 1776, 32997, 11, 1160, 16117, 15691, 92825, 26, 279, 198, 359, 939, 5389, 3026, 304, 279, 28036, 11, 3411, 4741, 311, 2500, 1938, 315, 90926, 11, 422, 198, 1962, 315, 6784, 26, 42216, 86631, 449, 813, 11102, 6916, 26, 735, 2637, 28682, 323, 3831, 11, 719, 198, 7712, 287, 912, 33397, 2035, 304, 279, 1917, 13, 22703, 287, 927, 1124, 682, 574, 279, 198, 9514, 65167, 17882, 13, 1283, 8967, 369, 1077, 11, 430, 10280, 6693, 11, 682, 430, 279, 198, 1605, 3026, 1051, 539, 313, 998, 872, 23948, 8333, 11, 25775, 11, 59772, 11, 2410, 13, 32980, 35120, 31202, 11203, 1203, 389, 279, 67026, 323, 15746, 1077, 3663, 13, 330, 4599, 358, 574, 264, 10585, 1359, 568, 1071, 11, 330, 438, 10837, 3235, 279, 6825, 11, 8260, 2999, 13, 7639, 198, 64, 36157, 11, 358, 2646, 3463, 358, 4265, 10457, 1618, 10307, 430, 6134, 311, 1518, 1461, 2586, 198, 258, 13, 82251, 956, 433, 279, 15123, 315, 264, 1917, 11, 90064, 30, 1102, 198, 467, 956, 1790, 315, 264, 10280, 311, 499, 11, 3060, 1210, 76996, 23114, 1461, 813, 6693, 25309, 15600, 11, 323, 11, 1606, 1077, 6548, 10409, 709, 198, 4291, 24014, 1457, 323, 1243, 520, 813, 1520, 33028, 11, 1364, 574, 539, 779, 10151, 1285, 439, 198, 32158, 2643, 617, 1027, 13, 3277, 832, 46605, 1285, 1047, 8208, 1523, 813, 13272, 11, 568, 31645, 198, 455, 520, 1077, 70205, 2740, 13, 1666, 1790, 439, 574, 3284, 11, 279, 8952, 66529, 389, 430, 10280, 6187, 13, 578, 198, 56544, 288, 4024, 922, 304, 7878, 4251, 78878, 449, 8314, 954, 315, 8770, 273, 998, 68, 304, 198, 50828, 3215, 75790, 11, 3815, 2478, 8679, 826, 13, 6193, 279, 8582, 27035, 11, 1405, 279, 198, 74, 49721, 1051, 7559, 11, 9041, 9017, 38245, 279, 1672, 80227, 54097, 315, 938, 15067, 198, 66467, 10786, 13, 7357, 26741, 1047, 1202, 93484, 315, 305, 8788, 13, 763, 279, 13658, 11, 3600, 198, 52898, 5762, 304, 279, 83249, 63891, 13, 33442, 11843, 4825, 1903, 872, 6435, 5208, 3235, 75752, 323, 1523, 198, 68, 3532, 3046, 13, 30088, 3916, 39428, 889, 1051, 3025, 311, 4321, 1344, 5795, 3235, 304, 28954, 198, 3306, 32997, 13, 42251, 279, 7029, 14365, 315, 279, 46979, 198, 1820, 13587, 11843, 4825, 1051, 28902, 304, 264, 5347, 292, 7219, 13, 43474, 1124, 11, 26435, 369, 198, 1820, 13402, 11, 1051, 279, 39736, 89069, 11, 279, 2015, 49201, 11, 323, 6295, 3842, 11, 889, 198, 11513, 588, 279, 51632, 13, 1952, 832, 3185, 315, 279, 60779, 11, 3221, 279, 4156, 11, 7731, 279, 37465, 304, 7123, 11, 304, 198, 5192, 13391, 19079, 323, 7878, 45233, 13, 1952, 279, 1023, 3185, 1047, 1027, 4694, 264, 198, 2050, 369, 279, 5687, 13, 578, 2655, 288, 14980, 1203, 2403, 279, 7147, 11, 5644, 311, 198, 6236, 704, 1990, 48041, 292, 826, 11, 439, 433, 1051, 313, 2000, 264, 36213, 477, 459, 51632, 198, 6797, 11, 439, 279, 1162, 2643, 387, 13, 6193, 4395, 2967, 6879, 279, 1306, 1773, 4481, 9096, 315, 10280, 13658, 13, 578, 37465, 29340, 11, 323, 76996, 29340, 449, 1124, 11, 1077, 7878, 3995, 7899, 16448, 198, 49532, 279, 2800, 13, 26541, 12688, 40120, 3782, 1555, 279, 61152, 2427, 448, 198, 28176, 323, 559, 606, 389, 1077, 17104, 74820, 3663, 11, 1077, 11113, 34801, 62626, 11, 1077, 198, 11600, 11, 2744, 1120, 264, 2697, 1384, 894, 13, 7639, 11, 84244, 287, 2403, 279, 7147, 11, 4028, 279, 83249, 11, 1766, 813, 6548, 198, 496, 17718, 9017, 1077, 15320, 13, 3639, 198, 64, 81631, 369, 5496, 323, 369, 23871, 1364, 1047, 0, 578, 97054, 63584, 1543, 1373, 279, 11266, 29400, 1473, 1, 1016, 283, 34143, 10456, 71329, 11, 323, 38674, 304, 5118, 488, 26, 9093, 4359, 11, 1524, 198, 27520, 4359, 11, 52677, 459, 85079, 40344, 449, 279, 5707, 315, 16089, 2136, 3485, 26236, 87819, 1210, 3005, 574, 1695, 11, 323, 1364, 1047, 1027, 459, 85079, 449, 279, 5707, 315, 198, 6200, 329, 2136, 13, 1628, 568, 27567, 16366, 10868, 574, 26139, 13, 5414, 5655, 22253, 7899, 11, 539, 2744, 837, 11, 712, 25111, 198, 412, 3485, 279, 5222, 315, 279, 2678, 2942, 13, 3279, 1047, 1027, 264, 1695, 10868, 311, 198, 40617, 26, 568, 1047, 1027, 264, 1695, 4538, 13, 7639, 596, 17764, 35534, 4059, 82294, 3201, 505, 279, 2532, 311, 279, 6945, 315, 813, 198, 27510, 927, 813, 10868, 596, 39682, 291, 18496, 11, 311, 279, 6825, 11, 311, 735, 2637, 311, 279, 198, 29652, 889, 1047, 16436, 311, 32918, 1461, 1606, 1364, 1550, 539, 7095, 1461, 11, 311, 198, 77264, 22983, 1566, 315, 682, 13, 1283, 6656, 264, 2697, 323, 10837, 813, 6548, 3235, 279, 1584, 198, 1073, 37465, 13, 1666, 422, 1364, 1051, 17371, 315, 813, 36752, 11, 1364, 30831, 198, 1964, 2010, 323, 65657, 9017, 1461, 13, 578, 37465, 29340, 25, 27567, 257, 330, 46, 27823, 9576, 315, 96058, 0, 32285, 408, 311, 603, 11, 584, 24739, 280, 415, 11514, 704, 1057, 7589, 11, 323, 3810, 304, 345, 415, 2893, 9405, 304, 603, 311, 11477, 1210, 578, 13587, 11843, 4825, 323, 5804, 3916, 39428, 934, 7403, 291, 279, 11537, 4339, 13, 3279, 596, 198, 89945, 28691, 36014, 449, 55349, 2136, 13, 578, 11452, 11, 11961, 264, 2697, 10980, 449, 1077, 6206, 48135, 304, 1077, 24301, 323, 198, 906, 661, 449, 279, 16066, 315, 279, 1917, 11, 8036, 1077, 6548, 323, 34793, 13, 578, 10280, 6693, 1047, 7263, 76996, 4376, 264, 21030, 21258, 13, 735, 13, 3288, 1077, 198, 64, 15310, 93297, 1162, 449, 1077, 1647, 13255, 11, 46993, 264, 27306, 18327, 13, 2030, 279, 8352, 315, 21258, 11, 927, 902, 76996, 596, 6548, 1047, 2840, 13111, 11, 574, 264, 198, 47991, 3830, 315, 61741, 13160, 304, 2999, 13, 7639, 596, 24166, 12, 1787, 4477, 11, 330, 3915, 264, 198, 37569, 1210, 350, 40458, 304, 279, 8579, 61607, 315, 1077, 34801, 62626, 574, 832, 315, 279, 61741, 430, 198, 10924, 12684, 13, 7639, 574, 8748, 369, 76996, 304, 279, 198, 6133, 83536, 13, 1198, 32158, 65657, 1523, 311, 279, 16392, 198, 32158, 28670, 13, 330, 791, 3885, 1304, 279, 1455, 70960, 2766, 315, 1933, 304, 279, 26741, 1210, 330, 7009, 527, 539, 904, 279, 2753, 10705, 1606, 358, 1097, 20806, 1023, 1274, 617, 264, 198, 331, 685, 311, 4774, 1124, 1210, 9636, 682, 813, 8485, 2676, 568, 574, 2917, 13610, 3722, 1748, 449, 1077, 13, 2052, 279, 5128, 198, 89325, 288, 568, 1053, 617, 1903, 311, 22770, 22983, 1234, 279, 13463, 8636, 198, 15145, 1077, 26875, 34522, 13, 2684, 1051, 1690, 2574, 568, 4934, 311, 2019, 311, 1077, 13, 1283, 4934, 311, 3371, 1077, 198, 9210, 568, 574, 14931, 1077, 6691, 1047, 8636, 26, 430, 279, 6825, 574, 4384, 2085, 198, 1964, 26, 430, 568, 7111, 4741, 311, 1521, 7446, 16659, 449, 1077, 439, 264, 27823, 198, 1543, 311, 813, 6596, 1603, 813, 36959, 13, 3639, 568, 2216, 1071, 574, 311, 68270, 198, 91627, 989, 3508, 1364, 1047, 1047, 1077, 10280, 14177, 13, 76996, 91772, 1461, 11, 4376, 86324, 11, 4376, 13194, 13, 2209, 433, 3958, 369, 26434, 369, 603, 311, 387, 1695, 198, 30562, 7673, 25681, 304, 1077, 6548, 436, 37588, 198, 1820, 38736, 315, 95046, 430, 2744, 1776, 900, 291, 304, 1461, 13, 330, 5159, 1841, 596, 1027, 68941, 304, 264, 12056, 3696, 2130, 19441, 2533, 4216, 420, 6693, 345, 438, 358, 617, 3279, 596, 86652, 304, 264, 20328, 1108, 13, 10435, 389, 701, 2574, 323, 2586, 369, 264, 198, 1425, 1210, 1283, 26253, 22770, 22983, 1436, 6865, 1148, 568, 1071, 26, 311, 387, 3738, 315, 433, 11, 568, 198, 14991, 9824, 398, 9408, 813, 7899, 264, 490, 19636, 13, 15469, 4024, 1203, 311, 279, 15197, 13, 3005, 574, 311, 387, 1949, 430, 13658, 3156, 4848, 297, 63510, 280, 8248, 1364, 1047, 19487, 311, 733, 2162, 13, 18165, 311, 832, 11, 568, 596, 449, 1077, 1457, 1210, 578, 198, 89945, 54097, 315, 279, 8952, 11, 73259, 839, 449, 279, 41466, 315, 34697, 323, 3596, 13553, 198, 258, 279, 83249, 26, 1903, 1077, 85184, 13, 1628, 345, 9620, 3422, 11, 422, 735, 13, 1051, 449, 46993, 27567, 12348, 596, 37913, 11, 4536, 956, 433, 7673, 330, 3112, 3686, 11, 499, 3136, 311, 26032, 757, 323, 1755, 757, 311, 7692, 9135, 330, 4054, 315, 279, 1455, 50189, 2574, 922, 62461, 374, 78504, 1457, 323, 198, 3473, 1210, 5810, 574, 1077, 2362, 4333, 323, 198, 37569, 10371, 311, 1935, 1077, 704, 369, 264, 53121, 12141, 13, 578, 30462, 53848, 198, 1073, 12822, 2403, 11447, 85007, 709, 304, 76996, 13, 22770, 22983, 1047, 8208, 555, 430, 892, 313, 46696, 449, 12491, 304, 1077, 4851, 323, 3776, 198, 5919, 12959, 13, 3005, 7020, 1633, 1664, 1148, 279, 4360, 1053, 387, 13, 76996, 1053, 6678, 198, 4291, 1461, 11, 323, 568, 1053, 3371, 1077, 1268, 17104, 1364, 7111, 449, 279, 3805, 389, 198, 1964, 3663, 323, 279, 12056, 922, 1077, 13, 578, 21706, 8050, 11633, 315, 279, 2697, 20328, 1108, 198, 41450, 2571, 1124, 3345, 3871, 13, 1283, 1053, 198, 22794, 76996, 596, 1450, 59772, 398, 323, 15648, 304, 1077, 6548, 13, 3011, 574, 813, 1749, 512, 998, 1514, 520, 3021, 28846, 1093, 459, 6264, 19995, 8334, 11, 3156, 5115, 15187, 279, 198, 89851, 12504, 323, 279, 8137, 574, 1070, 13, 578, 10280, 28361, 1047, 539, 8636, 704, 304, 279, 26741, 994, 22770, 22983, 4024, 198, 1445, 311, 433, 13, 1952, 1855, 96803, 2007, 574, 459, 19087, 11, 323, 30488, 433, 264, 6857, 198, 1073, 39640, 268, 36516, 323, 264, 48135, 4251, 1450, 7197, 62626, 13, 2684, 1051, 8314, 954, 315, 198, 8619, 398, 38067, 922, 11, 2288, 11, 323, 279, 1306, 1773, 4481, 2262, 315, 58785, 45405, 198, 438, 10054, 12, 47100, 13, 578, 61983, 3828, 889, 6476, 279, 63137, 4671, 7018, 1523, 279, 46979, 1139, 279, 198, 1637, 13, 3005, 574, 44865, 449, 21847, 11, 430, 837, 267, 35491, 11, 323, 449, 279, 198, 86258, 76978, 315, 279, 89358, 26741, 311, 9096, 13, 3005, 574, 5128, 304, 264, 3995, 11, 67163, 1648, 11, 323, 1606, 311, 1077, 10280, 198, 16514, 264, 19309, 323, 8967, 3987, 323, 279, 11471, 315, 279, 3995, 10425, 11, 1364, 198, 51908, 71414, 2574, 13, 578, 26741, 7731, 709, 11, 27569, 430, 433, 574, 539, 279, 81276, 11, 31645, 4028, 198, 1527, 4950, 311, 4950, 13, 578, 47027, 261, 11, 6832, 836, 574, 27738, 9336, 11, 574, 264, 16615, 11, 16025, 3828, 449, 264, 198, 4930, 11, 14618, 19689, 13, 3005, 8774, 709, 264, 4401, 5844, 857, 3904, 315, 2678, 3137, 311, 198, 1820, 4731, 13, 330, 5966, 10280, 1359, 1364, 1071, 24261, 3210, 11, 330, 906, 4024, 704, 1139, 279, 3224, 198, 258, 264, 18137, 2695, 6241, 323, 1047, 264, 1972, 892, 13, 358, 1541, 956, 1440, 1148, 358, 1097, 1618, 369, 345, 3852, 5269, 13, 330, 73819, 323, 63237, 11, 1332, 346, 4447, 323, 50625, 4447, 11, 3116, 13124, 315, 19692, 26, 430, 596, 198, 1820, 3460, 315, 9041, 584, 617, 709, 304, 1057, 961, 315, 279, 1917, 13, 3277, 358, 1781, 315, 198, 12840, 358, 7731, 1523, 311, 311, 11477, 313, 9135, 3005, 1047, 264, 28254, 5201, 369, 22770, 22983, 11, 323, 1077, 63426, 304, 279, 8952, 198, 67, 14657, 291, 505, 76996, 596, 304, 430, 433, 574, 311, 29960, 349, 1077, 2307, 22625, 11, 1418, 198, 68753, 3520, 596, 1047, 1027, 311, 2512, 369, 1077, 6978, 13, 56124, 22770, 22983, 34386, 11, 1364, 99507, 264, 2697, 58587, 13, 3005, 1047, 887, 398, 198, 6200, 3340, 279, 2440, 315, 264, 16088, 16893, 389, 279, 1203, 315, 1077, 1450, 11, 323, 574, 198, 26121, 33024, 264, 35113, 323, 5425, 1481, 3233, 389, 433, 13, 330, 40, 5895, 422, 499, 617, 14000, 2555, 1359, 1364, 1071, 11, 6548, 389, 279, 2440, 13, 330, 40, 617, 14000, 430, 279, 2380, 16405, 63510, 39653, 527, 539, 2728, 1359, 1071, 198, 77264, 22983, 46473, 26, 323, 9083, 27738, 9336, 11, 2103, 30929, 323, 84273, 11, 1903, 279, 198, 1067, 82, 315, 279, 26741, 13, 330, 40, 2846, 912, 58587, 1359, 1364, 1071, 11, 10917, 279, 35788, 389, 279, 2007, 13, 330, 2746, 499, 2834, 956, 198, 4151, 11, 499, 2834, 956, 13, 1666, 433, 574, 539, 2631, 430, 24014, 387, 12715, 389, 279, 3335, 11, 22770, 22983, 198, 35778, 912, 6666, 311, 420, 13, 9083, 27738, 9336, 2064, 15556, 449, 12939, 198, 438, 1095, 1077, 16757, 2610, 1077, 11157, 13, 362, 1450, 9508, 311, 2339, 22770, 22983, 596, 4851, 323, 3412, 433, 13, 21347, 459, 2362, 4333, 3250, 956, 1304, 499, 1427, 520, 264, 3828, 439, 422, 499, 198, 87692, 311, 1935, 264, 23556, 704, 315, 1077, 13, 4488, 856, 3492, 11, 9083, 36627, 11, 1364, 3358, 198, 37593, 6381, 1077, 4967, 26, 1364, 3358, 32918, 1461, 13, 358, 6562, 1359, 20536, 279, 198, 20493, 367, 261, 24261, 3210, 11, 330, 9210, 1063, 1695, 31348, 12637, 1093, 430, 1053, 198, 23609, 264, 27555, 311, 757, 13, 358, 1097, 439, 28360, 439, 264, 27275, 25675, 11, 719, 198, 40, 3077, 2751, 1742, 1210, 3005, 574, 1317, 323, 7589, 9373, 11, 719, 1364, 28670, 1077, 198, 75, 92916, 11, 5986, 2269, 15154, 15447, 449, 264, 3738, 30296, 13, 5340, 65670, 24573, 198, 41450, 617, 26435, 1077, 304, 86115, 6307, 311, 2489, 1077, 6548, 11, 323, 449, 1317, 198, 73, 1037, 67301, 11, 323, 1903, 1077, 264, 11401, 13, 578, 63137, 380, 1047, 3970, 279, 24014, 389, 32980, 198, 49, 8477, 31202, 596, 4251, 54888, 11, 323, 1047, 33341, 1139, 18147, 1228, 287, 11, 16267, 788, 4731, 13, 330, 40, 2846, 17510, 19101, 323, 1364, 596, 63134, 1359, 2854, 2106, 279, 198, 1637, 1234, 1202, 11745, 13, 330, 52182, 11, 1268, 358, 4265, 1093, 311, 15612, 0, 1442, 358, 3596, 636, 704, 315, 420, 1181, 8301, 37002, 9135, 578, 16088, 10398, 352, 11203, 520, 22770, 22983, 596, 46811, 26, 30488, 433, 279, 3830, 315, 9382, 13, 22770, 22983, 7020, 433, 1523, 311, 279, 43957, 315, 198, 1964, 57034, 8271, 13, 1666, 40605, 439, 279, 3814, 8272, 279, 1938, 11, 1364, 574, 198, 18310, 1077, 1847, 13, 3005, 1047, 5675, 2736, 11, 7389, 27567, 2746, 1364, 1436, 636, 76996, 704, 315, 279, 8952, 11, 433, 1053, 40821, 2574, 13, 3005, 1765, 76, 4147, 559, 4361, 67, 398, 430, 389, 279, 6825, 872, 12034, 1051, 7029, 198, 391, 472, 13, 1102, 574, 1618, 430, 814, 2322, 389, 4279, 5015, 13, 578, 61983, 63137, 43467, 4671, 7018, 704, 315, 279, 26741, 26, 279, 35612, 315, 279, 198, 22928, 12688, 93722, 23183, 1523, 13, 2468, 4330, 297, 63510, 22770, 22983, 3288, 9083, 198, 54, 569, 9336, 311, 1176, 87117, 11, 311, 279, 13051, 315, 430, 56452, 14792, 198, 9164, 13, 578, 26741, 11203, 2103, 477, 75371, 671, 332, 30666, 13, 10280, 574, 927, 345, 438, 1070, 1051, 912, 11714, 16064, 311, 1427, 4741, 311, 13, 22770, 22983, 6688, 279, 4330, 16405, 63510, 39653, 13, 5112, 1364, 7731, 1523, 520, 279, 2007, 198, 52759, 279, 6134, 11, 449, 279, 35788, 304, 4156, 315, 1077, 13, 2684, 527, 3738, 11555, 198, 9210, 527, 520, 1176, 5865, 315, 279, 8271, 26, 1306, 264, 1317, 892, 279, 50112, 198, 32089, 5097, 1124, 709, 323, 33822, 1124, 1139, 14385, 4661, 9651, 13, 19292, 1606, 369, 279, 1566, 2305, 1364, 1047, 2884, 279, 3245, 779, 3629, 304, 198, 1964, 4059, 11, 1202, 5150, 5178, 574, 4661, 2085, 17371, 3463, 13, 22770, 22983, 3952, 264, 16893, 505, 1077, 16088, 87041, 11, 323, 11, 4477, 264, 502, 198, 1530, 369, 433, 11, 3347, 291, 433, 927, 279, 2362, 832, 13, 5112, 1364, 48501, 433, 369, 832, 198, 1073, 279, 1890, 1404, 389, 279, 16088, 35788, 13, 763, 279, 18397, 48840, 11, 520, 279, 47027, 388, 6, 2007, 11, 9083, 27738, 9336, 574, 198, 83, 30855, 13, 330, 22404, 19704, 757, 1359, 1364, 1071, 11, 330, 2727, 369, 279, 3224, 323, 279, 4382, 2324, 1306, 198, 576, 13, 2435, 1781, 358, 2846, 1193, 264, 47027, 261, 323, 1541, 956, 1518, 4205, 11, 719, 358, 3077, 198, 23196, 6548, 304, 856, 2010, 13, 17882, 11, 323, 1364, 198, 339, 15872, 358, 1541, 956, 1518, 433, 13, 2030, 2646, 4059, 26, 358, 7318, 11, 1077, 709, 311, 11477, 369, 264, 2478, 198, 1073, 279, 503, 337, 2641, 1364, 706, 2728, 757, 1210, 46982, 279, 18397, 48840, 13326, 323, 40883, 3995, 3278, 3782, 323, 30912, 345, 71, 561, 1570, 477, 41308, 398, 439, 872, 6776, 574, 11, 323, 4024, 389, 872, 1648, 198, 33871, 13, 763, 872, 6206, 814, 5762, 279, 7039, 11, 539, 2744, 315, 2324, 323, 4648, 198, 66372, 11, 719, 315, 14553, 505, 6784, 11, 315, 8541, 29668, 11, 315, 11113, 67026, 11, 323, 198, 66, 8772, 315, 3090, 311, 98208, 23726, 13, 763, 872, 6548, 11, 439, 304, 76996, 596, 11, 27724, 279, 198, 4238, 315, 2532, 13, 2030, 1618, 323, 1070, 832, 1766, 3278, 11, 1093, 22770, 22983, 323, 9083, 27738, 9336, 345, 14965, 1047, 37104, 872, 348, 2328, 11, 889, 436, 5805, 2403, 279, 78316, 3633, 315, 279, 198, 14789, 11, 1202, 9669, 11, 1202, 26762, 58023, 13, 76020, 477, 779, 2403, 1403, 313, 69, 39724, 889, 7111, 704, 389, 279, 1917, 449, 279, 198, 69, 686, 1752, 34522, 315, 1884, 889, 617, 3970, 2324, 311, 1202, 43957, 11, 323, 11, 449, 279, 198, 65, 8714, 8830, 315, 5150, 3729, 11, 2103, 1766, 433, 1695, 13, 76020, 889, 198, 52898, 6975, 477, 1047, 9687, 539, 311, 4128, 16038, 872, 4335, 70181, 291, 198, 68075, 505, 279, 1377, 370, 315, 279, 14708, 13, 1628, 279, 33517, 11, 889, 1766, 279, 1633, 198, 2445, 372, 315, 279, 18340, 5153, 539, 2288, 78442, 369, 8541, 29668, 323, 2512, 11, 1095, 22770, 22983, 198, 438, 11, 304, 32415, 6767, 11, 279, 502, 47027, 261, 7636, 13, 2435, 1436, 539, 617, 198, 3415, 7725, 872, 8125, 13, 578, 87117, 48840, 574, 10409, 449, 872, 8579, 24196, 11, 279, 23941, 273, 315, 872, 198, 68075, 11, 279, 30103, 309, 315, 872, 31161, 4251, 19079, 13, 3277, 22770, 22983, 3782, 304, 11, 1364, 44865, 7000, 315, 1124, 13, 2435, 1550, 539, 1093, 1077, 345, 438, 1364, 7020, 433, 13, 13538, 1077, 11, 4619, 315, 279, 58905, 87117, 17203, 11, 1364, 574, 9298, 279, 198, 83543, 10398, 352, 439, 1364, 1047, 2163, 433, 13, 330, 40, 8101, 358, 3077, 8521, 1077, 1359, 1364, 1071, 311, 11937, 13, 6385, 1633, 13836, 574, 14363, 449, 8850, 315, 1148, 1364, 1047, 2884, 13, 96424, 53694, 23440, 1432, 42, 13, 5602, 76996, 369, 1193, 264, 4545, 389, 10280, 6187, 13, 1115, 574, 994, 279, 8485, 198, 56492, 20328, 1108, 1047, 10717, 304, 4156, 315, 279, 3838, 13, 76996, 1047, 73973, 12164, 18007, 304, 369, 264, 4545, 13, 46993, 596, 1370, 9761, 574, 198, 75593, 449, 4027, 4238, 323, 50380, 449, 70457, 323, 449, 279, 1206, 1683, 315, 1077, 198, 82694, 1824, 8772, 13, 735, 2637, 84244, 287, 1280, 337, 4501, 304, 4156, 315, 279, 4027, 11, 1047, 6656, 311, 1518, 76996, 198, 258, 279, 70458, 11, 323, 514, 10395, 311, 813, 7693, 13, 330, 40, 649, 956, 2586, 304, 1359, 1364, 39169, 13, 358, 1097, 704, 198, 82, 64069, 12, 50780, 449, 2999, 13, 330, 27264, 1461, 304, 369, 264, 10747, 315, 15600, 1359, 46993, 2663, 704, 13, 330, 8586, 596, 71322, 198, 27588, 65670, 323, 6691, 323, 1524, 42216, 9135, 46993, 1047, 20330, 2391, 279, 1566, 5672, 11, 719, 1364, 574, 10917, 709, 264, 34300, 198, 7096, 13, 76996, 10837, 311, 279, 4156, 6134, 323, 2663, 25, 330, 10149, 499, 2586, 304, 369, 264, 10747, 315, 198, 82694, 7673, 1666, 76996, 6656, 1203, 1139, 279, 3838, 11, 1364, 2322, 42216, 13, 1283, 1047, 2586, 704, 198, 258, 279, 14321, 11, 323, 1047, 8036, 279, 6134, 1139, 279, 1370, 9761, 4920, 1461, 13, 5414, 6916, 198, 16514, 2103, 304, 12786, 21719, 11, 323, 70955, 22340, 304, 264, 8485, 41044, 88733, 13, 578, 5222, 315, 43214, 3782, 1555, 279, 6134, 38678, 398, 13, 578, 8334, 596, 3663, 574, 198, 33222, 449, 1461, 13, 330, 56351, 304, 1063, 5627, 11, 719, 315, 3388, 313, 1875, 39469, 527, 814, 2133, 311, 14816, 7673, 330, 1548, 3250, 956, 2873, 311, 19065, 499, 26, 568, 2795, 433, 596, 682, 304, 279, 1847, 1210, 330, 68753, 3520, 11, 1587, 46993, 1440, 430, 358, 574, 539, 7636, 430, 3814, 7673, 330, 2746, 1364, 61637, 11, 433, 374, 539, 1606, 315, 4205, 279, 8334, 706, 1071, 13, 4470, 315, 279, 4027, 4238, 11, 3201, 505, 279, 70457, 323, 279, 43214, 11, 42216, 596, 198, 1594, 8710, 24634, 323, 305, 16094, 569, 13, 1283, 2231, 813, 1949, 1450, 389, 76996, 596, 17308, 13, 330, 40, 574, 7422, 430, 8530, 422, 358, 4024, 3201, 313, 1875, 34728, 1053, 387, 66275, 398, 11, 8434, 956, 433, 7673, 330, 2746, 46993, 1053, 1193, 2019, 2555, 323, 636, 433, 927, 449, 0, 3005, 3250, 956, 198, 82, 22537, 26, 358, 1781, 1364, 596, 2216, 4560, 311, 387, 3169, 13, 3005, 10800, 28639, 1475, 892, 358, 5916, 1077, 1450, 1210, 2052, 279, 3177, 1047, 8636, 704, 315, 76996, 596, 3663, 13, 9601, 574, 17936, 11, 1306, 198, 543, 313, 2017, 94179, 13, 3861, 1550, 5076, 2574, 11, 323, 1023, 1274, 16654, 26, 477, 198, 606, 574, 1695, 11, 439, 1077, 6691, 1047, 1027, 11, 323, 574, 2163, 40666, 11, 264, 57896, 11, 477, 198, 4908, 71322, 5340, 65670, 13, 20695, 1051, 779, 2204, 505, 198, 12840, 814, 9508, 311, 387, 25, 46993, 7953, 279, 6134, 11, 51513, 15600, 323, 198, 4355, 7595, 287, 449, 1077, 4851, 304, 67101, 26, 42216, 30488, 1077, 11, 14867, 16117, 26435, 198, 438, 289, 62837, 13, 578, 1193, 832, 1364, 3463, 2216, 2262, 291, 574, 735, 13, 1283, 9508, 198, 998, 3351, 779, 73170, 304, 813, 2697, 27605, 13, 1283, 574, 2744, 779, 24981, 11, 779, 198, 59502, 13, 1442, 2324, 5762, 912, 36394, 369, 1461, 11, 520, 3325, 433, 5762, 912, 43957, 13, 330, 3947, 596, 1193, 832, 3245, 11, 42216, 1359, 1364, 1071, 29059, 989, 13, 330, 88960, 35120, 31202, 198, 285, 2133, 311, 617, 813, 6140, 13, 1442, 21739, 304, 279, 1917, 649, 3665, 1461, 11, 7639, 198, 92493, 649, 1210, 578, 3177, 315, 430, 8982, 574, 304, 1077, 6548, 994, 1364, 4024, 704, 311, 279, 20328, 1108, 198, 33871, 13, 735, 13, 8272, 1077, 704, 323, 61218, 279, 71268, 304, 15884, 922, 1077, 13, 2209, 1070, 904, 6140, 315, 3515, 499, 2162, 369, 87117, 7673, 358, 1097, 311, 733, 389, 14523, 520, 4848, 1578, 35277, 82, 6548, 11, 1364, 1550, 539, 1518, 433, 13, 1283, 65682, 1124, 198, 1885, 37163, 398, 505, 279, 65442, 11, 323, 4024, 4856, 17345, 1203, 1139, 279, 198, 7830, 13, 330, 10156, 1268, 1690, 3026, 527, 304, 3021, 449, 499, 11, 76996, 7673, 4691, 7639, 11, 439, 86652, 198, 47823, 709, 279, 6825, 13, 330, 2822, 832, 430, 358, 1440, 315, 11, 7389, 313, 1875, 1, 66457, 13, 11115, 313, 1875, 31437, 358, 8967, 1359, 1364, 1071, 449, 38929, 11, 330, 285, 430, 7389, 832, 14921, 1633, 198, 82279, 3026, 11, 323, 430, 4536, 956, 2216, 3021, 1210, 330, 1687, 3358, 5387, 704, 13142, 46506, 12669, 323, 7182, 313, 2000, 11, 315, 3388, 11, 358, 1097, 1633, 198, 82279, 13, 10699, 374, 304, 3021, 449, 499, 28858, 2009, 77638, 818, 30, 5884, 315, 279, 2655, 288, 520, 198, 1820, 8952, 7673, 2009, 77638, 818, 374, 539, 304, 3021, 449, 757, 1210, 2684, 574, 1778, 100226, 304, 1077, 7899, 430, 17882, 574, 51512, 13, 735, 2637, 9191, 1109, 5678, 323, 810, 25165, 11, 1047, 2744, 1047, 459, 10535, 33464, 198, 2000, 3278, 13, 1283, 1047, 1027, 42762, 34386, 555, 1124, 11, 719, 279, 2144, 1047, 14958, 13, 1628, 7639, 810, 1109, 24740, 430, 1457, 11, 520, 1566, 11, 568, 1047, 1027, 10791, 13, 330, 8161, 956, 499, 2216, 3152, 430, 499, 527, 304, 3021, 449, 2009, 77638, 818, 7673, 358, 1097, 539, 304, 3021, 449, 21739, 26, 358, 9167, 956, 892, 198, 998, 387, 304, 3021, 13, 2100, 8369, 1550, 279, 5811, 3719, 430, 198, 20670, 5946, 2085, 9298, 264, 6278, 57859, 46229, 11, 2875, 323, 4856, 198, 89945, 743, 11, 20558, 1555, 264, 12056, 3696, 2130, 389, 4579, 11, 323, 15691, 304, 813, 198, 10888, 264, 19371, 44221, 660, 18012, 9145, 13, 2030, 279, 62705, 43578, 555, 323, 2163, 1461, 22095, 6953, 752, 345, 20986, 56949, 3725, 1306, 1124, 13, 3279, 596, 4059, 11, 1193, 264, 40146, 323, 198, 258, 81987, 6468, 23023, 13, 4314, 2574, 430, 3782, 779, 6847, 311, 7639, 11, 279, 198, 2715, 12181, 315, 3278, 11, 8485, 2697, 35048, 2805, 13757, 1093, 279, 39098, 198, 1073, 86652, 323, 279, 20328, 1108, 11, 1047, 2646, 1027, 813, 13, 1442, 1070, 574, 904, 38678, 198, 2762, 479, 11, 433, 574, 520, 5678, 13, 1283, 1047, 9408, 279, 8334, 5076, 313, 383, 1047, 198, 2629, 3948, 1461, 311, 387, 45743, 13, 55777, 279, 9145, 1579, 704, 315, 279, 34738, 82, 11, 568, 198, 28010, 813, 6435, 5208, 709, 279, 6825, 13, 2468, 2555, 1306, 1403, 297, 63510, 430, 3814, 11, 735, 13, 2231, 1523, 813, 13961, 198, 438, 34793, 13, 1283, 1047, 539, 1027, 3025, 311, 6212, 2533, 33433, 13, 763, 813, 198, 67, 57881, 2427, 785, 568, 1047, 7731, 555, 279, 2678, 4027, 11, 7422, 13, 578, 2262, 315, 813, 198, 3983, 2478, 4038, 389, 279, 6825, 574, 19019, 7231, 1648, 311, 59322, 13, 1283, 198, 14965, 1047, 8967, 311, 4018, 5678, 1022, 505, 2324, 1766, 5678, 1578, 304, 3345, 198, 22794, 449, 433, 26, 813, 1608, 10470, 574, 5655, 449, 433, 13, 1789, 279, 1176, 892, 11, 568, 1047, 22088, 311, 3488, 279, 24278, 315, 1148, 568, 1047, 198, 10655, 13, 1102, 1047, 4529, 25775, 11, 4359, 7020, 345, 998, 3041, 709, 4395, 323, 2586, 3201, 13, 763, 264, 1648, 11, 433, 1053, 617, 4529, 810, 198, 66, 61140, 311, 617, 20186, 13, 1283, 1047, 3463, 11, 520, 1176, 11, 430, 568, 1436, 198, 21828, 1523, 420, 3021, 369, 76996, 13, 578, 198, 6258, 511, 306, 5916, 315, 1077, 1450, 389, 813, 6916, 11, 279, 4545, 994, 568, 1047, 5762, 1077, 198, 258, 813, 11977, 1306, 1077, 6691, 596, 4648, 11, 279, 16579, 2678, 19015, 315, 1077, 198, 4310, 311, 279, 2697, 3838, 313, 543, 1521, 743, 813, 6680, 389, 4027, 13, 9636, 813, 11594, 28362, 735, 13, 21095, 1690, 26885, 1884, 12688, 198, 14097, 313, 2017, 813, 18496, 323, 48033, 520, 279, 5274, 11, 304, 813, 3130, 7636, 345, 4291, 5340, 65670, 9293, 7878, 38586, 82, 7953, 279, 17071, 11, 1524, 555, 198, 19875, 483, 596, 4027, 11, 449, 46993, 1120, 4028, 11, 11961, 304, 21847, 323, 198, 14658, 287, 813, 25165, 5643, 323, 24981, 6548, 13, 1283, 1047, 264, 2697, 6945, 315, 76996, 313, 64, 10885, 64630, 430, 568, 1047, 4529, 198, 40617, 726, 13, 1102, 8710, 76996, 28382, 264, 1450, 11, 902, 1047, 1027, 704, 315, 2134, 994, 198, 1820, 6382, 1047, 1027, 48110, 11, 323, 11509, 389, 264, 32366, 4051, 1968, 198, 8370, 1053, 617, 1027, 5115, 264, 2237, 1047, 568, 5762, 279, 6382, 7833, 13, 35053, 433, 574, 76996, 11, 1077, 7013, 41442, 922, 1077, 11, 6548, 3411, 198, 412, 11, 28682, 23726, 37163, 13, 3277, 1364, 574, 539, 520, 2162, 11, 433, 7731, 389, 735, 3238, 82, 198, 69334, 261, 11, 463, 7069, 2403, 813, 37204, 12913, 13, 3277, 1364, 574, 304, 279, 3838, 11, 433, 198, 6789, 1234, 279, 9160, 1824, 1136, 290, 13, 9220, 297, 63510, 304, 279, 6693, 11, 1243, 11, 323, 735, 13, 304, 813, 32623, 2427, 785, 11, 449, 279, 198, 27581, 463, 7069, 11, 539, 2403, 279, 37204, 12913, 11, 719, 2403, 813, 29062, 11, 1405, 198, 383, 1436, 1518, 433, 13, 1283, 7731, 4741, 304, 813, 10716, 11, 813, 6206, 48135, 2212, 813, 22095, 11, 323, 198, 7349, 291, 520, 433, 13, 1283, 574, 4560, 311, 6945, 279, 76996, 315, 279, 10512, 198, 258, 813, 2362, 2324, 313, 85134, 311, 1505, 264, 2035, 369, 1077, 13, 2684, 1047, 1027, 2478, 3278, 304, 813, 2362, 2324, 13, 2684, 1047, 1027, 3278, 889, 1047, 42777, 369, 1461, 11, 719, 568, 2231, 1124, 198, 318, 23557, 398, 704, 315, 813, 4059, 13, 35403, 198, 15145, 568, 1047, 568, 4234, 813, 1317, 14535, 704, 315, 279, 10716, 11, 1364, 574, 53728, 520, 198, 26301, 6134, 4994, 13, 35120, 31202, 574, 11509, 304, 279, 4827, 14321, 345, 64, 559, 88548, 922, 1077, 28004, 13, 330, 40, 3077, 1047, 3492, 311, 733, 311, 279, 8952, 1359, 1364, 1071, 13, 330, 40, 3463, 7344, 499, 4265, 198, 3427, 449, 757, 13, 1102, 5084, 439, 422, 358, 649, 956, 2559, 433, 7636, 13, 330, 11787, 499, 16984, 311, 4822, 304, 279, 3838, 7636, 7673, 1283, 10837, 709, 279, 62862, 311, 813, 3130, 323, 1344, 2234, 389, 1063, 17895, 13, 35120, 31202, 596, 274, 5598, 1047, 3719, 3428, 4647, 598, 26, 46993, 14980, 198, 8823, 16117, 927, 1077, 13, 330, 40, 1097, 50136, 14931, 1359, 1364, 1071, 313, 1, 466, 37852, 14931, 0, 3277, 358, 1781, 6832, 198, 60672, 682, 420, 374, 9135, 35120, 31202, 2231, 704, 264, 990, 71845, 6901, 1450, 323, 10791, 46993, 596, 198, 69, 14437, 13, 358, 8101, 499, 323, 358, 198, 8154, 2752, 1855, 1023, 13, 735, 13, 2646, 29695, 279, 6237, 304, 279, 2678, 13147, 26741, 311, 902, 32980, 198, 32345, 1027, 4529, 13, 9636, 279, 4251, 13001, 813, 8334, 819, 7216, 7111, 198, 496, 526, 989, 1317, 13, 2684, 574, 264, 1912, 2212, 279, 4950, 313, 6102, 17882, 11, 1403, 477, 198, 28956, 2655, 288, 11, 279, 3814, 29198, 389, 14523, 11, 323, 279, 11452, 13, 84854, 1120, 4871, 279, 6134, 389, 264, 7833, 10716, 574, 76996, 313, 21470, 264, 198, 68753, 3520, 439, 568, 2646, 1047, 3970, 1603, 11, 1077, 3663, 1933, 1752, 11, 1077, 6548, 7029, 198, 438, 653, 66154, 11, 1077, 6206, 86034, 2454, 304, 1077, 24301, 13, 3277, 568, 14980, 30488, 1077, 345, 32158, 1550, 539, 3351, 477, 1427, 709, 13, 578, 1912, 2212, 279, 4950, 1047, 85156, 311, 198, 329, 1800, 18083, 13, 8442, 76996, 323, 735, 13, 14958, 555, 198, 1820, 6134, 11, 25181, 11, 7636, 13, 330, 2675, 2011, 539, 1935, 433, 1093, 430, 11, 25237, 13, 2030, 11, 1306, 198, 543, 11, 304, 430, 3044, 313, 1875, 2181, 574, 1077, 1176, 6677, 430, 568, 574, 1070, 13, 6385, 7899, 574, 33201, 661, 11, 4225, 1191, 1752, 13, 330, 7009, 2019, 358, 6688, 1461, 279, 5076, 16088, 26, 430, 568, 596, 23069, 26, 430, 358, 198, 66206, 1126, 291, 1461, 1210, 358, 3782, 389, 14523, 520, 4848, 297, 63510, 323, 6688, 279, 198, 2106, 292, 1572, 13, 3277, 279, 3814, 29198, 3782, 389, 520, 8254, 11, 4395, 574, 682, 198, 1315, 13, 578, 16088, 10398, 352, 574, 1120, 439, 433, 1288, 387, 13, 358, 198, 42700, 311, 2019, 1695, 45427, 311, 1461, 323, 568, 313, 383, 574, 33173, 13, 358, 3287, 956, 3041, 1461, 198, 73254, 719, 1148, 574, 389, 279, 35788, 1359, 1364, 8220, 281, 635, 7162, 13, 330, 40, 7111, 520, 198, 1820, 2440, 26, 358, 2744, 1427, 1210, 3296, 264, 32931, 315, 279, 1912, 2212, 279, 4950, 11, 735, 3238, 82, 6548, 7111, 369, 264, 4545, 198, 20384, 398, 1139, 22770, 22983, 596, 13, 4702, 369, 264, 4545, 26, 1243, 279, 13734, 8036, 709, 198, 33871, 13, 1102, 574, 1664, 369, 22770, 22983, 430, 433, 1550, 13, 3005, 7111, 439, 422, 1364, 1047, 198, 29412, 264, 20457, 313, 35187, 1077, 6548, 11, 1524, 312, 41189, 13, 330, 1991, 1063, 832, 311, 198, 23609, 1077, 2035, 1210, 4740, 682, 11, 279, 9546, 315, 420, 893, 304, 420, 3130, 198, 266, 1778, 264, 892, 8967, 4400, 13, 1283, 574, 76996, 596, 4333, 11, 430, 574, 682, 13, 1102, 574, 279, 8334, 596, 58764, 3044, 430, 574, 13353, 1077, 198, 265, 53305, 1139, 31926, 13, 330, 40, 1097, 682, 1314, 1359, 1364, 40001, 4028, 279, 4950, 311, 279, 11452, 13, 1283, 1047, 2884, 4395, 568, 7020, 2085, 198, 1407, 13, 578, 8334, 11, 436, 22485, 369, 459, 9888, 11, 1053, 90417, 1578, 1139, 91132, 269, 13, 3161, 264, 9498, 893, 814, 1436, 617, 6818, 810, 71920, 11193, 313, 29177, 198, 19553, 9770, 1461, 311, 813, 7693, 323, 15203, 1461, 922, 11, 1436, 617, 31394, 1461, 198, 4291, 1168, 15889, 53984, 73307, 304, 10054, 55051, 13, 2030, 279, 36850, 291, 2547, 389, 279, 4950, 198, 29177, 2559, 912, 1778, 56375, 6514, 13, 1102, 574, 2009, 77638, 818, 11, 1306, 682, 11, 889, 6924, 32980, 35120, 31202, 596, 2324, 13, 1789, 11, 994, 198, 28221, 323, 37465, 1047, 39019, 682, 872, 5070, 11, 568, 25319, 4741, 198, 4291, 264, 11594, 3492, 430, 7263, 279, 2655, 288, 311, 872, 7693, 100174, 13, 2684, 574, 264, 502, 6514, 369, 1778, 5157, 313, 275, 1047, 1027, 6818, 22917, 13, 330, 22170, 433, 11, 369, 32450, 596, 25491, 1359, 568, 1071, 13, 578, 41705, 574, 539, 304, 279, 3838, 313, 25849, 387, 1327, 53471, 1534, 11, 13118, 11, 520, 198, 4354, 11, 315, 21448, 323, 10548, 505, 279, 10565, 48840, 13, 735, 13, 1550, 279, 990, 11, 813, 198, 4930, 19779, 711, 83, 323, 10151, 1285, 313, 3556, 18083, 13, 35120, 31202, 1168, 3903, 555, 279, 4950, 198, 4291, 1077, 3663, 28016, 26, 1418, 76996, 7731, 11, 294, 28109, 323, 99774, 12616, 11, 389, 1077, 198, 56492, 10716, 4871, 279, 6134, 26, 1418, 3814, 37465, 9165, 59091, 291, 3235, 279, 198, 6133, 83536, 11, 323, 279, 3814, 3821, 1543, 45135, 9850, 13850, 505, 4994, 279, 198, 11020, 13, 3277, 279, 1403, 2294, 77292, 430, 1051, 279, 13147, 26741, 11276, 198, 32345, 6656, 505, 41585, 42852, 279, 3130, 311, 18004, 77292, 304, 279, 198, 57902, 1251, 3177, 26, 32980, 35120, 31202, 9107, 813, 6548, 323, 12570, 279, 1176, 198, 5880, 430, 13160, 813, 471, 505, 279, 6453, 34681, 13, 3277, 433, 574, 2867, 430, 279, 8334, 1053, 3974, 11, 735, 13, 16392, 31161, 398, 505, 279, 198, 2788, 3002, 323, 4024, 927, 311, 76996, 596, 10716, 13, 330, 1548, 596, 682, 1314, 1457, 1359, 568, 1071, 313, 1, 300, 682, 1314, 439, 568, 649, 387, 11, 8009, 58725, 9135, 2650, 15234, 430, 499, 1288, 1440, 1778, 264, 3245, 13, 578, 2655, 288, 11, 7556, 4315, 5694, 11, 1047, 82294, 1523, 311, 872, 198, 67, 5859, 48840, 369, 4216, 11033, 13, 17882, 574, 7231, 264, 2478, 1566, 11470, 198, 300, 311, 279, 8334, 596, 2512, 3238, 82, 1450, 323, 198, 28708, 433, 311, 1077, 23726, 13, 578, 11245, 72498, 315, 279, 3814, 11, 315, 4038, 13118, 345, 67643, 3201, 1603, 1077, 4382, 2211, 676, 13, 330, 5159, 25237, 11, 856, 25237, 1359, 568, 1071, 9222, 74, 1570, 13, 330, 78403, 430, 358, 649, 656, 313, 2000, 198, 9514, 313, 266, 904, 892, 313, 1875, 2181, 574, 1306, 76996, 1047, 1922, 418, 1093, 264, 11102, 3245, 311, 1077, 3130, 430, 198, 77264, 22983, 36627, 323, 735, 13, 3782, 3663, 311, 3663, 13, 32980, 574, 5115, 17371, 198, 1729, 430, 892, 11, 264, 2697, 6437, 2212, 279, 23726, 11, 719, 1062, 5485, 398, 71414, 13, 330, 7816, 2574, 649, 3621, 311, 264, 12637, 1109, 358, 3596, 7020, 1070, 574, 9135, 568, 198, 84359, 311, 813, 6691, 11, 323, 14976, 4856, 33012, 65923, 311, 1077, 24014, 323, 198, 936, 44357, 13, 330, 2675, 1051, 2744, 264, 1695, 8334, 11, 32980, 1359, 1364, 1071, 13, 330, 10156, 499, 636, 1664, 198, 268, 1409, 311, 2586, 2162, 13, 358, 3358, 1935, 2512, 315, 499, 279, 2800, 315, 856, 2324, 13, 1226, 690, 198, 456, 499, 264, 13587, 79781, 994, 499, 649, 387, 922, 11, 323, 358, 649, 1935, 499, 704, 304, 198, 1820, 6246, 994, 358, 2586, 505, 990, 1210, 330, 40, 3358, 387, 24023, 323, 499, 3358, 387, 74817, 12844, 11, 7643, 1210, 2009, 77638, 818, 374, 2133, 311, 636, 701, 7126, 3288, 709, 1578, 13, 3161, 53474, 36399, 198, 39428, 264, 1938, 323, 1148, 358, 1304, 11, 584, 3358, 636, 3235, 1210, 330, 12174, 11, 32980, 11, 422, 358, 1436, 1518, 499, 5108, 304, 279, 6134, 1578, 323, 53302, 198, 2846, 1605, 6, 323, 596, 13886, 6, 304, 832, 11745, 9135, 578, 6574, 1990, 22770, 22983, 323, 2009, 77638, 818, 574, 1633, 11594, 13, 3005, 1047, 1027, 198, 28936, 264, 3460, 315, 84951, 21455, 389, 279, 84827, 315, 813, 4059, 198, 82977, 682, 279, 3814, 13, 1102, 1053, 387, 5107, 311, 3371, 994, 568, 3604, 198, 74, 943, 1077, 13, 3277, 279, 47979, 369, 7366, 32980, 1203, 311, 279, 2466, 26741, 1047, 1027, 198, 28010, 11, 279, 1023, 37465, 2163, 279, 3130, 11, 323, 22770, 22983, 323, 279, 8334, 1051, 198, 998, 3522, 13, 735, 13, 10717, 1077, 389, 1077, 1648, 311, 279, 6134, 13, 22653, 288, 1618, 26, 856, 836, 374, 2009, 77638, 818, 1210, 330, 40, 617, 539, 3970, 499, 2533, 499, 2163, 800, 13, 330, 2822, 26, 358, 313, 40, 66529, 369, 264, 2478, 4038, 1210, 330, 40, 23289, 814, 656, 539, 1440, 430, 499, 1051, 313, 9210, 499, 617, 1047, 904, 198, 20281, 8952, 3217, 1210, 330, 40, 4985, 539, 3371, 1124, 11, 315, 3388, 1210, 1628, 8617, 11, 555, 4382, 27848, 14771, 11, 433, 574, 28902, 430, 1855, 1288, 198, 45734, 279, 1023, 596, 12410, 13, 2684, 1047, 1027, 264, 892, 11, 1120, 1603, 39493, 345, 9493, 1364, 1047, 1047, 832, 315, 1884, 30462, 50908, 430, 7170, 2586, 520, 198, 1820, 842, 315, 264, 1317, 3814, 13, 578, 8334, 574, 198, 1225, 3428, 11, 20781, 27027, 13, 6385, 3347, 41398, 4920, 1077, 11, 264, 4101, 315, 198, 9181, 5891, 833, 288, 323, 25429, 704, 11490, 37379, 11, 30462, 7692, 826, 11, 6435, 79157, 13, 3005, 1053, 617, 2728, 1475, 3987, 1364, 1047, 304, 279, 198, 14957, 11, 1120, 1243, 11, 369, 76996, 596, 25468, 3347, 13, 3005, 38674, 11937, 449, 430, 99469, 781, 44661, 430, 4131, 315, 4686, 198, 726, 5621, 899, 367, 13, 1628, 1364, 11953, 311, 1077, 3130, 279, 6677, 430, 279, 3814, 596, 14993, 1047, 198, 82850, 304, 46604, 313, 9210, 11, 8051, 32980, 35120, 31202, 1053, 3974, 11, 1364, 1047, 18661, 198, 42312, 555, 1148, 568, 1047, 16654, 13, 578, 4459, 3814, 1047, 6982, 1077, 279, 198, 61893, 33028, 315, 904, 44397, 15003, 311, 3243, 17882, 505, 813, 502, 68078, 13, 3005, 198, 32345, 14792, 1461, 304, 279, 51902, 11, 10307, 76996, 596, 73342, 7216, 198, 300, 1364, 1903, 1077, 1648, 709, 279, 32249, 311, 1077, 3130, 13, 15037, 11, 304, 682, 813, 3347, 198, 78, 1653, 1439, 311, 1077, 11, 1047, 1364, 3970, 430, 1427, 304, 813, 6548, 13, 96424, 1630, 5511, 1432, 1271, 5340, 65670, 24573, 11, 76996, 596, 11914, 315, 27219, 2919, 6, 25288, 3782, 198, 300, 264, 12523, 13, 735, 13, 14760, 279, 3754, 311, 1077, 430, 11714, 1603, 279, 892, 369, 198, 68753, 3520, 596, 19163, 13, 578, 2697, 14048, 574, 11821, 304, 5340, 65670, 596, 44202, 13, 45563, 1047, 198, 64, 13438, 1457, 11, 264, 2697, 58069, 3828, 7086, 386, 25877, 13, 1628, 5340, 65670, 1047, 198, 34500, 291, 389, 279, 6825, 279, 19297, 315, 1306, 1773, 4481, 11033, 13, 1102, 574, 198, 2017, 279, 1306, 1773, 4481, 11033, 430, 735, 13, 1903, 813, 17480, 13, 330, 3923, 656, 499, 3152, 555, 5605, 1364, 374, 5108, 2162, 369, 27219, 2919, 30, 330, 2688, 5986, 11, 8051, 1364, 374, 539, 5115, 1664, 13, 578, 2144, 374, 11, 5340, 65670, 1359, 313, 2000, 198, 275, 574, 330, 27588, 65670, 1, 323, 330, 42, 1210, 555, 420, 892, 11, 313, 1, 19041, 706, 1027, 264, 3460, 315, 198, 87698, 12, 4575, 1748, 709, 520, 279, 8952, 13, 1102, 12775, 956, 19543, 14243, 11, 719, 313, 1875, 27588, 65670, 2231, 1523, 279, 39571, 273, 1355, 33076, 304, 1077, 1450, 323, 45135, 4028, 520, 1461, 13, 330, 3947, 574, 264, 16930, 922, 279, 16088, 11, 323, 1364, 574, 36197, 26, 430, 596, 198, 543, 1210, 330, 8100, 4265, 2731, 2586, 2162, 323, 4822, 2162, 1359, 1071, 5340, 65670, 20193, 13, 330, 40, 3987, 433, 198, 73014, 956, 636, 304, 279, 16064, 13, 1115, 8679, 28936, 2626, 374, 264, 15526, 3460, 315, 198, 1626, 13, 3861, 3492, 2403, 499, 477, 904, 315, 701, 3070, 11, 323, 279, 13734, 596, 1022, 198, 57839, 61752, 775, 1210, 330, 3947, 596, 4400, 2403, 76996, 1359, 735, 13, 31256, 1077, 13, 1102, 5084, 433, 596, 264, 198, 61031, 5030, 315, 26434, 13, 94192, 1903, 264, 16930, 11, 323, 814, 4250, 1095, 198, 21470, 264, 3245, 733, 555, 13, 2030, 568, 13919, 11, 439, 358, 656, 11, 430, 433, 574, 539, 76996, 1210, 4452, 5340, 65670, 1047, 71836, 11937, 2403, 279, 3828, 596, 19163, 11, 682, 1364, 198, 32345, 8967, 311, 2019, 30010, 994, 1364, 5602, 76996, 596, 4319, 839, 6548, 323, 67163, 198, 33780, 13, 1628, 3952, 1077, 1867, 751, 291, 198, 79548, 316, 13, 1789, 279, 892, 520, 3325, 11, 76996, 596, 1917, 1047, 8208, 311, 9863, 922, 1077, 13, 2052, 198, 1964, 34300, 11412, 3935, 315, 2532, 54434, 1603, 1077, 64935, 13, 3277, 46993, 1053, 617, 3970, 1077, 11, 1364, 8774, 1077, 6134, 16447, 323, 4691, 198, 2000, 1120, 430, 832, 11714, 7636, 13, 2030, 1306, 5340, 65670, 1047, 22311, 11, 323, 198, 44, 25877, 11, 279, 58069, 11, 1047, 1922, 418, 704, 311, 279, 9309, 311, 8232, 264, 6661, 1203, 311, 198, 6600, 20786, 11, 76996, 42644, 6312, 291, 1077, 6134, 323, 34793, 304, 279, 2697, 8582, 14321, 13, 5340, 65670, 11, 1077, 2010, 304, 264, 43713, 11, 1077, 3663, 15884, 9439, 12, 846, 3690, 11, 1047, 8208, 198, 998, 4950, 26, 719, 735, 3238, 82, 3177, 11, 439, 13783, 11, 574, 49025, 927, 279, 1380, 316, 13, 76996, 198, 10462, 59091, 291, 311, 279, 6134, 13, 330, 11356, 358, 2586, 304, 323, 3137, 311, 499, 7673, 1283, 6656, 323, 3952, 264, 4062, 10795, 315, 279, 3130, 13, 578, 6945, 574, 2403, 198, 1820, 37204, 12913, 13, 2030, 568, 3952, 279, 5326, 323, 5762, 279, 6134, 7029, 13, 76996, 3782, 304, 323, 7731, 1523, 555, 279, 4027, 13, 3296, 1694, 1008, 69596, 568, 9152, 311, 198, 3306, 575, 279, 2697, 6945, 927, 323, 1234, 279, 3830, 1603, 1364, 5602, 433, 13, 1102, 374, 198, 67, 5599, 83, 1285, 422, 1364, 1053, 617, 15393, 1202, 26431, 11, 1047, 1364, 3970, 433, 13, 330, 40, 3077, 1027, 7422, 2574, 927, 1359, 1364, 1071, 13, 330, 2181, 5084, 311, 757, 358, 4265, 2731, 198, 1962, 733, 1203, 1210, 330, 4897, 1053, 387, 46337, 11, 8434, 956, 433, 11, 994, 499, 617, 2884, 779, 1664, 30, 1628, 345, 9620, 3422, 11, 2533, 499, 527, 539, 16390, 11, 76996, 313, 1875, 7189, 3287, 956, 656, 433, 9135, 358, 649, 956, 2567, 389, 26, 430, 596, 682, 1070, 374, 311, 433, 13, 2052, 198, 4354, 3814, 11, 304, 279, 13147, 26741, 11, 358, 6612, 433, 2133, 13, 358, 198, 99020, 5605, 311, 7182, 25, 364, 2675, 3287, 956, 656, 433, 11, 499, 3287, 956, 656, 433, 7112, 323, 682, 279, 198, 1712, 2555, 4871, 315, 757, 574, 5605, 11, 364, 2688, 1457, 11, 8530, 26, 719, 36113, 198, 9514, 1253, 30251, 45773, 735, 2637, 889, 1047, 93469, 682, 420, 704, 369, 5678, 323, 1047, 2586, 311, 279, 198, 25030, 3242, 13559, 0, 330, 1271, 733, 389, 1093, 420, 11, 8430, 430, 832, 706, 2324, 323, 4648, 304, 832, 596, 1450, 345, 438, 1243, 8530, 1063, 1938, 311, 1304, 264, 16930, 1093, 430, 9135, 3005, 7111, 709, 520, 198, 40617, 369, 75, 1540, 398, 13, 330, 40, 1097, 1120, 539, 34300, 3403, 11, 735, 1210, 330, 29089, 77, 956, 433, 387, 1437, 7403, 311, 2567, 389, 30, 6385, 1917, 574, 304, 9863, 922, 1077, 11, 323, 1364, 6612, 7636, 304, 264, 7029, 323, 198, 3274, 2035, 13, 1628, 11, 1606, 1077, 50581, 1051, 15107, 259, 2784, 3156, 814, 1051, 198, 2359, 311, 10885, 11, 76996, 6656, 389, 1461, 559, 4361, 65923, 13, 330, 40, 1781, 499, 527, 682, 16984, 358, 690, 2586, 1203, 311, 4822, 13, 37558, 2216, 198, 86, 1821, 757, 12660, 313, 258, 682, 279, 1917, 0, 2876, 520, 279, 8952, 11, 539, 1618, 11, 539, 198, 3852, 2050, 13, 330, 4599, 499, 2019, 430, 19093, 6944, 499, 1359, 1071, 735, 2637, 539, 1633, 43071, 11, 330, 40, 313, 40, 198, 27963, 499, 527, 3339, 264, 16930, 1210, 578, 1193, 1732, 198, 14965, 3596, 2216, 4934, 757, 574, 856, 6691, 11, 323, 358, 4024, 3201, 323, 2163, 1077, 9135, 3005, 48548, 813, 3663, 15499, 11, 323, 11, 5403, 1070, 2555, 1364, 1550, 539, 198, 8154, 2752, 11, 1364, 220, 15187, 13, 330, 2822, 26, 358, 656, 539, 3152, 13142, 46506, 12669, 1210, 1442, 568, 1047, 1766, 904, 51475, 304, 1077, 3663, 11, 568, 1053, 617, 8208, 389, 198, 25662, 16117, 26, 719, 1077, 10321, 6548, 19201, 1461, 13, 330, 2746, 499, 3152, 7639, 17882, 1359, 1071, 76996, 11, 330, 9514, 527, 11622, 5076, 13, 1283, 596, 539, 198, 258, 3021, 449, 757, 313, 1962, 11, 430, 374, 11, 904, 810, 1109, 568, 374, 304, 3021, 449, 264, 198, 3055, 5797, 7724, 13, 1283, 13452, 311, 387, 449, 757, 313, 2319, 11, 358, 1440, 430, 26, 719, 430, 3250, 956, 198, 14622, 313, 73254, 775, 13, 5884, 5269, 11, 1306, 420, 64935, 313, 1875, 25938, 374, 912, 64935, 11, 1716, 1210, 330, 1548, 3358, 1781, 757, 84834, 11, 520, 279, 3325, 13, 330, 2675, 2019, 568, 13452, 311, 387, 449, 499, 13, 76996, 1047, 1027, 11961, 304, 264, 3428, 10716, 555, 279, 4027, 13, 3005, 16392, 449, 264, 198, 82, 39095, 25429, 7351, 13, 763, 279, 6179, 2786, 315, 279, 14048, 11, 1364, 345, 32345, 12263, 735, 13, 304, 1077, 32623, 2427, 785, 323, 1776, 32997, 26, 323, 1457, 1364, 14980, 198, 15145, 1461, 11, 264, 35279, 3995, 7216, 11, 43789, 287, 279, 61607, 315, 1077, 65510, 198, 582, 2177, 1077, 17659, 13, 330, 40, 24863, 1461, 11, 735, 13, 1359, 1364, 1071, 72702, 2740, 13, 330, 4599, 358, 1518, 1461, 5108, 11, 358, 1390, 198, 998, 636, 1523, 323, 1095, 1461, 4321, 389, 757, 13, 3842, 7882, 311, 279, 14150, 13, 358, 198, 33134, 279, 1633, 1648, 568, 25562, 369, 279, 39736, 13, 3277, 358, 1518, 1461, 304, 279, 198, 3376, 1113, 48840, 11, 7155, 323, 19858, 1418, 1475, 832, 775, 374, 1344, 5100, 291, 323, 198, 40541, 1639, 11, 568, 313, 383, 5992, 1093, 264, 10087, 1210, 5112, 11, 4376, 51975, 315, 1077, 704, 58838, 11, 1364, 6656, 1077, 1203, 311, 1461, 323, 14980, 198, 70, 6795, 520, 279, 2678, 11756, 4027, 13, 1102, 574, 439, 1664, 369, 735, 13, 430, 1364, 1550, 539, 198, 4151, 813, 3663, 13, 1789, 430, 832, 4545, 279, 46996, 430, 574, 304, 1461, 559, 606, 304, 198, 26301, 6548, 13, 1283, 65657, 2212, 279, 559, 79736, 2697, 3130, 11, 520, 279, 30811, 3252, 4950, 345, 1820, 37204, 12913, 11, 279, 281, 2910, 1136, 290, 11, 279, 2362, 42390, 2442, 18033, 44402, 1234, 902, 198, 3561, 992, 67, 1047, 34833, 1903, 813, 23634, 11, 520, 813, 13365, 43686, 2007, 11, 39682, 291, 449, 198, 87820, 323, 6603, 11, 520, 279, 2217, 304, 279, 18327, 315, 813, 1866, 16615, 7216, 345, 34152, 16771, 323, 76656, 13, 330, 2675, 2351, 2771, 433, 596, 539, 198, 4345, 313, 6200, 309, 414, 11, 76996, 7673, 6385, 7899, 574, 296, 43435, 11, 323, 568, 7020, 1243, 430, 198, 32158, 574, 31299, 13, 92014, 11, 315, 3388, 11, 3734, 304, 279, 12625, 198, 1073, 832, 596, 33044, 11, 1051, 539, 31308, 389, 279, 6825, 13, 330, 36227, 757, 264, 1450, 7197, 62626, 1359, 1071, 76996, 304, 264, 296, 43435, 16630, 11, 323, 279, 2697, 198, 23296, 574, 11102, 1139, 1418, 735, 13, 27600, 1555, 264, 44402, 27580, 13, 5112, 1473, 12348, 596, 682, 927, 11, 90064, 11, 2533, 420, 13, 1442, 568, 4265, 2216, 42777, 568, 4265, 617, 2586, 198, 2017, 311, 45427, 13, 6984, 304, 264, 12960, 1364, 3782, 40605, 311, 1077, 25288, 13, 3005, 1053, 2646, 198, 3427, 1203, 11, 1364, 1071, 92925, 13, 3005, 574, 25226, 11, 1047, 1027, 61456, 198, 4575, 2656, 13, 1442, 814, 1436, 1781, 1778, 264, 3245, 922, 1077, 11, 1364, 3287, 956, 1390, 311, 198, 1395, 304, 872, 2362, 8952, 13, 735, 13, 29440, 1077, 11, 7064, 2718, 65464, 323, 84072, 13, 358, 617, 2728, 1461, 813, 39653, 22700, 315, 3115, 1210, 330, 15546, 775, 1047, 2680, 311, 279, 16088, 33044, 7673, 330, 77264, 22983, 36627, 11953, 279, 7039, 11, 315, 3388, 13, 358, 574, 1022, 14523, 505, 3116, 198, 998, 4848, 13, 3277, 22770, 22983, 2163, 279, 26741, 11, 279, 47027, 261, 1053, 617, 1124, 1210, 330, 12389, 499, 2944, 311, 1781, 430, 3060, 832, 315, 1521, 7724, 1053, 6562, 499, 198, 71, 2227, 7673, 330, 4155, 8996, 1359, 6137, 76996, 92468, 398, 26, 323, 1243, 11, 13598, 198, 71, 388, 491, 11, 313, 1, 39870, 313, 8248, 430, 596, 4856, 27873, 1210, 330, 40, 3077, 7170, 3463, 430, 22770, 22983, 313, 8248, 358, 1097, 2771, 1364, 374, 14268, 198, 42000, 449, 757, 13, 7570, 422, 1364, 313, 333, 1364, 313, 1875, 58841, 7673, 17882, 11, 358, 1541, 956, 4510, 313, 10445, 11, 735, 2637, 1364, 8434, 956, 0, 330, 60978, 1126, 11, 315, 3388, 1359, 1071, 735, 2637, 330, 258, 14944, 11, 90064, 13, 358, 2846, 1193, 4560, 311, 1505, 704, 6832, 16930, 433, 574, 1210, 32862, 1306, 430, 1364, 1071, 1695, 45427, 323, 4024, 704, 13, 3005, 6656, 304, 279, 198, 11020, 3195, 323, 31645, 18659, 38662, 1203, 520, 1461, 13, 330, 2675, 617, 2884, 757, 264, 2763, 315, 1695, 13, 3161, 264, 4062, 7351, 430, 574, 832, 315, 1077, 81750, 11, 76996, 15187, 8036, 198, 1820, 6134, 323, 43578, 1203, 1139, 279, 3130, 13, 735, 2637, 11011, 279, 6134, 3345, 345, 61665, 1364, 1047, 8208, 11, 323, 12504, 17345, 1139, 264, 10716, 13, 1071, 76996, 15187, 505, 4920, 1461, 345, 438, 11, 58218, 927, 11, 1364, 59334, 1461, 389, 279, 41803, 13, 578, 1828, 9888, 279, 6134, 1047, 8036, 4920, 1077, 11, 323, 735, 13, 574, 2163, 7636, 198, 998, 1778, 289, 62837, 2136, 323, 59045, 439, 279, 11714, 1047, 7263, 1461, 13, 1952, 9017, 6693, 11, 5340, 65670, 11, 889, 46498, 719, 2800, 16117, 304, 1077, 43713, 345, 86, 3448, 291, 311, 279, 81234, 315, 813, 3177, 927, 279, 1380, 316, 13, 330, 40, 6562, 499, 8434, 956, 733, 311, 198, 26894, 323, 1095, 701, 3177, 8395, 9135, 735, 2637, 1765, 76, 3876, 279, 43713, 323, 9439, 12932, 11, 1047, 279, 14413, 539, 311, 1825, 813, 198, 11020, 13, 330, 40, 1097, 539, 33173, 11, 5340, 65670, 11, 323, 358, 1097, 14931, 922, 279, 3177, 13, 13538, 568, 56807, 3384, 279, 3177, 11, 568, 15203, 927, 311, 279, 2362, 96637, 323, 198, 48554, 291, 5678, 304, 279, 9168, 13, 9220, 22178, 2085, 6212, 323, 1790, 18547, 198, 32345, 3309, 389, 1461, 13, 1283, 7111, 2362, 11, 305, 16094, 569, 26, 72600, 19781, 13, 49693, 750, 568, 198, 5807, 1636, 5678, 449, 17882, 11, 74820, 449, 2450, 11, 38732, 11, 92906, 28022, 345, 60301, 85943, 306, 13, 12334, 1047, 810, 7995, 3309, 1461, 279, 3987, 33028, 198, 1073, 813, 3021, 369, 76996, 1109, 1077, 1695, 45427, 21735, 13, 1283, 24465, 264, 1317, 11745, 323, 45374, 198, 998, 2073, 676, 304, 279, 6453, 13, 13142, 46506, 12669, 3782, 311, 1518, 76996, 279, 1828, 1938, 13, 3005, 1053, 617, 31890, 198, 40617, 422, 1364, 1436, 11, 719, 386, 25877, 1047, 64260, 291, 1461, 709, 311, 279, 52319, 48840, 293, 3023, 13603, 198, 15145, 1364, 1047, 892, 311, 12731, 13, 3005, 1047, 539, 3970, 279, 8334, 369, 1403, 4038, 345, 438, 279, 2349, 304, 1461, 89815, 1077, 13, 1283, 574, 65355, 11, 4856, 91425, 345, 26121, 455, 38662, 1664, 26435, 13, 1364, 1071, 11, 323, 1243, 25, 330, 76936, 956, 499, 2503, 1523, 7673, 1283, 13859, 266, 1534, 5678, 11, 439, 568, 1047, 430, 198, 9471, 279, 5651, 1603, 994, 568, 1047, 4691, 76996, 311, 32918, 1461, 13, 1283, 9076, 912, 21349, 43213, 8996, 280, 8248, 11, 1306, 10795, 287, 1077, 27851, 11, 1077, 3776, 65510, 11, 279, 5238, 2212, 1077, 198, 58331, 25, 27567, 22336, 2351, 539, 2133, 1203, 311, 430, 2035, 11, 315, 3388, 7673, 330, 12487, 18570, 596, 2751, 311, 10491, 369, 499, 13, 578, 3245, 369, 499, 311, 656, 374, 311, 198, 60021, 1314, 1618, 11, 76996, 13, 37558, 1618, 198, 41450, 3596, 62742, 499, 315, 4560, 311, 10102, 21739, 1210, 763, 34781, 315, 11937, 11, 76996, 31645, 264, 2697, 13, 1102, 574, 264, 16930, 922, 279, 198, 2106, 292, 1572, 13, 5414, 3021, 574, 32227, 45743, 11, 369, 568, 62639, 16038, 1077, 8835, 439, 422, 1364, 198, 32345, 539, 22066, 13, 330, 2675, 3041, 757, 279, 3492, 323, 358, 3358, 733, 323, 636, 701, 2574, 26, 358, 3077, 2751, 264, 1841, 315, 198, 2465, 1866, 1457, 1210, 330, 4071, 11, 13142, 11, 814, 617, 1193, 2884, 1148, 814, 3463, 574, 1314, 13, 91395, 1903, 198, 275, 11, 1070, 574, 264, 16930, 1210, 330, 2675, 1541, 956, 3152, 430, 499, 527, 2133, 311, 2559, 369, 420, 3460, 315, 3245, 30, 7357, 892, 1063, 24931, 3727, 264, 16930, 11, 527, 814, 2133, 311, 19065, 433, 389, 198, 9514, 7673, 358, 649, 956, 3137, 311, 499, 198, 333, 499, 16135, 1093, 264, 25294, 682, 279, 892, 1210, 6385, 5030, 8838, 2269, 533, 16630, 1047, 1202, 2515, 13, 1283, 11084, 1139, 279, 3130, 11, 719, 198, 383, 2103, 88106, 291, 264, 10716, 13, 330, 40, 8101, 499, 3077, 1027, 20910, 3249, 499, 9167, 956, 6755, 505, 757, 1359, 568, 1071, 13, 330, 40, 3077, 3970, 499, 810, 1109, 499, 3077, 3970, 757, 1210, 578, 4623, 315, 78739, 374, 2744, 2109, 773, 52985, 11, 323, 198, 998, 617, 264, 18010, 31657, 2744, 304, 279, 1022, 287, 11, 439, 433, 1051, 11, 574, 198, 4338, 89405, 287, 13, 330, 40, 6562, 499, 1053, 387, 1120, 264, 2697, 2766, 38761, 11, 13142, 13, 1102, 596, 779, 30571, 315, 198, 9514, 11, 2216, 13, 1102, 596, 539, 1606, 499, 2512, 369, 757, 26, 433, 596, 2216, 1606, 499, 198, 10727, 369, 6261, 1210, 330, 2675, 649, 956, 1427, 520, 757, 323, 2019, 430, 11, 34989, 1210, 1283, 10837, 813, 14654, 2212, 813, 37204, 313, 276, 2362, 31257, 26, 719, 279, 37204, 574, 198, 1225, 20784, 13, 330, 40, 2846, 1120, 12459, 856, 4851, 704, 369, 499, 11, 323, 430, 596, 279, 8206, 13, 98325, 358, 733, 11, 1274, 2019, 11, 364, 3947, 596, 279, 12637, 76996, 198, 2732, 6656, 1523, 994, 1364, 4024, 311, 279, 8952, 3238, 358, 3077, 2751, 779, 358, 2567, 1022, 198, 1820, 6825, 439, 1790, 439, 358, 649, 1210, 1115, 8545, 11, 12304, 8334, 574, 539, 198, 1820, 5679, 3640, 398, 37513, 12822, 1364, 1047, 2744, 3967, 13, 1102, 9508, 311, 1077, 198, 9210, 568, 574, 20781, 62200, 313, 9210, 30456, 813, 11594, 11827, 323, 15884, 198, 265, 14655, 7899, 1070, 47747, 48259, 2555, 61754, 11, 2555, 1364, 1436, 198, 1962, 37586, 449, 13, 330, 4071, 1148, 656, 499, 1390, 757, 311, 656, 30, 1442, 499, 4265, 198, 3323, 2503, 1523, 313, 1875, 7189, 1390, 499, 311, 2586, 2162, 13, 358, 1120, 1390, 198, 9514, 311, 2586, 1203, 11, 779, 430, 2574, 690, 387, 279, 1648, 814, 1511, 311, 387, 13, 4800, 198, 9210, 814, 617, 6656, 499, 704, 313, 1875, 32408, 3077, 2884, 4400, 315, 279, 3460, 13, 330, 18433, 499, 3021, 279, 8952, 11, 477, 1606, 499, 3021, 18570, 8599, 198, 4291, 279, 8952, 7673, 76996, 574, 27461, 19021, 555, 420, 892, 11, 19021, 323, 54317, 13, 3005, 1047, 198, 2063, 1555, 779, 1790, 430, 1475, 32015, 574, 31299, 304, 25429, 8835, 13, 330, 2746, 433, 690, 1304, 499, 3619, 2574, 904, 2731, 1359, 1364, 39169, 11, 330, 40, 1097, 198, 9738, 1203, 369, 2225, 8125, 9135, 2030, 1077, 4339, 9508, 11, 29392, 198, 268, 1409, 11, 311, 24981, 1461, 13, 330, 12487, 11, 439, 3117, 439, 358, 1097, 11920, 11, 433, 596, 682, 927, 11, 374, 433, 7673, 58801, 25, 27567, 22336, 1781, 46993, 706, 1077, 6206, 2539, 449, 42216, 11, 1541, 956, 499, 30, 8489, 345, 333, 499, 1935, 7639, 17882, 11, 499, 2351, 2133, 311, 617, 810, 12544, 1109, 46993, 198, 2099, 57636, 315, 13, 358, 649, 3371, 499, 1063, 2574, 922, 1461, 1457, 430, 690, 1304, 198, 9514, 1781, 11157, 1210, 330, 11769, 3492, 430, 499, 2019, 5039, 757, 1268, 1314, 358, 1097, 304, 539, 78849, 499, 345, 41444, 1359, 1364, 1071, 13, 330, 13058, 3026, 656, 539, 2019, 1884, 2574, 922, 1855, 1023, 1234, 198, 3852, 13463, 13, 358, 1541, 956, 1390, 499, 311, 198, 2063, 1203, 3156, 499, 617, 15042, 709, 1210, 1283, 574, 1633, 4251, 11, 719, 568, 13061, 709, 813, 9072, 323, 4024, 311, 279, 6134, 13, 330, 40, 8101, 358, 6912, 14599, 1359, 568, 1071, 13, 330, 40, 3077, 1027, 19762, 311, 733, 3201, 11, 719, 6691, 198, 33371, 1778, 264, 64864, 313, 40, 3358, 539, 38292, 499, 904, 810, 1210, 1283, 8813, 304, 813, 18301, 323, 11, 23062, 704, 264, 2678, 3830, 11, 5762, 433, 9017, 198, 1964, 13, 330, 3561, 992, 67, 1359, 568, 1071, 64702, 398, 13, 4427, 13305, 10791, 198, 40617, 304, 279, 6246, 11, 323, 358, 7263, 1461, 2162, 1210, 1283, 2163, 1077, 11509, 1070, 8982, 1752, 449, 13051, 11, 449, 279, 3830, 304, 1077, 198, 10888, 11, 323, 10837, 1523, 279, 32249, 323, 704, 1139, 279, 6825, 13, 2468, 279, 4579, 315, 198, 1820, 7504, 568, 4661, 79990, 449, 2999, 13, 358, 2846, 16089, 198, 9514, 3077, 1903, 433, 709, 1210, 96424, 30388, 1432, 67288, 31467, 1202, 43789, 14297, 430, 1060, 13, 5587, 574, 99945, 9439, 280, 17206, 5936, 1766, 279, 19795, 2103, 20268, 323, 279, 61316, 1414, 4336, 93858, 449, 198, 560, 13, 2030, 520, 5209, 11477, 1070, 574, 10683, 304, 279, 3805, 13, 763, 279, 66385, 315, 279, 198, 71127, 11, 5804, 3916, 39428, 7731, 389, 279, 79509, 323, 15746, 369, 10773, 1354, 13, 578, 198, 69, 35624, 11, 902, 1047, 20268, 704, 11, 574, 1694, 52834, 13, 5810, 323, 1070, 389, 198, 1637, 3321, 1355, 3385, 62036, 3153, 9107, 872, 342, 8039, 88, 96740, 311, 279, 7160, 13, 5340, 65670, 1047, 8208, 22917, 369, 264, 16706, 8577, 304, 5587, 323, 3782, 1203, 82787, 198, 4291, 502, 6848, 11, 1646, 342, 43456, 11, 323, 7878, 36232, 13, 3005, 11953, 704, 323, 198, 501, 7719, 19837, 389, 1077, 13219, 596, 25165, 11, 323, 4024, 1203, 311, 1077, 990, 449, 264, 198, 1897, 20659, 315, 14523, 2884, 13, 362, 10824, 315, 14425, 66, 4881, 323, 12056, 389, 279, 5015, 198, 32345, 2728, 1077, 459, 20343, 369, 264, 65510, 13, 3005, 24465, 433, 304, 47218, 389, 459, 198, 268, 19384, 389, 1077, 1648, 1203, 304, 279, 8761, 1841, 13, 32171, 56310, 11, 3515, 1903, 1695, 2391, 279, 4251, 6763, 11, 1047, 1027, 3288, 311, 198, 1820, 10683, 24428, 82, 13, 578, 1938, 1364, 198, 63578, 76996, 3769, 369, 264, 4382, 4251, 65510, 11, 1364, 574, 1633, 6380, 13, 9843, 198, 64, 6130, 7263, 1077, 264, 15860, 315, 9036, 3714, 288, 13, 2052, 1938, 1364, 8774, 1124, 1234, 198, 1820, 5663, 304, 264, 9168, 315, 3090, 11, 323, 520, 11714, 1364, 3952, 1124, 311, 32980, 198, 49, 8477, 31202, 11, 2103, 21078, 38097, 304, 279, 8952, 13, 1952, 76996, 11, 389, 735, 2637, 323, 389, 46993, 279, 12688, 1047, 2163, 1202, 1906, 17345, 13, 46993, 11, 1373, 4345, 287, 1077, 2324, 311, 502, 4787, 11, 574, 1099, 7403, 11, 810, 198, 61665, 1285, 3238, 82, 19351, 11, 1364, 198, 32345, 2728, 709, 279, 330, 35, 1412, 434, 1, 323, 574, 5403, 1972, 6603, 13, 3005, 574, 7422, 198, 8110, 11555, 11, 2288, 11, 369, 279, 1176, 892, 304, 1077, 2324, 13, 76996, 11, 439, 28682, 439, 3596, 11, 1047, 5675, 264, 2697, 315, 279, 12164, 685, 505, 1077, 198, 58331, 26, 1077, 7899, 1047, 5655, 6901, 13, 11208, 1364, 1047, 1027, 264, 5128, 3828, 11, 1364, 198, 16514, 1457, 17104, 13, 3005, 574, 1203, 304, 279, 8952, 1578, 11, 420, 892, 304, 279, 198, 5988, 596, 26741, 13, 735, 2637, 2133, 304, 832, 1938, 311, 1935, 32980, 35120, 31202, 264, 14351, 198, 1073, 14098, 11, 5602, 1077, 1070, 449, 264, 1716, 304, 1077, 11977, 11, 323, 264, 3177, 304, 1077, 198, 58331, 430, 568, 1047, 2646, 3970, 1603, 13, 1102, 13194, 1461, 11, 4856, 313, 58174, 1694, 439, 198, 20670, 1051, 449, 1461, 13, 3161, 279, 8736, 315, 10683, 279, 2697, 3838, 520, 8270, 5447, 3952, 389, 7878, 198, 33078, 13, 40788, 648, 574, 3838, 1824, 60051, 449, 2294, 17879, 2136, 13, 3005, 198, 26121, 392, 2788, 89341, 11, 3952, 1523, 279, 4335, 54499, 11, 323, 2231, 1124, 709, 1578, 198, 72408, 398, 70181, 291, 13, 1102, 574, 439, 422, 1364, 1766, 304, 33638, 5820, 323, 36709, 264, 198, 1864, 9497, 369, 1077, 6316, 300, 1918, 13, 578, 89646, 3752, 315, 279, 2697, 198, 7830, 1047, 1027, 2403, 433, 13, 5124, 3886, 1047, 264, 2697, 3703, 323, 198, 82, 2841, 279, 1888, 12046, 1105, 568, 1436, 3780, 26, 719, 568, 64770, 2872, 67, 1918, 313, 32345, 198, 82850, 3967, 311, 26122, 311, 4662, 311, 13305, 1234, 17510, 19101, 323, 311, 3026, 889, 1047, 198, 46584, 927, 485, 360, 3640, 13, 578, 3492, 4024, 922, 430, 5124, 3886, 596, 574, 912, 2035, 198, 2000, 264, 1695, 892, 13, 7570, 40788, 648, 596, 16553, 323, 289, 81635, 4745, 2403, 420, 198, 10888, 98450, 13, 3296, 279, 6278, 315, 5936, 279, 3838, 1824, 60051, 574, 2884, 13, 3861, 477, 1403, 9048, 198, 4581, 552, 1047, 2586, 704, 11, 294, 1619, 11163, 2718, 323, 289, 1619, 70351, 11, 323, 1047, 8208, 198, 1445, 311, 279, 3363, 1578, 13, 578, 198, 43791, 32720, 709, 11, 10773, 1354, 10409, 279, 12690, 449, 872, 50380, 10683, 11936, 345, 438, 2103, 2626, 8738, 41630, 13, 3296, 279, 1176, 1938, 315, 3297, 11, 40788, 648, 596, 6316, 300, 1918, 1047, 3719, 44743, 13, 5124, 3886, 11, 5108, 304, 505, 279, 4216, 7625, 10789, 11, 1766, 1077, 198, 82, 15154, 304, 279, 9979, 11, 1077, 3663, 28016, 304, 1077, 21726, 263, 13, 1283, 2231, 1523, 279, 198, 76, 34263, 2320, 6341, 323, 11, 2133, 927, 311, 1077, 11, 2231, 264, 1450, 389, 1077, 2010, 13, 330, 40, 8101, 1070, 596, 912, 16930, 11, 1243, 7673, 330, 3947, 596, 912, 16930, 1359, 1071, 8009, 40788, 648, 1139, 1077, 21726, 263, 13, 1283, 30280, 1523, 323, 59334, 279, 1203, 315, 1077, 13272, 13, 5112, 11, 994, 1364, 4745, 311, 198, 73216, 268, 11, 568, 9165, 59091, 291, 2212, 279, 9979, 11, 44033, 279, 14403, 1139, 64883, 345, 438, 97139, 291, 279, 43212, 11, 3318, 1749, 2740, 304, 813, 8987, 1648, 13, 578, 198, 82694, 12934, 48726, 1047, 66185, 9235, 13, 5112, 25, 27567, 91724, 499, 1390, 311, 1518, 264, 10896, 7673, 330, 40, 4265, 2731, 1518, 18570, 1359, 1364, 1071, 11, 2085, 3411, 709, 13, 330, 3112, 313, 15357, 956, 198, 27963, 358, 2846, 59771, 499, 13, 1666, 3117, 439, 198, 9210, 5900, 11, 358, 3077, 4934, 264, 1716, 1314, 3235, 13, 1102, 4536, 956, 279, 12544, 358, 1097, 198, 83360, 315, 3060, 1210, 1283, 1903, 1063, 15600, 198, 566, 6370, 1570, 323, 14198, 291, 1077, 264, 6710, 315, 23211, 13, 3277, 568, 1047, 2231, 1124, 389, 832, 198, 408, 315, 279, 9979, 2007, 11, 568, 4024, 927, 311, 1077, 1578, 13, 330, 40, 8101, 358, 4265, 22525, 311, 617, 3463, 315, 420, 1603, 11, 719, 682, 358, 3463, 315, 198, 16514, 4560, 311, 636, 264, 2697, 23871, 704, 315, 2324, 13, 1628, 1359, 313, 383, 31511, 48259, 198, 1964, 6916, 11, 313, 1, 300, 3117, 439, 358, 1097, 11920, 11, 433, 596, 1027, 5922, 1418, 11, 40788, 648, 13, 2360, 198, 60065, 1148, 358, 3077, 1047, 311, 656, 11, 358, 3077, 2744, 7111, 4741, 311, 5108, 1203, 198, 6881, 311, 499, 304, 279, 11714, 13, 10926, 358, 1541, 956, 2019, 433, 3403, 11, 719, 358, 8101, 499, 198, 33134, 358, 2733, 433, 682, 1314, 1210, 17586, 3411, 709, 11, 1364, 9277, 1077, 1450, 927, 813, 13, 330, 40, 8101, 584, 3940, 5076, 1359, 568, 4024, 389, 13, 330, 2675, 649, 956, 1977, 23871, 389, 198, 12840, 4536, 956, 1314, 13, 1472, 323, 358, 649, 10299, 1664, 3403, 26, 719, 1457, 430, 1070, 596, 198, 9738, 311, 387, 2500, 11, 433, 5992, 2204, 11, 17354, 1210, 4740, 430, 6693, 40788, 648, 3952, 709, 1077, 23104, 43132, 2740, 13, 578, 3987, 315, 198, 27510, 5812, 7064, 660, 449, 3776, 18809, 315, 18710, 13, 3005, 29340, 520, 1077, 198, 1816, 11, 311, 21165, 704, 1139, 11210, 24014, 13, 5124, 3886, 1047, 2728, 709, 813, 56226, 198, 27243, 26, 719, 279, 72998, 889, 3782, 311, 8270, 5447, 1550, 539, 2586, 1203, 13, 3277, 11, 520, 1566, 11, 568, 3952, 279, 15580, 323, 80411, 323, 23980, 922, 279, 3224, 198, 2000, 10373, 11, 568, 574, 2288, 3389, 13, 7089, 56226, 5794, 1047, 1027, 1603, 1461, 280, 939, 60530, 655, 552, 323, 41245, 2402, 1051, 2736, 1694, 743, 704, 13, 578, 2132, 8323, 198, 263, 813, 21515, 1053, 387, 4245, 304, 5887, 13, 3296, 279, 6278, 315, 3297, 814, 1051, 198, 1658, 1201, 398, 709, 2403, 433, 13, 5124, 3886, 520, 1566, 76035, 311, 2231, 279, 6671, 1139, 198, 5880, 13, 330, 1687, 2351, 539, 3339, 1695, 11, 58769, 1359, 568, 1071, 13, 1226, 527, 2288, 15326, 26, 430, 596, 1148, 596, 279, 5030, 449, 603, 1210, 3161, 682, 1077, 85536, 11, 40788, 648, 574, 53108, 48080, 315, 2324, 13, 330, 1687, 3358, 617, 311, 2567, 264, 3460, 315, 9689, 1359, 568, 1071, 32703, 989, 13, 330, 69633, 311, 16470, 198, 9210, 4131, 3235, 11, 323, 313, 333, 9875, 1390, 311, 4822, 927, 45427, 313, 1875, 51, 484, 648, 596, 4251, 3663, 6656, 97119, 13, 330, 2746, 433, 596, 3958, 9282, 11, 323, 814, 2351, 12502, 313, 1875, 72059, 527, 584, 311, 1440, 422, 814, 527, 12502, 477, 539, 7673, 2030, 279, 198, 47965, 4090, 574, 539, 2753, 30883, 13, 2684, 1051, 1403, 477, 2380, 9662, 84539, 12295, 198, 263, 279, 2132, 6558, 13, 1283, 6137, 311, 1304, 76220, 18726, 439, 311, 872, 198, 69, 399, 11218, 13, 9843, 568, 2751, 264, 49639, 505, 459, 47644, 3838, 11, 323, 6818, 198, 998, 10477, 433, 505, 1077, 13, 3005, 27724, 433, 304, 279, 9979, 198, 267, 1009, 13, 5124, 3886, 5678, 574, 51975, 26, 719, 279, 4623, 44898, 1461, 13, 7089, 1274, 198, 69, 1617, 6901, 389, 279, 91223, 29646, 315, 3823, 7138, 13, 9220, 8931, 3201, 11, 389, 279, 1023, 198, 8714, 11, 574, 264, 586, 3838, 430, 1047, 4272, 6702, 279, 6506, 5899, 16579, 11441, 198, 26466, 279, 1060, 1603, 13, 1283, 574, 539, 439, 3995, 439, 568, 1047, 1027, 26, 1070, 574, 279, 20900, 315, 10494, 198, 26301, 7555, 313, 383, 1047, 2646, 5535, 1077, 311, 733, 1139, 279, 23693, 26741, 520, 279, 198, 300, 28433, 13, 4800, 430, 1070, 574, 2133, 311, 387, 264, 1716, 11, 1070, 1053, 387, 2380, 198, 16455, 18222, 5304, 1461, 13, 3861, 3814, 11, 1306, 40788, 648, 574, 33173, 11, 568, 43578, 12248, 16117, 1139, 813, 198, 33678, 6509, 323, 704, 311, 279, 33419, 11, 1405, 568, 59207, 291, 709, 279, 15580, 449, 23418, 198, 69, 14437, 13, 40788, 648, 2646, 9687, 315, 430, 33433, 95440, 311, 279, 330, 34, 4763, 7278, 16344, 43352, 20375, 8931, 3201, 13, 35270, 1529, 28109, 304, 1475, 3321, 26, 264, 21030, 79648, 1051, 198, 29836, 291, 1603, 279, 33419, 13, 5659, 279, 3703, 3782, 198, 1820, 503, 2222, 315, 29247, 323, 17813, 11, 71414, 10652, 13, 3277, 5124, 3886, 6656, 279, 15580, 596, 2010, 1203, 9017, 8270, 5447, 11, 813, 198, 38824, 574, 1903, 709, 13, 1283, 1053, 52259, 279, 8582, 12295, 26, 568, 1053, 4546, 264, 198, 2308, 19393, 505, 6424, 313, 45010, 1274, 4934, 9709, 21662, 26, 568, 1436, 636, 264, 198, 5686, 25417, 27374, 15038, 13, 3277, 1364, 1766, 1461, 198, 67, 47737, 11, 1364, 1903, 279, 30485, 430, 1077, 3044, 4541, 33337, 13, 3005, 198, 29177, 539, 5387, 1461, 11, 719, 1364, 1053, 539, 4822, 304, 279, 89266, 660, 2697, 198, 7830, 13, 3277, 11, 264, 2046, 1306, 5124, 3886, 596, 4034, 311, 279, 330, 34, 4763, 7278, 16344, 1359, 459, 198, 12527, 479, 5355, 11721, 505, 6424, 449, 279, 502, 14891, 11, 40788, 648, 198, 95918, 704, 311, 1148, 1047, 1027, 279, 33508, 48840, 315, 279, 2362, 33419, 323, 1070, 198, 34500, 291, 11937, 13, 330, 40, 1097, 539, 9564, 499, 1359, 1364, 3309, 1461, 13, 330, 40, 1541, 956, 1524, 1440, 430, 358, 1097, 198, 2067, 6605, 499, 13, 2030, 358, 1097, 539, 2133, 311, 617, 4205, 311, 656, 449, 433, 11, 323, 198, 9210, 596, 10269, 1210, 2100, 433, 7077, 430, 735, 2637, 3339, 264, 10683, 90413, 311, 1518, 40788, 648, 345, 90963, 12025, 13382, 304, 279, 5754, 13, 578, 9282, 574, 8369, 11, 323, 568, 11953, 198, 26301, 59792, 23724, 927, 813, 6916, 13, 578, 2697, 3838, 574, 90256, 26, 264, 21030, 198, 28172, 3454, 82, 1051, 43335, 304, 279, 33419, 17884, 13, 578, 3703, 574, 39313, 11, 323, 264, 198, 2308, 19393, 304, 264, 4251, 23724, 574, 27890, 21662, 449, 279, 16736, 85273, 198, 1073, 813, 3169, 13, 2684, 1051, 12920, 1234, 279, 12690, 389, 279, 37125, 11, 323, 264, 502, 198, 7908, 389, 279, 18618, 13, 7570, 5124, 3886, 39786, 264, 502, 1427, 315, 44202, 13, 6193, 813, 78140, 27674, 315, 13179, 198, 42, 13, 20802, 2555, 315, 279, 3446, 13, 330, 40, 2846, 539, 12691, 315, 433, 11, 4491, 13, 358, 3077, 2586, 311, 656, 264, 1695, 1690, 2574, 198, 1820, 1566, 1060, 477, 779, 430, 358, 2646, 3463, 358, 1053, 656, 13, 5629, 358, 3952, 40788, 648, 3201, 505, 1077, 1695, 2361, 11, 323, 1306, 198, 9210, 4400, 4024, 1314, 13, 5112, 1070, 1051, 2574, 5108, 389, 75857, 383, 7111, 520, 198, 42, 13, 74884, 13610, 313, 1, 9210, 8967, 810, 20900, 13, 358, 1053, 387, 16089, 422, 499, 8434, 956, 198, 37890, 4205, 922, 433, 520, 18083, 13, 330, 40, 3358, 539, 6604, 315, 433, 11, 315, 3388, 1210, 1102, 574, 1243, 11, 994, 735, 13, 4691, 369, 40788, 648, 11, 430, 4491, 13, 5124, 3886, 596, 31175, 67391, 198, 17106, 373, 810, 10186, 13, 330, 8100, 8434, 956, 2559, 369, 433, 1359, 568, 1071, 13, 330, 8100, 7882, 704, 279, 1938, 358, 24330, 198, 1820, 12295, 50007, 323, 2751, 279, 27374, 1210, 358, 313, 40, 3358, 1935, 499, 198, 412, 1070, 11, 422, 499, 1053, 1093, 311, 1518, 1077, 1210, 735, 13, 559, 4361, 67, 398, 1765, 76, 4147, 430, 40788, 648, 1053, 10932, 311, 1518, 1461, 7636, 11, 1234, 198, 1820, 13463, 13, 330, 40, 8101, 358, 649, 1505, 1077, 1359, 568, 1071, 11, 323, 16392, 505, 279, 2697, 2007, 13, 330, 2746, 499, 313, 333, 499, 649, 2019, 4205, 311, 1520, 757, 704, 11, 28146, 11, 358, 4265, 15763, 433, 13, 5046, 3388, 11, 1364, 31869, 1268, 358, 1097, 16625, 13, 2030, 313, 36046, 422, 499, 1053, 198, 73457, 1077, 430, 279, 6825, 3250, 956, 1440, 313, 1875, 7189, 3358, 656, 682, 358, 649, 1359, 735, 13, 19487, 11, 323, 8272, 279, 1853, 311, 279, 33419, 13, 578, 2697, 33508, 48840, 198, 16514, 1633, 10882, 13, 362, 4251, 11245, 4950, 304, 264, 9309, 11, 264, 10269, 2007, 449, 198, 64, 18327, 3485, 433, 11, 264, 67724, 79781, 11, 323, 264, 52319, 84315, 24330, 279, 198, 3039, 13, 330, 40, 8434, 956, 2559, 369, 433, 1359, 1364, 1071, 5042, 26, 330, 708, 1618, 358, 1097, 13, 2684, 1694, 719, 832, 198, 524, 2196, 1363, 14924, 25, 11208, 374, 10455, 30, 22559, 449, 264, 3254, 3492, 13, 128009, 128006, 78191, 128007, 271]
x2 = [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 35628, 397, 8586, 304, 279, 27394, 568, 1436, 3412, 279, 6671, 369, 264, 4545, 13, 1442, 568, 198, 29177, 636, 76996, 1139, 279, 3838, 11, 279, 2800, 1053, 539, 5030, 13, 48600, 11, 279, 198, 13616, 444, 88, 574, 1633, 6453, 13, 2009, 77638, 818, 11, 323, 568, 8964, 889, 499, 527, 1633, 198, 9336, 11, 13118, 1210, 58072, 956, 279, 6825, 9567, 279, 8732, 10481, 82, 198, 1820, 1023, 1938, 30, 1628, 358, 4510, 279, 35120, 31202, 82, 527, 304, 23383, 315, 53474, 36399, 198, 39428, 264, 1938, 323, 24779, 9096, 323, 11594, 1555, 499, 11, 4491, 13, 1472, 2351, 279, 1455, 5526, 893, 389, 279, 6825, 1210, 17882, 374, 1618, 311, 1518, 198, 22479, 6691, 313, 1875, 1, 47439, 1359, 1071, 76996, 13, 17882, 374, 264, 1633, 2294, 1732, 11, 735, 2637, 779, 387, 198, 10097, 635, 311, 1461, 1210, 7639, 1047, 436, 37588, 520, 279, 5222, 315, 2009, 77638, 818, 596, 7899, 11, 539, 311, 38141, 345, 1073, 3388, 11, 719, 311, 5044, 13, 17586, 904, 10186, 2944, 11, 568, 574, 1203, 304, 198, 95509, 11, 490, 31557, 279, 3224, 19795, 11, 323, 30488, 1461, 27567, 61067, 14518, 1285, 3814, 9135, 330, 791, 4059, 596, 264, 22999, 3245, 11, 4536, 956, 433, 13, 763, 279, 198, 86258, 2533, 9083, 5874, 4024, 1555, 430, 3321, 358, 3077, 1027, 311, 20437, 323, 198, 1445, 0, 735, 13, 17948, 264, 2489, 449, 813, 24981, 6206, 13, 4800, 430, 279, 3245, 1047, 2586, 11, 568, 198, 16514, 16089, 311, 3663, 433, 13, 763, 279, 61363, 11, 813, 11594, 5643, 2840, 13111, 2403, 279, 198, 9471, 13, 330, 32576, 856, 7899, 3952, 499, 1203, 311, 20437, 1210, 5348, 2136, 1047, 58842, 389, 1124, 1578, 11, 3734, 198, 2000, 279, 41630, 37066, 315, 735, 13, 578, 19228, 1828, 6134, 617, 264, 3958, 14464, 315, 11961, 1120, 4871, 279, 198, 2407, 127324, 1210, 358, 3358, 3137, 311, 499, 11, 422, 499, 3358, 198, 47965, 2103, 13, 330, 40, 3077, 1027, 1618, 313, 258, 279, 3363, 11, 358, 3152, 313, 2000, 264, 1060, 13, 4418, 956, 198, 41119, 433, 313, 2356, 77638, 818, 13, 358, 3077, 2751, 264, 2361, 304, 279, 6962, 5274, 11, 57460, 950, 13, 358, 617, 2944, 311, 1781, 358, 2846, 2133, 311, 387, 7882, 198, 455, 13, 3011, 690, 387, 17510, 11, 7344, 17510, 38502, 1210, 17882, 75940, 11, 719, 568, 1766, 912, 26613, 4339, 13, 8442, 264, 961, 315, 1148, 735, 627, 84359, 2751, 311, 1461, 13, 1789, 264, 4545, 568, 574, 1203, 304, 264, 11495, 28913, 11, 323, 420, 198, 1543, 4028, 505, 1461, 313, 275, 574, 539, 92495, 0, 330, 2181, 596, 539, 2653, 990, 11, 323, 433, 596, 6220, 13, 1442, 358, 1304, 264, 16930, 1070, 596, 912, 2324, 198, 71, 8764, 389, 433, 13, 9843, 358, 1903, 264, 1529, 8154, 11, 264, 2305, 477, 1403, 4227, 13, 1102, 2853, 757, 2380, 11441, 704, 315, 856, 1866, 18301, 13, 17882, 596, 7899, 8710, 430, 568, 574, 810, 1109, 9850, 13850, 26, 568, 574, 198, 782, 15456, 398, 7882, 13, 3277, 264, 1060, 198, 42700, 555, 313, 1820, 89862, 1047, 8208, 1523, 11, 323, 19093, 7020, 719, 1148, 499, 1051, 389, 198, 275, 313, 906, 6688, 709, 13, 358, 313, 258, 5651, 584, 2231, 709, 264, 21354, 369, 499, 520, 279, 7926, 13, 358, 198, 42700, 1523, 369, 279, 313, 2000, 279, 3600, 1210, 330, 10267, 433, 4822, 1359, 1071, 735, 13, 30666, 13, 330, 40, 2846, 5710, 439, 3117, 439, 279, 7926, 5900, 345, 3852, 5269, 13, 1628, 11, 369, 32450, 596, 25491, 345, 15357, 956, 387, 14931, 369, 757, 13, 358, 2846, 810, 2262, 291, 1109, 358, 3077, 1027, 369, 264, 1317, 198, 1712, 1210, 578, 5895, 304, 17882, 596, 7899, 574, 7231, 1648, 311, 66510, 13, 8595, 11, 1695, 1283, 402, 729, 11, 893, 11, 358, 1550, 701, 198, 9446, 311, 11477, 11, 323, 358, 3077, 1027, 41442, 922, 433, 3596, 2533, 1210, 3277, 430, 198, 71, 680, 6901, 358, 6688, 709, 13, 2052, 264, 893, 304, 1057, 4913, 706, 374, 264, 3738, 1749, 345, 90153, 313, 6797, 433, 1148, 499, 1093, 11, 313, 438, 10082, 304, 5678, 13, 358, 5675, 856, 198, 726, 91980, 26, 430, 596, 682, 13, 1789, 922, 264, 1060, 358, 574, 198, 15770, 19212, 14931, 369, 7182, 13, 330, 2746, 1475, 46098, 6688, 709, 1606, 568, 5675, 5157, 313, 40, 3077, 1120, 3309, 499, 358, 198, 23770, 701, 5784, 311, 11477, 13, 2684, 574, 1120, 264, 6140, 369, 279, 893, 11, 323, 358, 198, 91453, 856, 25775, 304, 856, 6206, 323, 6818, 433, 13, 735, 13, 16392, 4856, 584, 6751, 323, 94972, 813, 13961, 927, 279, 44952, 13881, 13, 28905, 304, 1450, 11, 568, 14980, 37874, 704, 520, 279, 60243, 32329, 355, 5021, 449, 1202, 27631, 198, 1073, 9958, 13, 12361, 315, 279, 6825, 449, 1202, 11594, 15316, 11, 568, 5602, 279, 3026, 198, 383, 1047, 3967, 323, 6575, 449, 323, 15972, 11, 813, 4885, 889, 12570, 813, 198, 11789, 11, 889, 1047, 10456, 1461, 11, 1690, 315, 1124, 11, 20802, 922, 264, 40907, 198, 99793, 743, 304, 264, 7147, 315, 279, 2362, 7926, 26, 568, 5602, 872, 55349, 12580, 323, 198, 77731, 6548, 13, 1283, 6755, 27567, 1548, 6755, 279, 8579, 23941, 273, 315, 76996, 596, 8679, 439, 1364, 3782, 1139, 279, 2697, 198, 3039, 4920, 1124, 13, 96424, 96860, 1432, 32, 2478, 2919, 1306, 17882, 596, 18324, 315, 735, 2637, 1403, 1455, 13548, 2574, 198, 71, 680, 6901, 311, 76996, 13, 3861, 574, 430, 46993, 4691, 1077, 311, 387, 43165, 315, 16044, 198, 266, 1077, 13306, 13, 3005, 574, 11928, 11, 323, 198, 43068, 1077, 2107, 13, 9393, 1364, 1436, 539, 636, 2162, 430, 3814, 11, 323, 1606, 279, 2697, 3838, 198, 32345, 912, 21186, 11, 1364, 6267, 279, 3754, 311, 1077, 6691, 323, 3288, 264, 5296, 311, 2009, 198, 44, 2303, 818, 1473, 1170, 946, 735, 2637, 313, 40, 1097, 11928, 11, 323, 8871, 374, 389, 856, 2010, 520, 420, 9568, 13, 358, 1097, 439, 198, 64589, 315, 433, 439, 422, 433, 1051, 264, 71812, 11, 323, 439, 422, 358, 1047, 2884, 2555, 311, 198, 5919, 5976, 433, 11, 4619, 315, 1120, 16026, 430, 54108, 358, 4985, 13, 358, 1097, 4477, 198, 576, 389, 279, 44402, 11, 779, 430, 994, 358, 12157, 856, 6548, 358, 1253, 1518, 1102, 13, 358, 1097, 198, 2642, 14152, 1120, 1457, 358, 1097, 7422, 810, 315, 279, 2107, 1109, 315, 1148, 433, 3445, 13, 15668, 5246, 358, 4985, 21818, 1523, 323, 1501, 433, 311, 279, 26741, 13, 358, 4985, 733, 311, 279, 6134, 994, 279, 3814, 29198, 374, 13326, 15038, 11, 323, 198, 413, 682, 2212, 323, 1095, 1124, 1518, 433, 11, 2085, 5605, 264, 3492, 13, 1472, 617, 1027, 1633, 1695, 311, 757, 11, 25237, 735, 13, 1102, 374, 499, 889, 617, 1903, 3284, 198, 576, 23871, 315, 10705, 311, 45427, 13, 358, 1097, 26455, 7182, 311, 387, 1633, 1695, 345, 438, 539, 779, 46604, 11, 323, 311, 3021, 856, 14207, 95054, 8051, 358, 617, 7000, 1457, 13, 9083, 36627, 706, 1120, 40588, 7913, 757, 1455, 47626, 11, 323, 358, 1097, 2771, 8009, 198, 41444, 706, 2225, 73695, 323, 25565, 13, 735, 13, 1766, 279, 5296, 389, 279, 14321, 2007, 994, 568, 2751, 2162, 430, 3814, 11, 323, 198, 7063, 4588, 433, 50007, 311, 1373, 13, 33739, 38678, 3987, 568, 2643, 617, 1047, 430, 198, 1964, 12822, 1053, 5471, 1077, 26586, 568, 7020, 1457, 574, 927, 13, 3161, 279, 198, 9638, 304, 813, 1450, 11, 568, 7731, 555, 813, 2007, 323, 7111, 8469, 1139, 279, 4384, 198, 42820, 13, 2030, 810, 323, 810, 279, 2324, 315, 279, 8952, 1053, 2995, 2177, 1077, 13, 1283, 198, 20370, 76, 4147, 11, 2288, 11, 1633, 559, 4361, 67, 398, 11, 430, 11, 1047, 568, 3596, 1047, 264, 3987, 430, 1364, 198, 46068, 2586, 311, 2512, 369, 1461, 11, 813, 1633, 9546, 304, 279, 2697, 3838, 198, 76, 3748, 660, 2403, 1461, 13, 2684, 574, 7000, 315, 279, 41919, 315, 25768, 280, 383, 574, 2744, 1070, 11, 1093, 45563, 13, 3277, 1364, 9107, 279, 6134, 11, 1364, 2663, 198, 1, 59978, 1, 505, 279, 14321, 13, 1442, 24101, 1550, 539, 4320, 11, 1364, 2663, 1461, 11, 304, 1790, 198, 1820, 1890, 7899, 13, 1283, 1047, 5918, 264, 7147, 315, 19675, 430, 1047, 449, 52784, 1524, 17882, 596, 198, 2827, 31756, 323, 8835, 13, 2030, 52876, 19675, 4131, 1193, 449, 892, 280, 438, 568, 574, 3995, 13, 4800, 323, 1243, 682, 813, 46616, 1589, 26902, 1603, 264, 198, 6519, 290, 430, 11, 994, 568, 76035, 311, 3663, 433, 11, 36014, 1461, 555, 1202, 1633, 8333, 13, 1628, 430, 1938, 682, 813, 43132, 42914, 4024, 1523, 1603, 76996, 596, 6661, 13, 11699, 1633, 198, 1658, 1201, 2136, 323, 37805, 13194, 313, 1962, 430, 568, 1550, 539, 1390, 1077, 37805, 280, 8248, 568, 1589, 4234, 779, 1790, 810, 13, 1283, 22982, 5678, 3663, 1523, 389, 279, 4950, 11, 449, 198, 1820, 5684, 33745, 304, 813, 1450, 13, 76996, 596, 6661, 574, 539, 279, 1193, 832, 568, 4036, 430, 1938, 13, 3277, 11, 304, 198, 2376, 311, 45563, 596, 75400, 11, 568, 16392, 17345, 323, 10235, 369, 14177, 11, 568, 198, 15456, 459, 653, 57228, 35498, 389, 279, 2007, 13, 1102, 574, 505, 7639, 17882, 25, 27567, 1170, 946, 11396, 11672, 56, 4031, 11, 313, 40, 617, 1027, 2133, 2212, 304, 264, 3460, 315, 90409, 682, 1938, 13, 578, 198, 34210, 430, 358, 1193, 6755, 701, 7899, 323, 80987, 5602, 499, 1566, 3814, 706, 198, 28010, 279, 4459, 3245, 1524, 810, 50204, 13, 358, 617, 264, 8430, 315, 61353, 2826, 922, 4560, 311, 1518, 499, 1578, 779, 5246, 13, 358, 2846, 198, 11130, 311, 5201, 701, 513, 9134, 13, 2030, 1070, 527, 1063, 2574, 430, 617, 2751, 198, 998, 387, 14407, 13, 1472, 1071, 1566, 3814, 430, 2574, 1051, 330, 64750, 1, 449, 499, 13, 3234, 499, 1440, 904, 893, 304, 1057, 198, 97235, 889, 706, 539, 30, 1628, 11, 369, 8850, 499, 1781, 358, 656, 539, 1440, 1148, 358, 1097, 198, 83, 30855, 922, 11, 279, 3245, 574, 52844, 291, 704, 520, 279, 3314, 13581, 994, 279, 198, 7998, 315, 279, 21354, 3782, 709, 13, 10846, 44954, 2751, 709, 323, 1071, 25, 330, 38, 306, 42407, 345, 543, 315, 603, 3974, 810, 477, 2753, 304, 9168, 15316, 13, 6914, 1461, 889, 374, 2085, 198, 8890, 3036, 4315, 603, 2571, 279, 1176, 9998, 9135, 358, 3952, 856, 2697, 1841, 323, 23980, 2212, 279, 198, 11389, 19795, 11, 323, 279, 43726, 358, 4024, 279, 810, 55588, 701, 2361, 198, 17106, 373, 13, 358, 2846, 539, 2133, 311, 3350, 904, 5868, 922, 279, 1917, 33921, 3026, 1093, 198, 9514, 11, 8051, 433, 596, 837, 3403, 13, 1472, 3318, 304, 198, 64, 6962, 5274, 11, 1418, 2362, 507, 68070, 5169, 293, 2234, 645, 323, 60884, 11, 323, 358, 14993, 3235, 198, 263, 1148, 358, 9687, 505, 499, 0, 1102, 5097, 25775, 311, 3094, 1523, 505, 279, 99589, 499, 14980, 389, 13, 2100, 433, 596, 198, 1962, 66275, 560, 430, 706, 743, 499, 1523, 1618, 13, 578, 1176, 11, 323, 1888, 11, 374, 369, 499, 311, 733, 1203, 13, 2360, 832, 706, 4529, 701, 2035, 11, 1606, 912, 832, 1436, 656, 279, 990, 13, 2030, 422, 198, 9210, 596, 704, 315, 279, 3488, 11, 313, 438, 1193, 499, 1440, 430, 11, 369, 1193, 499, 1440, 198, 1820, 13363, 11, 313, 1820, 1828, 1888, 3245, 374, 420, 11, 323, 304, 682, 75372, 358, 1304, 279, 198, 82, 43216, 13, 12040, 279, 3314, 40786, 1234, 701, 3118, 836, 11, 323, 994, 499, 3077, 2751, 701, 198, 64686, 11, 2586, 304, 449, 757, 13, 358, 3358, 387, 3794, 264, 198, 89719, 14254, 810, 1109, 358, 3041, 13, 1102, 374, 264, 22999, 2144, 430, 264, 893, 889, 374, 11112, 653, 57755, 43629, 922, 198, 65762, 374, 3629, 279, 13836, 315, 16044, 311, 1023, 3026, 13, 578, 14992, 17882, 345, 89894, 813, 72974, 34504, 323, 539, 2288, 14572, 16252, 398, 11, 574, 3339, 459, 198, 26133, 430, 8967, 813, 17139, 56528, 11, 323, 3815, 433, 26085, 3725, 11, 449, 813, 198, 58331, 1825, 13, 1102, 574, 1093, 7639, 311, 1304, 1778, 459, 3085, 11, 1093, 1461, 311, 1304, 433, 198, 300, 422, 568, 1051, 10371, 264, 4799, 323, 539, 2389, 14782, 832, 13, 2030, 279, 3085, 2163, 198, 40617, 653, 4307, 291, 13, 1283, 1047, 47712, 5678, 304, 279, 8335, 11, 323, 1766, 5678, 198, 53757, 287, 13, 2360, 21354, 389, 279, 7926, 7147, 1436, 2349, 430, 13, 1628, 994, 345, 5185, 430, 3814, 11, 17882, 1766, 1461, 389, 279, 44952, 323, 3779, 14638, 311, 198, 14819, 11, 279, 6671, 14958, 35957, 13, 1283, 15393, 1202, 3987, 33028, 198, 9493, 735, 13, 326, 9381, 1139, 70205, 950, 28485, 13, 330, 40, 2846, 539, 11112, 29948, 1405, 358, 1097, 11, 499, 1440, 11, 7639, 1359, 568, 1071, 13, 330, 40, 3077, 198, 93613, 2380, 42120, 11012, 323, 264, 3070, 315, 83190, 420, 7474, 11, 9087, 198, 998, 3197, 264, 490, 46651, 2933, 11, 39061, 304, 27397, 7147, 77819, 369, 279, 3130, 1120, 198, 42450, 11, 313, 23770, 499, 5406, 433, 30, 313, 438, 8040, 264, 8334, 42070, 449, 264, 5041, 198, 9210, 62990, 2212, 279, 16120, 1093, 264, 4349, 645, 59904, 2212, 264, 12786, 396, 9135, 330, 2746, 499, 2351, 2133, 311, 387, 70946, 313, 1875, 46240, 25237, 12637, 1359, 1071, 735, 13, 30666, 11, 330, 333, 358, 1047, 912, 5647, 315, 28485, 11, 358, 1288, 198, 3427, 50007, 311, 45427, 11, 2543, 389, 279, 6962, 11, 323, 1304, 264, 357, 531, 20857, 20396, 198, 18614, 64482, 13, 3296, 279, 1648, 11, 430, 596, 2555, 358, 29695, 9135, 22395, 856, 1023, 7640, 11, 358, 53221, 279, 1370, 9761, 369, 198, 64465, 3177, 13, 578, 42447, 4791, 15502, 25283, 1063, 25396, 41476, 439, 13306, 198, 53430, 82, 11, 323, 313, 1875, 92493, 16392, 323, 1344, 2234, 813, 36213, 1139, 279, 16763, 13, 735, 13, 16392, 449, 1461, 11, 323, 682, 279, 56089, 8430, 315, 279, 7274, 574, 198, 52905, 9988, 1139, 813, 1566, 2478, 4339, 13, 330, 40, 2846, 539, 439, 653, 911, 21508, 439, 499, 1781, 11, 7639, 1359, 568, 1071, 13, 330, 40, 313, 9514, 3077, 9087, 198, 64, 2763, 13, 358, 2846, 439, 1664, 1022, 439, 358, 23528, 311, 387, 11, 323, 198, 58234, 13, 17882, 596, 16907, 8622, 16348, 488, 2231, 735, 13, 304, 264, 22999, 2361, 313, 2414, 1461, 345, 300, 433, 1051, 11, 449, 264, 18255, 68078, 13, 76996, 596, 26875, 4225, 266, 4090, 369, 198, 1820, 3995, 46098, 574, 7982, 13, 1628, 1405, 1603, 198, 383, 2643, 617, 6612, 35516, 304, 2133, 311, 279, 3160, 315, 10163, 1077, 11, 1457, 198, 26301, 6206, 1051, 17791, 13, 4497, 1109, 3131, 568, 1047, 198, 83261, 76996, 1203, 311, 279, 8952, 304, 813, 1841, 13, 2009, 77638, 818, 11, 48270, 5795, 520, 198, 30115, 2543, 11, 1766, 5678, 13176, 1403, 27548, 11, 832, 719, 2697, 2731, 198, 54895, 279, 1023, 13, 578, 31017, 2643, 1629, 264, 23583, 3388, 11, 13696, 304, 198, 5730, 8950, 313, 64, 1060, 315, 23871, 369, 1077, 11, 323, 1243, 1148, 11103, 449, 198, 6102, 11, 439, 568, 7020, 1461, 11, 1053, 40605, 3152, 25, 40320, 826, 3201, 11, 79157, 1285, 198, 4310, 311, 1077, 11, 4225, 27900, 1385, 11, 58701, 13, 2582, 11, 433, 2643, 387, 2753, 6129, 719, 198, 60301, 18813, 43251, 369, 1077, 13, 7639, 2643, 2571, 28930, 311, 279, 29592, 345, 79, 1759, 361, 1077, 369, 264, 892, 11, 313, 42, 13, 1047, 3970, 1461, 656, 420, 11, 313, 438, 1243, 11, 7982, 198, 83, 2757, 11, 2349, 311, 1063, 502, 33464, 13, 763, 3060, 1162, 11, 568, 1436, 1193, 3868, 198, 438, 3821, 11, 12459, 813, 4851, 704, 2391, 279, 1317, 59938, 994, 24101, 1373, 198, 1964, 330, 44653, 61399, 1, 50007, 323, 568, 7731, 7636, 449, 813, 13961, 389, 279, 198, 13616, 444, 88, 13, 76996, 4024, 389, 3814, 14523, 20193, 1306, 1077, 26586, 13, 2052, 315, 1077, 198, 1382, 398, 3995, 2324, 1047, 1027, 18255, 1139, 1403, 5596, 25, 1938, 11, 994, 832, 198, 51908, 477, 6575, 11, 323, 3814, 11, 994, 832, 46498, 13, 4800, 1364, 574, 48984, 311, 198, 64, 1373, 4345, 479, 25, 832, 6575, 304, 279, 3814, 323, 46498, 304, 279, 1938, 13, 2468, 279, 842, 315, 1077, 1176, 3814, 1934, 76996, 198, 17249, 1148, 1364, 1436, 6227, 315, 264, 2697, 33487, 315, 83048, 596, 13, 3005, 198, 17249, 433, 311, 279, 842, 315, 1077, 4689, 1934, 11, 902, 574, 311, 279, 2515, 430, 198, 73826, 1047, 1027, 11594, 2391, 279, 3814, 3734, 279, 12818, 13, 330, 3112, 1587, 433, 539, 2873, 2653, 311, 499, 345, 415, 3277, 682, 279, 13180, 374, 2867, 323, 6437, 345, 415, 1628, 358, 1288, 1093, 779, 1790, 311, 1514, 345, 415, 2057, 617, 311, 733, 311, 4950, 555, 1938, 7673, 578, 1938, 18328, 7077, 389, 279, 1934, 11, 323, 574, 5115, 26681, 1534, 13, 330, 2746, 279, 3814, 37465, 527, 311, 8493, 872, 892, 3339, 709, 32349, 1359, 1364, 198, 84359, 5425, 398, 11, 330, 906, 4265, 2731, 2349, 420, 8952, 1139, 264, 3995, 23628, 1270, 325, 1083, 661, 13, 1442, 1364, 6944, 311, 29011, 922, 279, 12248, 304, 279, 8761, 11, 1364, 198, 5562, 656, 779, 304, 6300, 1376, 1210, 330, 40, 1541, 956, 1781, 1364, 1903, 433, 709, 1359, 1071, 279, 11452, 11, 4560, 539, 311, 15648, 13, 330, 40, 3077, 6755, 2555, 1093, 433, 15038, 11, 323, 11, 1148, 449, 279, 8798, 323, 279, 198, 53318, 315, 9629, 11, 358, 1541, 956, 1518, 1268, 904, 315, 1124, 636, 904, 6212, 1210, 2030, 11, 1606, 26434, 2011, 387, 13468, 11, 1364, 6267, 389, 279, 21818, 279, 198, 78191, 11953, 2212, 25, 330, 5618, 9502, 3814, 6821, 304, 61801, 1210, 3005, 259, 26902, 1139, 1077, 3428, 4950, 520, 11888, 297, 63510, 198, 258, 279, 6693, 11, 1884, 2919, 11, 449, 1077, 70960, 7013, 63266, 20716, 4591, 1523, 198, 1964, 1203, 323, 1077, 34296, 1071, 11, 323, 7214, 1077, 4642, 3995, 4059, 198, 28611, 449, 5448, 313, 19875, 483, 596, 13306, 11, 2999, 13, 7639, 12579, 279, 6134, 315, 1077, 198, 820, 26741, 323, 1364, 539, 1070, 11, 13142, 313, 17206, 40788, 648, 11, 6832, 3446, 574, 1457, 279, 198, 82, 42725, 315, 279, 6825, 13, 362, 2478, 4038, 1603, 1364, 1053, 539, 617, 42777, 198, 998, 1781, 315, 40788, 648, 13, 3005, 1053, 617, 22311, 1077, 1139, 279, 4363, 315, 198, 58174, 19101, 1474, 592, 15548, 456, 13, 2030, 279, 6825, 596, 45593, 1051, 539, 10168, 198, 68753, 3520, 596, 11555, 1457, 13, 3005, 87420, 927, 40788, 648, 264, 2294, 3568, 11, 323, 927, 198, 87643, 323, 1077, 3169, 13, 1952, 1077, 1176, 3814, 389, 14523, 11, 264, 3828, 1047, 1027, 7263, 304, 505, 279, 17569, 13, 3005, 1047, 4529, 264, 21109, 313, 77, 43440, 7020, 1120, 1148, 13, 3277, 279, 2655, 288, 1047, 198, 376, 1142, 311, 1505, 704, 11, 1364, 1047, 1193, 1071, 25, 330, 3923, 596, 279, 1005, 7673, 1884, 58302, 994, 1364, 1436, 539, 636, 198, 998, 6212, 13, 9029, 1051, 3169, 313, 5794, 1051, 3169, 11, 2216, 11, 313, 438, 3686, 11, 369, 1063, 198, 20489, 477, 1023, 11, 1884, 2574, 1047, 311, 387, 13, 4740, 264, 892, 76996, 1053, 656, 3059, 5052, 3725, 13, 2030, 555, 2380, 297, 63510, 1364, 574, 198, 33222, 709, 323, 32623, 13, 68502, 315, 198, 26894, 6267, 42902, 82, 2212, 1077, 6548, 323, 7577, 1063, 315, 1077, 10107, 1933, 13, 28232, 2380, 323, 3116, 297, 63510, 304, 279, 6693, 1364, 574, 43206, 389, 198, 67, 20850, 555, 264, 4832, 52819, 315, 6212, 13, 2684, 574, 264, 16750, 369, 21811, 389, 198, 67, 20850, 13, 578, 2362, 3814, 3821, 1543, 1047, 264, 1648, 315, 64802, 709, 389, 832, 16387, 7113, 13, 578, 3814, 37465, 37287, 814, 2643, 5043, 268, 264, 29519, 389, 1461, 0, 48600, 11, 520, 3116, 3782, 4216, 1474, 80675, 20472, 26, 430, 436, 37588, 1077, 13, 1628, 198, 10924, 430, 3782, 279, 1206, 1683, 315, 4216, 14403, 2695, 351, 2439, 323, 279, 16392, 82757, 315, 198, 67, 6513, 927, 279, 77395, 13, 94666, 304, 279, 3814, 11, 3131, 520, 87117, 323, 1578, 9017, 198, 67, 6513, 11, 1364, 54244, 3831, 3776, 11033, 13, 2030, 1306, 264, 2046, 477, 1403, 1077, 50581, 198, 52898, 41398, 259, 2784, 439, 264, 925, 13, 6385, 8216, 574, 304, 264, 2678, 3130, 3345, 311, 1077, 2380, 78251, 13, 2030, 1364, 7731, 198, 1225, 2697, 11, 439, 264, 5030, 315, 2144, 13, 6385, 12014, 574, 8987, 389, 1077, 280, 32158, 1903, 21420, 20101, 13, 578, 3389, 7474, 22178, 1051, 5052, 1285, 11, 34653, 819, 280, 1820, 6453, 6901, 78251, 41398, 3201, 1093, 76304, 82, 505, 279, 5213, 3177, 3221, 198, 1820, 6134, 13, 1628, 505, 704, 315, 1521, 76304, 82, 3782, 6896, 69733, 24196, 11, 83372, 198, 31061, 3808, 11, 279, 87211, 315, 264, 10747, 389, 264, 96803, 11, 902, 574, 279, 8450, 315, 198, 339, 874, 13, 2057, 1124, 11, 8530, 1120, 198, 64, 2697, 76656, 449, 892, 323, 1790, 2532, 11, 279, 87211, 10747, 8967, 539, 779, 198, 59178, 50690, 439, 91904, 13, 330, 8161, 956, 7940, 1093, 430, 11, 1716, 26, 814, 2351, 539, 66302, 291, 11, 499, 1440, 1210, 330, 4071, 422, 499, 617, 264, 34653, 323, 527, 98208, 313, 1875, 1, 1016, 874, 88, 4400, 0, 330, 12487, 1359, 76996, 1053, 2019, 11, 16448, 594, 9887, 11, 330, 20670, 527, 2133, 311, 1518, 757, 1210, 22547, 1870, 279, 9191, 7724, 5602, 430, 1364, 1053, 539, 3665, 11937, 13, 2435, 198, 44336, 1077, 1633, 1790, 11, 323, 814, 11, 2288, 11, 1047, 3940, 304, 449, 10032, 7693, 198, 438, 28682, 6206, 26, 719, 279, 16579, 323, 832, 18651, 315, 872, 2532, 198, 32345, 61508, 1124, 9235, 13, 2435, 1051, 11297, 11, 7155, 62166, 11, 4062, 100051, 198, 55377, 1572, 11, 3815, 872, 1888, 11, 315, 3388, 11, 719, 61469, 505, 76996, 304, 430, 198, 50828, 2532, 574, 315, 279, 4059, 11, 1418, 11074, 574, 315, 279, 4851, 13, 2057, 1124, 345, 92724, 574, 264, 3245, 311, 387, 12715, 389, 264, 1934, 26, 311, 76996, 11, 433, 574, 5439, 198, 263, 279, 29679, 315, 1077, 13836, 13, 22770, 22983, 36627, 4024, 389, 3814, 14523, 520, 279, 1890, 892, 313, 1964, 1566, 3814, 198, 8095, 11, 439, 433, 574, 76996, 596, 1176, 13, 3005, 1047, 198, 14215, 315, 279, 2380, 78251, 389, 279, 6558, 1120, 3770, 76996, 11, 323, 315, 279, 198, 1637, 1139, 902, 682, 13147, 5157, 1051, 4529, 13, 1102, 574, 264, 5107, 198, 8095, 11, 8530, 279, 1455, 5107, 304, 279, 3838, 13, 2522, 8362, 989, 264, 3814, 4024, 198, 1729, 2085, 1202, 33622, 477, 51632, 1162, 13, 31137, 258, 6751, 11, 279, 13147, 26741, 198, 32345, 1202, 1866, 3814, 29198, 13, 7984, 660, 198, 54803, 811, 323, 17563, 1047, 79266, 279, 4967, 35789, 13, 22770, 22983, 11, 2728, 198, 4429, 14523, 11, 16632, 70756, 1077, 28004, 13, 330, 40, 3077, 2744, 1047, 2574, 5128, 2653, 1618, 1359, 1364, 29786, 27851, 13, 330, 4599, 358, 733, 704, 11, 358, 3358, 3060, 387, 40883, 3403, 311, 1629, 264, 4459, 8952, 198, 15698, 71, 6601, 11, 477, 358, 3358, 387, 11953, 704, 7693, 1176, 1210, 3005, 7020, 1077, 2731, 1109, 1364, 7020, 198, 1820, 1023, 37465, 13, 15344, 69074, 1051, 15320, 40986, 323, 9455, 198, 1964, 520, 264, 4814, 13, 9843, 520, 3325, 1475, 3814, 11, 9083, 36627, 1053, 6865, 264, 198, 3594, 108593, 505, 279, 1203, 62862, 430, 8599, 279, 1403, 27035, 11, 323, 345, 9738, 704, 11, 1053, 1518, 76996, 596, 74820, 3663, 323, 10284, 14425, 11059, 2107, 198, 65, 2518, 927, 279, 37649, 3880, 607, 13, 330, 40, 2846, 36075, 3725, 14931, 311, 20753, 499, 1359, 1364, 1053, 2019, 11, 330, 8248, 2100, 9976, 12, 4516, 198, 55767, 956, 617, 264, 34653, 9061, 5233, 477, 11, 330, 40, 3077, 264, 5333, 1618, 889, 41716, 1077, 198, 83543, 1210, 5112, 1053, 1833, 11295, 4860, 323, 18813, 11295, 11503, 13, 24191, 439, 22770, 22983, 99733, 323, 38569, 279, 3828, 32115, 11, 433, 2646, 198, 14310, 8293, 311, 1077, 311, 26122, 1077, 13291, 13, 19292, 279, 46968, 889, 2567, 198, 1820, 2294, 3335, 690, 2231, 430, 311, 1077, 6807, 13, 76996, 5602, 1077, 1176, 4648, 20193, 1306, 1364, 4024, 389, 3814, 14523, 13, 1102, 574, 198, 1820, 1455, 17936, 3217, 315, 682, 1077, 2324, 26, 323, 3686, 11, 439, 4648, 5900, 11, 433, 198, 16514, 11594, 3403, 13, 2100, 53722, 574, 433, 430, 76996, 11, 449, 735, 3238, 82, 2697, 3821, 198, 258, 1450, 11, 574, 539, 2771, 7041, 994, 433, 7077, 13, 578, 3177, 574, 1633, 5213, 198, 30998, 485, 279, 2697, 4264, 13, 3861, 4545, 279, 11071, 574, 934, 85040, 10284, 198, 8154, 279, 14993, 369, 11745, 11, 279, 1828, 433, 574, 2103, 13, 3011, 2324, 11, 779, 4754, 11, 779, 28040, 264, 198, 1626, 11, 1436, 842, 779, 5335, 8129, 13610, 11, 430, 279, 1317, 8209, 1288, 30754, 198, 33222, 304, 420, 61510, 2987, 313, 275, 9508, 311, 1077, 430, 1364, 1436, 539, 2559, 198, 275, 13, 18152, 311, 682, 1077, 1023, 502, 5435, 315, 5496, 574, 420, 832, 315, 23069, 13, 3005, 1903, 21294, 11, 315, 3388, 11, 902, 279, 47626, 37465, 29695, 311, 198, 11998, 313, 18275, 1354, 2163, 922, 11, 6103, 389, 1077, 7576, 13, 3005, 97139, 291, 1077, 198, 700, 76, 21037, 304, 4106, 3090, 832, 3814, 11, 323, 89815, 459, 958, 818, 555, 11889, 198, 40617, 3492, 430, 10455, 84164, 556, 596, 9499, 574, 264, 7895, 323, 5899, 12628, 13, 3005, 1095, 264, 1624, 404, 1245, 8893, 12731, 505, 279, 26741, 2500, 3814, 323, 733, 198, 1334, 1570, 1523, 279, 4027, 12, 13295, 1603, 1364, 11352, 1148, 1047, 7077, 0, 5112, 1364, 39575, 11937, 555, 16706, 1523, 279, 11245, 62862, 323, 198, 81088, 279, 91740, 1203, 3254, 37240, 13, 1789, 46993, 596, 13306, 279, 6825, 22982, 1022, 1202, 1377, 370, 69260, 323, 19655, 198, 64, 13306, 60718, 13, 763, 279, 7314, 433, 574, 9850, 13850, 922, 1063, 315, 279, 198, 15216, 13, 330, 2127, 1384, 1251, 505, 279, 3838, 6134, 311, 279, 44962, 11046, 11, 323, 264, 79680, 9135, 35120, 31202, 11, 889, 574, 9455, 24981, 14740, 520, 279, 52290, 89, 198, 7830, 13, 330, 3112, 2500, 1384, 1251, 520, 279, 8993, 11, 449, 264, 2579, 28954, 9135, 35120, 31202, 1047, 11721, 2162, 323, 574, 3339, 709, 802, 265, 1590, 315, 2800, 323, 198, 265, 38475, 13, 330, 10445, 656, 814, 2610, 364, 336, 422, 814, 1541, 956, 7095, 364, 336, 7673, 2030, 279, 6420, 315, 279, 86219, 1047, 1027, 34707, 13, 1102, 32230, 311, 198, 40617, 1690, 2574, 430, 1051, 2731, 25565, 13, 1283, 16392, 323, 1156, 363, 839, 520, 813, 198, 48841, 13, 330, 2675, 3371, 32980, 2555, 369, 757, 1359, 568, 4224, 277, 839, 13, 330, 2675, 3371, 1461, 994, 568, 198, 325, 288, 813, 7126, 11689, 1523, 8761, 11, 323, 568, 274, 1468, 258, 6, 709, 1070, 7636, 389, 198, 9210, 35528, 11, 358, 1390, 1461, 311, 3009, 323, 3820, 757, 709, 994, 358, 65350, 1461, 13, 2206, 198, 81749, 11, 1418, 856, 4538, 2064, 6572, 2212, 304, 264, 1841, 0, 330, 2675, 1095, 757, 6865, 315, 1461, 5754, 2902, 788, 258, 518, 323, 198, 40, 3358, 5622, 1461, 9135, 578, 13306, 574, 311, 387, 520, 4330, 297, 63510, 13, 1115, 11, 304, 5196, 11, 711, 1142, 682, 198, 48126, 6055, 315, 279, 6825, 11, 902, 574, 3060, 12502, 304, 279, 1633, 4216, 198, 57902, 1251, 520, 279, 16879, 8993, 477, 520, 8223, 297, 63510, 304, 279, 11714, 520, 198, 1820, 86464, 13, 2684, 574, 2555, 54317, 922, 4330, 297, 63510, 13, 1102, 1047, 264, 55641, 8430, 430, 8530, 1778, 264, 198, 5730, 8950, 574, 539, 5115, 5897, 13, 578, 3488, 315, 1148, 311, 10051, 6244, 11, 369, 279, 3026, 11, 459, 55349, 832, 13, 3279, 198, 417, 59100, 291, 459, 2362, 3776, 13091, 377, 23283, 266, 323, 1047, 264, 330, 53, 1, 315, 3776, 6730, 1347, 292, 743, 198, 258, 279, 28705, 13, 45236, 11, 279, 10707, 3913, 11, 49959, 264, 4018, 14075, 11, 323, 11021, 264, 198, 943, 49136, 311, 10051, 449, 433, 13, 578, 47842, 9976, 1773, 3635, 2363, 8479, 889, 84569, 520, 198, 26353, 42, 5633, 518, 323, 889, 11, 555, 2944, 315, 813, 95623, 290, 11, 574, 73170, 48080, 315, 198, 1820, 28361, 2212, 1461, 11, 28670, 264, 46664, 8679, 1355, 3159, 11, 323, 6646, 198, 40617, 726, 311, 279, 842, 315, 813, 2919, 279, 1193, 10489, 1651, 2757, 893, 304, 279, 198, 65294, 13, 578, 14992, 17882, 574, 311, 387, 832, 315, 279, 603, 71, 388, 13, 3277, 279, 32594, 3782, 198, 412, 449, 279, 4756, 1160, 323, 420, 574, 11352, 11, 439, 1664, 439, 430, 198, 68753, 3520, 574, 279, 43165, 315, 16044, 11, 1070, 574, 264, 12742, 934, 1553, 1555, 279, 198, 71127, 4967, 35789, 13, 362, 47027, 261, 574, 19144, 311, 1505, 704, 198, 4581, 24553, 82, 13, 1102, 574, 279, 1938, 315, 279, 13306, 1243, 11, 323, 76996, 11, 889, 1047, 198, 1962, 1027, 311, 4950, 520, 682, 11, 574, 11961, 304, 264, 40798, 3321, 304, 279, 98632, 10843, 198, 28192, 327, 11, 46479, 1077, 7013, 13, 330, 40, 313, 40, 1120, 5895, 1359, 1364, 1071, 11, 330, 333, 499, 1053, 1095, 1063, 315, 279, 7724, 2586, 198, 258, 311, 1518, 499, 994, 499, 2351, 26435, 7673, 330, 2181, 596, 1384, 3725, 57399, 11, 4536, 956, 433, 30, 330, 11787, 499, 2133, 311, 4321, 1523, 279, 60779, 449, 1461, 7673, 2435, 1047, 264, 89955, 1566, 3814, 11, 719, 315, 3388, 358, 574, 539, 198, 19041, 13, 578, 47027, 261, 1047, 1027, 42075, 311, 1505, 704, 1023, 2574, 26, 779, 1364, 743, 198, 998, 990, 449, 264, 8571, 520, 76996, 596, 7013, 13, 330, 1548, 596, 1384, 3725, 1695, 31348, 11, 4536, 956, 568, 7673, 3005, 574, 539, 48080, 315, 279, 5528, 315, 279, 2978, 13, 1442, 198, 576, 3828, 574, 53226, 1077, 27567, 7189, 3358, 617, 311, 1781, 430, 927, 1359, 1364, 1071, 11, 449, 264, 2840, 396, 315, 95046, 304, 198, 1964, 6548, 13, 330, 4599, 499, 1440, 264, 1732, 50136, 1664, 11, 499, 20781, 1440, 3508, 198, 383, 596, 1695, 31348, 477, 539, 1210, 330, 40, 23289, 1359, 1071, 279, 47027, 261, 11, 4401, 279, 1317, 69864, 315, 76996, 596, 198, 51729, 1555, 1077, 19779, 11, 330, 9210, 994, 499, 527, 520, 2162, 499, 1518, 1461, 3629, 1210, 76996, 2751, 1022, 279, 3321, 1355, 484, 11, 323, 11, 4737, 279, 47027, 261, 37163, 398, 555, 198, 1820, 28004, 11, 17011, 1077, 9017, 279, 6134, 13, 330, 2675, 733, 1203, 311, 279, 7724, 1359, 1364, 1071, 11, 330, 438, 3371, 1124, 311, 2586, 304, 323, 1518, 198, 2727, 994, 358, 1097, 26435, 11, 323, 3371, 1124, 420, 25, 358, 1541, 956, 1440, 3508, 358, 1097, 311, 198, 19599, 1523, 279, 60779, 449, 2999, 13, 3005, 83198, 279, 47027, 261, 704, 1139, 279, 14321, 323, 16447, 279, 6134, 4920, 198, 1964, 13, 3011, 1984, 304, 1202, 49017, 8813, 22770, 22983, 36627, 13, 3005, 11, 2288, 11, 1047, 539, 46498, 2391, 279, 1938, 13, 3277, 279, 47027, 261, 889, 198, 32345, 7263, 1077, 279, 1934, 1047, 8208, 704, 11, 1364, 11203, 304, 1077, 1317, 4251, 198, 9471, 2427, 785, 11, 6206, 57220, 7018, 1234, 1077, 2010, 11, 323, 45135, 520, 279, 35684, 12970, 198, 346, 8138, 315, 1077, 2697, 3130, 13, 3005, 5602, 1070, 76996, 304, 1077, 4251, 8679, 2133, 1523, 279, 60779, 315, 279, 198, 65294, 26, 1364, 5602, 279, 1912, 2212, 279, 51943, 26, 323, 11, 439, 22225, 439, 1364, 11203, 198, 19041, 11, 1364, 7020, 430, 7639, 17882, 596, 6548, 1053, 387, 11, 539, 389, 279, 42447, 11, 719, 198, 263, 279, 3828, 889, 14980, 30488, 1077, 13, 578, 22999, 3245, 574, 430, 22770, 22983, 6612, 430, 1364, 1436, 3009, 279, 13306, 198, 333, 1364, 4934, 311, 13, 3005, 4265, 7077, 389, 264, 2766, 315, 2038, 313, 35676, 264, 13306, 198, 32345, 1027, 10717, 369, 2753, 13, 1102, 4856, 44898, 1077, 311, 1781, 315, 23351, 198, 1820, 13306, 11, 779, 430, 76996, 323, 7639, 1053, 539, 4321, 1523, 279, 60779, 198, 998, 3522, 13, 2684, 3782, 11, 520, 1566, 11, 459, 6596, 1603, 279, 13306, 11, 264, 326, 620, 304, 279, 34653, 819, 198, 33078, 315, 279, 3766, 2305, 13, 763, 279, 52290, 89, 198, 74, 7606, 11, 59872, 315, 25485, 11, 366, 55319, 1227, 29, 3868, 388, 11, 10054, 12, 47100, 1949, 60719, 11, 323, 18083, 13, 763, 279, 74721, 11, 304, 279, 4219, 315, 264, 198, 15470, 11, 1603, 264, 27306, 17203, 902, 1047, 1027, 11953, 50007, 369, 1077, 198, 68244, 275, 11, 7731, 11, 389, 420, 1077, 1938, 315, 2919, 11, 279, 42447, 13, 2052, 279, 2132, 3446, 198, 32345, 1027, 10235, 369, 15051, 323, 18911, 13, 8956, 1705, 1051, 2103, 13326, 304, 279, 3130, 3770, 13, 426, 18746, 98250, 1051, 93858, 198, 263, 279, 2697, 62862, 11, 58218, 927, 520, 1855, 502, 10264, 315, 279, 29519, 323, 198, 74826, 6821, 311, 46993, 1555, 279, 8036, 6134, 25, 27567, 1, 14364, 23162, 3830, 11, 46993, 13, 3639, 690, 499, 198, 2099, 656, 449, 1124, 682, 7673, 5810, 596, 2500, 315, 279, 19228, 889, 6944, 311, 1518, 1268, 499, 198, 7349, 13, 3234, 2019, 499, 649, 956, 617, 904, 15613, 1457, 1210, 46993, 7731, 7636, 304, 279, 4219, 315, 1077, 11071, 13, 578, 61365, 98250, 1047, 1027, 198, 267, 944, 398, 37913, 311, 2586, 1139, 1077, 3130, 13, 330, 40, 9167, 956, 1047, 264, 6140, 311, 1781, 369, 264, 2305, 1359, 1364, 1071, 13, 330, 3112, 358, 3077, 2751, 198, 15031, 2574, 358, 3077, 2751, 311, 1781, 704, 1210, 2030, 11, 994, 76996, 3782, 11, 1364, 3288, 369, 1077, 13, 76996, 1766, 1077, 11961, 389, 264, 198, 267, 3168, 10716, 11, 304, 1077, 13306, 65510, 11, 449, 1077, 59355, 9041, 704, 389, 264, 2678, 198, 2752, 13, 1628, 11, 1306, 76996, 1047, 59334, 1077, 25, 27567, 7189, 3077, 264, 1695, 4059, 539, 311, 656, 433, 1210, 330, 2675, 2351, 19781, 323, 23418, 11, 430, 596, 682, 1210, 2030, 430, 4536, 956, 1148, 596, 5076, 449, 757, 13, 32445, 430, 59355, 198, 15031, 2035, 323, 2503, 1523, 1210, 46993, 574, 38048, 29465, 3640, 11, 264, 1633, 36301, 5916, 13, 76996, 3463, 198, 1347, 3422, 1288, 387, 4856, 28639, 13, 2030, 1234, 1077, 6548, 1051, 5238, 430, 76996, 198, 32345, 2646, 3970, 1070, 1603, 13, 330, 40, 2846, 539, 2133, 311, 387, 46337, 11, 76996, 13, 358, 3358, 733, 1555, 449, 433, 11, 315, 198, 12120, 13, 1102, 1053, 2231, 90411, 304, 1077, 25165, 422, 358, 1903, 264, 6237, 1457, 1210, 330, 20577, 1195, 6688, 813, 49683, 14177, 520, 279, 14438, 10349, 1566, 3814, 13, 94192, 2663, 7126, 709, 311, 11477, 323, 198, 84359, 430, 42216, 1047, 94972, 264, 16893, 315, 13378, 1139, 279, 27374, 13, 1283, 12775, 956, 198, 82850, 1618, 311, 11477, 1210, 1628, 439, 369, 279, 1023, 313, 66372, 433, 5828, 956, 42216, 889, 1550, 198, 275, 1210, 14853, 4038, 1603, 11, 8530, 11, 76996, 1436, 539, 617, 6981, 291, 1077, 26, 719, 198, 28956, 4038, 1047, 1903, 264, 2349, 304, 76996, 13, 578, 4684, 18680, 19219, 380, 4108, 198, 1073, 1077, 3828, 5812, 912, 5129, 19089, 369, 8206, 13, 3005, 2231, 1077, 11977, 2212, 198, 19875, 483, 596, 28004, 13, 330, 32, 893, 889, 21662, 374, 264, 11102, 312, 291, 1359, 1071, 46993, 13, 330, 4897, 596, 1148, 358, 2846, 198, 9738, 311, 32918, 323, 16025, 389, 279, 2800, 315, 856, 2324, 313, 64, 11102, 312, 291, 13, 3005, 2751, 709, 6288, 11, 323, 11, 28848, 1077, 1317, 78184, 5542, 4028, 279, 6558, 345, 53533, 291, 279, 6134, 13, 5112, 505, 4871, 1077, 45011, 425, 1364, 7263, 704, 323, 5762, 198, 998, 76996, 264, 6661, 13, 1102, 574, 1633, 2875, 26, 76996, 1373, 433, 520, 264, 34522, 25, 27567, 27264, 701, 3938, 10177, 422, 568, 8964, 264, 3828, 520, 220, 11702, 11556, 17569, 13, 14853, 4038, 1603, 11, 279, 17569, 1053, 617, 8967, 4400, 311, 76996, 13, 58795, 15187, 76996, 7020, 889, 279, 3828, 520, 220, 11702, 11556, 17569, 574, 13, 578, 198, 19716, 1364, 5762, 304, 1077, 1450, 574, 8952, 5684, 449, 279, 14836, 21830, 1022, 13, 578, 4459, 274, 541, 307, 3446, 11203, 1603, 1077, 25, 32171, 56310, 11, 449, 1077, 15792, 3663, 198, 438, 61243, 7013, 11, 323, 279, 17222, 389, 279, 6558, 315, 279, 26741, 30488, 1077, 0, 3861, 315, 279, 61365, 98250, 270, 61292, 65300, 389, 279, 6134, 4994, 13, 330, 14364, 9249, 29062, 1359, 1364, 2663, 12304, 398, 1555, 279, 6134, 13, 330, 2675, 1518, 1359, 46993, 1071, 33201, 6751, 13, 330, 40, 617, 4036, 2500, 9249, 198, 95857, 11, 323, 42216, 374, 63891, 0, 358, 3077, 2751, 311, 733, 1555, 449, 433, 11, 358, 198, 13066, 2972, 13, 578, 1193, 6811, 1990, 757, 323, 1023, 61365, 374, 430, 358, 1440, 198, 12840, 358, 2846, 3794, 13, 330, 2181, 596, 2288, 3389, 311, 656, 4205, 775, 13, 358, 1097, 539, 2133, 311, 3041, 420, 198, 37569, 5812, 4205, 311, 3137, 922, 1210, 3005, 13061, 709, 1077, 59355, 323, 743, 279, 22760, 295, 389, 1077, 2010, 13, 76996, 14980, 198, 4291, 279, 6661, 304, 1077, 6206, 3238, 82, 11503, 311, 1077, 4106, 3488, 198, 32345, 1027, 420, 25, 27567, 25938, 374, 912, 5647, 304, 3411, 1203, 7389, 433, 8779, 603, 311, 1427, 8469, 13, 3639, 701, 2697, 3828, 315, 279, 26741, 706, 1027, 374, 539, 779, 3062, 439, 1148, 198, 32158, 374, 2133, 311, 387, 1210, 330, 13461, 49780, 420, 311, 387, 837, 1359, 1364, 1071, 311, 46993, 14297, 11, 313, 1, 438, 433, 198, 18864, 1193, 387, 39270, 1306, 682, 11, 46993, 11, 313, 275, 596, 22225, 927, 323, 2884, 198, 4291, 13, 1102, 596, 539, 42216, 596, 3347, 430, 10742, 499, 1457, 26, 433, 596, 813, 3938, 449, 198, 9514, 11, 4536, 956, 433, 7673, 362, 7200, 315, 294, 1412, 24201, 46793, 16392, 198, 4908, 264, 22760, 295, 505, 1077, 8579, 7013, 11, 323, 505, 433, 11, 42071, 311, 279, 842, 315, 198, 1964, 5542, 11, 11299, 11816, 1306, 11816, 315, 8579, 259, 41154, 13, 3005, 28902, 279, 22760, 295, 198, 10727, 3725, 449, 2678, 70723, 2442, 18033, 28042, 13, 5112, 1364, 16392, 323, 2231, 1077, 6206, 198, 263, 76996, 596, 28004, 13, 330, 791, 4382, 8206, 374, 1359, 1364, 1071, 30666, 11, 330, 9210, 358, 2643, 3412, 42216, 422, 198, 40, 42777, 313, 466, 37852, 13, 1102, 596, 856, 22519, 198, 9210, 596, 13194, 11, 4400, 775, 1210, 1628, 8617, 1550, 46993, 52290, 89, 733, 1523, 311, 1077, 13306, 13, 76996, 14980, 369, 264, 4545, 11, 1077, 6548, 389, 279, 6661, 1364, 5762, 13, 40612, 11, 304, 198, 1964, 502, 19675, 11, 1364, 1047, 9687, 1690, 15234, 2574, 13, 3861, 315, 1124, 574, 198, 576, 25, 430, 3278, 1093, 32171, 56310, 1550, 539, 34568, 872, 33218, 26, 430, 279, 198, 1889, 315, 279, 100225, 574, 330, 63007, 311, 279, 32232, 13412, 5233, 430, 832, 6476, 279, 198, 5924, 11, 323, 2834, 477, 5675, 11, 323, 422, 568, 5675, 11, 3952, 813, 16088, 13, 94192, 775, 304, 279, 8952, 889, 7020, 1077, 3446, 11, 315, 3388, 13, 13538, 2133, 63891, 11, 76996, 9277, 279, 6661, 304, 264, 829, 60496, 323, 743, 198, 11029, 311, 433, 449, 264, 2489, 13, 4427, 315, 279, 12164, 685, 1047, 8636, 704, 315, 1077, 6548, 13, 578, 54050, 11, 4869, 11, 574, 198, 74303, 22568, 555, 3738, 2574, 13, 1789, 2937, 11, 433, 27458, 279, 1384, 1251, 198, 300, 16168, 369, 279, 52547, 15051, 11, 323, 8710, 264, 31954, 311, 37085, 198, 258, 1234, 279, 3185, 994, 912, 832, 574, 3411, 13, 35120, 31202, 11112, 198, 1116, 2656, 311, 1935, 279, 64260, 596, 6916, 902, 574, 9076, 1077, 11, 323, 1071, 1364, 198, 8890, 22456, 1364, 574, 3025, 311, 4321, 709, 7636, 13, 32980, 35120, 31202, 3782, 11, 439, 58046, 3847, 813, 2361, 11, 304, 264, 4686, 198, 6583, 1386, 12844, 596, 28403, 315, 18012, 2107, 323, 91441, 11, 449, 279, 20858, 430, 574, 198, 26301, 3314, 5842, 48809, 927, 813, 4851, 13, 578, 6825, 3782, 10799, 7162, 11, 43169, 449, 264, 8547, 315, 27924, 439, 311, 198, 13066, 716, 13, 12540, 814, 2231, 2555, 389, 279, 45115, 1603, 814, 2163, 11, 304, 1162, 198, 3323, 10054, 12932, 323, 19692, 1051, 10434, 520, 279, 3838, 30, 2582, 574, 433, 1120, 439, 1664, 198, 998, 7095, 311, 15369, 11, 323, 11, 422, 279, 52290, 89, 87117, 19168, 46579, 11, 311, 2503, 198, 2996, 311, 264, 9439, 40459, 994, 814, 2751, 2162, 30, 2057, 735, 2637, 11961, 304, 279, 1203, 315, 279, 8993, 1990, 5340, 65670, 323, 24101, 11, 279, 198, 68467, 7113, 574, 76996, 313, 68753, 3520, 1193, 13, 1283, 15746, 1077, 1176, 7504, 1523, 279, 198, 2852, 273, 11, 5602, 1077, 46175, 733, 709, 439, 1364, 18661, 3273, 1082, 323, 12410, 11, 15746, 198, 1820, 55060, 315, 1077, 3995, 7216, 304, 1202, 342, 2933, 4341, 4251, 439, 1364, 5946, 1461, 198, 438, 4024, 4741, 3347, 279, 1317, 7123, 315, 1589, 44159, 13272, 82, 13, 4740, 1637, 568, 1436, 198, 1962, 6227, 279, 13306, 4717, 520, 682, 13, 578, 2532, 369, 1461, 574, 76996, 345, 74303, 1384, 291, 323, 1633, 6129, 11, 30488, 279, 51943, 13, 1102, 574, 76996, 889, 3782, 198, 2996, 279, 60779, 311, 279, 92906, 28022, 42400, 315, 279, 13306, 15572, 11, 76996, 198, 4291, 7639, 30488, 1077, 0, 1952, 813, 1314, 7731, 5340, 65670, 11, 3515, 8813, 279, 1176, 99589, 315, 1077, 198, 943, 7076, 13, 2435, 1051, 810, 1109, 198, 9210, 313, 20670, 1051, 92906, 28022, 13, 84854, 1070, 11, 1364, 6445, 16195, 6548, 198, 2017, 279, 8993, 11, 10409, 449, 4754, 61365, 13, 2057, 5340, 65670, 11, 1243, 11, 430, 6664, 13658, 574, 264, 3938, 315, 26762, 46793, 198, 438, 84064, 263, 11, 279, 16267, 315, 9886, 11, 38586, 93534, 287, 38586, 13, 2030, 311, 198, 57856, 11, 10307, 279, 22260, 449, 73500, 6548, 323, 19661, 1740, 940, 1529, 51220, 198, 75, 3153, 11, 574, 5108, 1077, 6596, 13, 84854, 1203, 304, 279, 62516, 11, 449, 1077, 6206, 198, 20557, 291, 927, 1077, 23788, 34740, 11, 1364, 1071, 264, 2697, 23788, 369, 1077, 7833, 198, 82279, 10003, 11, 13176, 704, 505, 279, 51943, 449, 2867, 11, 653, 2642, 14152, 6548, 13, 1666, 76996, 323, 7639, 24465, 3221, 279, 6134, 11, 13142, 46506, 12669, 11, 889, 1047, 1027, 198, 10276, 520, 279, 1203, 315, 279, 8993, 11, 6656, 6288, 323, 4024, 704, 13, 1283, 198, 267, 26902, 11, 4856, 11, 439, 422, 568, 1436, 539, 1518, 13, 96424, 94515, 1432, 791, 87117, 520, 279, 5929, 30304, 14894, 1047, 539, 1027, 279, 1566, 87117, 198, 77264, 22983, 36627, 323, 7639, 17882, 1047, 4529, 3871, 13, 22770, 22983, 1047, 198, 4626, 369, 1077, 20769, 264, 2678, 6424, 2949, 4228, 3937, 5620, 6138, 315, 198, 1820, 3363, 11, 323, 1403, 477, 2380, 3115, 2391, 1077, 1403, 5672, 1022, 14523, 17882, 198, 32345, 8208, 704, 311, 1518, 1077, 13, 1789, 3131, 430, 568, 1436, 1518, 76996, 11, 568, 5602, 22770, 22983, 11157, 13, 3005, 7020, 5115, 1664, 279, 3169, 315, 893, 449, 8884, 1364, 574, 14892, 313, 9210, 568, 198, 41450, 2343, 439, 2697, 439, 3284, 13, 2030, 1364, 7020, 11, 2288, 11, 430, 11, 1095, 1461, 1390, 264, 198, 1626, 3403, 11, 568, 1053, 2343, 904, 3430, 369, 433, 11, 1524, 11103, 13, 578, 1633, 85009, 269, 304, 1077, 3663, 574, 304, 1077, 4799, 13, 3005, 1053, 2231, 279, 3245, 198, 20322, 11, 323, 1501, 1884, 7893, 287, 37465, 11, 449, 872, 281, 1245, 6548, 323, 11714, 198, 652, 5184, 11, 264, 3245, 477, 1403, 13, 12220, 430, 4553, 20769, 568, 2646, 5602, 1077, 304, 4205, 810, 37067, 198, 54895, 279, 45648, 315, 4251, 37862, 27946, 398, 1825, 520, 279, 28691, 11, 43475, 198, 21621, 709, 311, 1501, 1077, 7731, 6577, 11977, 13, 2684, 1051, 912, 1023, 4580, 388, 520, 279, 198, 56492, 83408, 13, 3005, 7731, 369, 4207, 304, 279, 7474, 59938, 304, 279, 9518, 198, 17884, 10409, 449, 24149, 12690, 430, 77317, 279, 27834, 11, 15884, 198, 3950, 927, 264, 2363, 11, 719, 449, 1077, 27989, 6548, 2744, 389, 279, 5754, 13, 3005, 1373, 198, 33, 51520, 11, 80554, 11, 4593, 258, 22464, 68, 13, 9843, 568, 1766, 1077, 449, 264, 2363, 430, 1364, 198, 71, 561, 1570, 44133, 13, 1283, 29676, 389, 9298, 433, 11, 323, 27528, 433, 13, 1221, 7096, 291, 449, 433, 11, 1364, 1529, 53078, 323, 12504, 1077, 198, 58331, 13, 5414, 35072, 52671, 1766, 304, 433, 279, 1455, 1672, 80227, 315, 72739, 11, 439, 198, 32158, 1047, 10825, 13, 330, 40, 2733, 1778, 459, 43358, 994, 358, 1097, 449, 499, 1359, 1364, 1071, 13, 330, 40, 4934, 311, 1440, 264, 198, 56492, 810, 922, 279, 2574, 499, 656, 1210, 3011, 2231, 872, 5133, 389, 264, 502, 323, 11084, 8197, 13, 2684, 10924, 198, 383, 23781, 15243, 15173, 4619, 315, 27065, 13, 5414, 990, 11, 264, 19584, 2363, 311, 813, 3278, 1603, 345, 6789, 1825, 311, 1077, 13, 4800, 323, 1243, 872, 6721, 20954, 9670, 304, 2555, 198, 64750, 13, 358, 649, 3137, 198, 8845, 449, 499, 2085, 3060, 34734, 477, 49394, 1113, 499, 13, 1472, 527, 279, 198, 3646, 25530, 5333, 358, 1440, 313, 438, 832, 315, 279, 4509, 10462, 478, 1210, 1283, 1047, 10717, 279, 5780, 389, 279, 73024, 315, 264, 24898, 369, 279, 37996, 37864, 198, 77763, 315, 14933, 6322, 279, 1684, 13, 330, 2170, 1317, 439, 499, 3137, 8221, 1359, 1364, 1071, 11, 330, 40, 2733, 430, 1070, 374, 4400, 198, 35970, 304, 1057, 1694, 3871, 26, 719, 994, 499, 2019, 279, 1023, 3245, 313, 1875, 1, 3957, 433, 5076, 311, 3371, 264, 5128, 5333, 499, 50863, 1077, 7673, 1283, 38817, 5678, 2212, 304, 279, 10954, 323, 7731, 3411, 520, 1077, 13, 330, 791, 781, 899, 13744, 11013, 304, 279, 1917, 9135, 3005, 1047, 3685, 433, 369, 520, 3325, 264, 2046, 11, 719, 1077, 13051, 574, 1664, 2884, 13, 8489, 2884, 1101, 574, 1077, 21847, 2391, 279, 5105, 24998, 12141, 13, 2360, 11, 1364, 574, 539, 19021, 11, 1364, 1071, 13, 1102, 574, 1193, 430, 568, 1047, 743, 1077, 198, 83360, 13, 3277, 1364, 2751, 704, 315, 279, 1841, 11, 1364, 293, 1037, 1461, 1695, 45427, 323, 198, 19045, 1481, 9188, 13, 4740, 430, 4400, 1436, 617, 8774, 1461, 3201, 11, 323, 1364, 7020, 433, 13, 330, 1692, 18651, 2225, 8137, 323, 1514, 26, 9093, 568, 50243, 5333, 439, 279, 198, 3646, 11660, 315, 23939, 1210, 362, 42786, 315, 8137, 1047, 10862, 1139, 872, 198, 37195, 13, 1283, 3937, 3093, 704, 311, 279, 8961, 279, 1828, 1938, 11, 311, 387, 3309, 430, 9083, 36627, 198, 32345, 8208, 369, 264, 1317, 4321, 323, 1047, 539, 1071, 994, 1364, 1053, 387, 1203, 13, 7357, 893, 13452, 311, 1781, 430, 198, 383, 374, 264, 2766, 315, 264, 38736, 13, 7639, 23183, 813, 18623, 11, 323, 11, 9564, 813, 198, 7063, 4994, 279, 34799, 365, 13883, 25675, 11, 56696, 1529, 411, 989, 389, 4579, 304, 279, 198, 19612, 22770, 22983, 1047, 4529, 13, 1283, 1766, 1077, 11, 3663, 1523, 11, 1234, 264, 5021, 345, 20986, 28639, 323, 24634, 323, 18534, 682, 279, 6029, 315, 264, 15748, 10723, 198, 496, 50428, 13, 3005, 16392, 304, 22047, 994, 1364, 6755, 813, 3094, 11, 323, 92037, 264, 198, 5447, 477, 1403, 11, 449, 1077, 6206, 704, 1603, 1077, 13, 358, 313, 40, 617, 2751, 311, 198, 19553, 264, 2697, 892, 7636, 13, 1283, 7020, 433, 574, 1514, 12, 36022, 11, 719, 4856, 15262, 433, 26, 323, 11, 1606, 568, 574, 198, 84270, 439, 10151, 1285, 439, 1364, 574, 11, 568, 17948, 264, 2489, 389, 279, 38411, 315, 279, 5021, 198, 438, 3177, 291, 264, 36213, 1603, 568, 19089, 13, 330, 40, 574, 16984, 315, 420, 1359, 568, 1071, 11, 5737, 709, 13, 358, 1097, 539, 2216, 264, 40148, 11, 22770, 22983, 1210, 1102, 574, 279, 1176, 892, 568, 1047, 1511, 1077, 836, 13, 330, 61935, 1523, 323, 1095, 603, 3137, 2574, 927, 1210, 3005, 7731, 1523, 520, 264, 6220, 6138, 11, 323, 7111, 4028, 279, 2697, 33850, 198, 998, 1461, 449, 279, 1794, 655, 6548, 430, 1051, 1077, 2294, 9513, 13, 330, 2675, 649, 10150, 311, 387, 1633, 19858, 1359, 1364, 1071, 11, 330, 28753, 420, 374, 1193, 1514, 198, 998, 499, 26, 358, 1440, 433, 13, 358, 2846, 264, 1695, 11700, 323, 198, 1962, 313, 359, 266, 70178, 13, 2030, 1148, 374, 1514, 369, 499, 374, 539, 14647, 1514, 369, 198, 2727, 13, 1789, 279, 1176, 892, 11, 568, 1766, 5678, 35090, 304, 1077, 100226, 13, 1442, 1364, 39169, 313, 383, 574, 520, 198, 1820, 39324, 315, 904, 5333, 889, 39169, 13, 1115, 3460, 315, 3245, 4250, 733, 389, 11, 2999, 13, 3005, 1550, 16106, 1243, 313, 8110, 24014, 26, 323, 568, 4024, 927, 30488, 1077, 323, 3952, 1077, 198, 258, 813, 11977, 13, 1472, 1304, 757, 2733, 1093, 198, 64, 1156, 801, 3833, 11, 323, 358, 3077, 1193, 1027, 4737, 264, 2697, 2766, 315, 23871, 13, 7639, 11, 323, 59334, 1077, 1578, 389, 279, 23726, 13, 578, 832, 2449, 22770, 22983, 1047, 2163, 704, 315, 1077, 29217, 574, 11937, 13, 3005, 1047, 3967, 279, 893, 11, 1047, 4529, 279, 6671, 520, 1202, 6300, 907, 13, 2030, 198, 32158, 1047, 2163, 704, 420, 3062, 8331, 304, 279, 24524, 11, 313, 9210, 8331, 198, 8370, 304, 1475, 5133, 1990, 893, 323, 5333, 27667, 279, 198, 26378, 367, 11, 313, 1820, 5333, 13, 32745, 1077, 38714, 45715, 1047, 2586, 264, 502, 323, 33812, 2449, 13, 3005, 198, 14965, 11, 1093, 735, 13, 304, 813, 2697, 3130, 389, 279, 6825, 11, 1047, 2231, 16038, 3021, 323, 198, 1820, 2574, 34366, 11, 1766, 430, 433, 1053, 539, 387, 2231, 16038, 13, 3296, 279, 842, 315, 198, 1964, 2875, 20769, 22770, 22983, 36627, 574, 48295, 304, 3021, 449, 279, 14992, 198, 92493, 13, 2435, 8738, 311, 3449, 11, 539, 439, 3629, 439, 1603, 11, 719, 3131, 264, 2046, 345, 66372, 13, 578, 16659, 1051, 2539, 315, 8137, 1457, 26, 323, 422, 369, 279, 3828, 814, 198, 55437, 555, 420, 4367, 11, 814, 18661, 33464, 369, 279, 893, 13, 3005, 574, 559, 4361, 67, 198, 268, 1409, 311, 13383, 1077, 1866, 6671, 13, 3005, 198, 66, 1636, 11, 323, 568, 1550, 539, 13, 1102, 574, 682, 264, 1847, 1457, 11, 539, 11074, 13, 2052, 3278, 527, 42779, 26, 3278, 304, 3021, 527, 59594, 779, 13, 1666, 1664, 439, 198, 32158, 7020, 430, 813, 11939, 369, 1077, 574, 539, 279, 1972, 3245, 11, 779, 1101, 1364, 198, 8110, 1534, 430, 1070, 574, 7982, 709, 304, 813, 4851, 2555, 53007, 311, 279, 198, 8110, 3245, 369, 76996, 5874, 13, 53306, 14222, 6244, 44743, 1306, 264, 3137, 198, 20670, 1047, 927, 279, 87117, 2007, 520, 264, 3224, 5754, 37002, 279, 1938, 1306, 198, 19875, 483, 596, 13306, 13, 330, 4438, 574, 279, 13306, 313, 83, 3946, 638, 7673, 2684, 596, 2744, 2555, 57399, 311, 757, 304, 264, 893, 63143, 198, 40617, 726, 709, 369, 2324, 311, 832, 5333, 13, 330, 4897, 596, 539, 7041, 279, 7658, 323, 279, 1322, 764, 1441, 11, 374, 433, 7673, 2057, 1781, 315, 27397, 704, 315, 682, 279, 1917, 832, 5333, 345, 438, 4135, 287, 311, 8493, 279, 2800, 315, 832, 596, 2919, 449, 1077, 0, 10541, 313, 1875, 16366, 6548, 7111, 3347, 22770, 22983, 1139, 6138, 13, 330, 68753, 3520, 5874, 574, 832, 315, 279, 61365, 98250, 1359, 568, 1071, 40815, 398, 13, 330, 8100, 574, 198, 385, 899, 1291, 1109, 279, 42447, 1210, 330, 53040, 11, 719, 18754, 1359, 1071, 22770, 22983, 13, 358, 3077, 2216, 6818, 311, 198, 668, 613, 1077, 2574, 11, 719, 313, 9514, 1440, 313, 1, 3005, 70756, 1077, 28004, 13, 1442, 1070, 574, 264, 4483, 36244, 304, 813, 8071, 11, 568, 198, 588, 2230, 433, 62504, 398, 13, 2030, 11, 3131, 1578, 304, 279, 5780, 11, 568, 30280, 927, 323, 198, 631, 813, 41803, 2403, 11074, 13, 1472, 2351, 41690, 1359, 568, 1071, 506, 495, 18007, 13, 35053, 11, 8051, 568, 2643, 15648, 11, 279, 2217, 315, 76996, 11203, 1633, 198, 5669, 311, 813, 4851, 1884, 42774, 2919, 13, 76996, 3782, 1022, 3814, 14523, 279, 6278, 315, 6841, 13, 578, 3814, 14523, 1047, 198, 82850, 264, 892, 315, 55394, 9096, 311, 22770, 22983, 13, 7639, 1436, 4546, 76996, 1203, 311, 279, 8952, 304, 813, 1841, 13, 76996, 596, 4376, 96314, 520, 2162, 1051, 25975, 369, 57770, 552, 315, 80822, 389, 198, 77264, 22983, 596, 961, 13, 1952, 1778, 459, 13402, 11, 264, 2305, 1306, 279, 13306, 11, 1364, 198, 29177, 539, 6782, 11937, 13, 3005, 40001, 1077, 2362, 28391, 315, 47846, 11, 323, 198, 91453, 279, 259, 75143, 311, 264, 1486, 3221, 279, 842, 315, 279, 6825, 13, 4740, 93722, 198, 67643, 11, 1364, 14297, 15203, 279, 3160, 315, 279, 6825, 13, 46993, 323, 42216, 198, 32345, 539, 6052, 505, 872, 13306, 11879, 13, 578, 6841, 11714, 574, 198, 1962, 9439, 11, 323, 389, 279, 2697, 44952, 7731, 76996, 323, 2999, 13, 735, 13, 574, 198, 19041, 11, 2288, 11, 1047, 1364, 1193, 3967, 433, 11, 11961, 1203, 304, 279, 12737, 323, 5605, 198, 56492, 11, 813, 24981, 6548, 389, 76996, 596, 5643, 13, 3005, 4024, 389, 1523, 279, 6825, 304, 264, 74941, 198, 1073, 41690, 19788, 13, 4740, 430, 1403, 6848, 10837, 35135, 304, 22770, 22983, 596, 4059, 25, 832, 574, 311, 636, 198, 68753, 3520, 704, 315, 279, 1648, 11, 279, 1023, 574, 311, 1304, 17882, 30714, 311, 1077, 13, 763, 198, 1964, 4851, 1364, 7020, 430, 389, 279, 1176, 79101, 279, 2132, 13, 362, 2046, 3010, 1364, 1903, 279, 1890, 89706, 95440, 11, 719, 449, 264, 2204, 198, 1407, 13, 2030, 11509, 389, 279, 23162, 198, 11020, 9710, 315, 279, 2697, 3838, 574, 2009, 77638, 818, 13, 578, 60243, 32329, 355, 12690, 1051, 198, 55202, 520, 430, 892, 11, 21939, 13819, 3935, 11977, 40872, 311, 279, 6841, 13180, 13, 578, 198, 28451, 2922, 1141, 11, 902, 304, 279, 7474, 2163, 279, 76977, 304, 279, 12737, 11, 1457, 198, 939, 606, 1555, 279, 23962, 323, 22982, 1139, 3831, 16337, 2009, 77638, 818, 596, 16615, 198, 18257, 323, 743, 3663, 13, 3005, 4024, 389, 11, 89815, 11, 1077, 13326, 8271, 20225, 287, 93392, 13, 1102, 574, 279, 1176, 892, 198, 32158, 1047, 3967, 430, 735, 13, 12439, 304, 279, 5874, 3838, 13, 1102, 6688, 1077, 264, 5647, 315, 198, 1371, 81246, 323, 25114, 8850, 13, 3005, 1903, 1077, 1176, 11919, 927, 5081, 315, 1690, 2919, 311, 76996, 279, 198, 44018, 1938, 13, 2435, 2322, 304, 279, 35779, 48840, 304, 279, 31741, 1405, 279, 198, 28451, 17895, 369, 279, 26741, 6978, 574, 8774, 13, 5810, 11, 23255, 304, 49707, 198, 438, 11989, 291, 11, 3185, 555, 3185, 11203, 279, 98882, 67556, 304, 902, 198, 1820, 6978, 1047, 2322, 11677, 477, 17563, 13, 432, 2076, 323, 13413, 1918, 11, 1488, 339, 323, 198, 18883, 49958, 11, 11203, 4661, 31687, 13, 13759, 3201, 389, 279, 1023, 3185, 315, 279, 4251, 2695, 13883, 31741, 11, 3026, 1051, 198, 359, 10853, 30103, 6605, 43732, 315, 14403, 13, 57227, 82, 315, 40120, 3782, 1523, 279, 198, 5997, 277, 27896, 11, 31687, 872, 4251, 56737, 323, 13353, 279, 43732, 311, 15310, 13, 98325, 574, 279, 13901, 315, 279, 8952, 11, 902, 374, 2015, 13, 76996, 11, 305, 34552, 1203, 505, 3293, 1776, 2866, 311, 279, 62862, 10652, 198, 1073, 1077, 3814, 14523, 11, 31645, 520, 22770, 22983, 26085, 3725, 13, 330, 87643, 56310, 374, 2133, 704, 311, 11477, 13, 3277, 832, 43457, 1268, 5986, 1364, 574, 323, 1268, 584, 3463, 1364, 1436, 539, 198, 25859, 11, 433, 596, 4856, 264, 38586, 11, 4536, 956, 433, 7673, 76996, 25078, 449, 1063, 73082, 279, 37067, 36156, 2176, 67556, 304, 1077, 198, 10888, 13, 330, 8100, 649, 956, 733, 704, 304, 1884, 26, 358, 4985, 617, 311, 39580, 1077, 2555, 1210, 362, 198, 56492, 315, 279, 3177, 8636, 704, 315, 1077, 3663, 13, 330, 8100, 596, 1047, 264, 2653, 4465, 11, 323, 198, 32158, 706, 2834, 1359, 1364, 1071, 13, 330, 4071, 994, 358, 1781, 315, 1148, 1364, 596, 4762, 2133, 198, 1445, 311, 313, 1875, 77264, 22983, 70756, 1077, 28004, 13, 330, 2181, 596, 682, 304, 279, 1938, 596, 990, 1359, 1364, 13468, 1280, 14657, 4501, 13, 330, 2675, 649, 1935, 198, 49818, 709, 1139, 279, 9979, 323, 3041, 1124, 24981, 990, 1370, 287, 35267, 11, 477, 198, 631, 1124, 304, 279, 35189, 11245, 287, 13, 3005, 24465, 264, 6462, 505, 279, 35779, 323, 7111, 520, 433, 56949, 3725, 13, 330, 11649, 11, 1148, 656, 499, 1440, 922, 420, 30, 5810, 596, 264, 5333, 889, 3782, 304, 304, 264, 198, 9471, 70, 785, 323, 6857, 315, 1776, 32997, 13, 1628, 1457, 1364, 6944, 311, 733, 704, 304, 4376, 459, 198, 25704, 9135, 3005, 6656, 11, 389, 1077, 1648, 704, 315, 279, 35779, 48840, 11, 323, 6689, 264, 4062, 34522, 198, 266, 76996, 13, 330, 40, 7077, 311, 387, 389, 701, 8761, 279, 1023, 3814, 1359, 1364, 1071, 13, 330, 2675, 3974, 198, 582, 2177, 279, 8761, 505, 17882, 82, 518, 1541, 956, 499, 7673, 330, 40, 3463, 779, 26, 358, 1047, 6755, 499, 6604, 315, 279, 3838, 13, 4718, 313, 22479, 10868, 198, 16514, 11509, 389, 279, 7504, 1210, 1102, 4536, 956, 2216, 198, 1315, 311, 1650, 1461, 264, 3130, 261, 26, 568, 596, 832, 315, 279, 3070, 1457, 35277, 82, 836, 1047, 17948, 459, 2744, 27078, 44321, 304, 76996, 13, 578, 1403, 7724, 198, 42700, 9017, 279, 39736, 3871, 13, 3161, 264, 1633, 2697, 51475, 345, 68753, 3520, 15243, 315, 735, 13, 3005, 574, 18949, 520, 9083, 36627, 596, 11919, 16630, 345, 6200, 329, 430, 2574, 1051, 682, 1314, 1990, 1124, 1578, 13, 2468, 1077, 6558, 11, 1364, 198, 631, 264, 99149, 1450, 389, 279, 3828, 596, 6916, 13, 330, 40, 574, 16984, 358, 1047, 54221, 499, 477, 85334, 1503, 499, 1359, 1364, 1071, 13, 330, 40, 2846, 779, 198, 6200, 329, 433, 4536, 956, 779, 1210, 20695, 1051, 539, 2133, 904, 2288, 1664, 449, 735, 13, 3082, 11, 568, 1047, 4036, 813, 198, 75242, 520, 279, 5274, 11, 323, 449, 420, 3118, 3611, 10036, 768, 315, 17510, 38502, 198, 78289, 1590, 264, 2046, 568, 574, 3025, 311, 656, 3892, 2574, 13, 35120, 31202, 1457, 198, 86, 13883, 323, 11245, 291, 832, 1938, 264, 2046, 520, 279, 2697, 3838, 11, 779, 430, 45563, 198, 46068, 617, 810, 892, 311, 1427, 1306, 24101, 13, 1283, 1047, 7319, 1101, 279, 198, 6173, 315, 3300, 430, 568, 48582, 3288, 6460, 13, 578, 3245, 430, 7222, 839, 323, 10409, 1461, 449, 264, 5647, 198, 1073, 8060, 574, 7639, 17882, 596, 19451, 13, 1102, 574, 539, 9662, 18260, 26, 433, 574, 345, 28074, 291, 11, 21356, 49150, 11, 4661, 36237, 2335, 13, 2030, 568, 9539, 198, 25742, 291, 2009, 77638, 818, 596, 2361, 32677, 13, 2684, 574, 912, 837, 67128, 2200, 1990, 279, 1403, 3026, 26, 719, 1070, 574, 198, 7413, 1251, 311, 387, 6926, 15360, 11, 323, 31445, 264, 3738, 3392, 315, 198, 1658, 2538, 13, 17882, 6137, 311, 4546, 682, 813, 5435, 311, 2009, 77638, 818, 13, 2684, 1051, 1317, 198, 49512, 811, 304, 430, 2678, 8582, 3130, 13, 19292, 810, 1109, 832, 893, 477, 198, 22803, 889, 1550, 539, 1440, 315, 735, 3238, 82, 14209, 49009, 813, 2324, 311, 1461, 430, 4498, 13, 47124, 6137, 311, 2586, 304, 311, 1461, 198, 1527, 279, 14932, 25861, 13, 2057, 813, 1866, 59772, 574, 3779, 264, 502, 323, 198, 38548, 481, 15105, 13, 2030, 2009, 77638, 818, 11, 889, 1047, 1766, 44734, 422, 539, 198, 1834, 11, 574, 3131, 1578, 304, 5916, 449, 279, 990, 568, 10456, 13, 2684, 1051, 198, 15487, 994, 11, 3515, 8983, 13883, 264, 1162, 704, 3871, 323, 33740, 279, 1828, 198, 1316, 596, 990, 369, 7639, 11, 568, 1053, 4321, 369, 4207, 1139, 279, 3814, 704, 927, 279, 198, 71, 3385, 11, 11039, 813, 8209, 13, 578, 79422, 574, 389, 1461, 311, 387, 304, 279, 12314, 198, 1073, 2574, 1578, 13, 578, 3463, 315, 279, 6962, 5274, 323, 1202, 25114, 4883, 198, 82, 875, 6901, 1461, 13, 1102, 574, 389, 832, 315, 813, 1317, 23291, 430, 735, 13, 1766, 40788, 648, 13, 1102, 574, 6790, 1243, 11, 18004, 323, 7257, 11, 449, 264, 14739, 12056, 430, 5614, 311, 198, 30193, 439, 433, 11299, 13, 578, 3224, 19795, 1051, 39448, 6953, 752, 449, 27275, 11, 279, 5627, 579, 198, 22354, 12314, 449, 24905, 5294, 11141, 13, 578, 33201, 277, 1918, 315, 279, 47967, 430, 198, 34774, 13658, 32599, 813, 20247, 13, 1283, 1047, 93327, 311, 279, 842, 315, 279, 198, 28451, 24948, 1584, 11, 323, 3940, 813, 4321, 505, 1070, 13, 1666, 574, 813, 2587, 11, 568, 198, 86, 461, 912, 927, 68512, 11, 719, 264, 2875, 61221, 1234, 813, 23724, 13, 18024, 61752, 3235, 198, 1820, 5754, 568, 1047, 13061, 709, 264, 75642, 3833, 5679, 11, 323, 11, 439, 422, 304, 33638, 12876, 369, 198, 26380, 8396, 11, 433, 57677, 6702, 22489, 2915, 520, 813, 34460, 13, 31048, 8931, 505, 279, 842, 315, 279, 1841, 1584, 568, 1766, 264, 5754, 37002, 11, 323, 198, 90963, 304, 369, 264, 9168, 315, 93050, 13, 578, 5679, 198, 42700, 304, 449, 1461, 11, 323, 14980, 3411, 709, 1139, 813, 3663, 13, 1102, 574, 439, 422, 568, 198, 73563, 11, 719, 31156, 3249, 420, 55154, 11, 449, 279, 1156, 812, 315, 279, 5754, 198, 43887, 323, 279, 33965, 315, 70244, 927, 279, 5151, 13, 578, 3838, 574, 743, 304, 264, 34681, 520, 279, 4579, 315, 1403, 35231, 13, 17331, 279, 8770, 198, 1073, 279, 6790, 13658, 11, 433, 1047, 781, 25111, 82374, 1603, 1461, 13, 578, 6134, 198, 16514, 264, 19571, 11, 323, 568, 25319, 1139, 264, 2697, 14321, 9960, 449, 6892, 30193, 28954, 13, 2057, 279, 1314, 574, 279, 18397, 48840, 11, 279, 2007, 9960, 449, 264, 4251, 28392, 345, 438, 304, 1202, 4839, 4219, 459, 66637, 442, 3876, 15860, 315, 32720, 19837, 13, 2057, 279, 198, 2414, 11, 279, 14595, 1370, 9761, 315, 1778, 7634, 13, 1102, 2643, 617, 1027, 279, 1370, 9761, 198, 1073, 279, 5929, 30304, 14894, 304, 23329, 11, 72779, 659, 53034, 15512, 323, 682, 13, 6193, 198, 73826, 574, 21847, 323, 264, 47274, 2277, 22843, 315, 7878, 767, 97939, 13, 578, 3838, 198, 16514, 19738, 449, 502, 6308, 313, 1820, 30811, 3252, 2362, 27035, 559, 606, 449, 433, 11, 279, 198, 28404, 30103, 3690, 13, 2663, 735, 382, 3947, 1051, 6435, 60802, 50007, 11, 279, 15676, 315, 264, 44402, 27580, 345, 1820, 23941, 273, 315, 264, 5333, 596, 8679, 5108, 1523, 279, 32249, 13, 735, 2637, 11509, 198, 1371, 7770, 398, 389, 264, 28954, 86810, 430, 574, 279, 4219, 315, 279, 1370, 9761, 767, 97939, 345, 30972, 7069, 1022, 813, 61221, 13, 568, 1071, 311, 279, 64233, 8954, 389, 279, 198, 267, 1334, 5756, 13, 3005, 2231, 264, 1450, 2403, 279, 198, 11020, 6906, 311, 24981, 11937, 13, 40788, 648, 22225, 11, 719, 264, 502, 40788, 648, 0, 3161, 1077, 198, 51729, 46050, 6901, 2212, 1077, 3663, 11, 264, 7878, 6437, 523, 396, 89, 8679, 1825, 520, 279, 198, 87164, 11, 264, 3776, 72426, 15631, 389, 1077, 17659, 11, 1618, 574, 264, 40788, 648, 88018, 345, 13885, 7627, 810, 19411, 11, 1109, 568, 1047, 27569, 1077, 13, 2030, 1364, 1550, 539, 198, 3647, 458, 520, 1461, 13, 2684, 574, 2555, 922, 1077, 6548, 539, 20426, 279, 5679, 596, 198, 29199, 11, 97340, 11, 719, 34685, 13, 330, 11649, 11, 499, 3077, 1766, 757, 11, 4491, 13, 1628, 11, 994, 568, 5762, 704, 813, 1450, 345, 3647, 8138, 25, 330, 40, 1120, 1047, 311, 656, 433, 11, 4491, 13, 1472, 1427, 42727, 7060, 323, 313, 57621, 11, 40788, 648, 1210, 4946, 499, 617, 264, 10747, 315, 15600, 11, 477, 690, 499, 617, 2555, 775, 7673, 578, 31655, 315, 279, 6825, 574, 2103, 3831, 304, 40788, 648, 13, 578, 6825, 1550, 198, 1962, 29251, 315, 330, 34431, 775, 1210, 330, 3407, 94707, 9976, 1355, 14320, 1359, 1071, 2009, 77638, 818, 13, 330, 3112, 4985, 358, 3780, 264, 11989, 369, 499, 311, 198, 79, 3265, 7673, 1283, 574, 14931, 568, 1047, 1903, 279, 1529, 8154, 13, 469, 1325, 4501, 279, 6825, 323, 682, 430, 18713, 2692, 574, 264, 36366, 3917, 13, 1102, 574, 369, 420, 430, 1364, 1047, 48501, 198, 1820, 9043, 70, 992, 17025, 315, 1077, 2324, 520, 18083, 13, 26718, 2176, 596, 313, 2000, 420, 10160, 1355, 906, 418, 198, 56492, 3838, 11, 13413, 1570, 28360, 11, 72600, 40666, 13, 2684, 1051, 1403, 90016, 263, 198, 268, 27995, 3808, 927, 279, 26976, 301, 13, 578, 198, 1605, 574, 279, 5684, 1773, 980, 7555, 13, 735, 13, 31156, 1148, 22999, 31655, 315, 198, 726, 39130, 29875, 367, 1047, 9057, 40788, 648, 311, 5387, 279, 7555, 1070, 2073, 380, 75325, 13, 6984, 315, 1202, 2361, 315, 16044, 568, 5602, 279, 3828, 596, 49803, 315, 1077, 1866, 198, 47965, 4090, 13, 1952, 264, 23162, 28745, 11, 7041, 1990, 279, 1403, 9364, 11, 574, 198, 43063, 93484, 315, 32720, 19837, 13, 40788, 648, 7263, 279, 93050, 11, 2736, 9709, 11, 304, 264, 16615, 9168, 13, 735, 13, 1053, 198, 19553, 15236, 311, 6651, 433, 5678, 11, 719, 279, 93050, 574, 1695, 13, 330, 2675, 6688, 757, 264, 2543, 520, 1176, 1359, 1071, 40788, 648, 13, 330, 4071, 358, 1097, 1314, 16089, 311, 1518, 198, 9514, 11, 4491, 13, 4800, 430, 279, 19795, 527, 3958, 11, 19093, 4131, 1633, 1790, 13, 30070, 1457, 11, 735, 13, 323, 40788, 648, 11, 994, 814, 2322, 11, 1047, 2322, 10652, 750, 389, 279, 198, 5581, 5015, 315, 3691, 13, 2435, 912, 5129, 1047, 430, 11, 323, 1990, 1124, 2225, 198, 6789, 1093, 264, 22881, 872, 1566, 10652, 13, 4497, 19411, 433, 7995, 574, 345, 8248, 6380, 30, 2684, 574, 264, 289, 380, 31514, 922, 40788, 648, 596, 11013, 430, 743, 1461, 198, 86, 14518, 287, 13, 330, 1548, 596, 922, 279, 1888, 893, 389, 9578, 13, 1283, 596, 2646, 1071, 264, 5425, 3492, 311, 198, 2727, 313, 17206, 520, 1176, 11, 994, 358, 574, 7363, 19501, 323, 27207, 520, 1475, 5222, 1210, 330, 40, 27724, 264, 2763, 315, 5310, 940, 82, 994, 358, 1176, 3782, 11, 4401, 1022, 323, 26092, 198, 9493, 358, 6755, 1274, 2212, 279, 2035, 13, 1102, 1511, 311, 2873, 311, 757, 430, 1148, 198, 40, 4265, 2884, 574, 5439, 389, 856, 3663, 13, 40788, 648, 65657, 709, 520, 279, 1403, 9364, 927, 279, 26976, 301, 13, 330, 32148, 433, 374, 313, 9493, 568, 4131, 304, 19781, 11, 323, 358, 3077, 264, 16553, 5644, 477, 198, 15031, 41951, 13824, 323, 19335, 369, 813, 87117, 11, 323, 358, 1518, 1461, 3240, 311, 1427, 198, 4014, 291, 13, 1283, 13001, 813, 13961, 11, 323, 1690, 459, 11714, 568, 8779, 757, 449, 279, 198, 67, 21168, 13, 330, 40, 8434, 956, 733, 1203, 311, 1405, 358, 574, 11, 719, 358, 1097, 539, 6380, 11, 4491, 13, 1115, 2035, 374, 813, 11, 323, 568, 4265, 1093, 264, 8334, 311, 2586, 1139, 433, 198, 9493, 568, 596, 8208, 13, 422, 358, 1550, 617, 832, 26, 1148, 1053, 433, 387, 7673, 6, 82, 6548, 8272, 11074, 311, 279, 6945, 323, 279, 3596, 4354, 826, 30456, 13, 330, 3112, 1364, 313, 19041, 4536, 956, 904, 22199, 315, 1077, 313, 7673, 2684, 574, 912, 6425, 311, 40788, 648, 596, 3575, 13, 2009, 77638, 818, 11, 11509, 389, 279, 198, 383, 47601, 323, 3411, 1523, 520, 1077, 11, 15393, 430, 11, 1306, 682, 11, 40788, 648, 2011, 198, 1816, 704, 1077, 1866, 46035, 13, 2435, 15243, 3117, 1139, 279, 7982, 93722, 315, 279, 13658, 13, 40788, 648, 574, 198, 57932, 894, 369, 3754, 315, 279, 6825, 25, 2011, 1440, 315, 46993, 596, 13306, 11, 315, 198, 27588, 65670, 11, 315, 76996, 304, 1077, 8952, 13, 1628, 994, 568, 1047, 3309, 1077, 682, 11, 1364, 198, 37568, 21737, 11, 20700, 1077, 1450, 7197, 62626, 304, 1077, 19779, 13, 5112, 25, 27567, 1, 18293, 279, 3116, 315, 603, 1359, 1364, 1071, 15187, 11, 313, 1, 19875, 483, 52290, 89, 323, 76996, 198, 2732, 323, 9083, 5340, 65670, 323, 757, 11, 313, 438, 902, 832, 1053, 499, 617, 13061, 311, 198, 3427, 5076, 1093, 420, 30, 358, 8101, 11, 505, 279, 5992, 315, 2574, 11, 1455, 15687, 1053, 198, 19553, 3463, 433, 1053, 387, 279, 52290, 89, 3828, 13, 2435, 4265, 617, 13061, 5340, 65670, 198, 89734, 9497, 369, 279, 8952, 11, 323, 757, 369, 279, 8679, 28936, 11, 323, 433, 1053, 617, 198, 82850, 76996, 5874, 430, 2751, 12502, 323, 1047, 459, 35528, 13, 3005, 7111, 709, 520, 735, 13, 559, 4361, 67, 398, 13, 2435, 3287, 956, 1440, 757, 11, 323, 358, 198, 55603, 1124, 7556, 13, 2435, 1071, 76996, 5874, 574, 2133, 311, 32918, 2999, 13, 1666, 1364, 198, 52784, 1603, 1461, 1364, 7111, 709, 1139, 813, 3663, 13, 330, 2746, 499, 1093, 1077, 439, 1664, 439, 358, 1781, 499, 656, 11, 4491, 13, 2009, 77638, 818, 11, 499, 2834, 956, 1095, 198, 40617, 636, 1077, 1210, 330, 40, 1097, 16984, 430, 596, 539, 709, 311, 757, 11, 374, 433, 30, 3639, 1053, 358, 656, 449, 264, 7555, 345, 51, 484, 648, 7673, 358, 8101, 11, 304, 279, 198, 4930, 1629, 11, 430, 1053, 1797, 810, 1109, 3300, 1210, 3011, 574, 1148, 735, 13, 3952, 2162, 449, 1461, 1306, 813, 13123, 449, 40788, 648, 13, 1283, 198, 20949, 291, 433, 389, 813, 1648, 1203, 311, 279, 8761, 24948, 11, 439, 568, 28214, 2403, 198, 1820, 10160, 13, 468, 6241, 10398, 7977, 3235, 279, 5754, 1051, 198, 28611, 449, 3090, 323, 1047, 22088, 311, 31030, 13, 578, 11422, 1047, 6656, 311, 264, 198, 3696, 2299, 274, 19510, 430, 4018, 813, 3663, 13, 26924, 3195, 311, 279, 259, 75143, 1584, 11, 279, 5679, 198, 42286, 1022, 1139, 264, 555, 66815, 13, 578, 5679, 45135, 1306, 198, 40617, 11, 832, 4579, 9408, 13, 9843, 1578, 813, 6548, 1051, 1093, 40788, 648, 596, 11, 439, 1364, 1047, 198, 86, 4234, 1695, 1481, 9188, 505, 279, 45747, 13, 5414, 2010, 68265, 389, 813, 17659, 11, 735, 13, 9960, 8931, 315, 5754, 449, 813, 1317, 345, 2332, 24992, 18338, 11, 323, 21095, 813, 8209, 13, 15148, 40788, 648, 1314, 11, 1306, 682, 11, 323, 198, 32345, 568, 1027, 5076, 30, 8595, 1288, 568, 3369, 580, 5678, 11, 422, 433, 8967, 76996, 596, 198, 359, 71, 67391, 30, 8595, 539, 4287, 17882, 596, 3085, 323, 1212, 927, 1578, 30, 5112, 198, 333, 2574, 4024, 1664, 313, 1820, 62461, 574, 3831, 430, 13766, 88, 13658, 13, 1283, 198, 631, 433, 505, 1461, 520, 1566, 11, 1606, 315, 279, 29191, 430, 8996, 568, 1550, 198, 41450, 1304, 912, 2349, 304, 76996, 596, 17139, 5597, 13, 1442, 1364, 42777, 3403, 198, 2000, 17882, 11, 1364, 1053, 32918, 1461, 13, 96424, 53694, 1432, 20577, 1195, 323, 46993, 6052, 505, 872, 13306, 8577, 279, 1938, 735, 627, 4338, 21468, 40788, 648, 13, 24101, 5874, 1903, 1790, 315, 279, 19163, 11, 29676, 389, 198, 67, 4481, 369, 1124, 430, 3814, 520, 279, 2697, 3838, 11, 2011, 1520, 46993, 198, 81874, 1077, 490, 15655, 323, 31993, 1077, 13306, 21258, 922, 279, 13455, 13, 3005, 198, 16514, 53657, 1109, 1364, 1047, 1027, 369, 2919, 11, 810, 8173, 13, 578, 40164, 315, 198, 1820, 490, 46651, 2933, 10409, 1077, 449, 66939, 323, 264, 3460, 315, 41690, 66533, 369, 198, 68753, 3520, 11, 889, 1436, 617, 7000, 315, 1521, 2574, 13, 763, 264, 67163, 3460, 315, 1648, 345, 32158, 6691, 291, 46993, 304, 39381, 315, 1077, 1866, 10003, 13, 1628, 433, 574, 1077, 4062, 8071, 430, 42645, 291, 2555, 5076, 13, 9636, 1077, 28361, 574, 459, 1234, 3311, 315, 21137, 13, 24101, 11, 9257, 304, 71271, 422, 304, 4400, 775, 11, 6612, 433, 11, 323, 304, 10052, 311, 198, 15031, 8982, 315, 46993, 596, 430, 17948, 1077, 439, 2653, 11, 539, 5115, 27442, 345, 32158, 6688, 1077, 264, 22443, 83085, 11218, 13, 330, 12331, 4588, 2324, 5097, 264, 2697, 43468, 11, 856, 25237, 1359, 1364, 1071, 13, 330, 6153, 584, 198, 19553, 12439, 311, 13520, 369, 264, 1396, 315, 1667, 11, 433, 374, 539, 4228, 311, 3974, 198, 2000, 1063, 832, 775, 1210, 46993, 7833, 6901, 505, 279, 15600, 17203, 1364, 574, 77218, 13, 2030, 3249, 1288, 279, 5333, 656, 682, 279, 43468, 7673, 330, 29819, 527, 810, 743, 1359, 1071, 8009, 24101, 11, 889, 1047, 2646, 1027, 743, 304, 4205, 198, 258, 1077, 2324, 13, 330, 2181, 374, 16127, 369, 1124, 311, 3041, 304, 13, 1628, 11, 315, 3388, 11, 42216, 198, 285, 9191, 11, 323, 813, 26870, 313, 1875, 10227, 2753, 1071, 922, 42216, 596, 26870, 279, 2731, 1359, 70939, 46993, 13, 330, 40, 198, 41866, 311, 617, 12502, 264, 15860, 315, 26870, 1210, 3005, 6688, 927, 1077, 32165, 287, 11, 323, 7731, 1523, 1160, 16117, 555, 279, 4027, 11, 1418, 198, 57856, 7882, 922, 11, 13326, 449, 279, 2678, 7640, 430, 35072, 1077, 13, 19198, 5672, 315, 42216, 596, 8396, 304, 27862, 15055, 1047, 34386, 46993, 198, 998, 54716, 13, 3005, 7731, 449, 48135, 6206, 323, 7111, 1139, 264, 3938, 430, 198, 325, 16809, 311, 2997, 4400, 719, 42216, 25, 42216, 33173, 449, 813, 11013, 1825, 280, 20577, 1195, 81109, 1603, 17954, 11, 323, 6348, 23349, 3156, 568, 1047, 1047, 813, 198, 79217, 26, 42216, 46772, 1251, 927, 279, 17222, 13, 1628, 1070, 574, 264, 40130, 3185, 311, 279, 6945, 1109, 430, 13, 2684, 574, 264, 11376, 198, 1073, 42216, 64802, 30666, 1139, 813, 3130, 323, 16054, 1139, 279, 8987, 198, 26894, 11, 539, 315, 63463, 2136, 8530, 11, 719, 315, 7172, 13, 3005, 7020, 1457, 430, 433, 1053, 3621, 1578, 323, 1578, 11, 439, 1317, 439, 568, 198, 75, 2270, 13, 578, 6661, 1364, 1047, 4036, 389, 198, 1964, 13306, 1938, 574, 27724, 1139, 1077, 8271, 13, 2684, 1053, 387, 430, 304, 279, 198, 21733, 2288, 11, 4762, 13, 3005, 574, 3339, 264, 34300, 43789, 198, 266, 23871, 13, 2030, 430, 13658, 315, 279, 1176, 1938, 520, 2162, 1364, 574, 198, 69769, 1908, 13, 3005, 574, 16089, 994, 24101, 4024, 323, 2163, 1077, 7636, 555, 1077, 4027, 13, 2030, 994, 1364, 6755, 264, 3094, 304, 279, 14321, 11, 1364, 9107, 279, 6134, 11937, 13, 3005, 198, 32345, 11075, 311, 3449, 42216, 449, 264, 15648, 13, 92014, 7263, 4400, 280, 32158, 1047, 9687, 430, 2736, 13, 330, 31516, 27603, 315, 16267, 1359, 814, 2663, 7724, 1093, 279, 832, 389, 279, 17569, 13, 3005, 30315, 1418, 11, 449, 813, 1203, 311, 1077, 11, 568, 198, 939, 1982, 5678, 1093, 264, 2294, 5679, 13, 1283, 31645, 1523, 520, 1077, 11, 813, 47626, 6548, 18186, 13, 330, 2181, 596, 1695, 311, 387, 2162, 323, 311, 1518, 499, 1578, 13, 43549, 956, 499, 2586, 304, 311, 856, 198, 11029, 7673, 330, 2460, 279, 810, 2944, 3249, 499, 1288, 2586, 1359, 1364, 39169, 8485, 398, 11, 323, 5762, 279, 198, 11020, 7029, 13, 578, 2697, 1370, 9761, 574, 71414, 449, 4027, 323, 8579, 50552, 11, 10107, 449, 198, 83014, 348, 2315, 2539, 315, 19837, 13, 735, 13, 25319, 4871, 323, 3952, 264, 9200, 198, 48554, 315, 279, 3130, 13, 330, 26556, 603, 584, 617, 1903, 264, 5128, 1695, 2683, 315, 420, 11, 358, 198, 4291, 279, 5684, 323, 279, 19358, 11, 323, 499, 449, 701, 5128, 73271, 323, 198, 22479, 5128, 659, 1210, 46993, 5602, 813, 14765, 11, 323, 574, 198, 71, 680, 1291, 1109, 1364, 1047, 1027, 369, 5672, 13, 3005, 2231, 389, 279, 16579, 2697, 79991, 198, 438, 1099, 2492, 430, 1051, 264, 961, 315, 1077, 313, 28708, 1077, 46175, 1579, 11, 7111, 709, 520, 198, 40617, 449, 279, 2697, 33352, 2840, 3095, 430, 1364, 1047, 1766, 1051, 39928, 389, 198, 20577, 1195, 13, 3005, 3177, 291, 279, 9090, 2922, 1141, 311, 1304, 15600, 11, 24465, 704, 279, 1888, 10716, 198, 2000, 1461, 11, 323, 281, 12400, 264, 45738, 449, 1077, 1664, 1824, 1636, 15548, 6206, 13, 330, 3112, 1518, 11, 1618, 596, 264, 4579, 267, 1786, 1210, 330, 40, 1097, 73011, 21901, 315, 1694, 16993, 1142, 1359, 1071, 735, 2637, 323, 5115, 293, 1091, 291, 304, 198, 26301, 502, 16975, 315, 1664, 33851, 13, 1115, 574, 2731, 1109, 813, 4384, 3130, 198, 455, 31234, 11, 1109, 490, 31557, 3235, 3224, 19795, 11, 1109, 813, 1866, 11555, 13, 330, 5519, 198, 73457, 757, 682, 279, 26681, 315, 279, 6825, 1210, 330, 3947, 706, 1027, 912, 26681, 2533, 499, 4024, 3201, 1359, 1071, 735, 13, 1628, 11, 1606, 198, 9739, 574, 16089, 539, 311, 387, 2163, 311, 813, 1866, 11555, 11, 814, 32627, 520, 420, 198, 4590, 315, 40711, 28485, 13, 330, 89441, 1359, 1071, 2009, 77638, 818, 11, 330, 906, 617, 1027, 1633, 11594, 13, 358, 617, 1047, 856, 198, 52710, 9408, 323, 1097, 1457, 48041, 10332, 304, 17510, 38502, 11441, 264, 2046, 13, 4702, 994, 358, 1047, 682, 856, 6848, 8521, 369, 198, 69, 333, 15247, 11, 358, 636, 17510, 38502, 323, 617, 311, 312, 70208, 1124, 13, 330, 2181, 374, 1633, 29395, 481, 994, 832, 596, 8070, 9221, 264, 23104, 1359, 1071, 198, 19875, 483, 29059, 989, 13, 3005, 574, 9455, 304, 2009, 77638, 818, 2555, 430, 1364, 4460, 1120, 1243, 313, 64, 198, 32127, 488, 11, 264, 3460, 315, 6904, 2968, 11, 430, 1047, 4400, 311, 656, 449, 198, 383, 402, 1918, 13, 3005, 6612, 430, 1618, 574, 264, 893, 1364, 1436, 7095, 11, 4661, 2389, 579, 198, 258, 13, 3005, 15262, 813, 1317, 6206, 11, 813, 559, 79736, 719, 1664, 43321, 15447, 11, 813, 7060, 198, 5478, 449, 1202, 3831, 46175, 13, 3005, 2163, 1022, 1077, 2697, 7958, 811, 11, 313, 64, 198, 2248, 311, 813, 1866, 6996, 315, 1124, 11, 313, 438, 7731, 1203, 304, 1077, 10716, 11, 10307, 198, 1820, 4027, 13, 3277, 735, 13, 14896, 11, 568, 1436, 3137, 1664, 13, 578, 2650, 288, 1047, 1027, 311, 91366, 389, 198, 50828, 13306, 8577, 13, 1283, 7020, 91366, 26, 430, 6688, 1124, 264, 4279, 5015, 13, 1666, 369, 735, 2637, 568, 42762, 14333, 198, 1820, 2697, 4034, 313, 67, 4361, 5678, 520, 1566, 449, 23023, 704, 315, 813, 10716, 13, 330, 2675, 3077, 1027, 1633, 6555, 311, 2610, 757, 304, 11, 18083, 13, 330, 40, 3987, 499, 198, 14724, 2187, 757, 311, 2586, 1578, 13, 2030, 11, 315, 3388, 11, 499, 527, 2133, 311, 387, 1633, 198, 75593, 1210, 1102, 9508, 311, 46993, 1364, 1053, 2646, 387, 8485, 1578, 13, 3005, 1550, 539, 198, 53757, 1461, 311, 733, 3201, 13, 578, 5222, 315, 813, 5655, 7899, 6688, 1077, 264, 5647, 315, 198, 17476, 13, 3005, 15262, 279, 1206, 13671, 315, 279, 1450, 568, 5762, 704, 311, 1077, 11, 994, 520, 198, 4354, 568, 1903, 264, 3351, 9017, 279, 6134, 13, 86631, 358, 1097, 14931, 568, 13942, 1057, 2697, 4717, 1359, 1071, 2009, 77638, 818, 13, 1666, 568, 8036, 279, 6134, 4920, 1461, 11, 1070, 574, 264, 502, 3177, 304, 46993, 596, 198, 58331, 13, 20695, 1051, 539, 1314, 11, 719, 11, 1306, 682, 11, 814, 1051, 539, 75323, 13, 3861, 198, 46068, 2103, 617, 4885, 11, 2466, 323, 3831, 11, 24981, 315, 8071, 323, 7899, 13, 3277, 198, 20577, 1195, 3782, 2162, 11, 279, 15648, 1364, 6688, 1461, 574, 539, 9770, 13, 578, 1938, 596, 43844, 290, 1047, 1027, 3958, 369, 24101, 13, 2009, 77638, 818, 1766, 1077, 389, 279, 198, 66, 3102, 304, 279, 24411, 52319, 48840, 11, 323, 6688, 1077, 264, 4062, 34522, 315, 198, 680, 11310, 2711, 13, 3005, 574, 463, 7069, 709, 1579, 449, 67026, 11, 449, 264, 16893, 315, 198, 277, 13795, 92003, 30488, 1077, 13, 330, 10156, 313, 8846, 315, 11745, 1359, 1364, 281, 7719, 13, 76996, 313, 285, 198, 5065, 2162, 313, 998, 87117, 26, 323, 313, 1820, 3885, 313, 20577, 1195, 323, 313, 1875, 4897, 574, 439, 3117, 439, 1364, 2751, 13, 735, 2637, 3821, 304, 1450, 11, 1766, 1077, 28334, 15792, 345, 928, 88, 11, 42408, 13, 1283, 1047, 1027, 10235, 369, 1063, 1778, 13147, 11, 323, 568, 198, 104402, 4588, 1139, 813, 3130, 369, 1097, 4010, 5392, 275, 7853, 13, 3277, 568, 3782, 1203, 1364, 574, 4661, 198, 359, 64589, 13, 1283, 14760, 279, 48739, 198, 258, 264, 43713, 11, 323, 5762, 433, 927, 1077, 3663, 13, 4740, 264, 892, 279, 993, 10753, 31467, 345, 8248, 1077, 3044, 14958, 57376, 13, 5340, 65670, 11, 889, 1047, 2586, 2162, 555, 430, 892, 11, 7731, 555, 279, 27240, 323, 5762, 1077, 198, 82, 1601, 596, 1450, 13, 8442, 3131, 304, 279, 1828, 6596, 477, 779, 1550, 1364, 6604, 13, 5340, 65670, 574, 2288, 289, 62837, 311, 198, 24467, 279, 6721, 11827, 304, 902, 735, 13, 743, 311, 990, 927, 24101, 13, 330, 40, 3077, 1027, 264, 1633, 2653, 13219, 311, 1077, 1359, 1364, 1071, 13, 330, 2746, 499, 649, 6958, 1077, 198, 20322, 11, 358, 3358, 1456, 311, 1304, 709, 369, 433, 1210, 46993, 7731, 389, 279, 32249, 4994, 11, 60178, 323, 63628, 13, 2435, 1047, 198, 25526, 369, 76996, 26, 719, 279, 2697, 3838, 1047, 912, 21186, 11, 323, 279, 1984, 198, 16514, 6435, 304, 3794, 1022, 13, 3279, 3782, 26346, 287, 709, 279, 32249, 323, 1139, 279, 3130, 13, 330, 11649, 11, 420, 374, 12703, 11, 5340, 65670, 1359, 1071, 2999, 13, 330, 10445, 304, 279, 836, 315, 32450, 345, 9493, 358, 5828, 956, 2212, 11, 3287, 956, 499, 636, 2500, 10896, 13, 1442, 1364, 1047, 1047, 1063, 198, 309, 4010, 5392, 275, 7853, 313, 1875, 7189, 6688, 1077, 1063, 25719, 7853, 315, 1097, 4010, 1359, 1071, 735, 13, 30666, 13, 330, 3947, 574, 2216, 912, 198, 1712, 311, 3708, 369, 21739, 13, 3005, 4661, 4024, 1234, 520, 4376, 2320, 561, 4330, 1210, 7639, 1047, 8774, 813, 3492, 11, 323, 1524, 2999, 13, 1283, 198, 70, 525, 264, 4062, 34522, 520, 420, 16615, 3995, 893, 889, 12570, 779, 30666, 315, 1148, 198, 383, 1047, 2884, 369, 279, 14363, 5333, 11, 323, 4024, 389, 449, 813, 990, 13, 76996, 11721, 264, 2697, 1306, 4848, 11, 323, 505, 430, 4545, 279, 22047, 304, 198, 1820, 14363, 48840, 574, 520, 459, 842, 13, 3005, 7882, 46993, 505, 279, 32249, 345, 2940, 45563, 389, 1077, 12387, 1886, 2914, 2011, 46764, 927, 1077, 26, 743, 5340, 65670, 311, 198, 86, 34002, 1077, 6691, 596, 4950, 323, 3794, 433, 5644, 26, 9107, 11276, 11, 7263, 198, 1382, 323, 11594, 13, 1628, 1243, 11, 449, 4648, 304, 1077, 6548, 11, 1364, 3952, 709, 1077, 198, 3571, 30488, 1077, 6691, 13, 1115, 574, 912, 892, 369, 584, 55323, 26, 430, 1053, 198, 2063, 3010, 13, 9843, 1364, 6656, 311, 735, 2637, 11509, 3821, 3725, 30488, 1077, 13, 330, 40, 1781, 499, 617, 3967, 420, 369, 264, 1317, 892, 1359, 1364, 1071, 13, 1628, 11, 994, 568, 198, 23770, 539, 4320, 25, 330, 10445, 1550, 499, 1095, 757, 4822, 3201, 505, 1077, 30, 1102, 1053, 617, 198, 82850, 1778, 264, 2697, 892, 9135, 330, 1687, 1051, 4560, 311, 656, 1057, 1888, 369, 2225, 315, 499, 1359, 568, 20592, 13, 1102, 3782, 439, 264, 16106, 505, 279, 43957, 315, 279, 198, 29652, 596, 502, 3217, 13, 330, 8100, 706, 1047, 779, 2697, 315, 2324, 1359, 1364, 1071, 11, 927, 323, 927, 13, 330, 6153, 682, 11, 76996, 1359, 568, 1071, 11, 330, 1820, 6825, 3507, 2324, 25, 279, 1917, 374, 1193, 198, 35676, 14708, 13, 3005, 1047, 3021, 323, 2262, 11, 323, 1364, 198, 32345, 499, 1210, 24101, 8636, 264, 2697, 1306, 33433, 11, 264, 11594, 12579, 11, 779, 430, 1193, 76996, 198, 438, 279, 1403, 3026, 7020, 994, 1364, 4024, 3201, 13, 3842, 46368, 311, 279, 51902, 13, 12220, 682, 430, 1317, 11714, 1364, 1047, 7731, 3411, 1203, 927, 1667, 315, 198, 9181, 653, 15674, 2136, 288, 13, 578, 270, 1540, 315, 24101, 596, 90325, 10603, 1047, 2744, 7222, 839, 198, 258, 1077, 26000, 13, 3005, 1047, 1027, 2653, 11, 66637, 442, 3876, 11, 64788, 291, 13, 9843, 568, 3463, 1364, 574, 38678, 287, 11, 323, 198, 42700, 311, 1077, 13, 3234, 499, 1781, 499, 1436, 636, 1124, 682, 704, 315, 279, 3130, 323, 198, 1169, 757, 617, 1077, 7636, 369, 1120, 264, 2478, 4520, 7673, 1283, 23803, 279, 3130, 11, 323, 3952, 709, 813, 38494, 4994, 279, 6134, 13, 1628, 11, 439, 568, 198, 52784, 1070, 11, 568, 3463, 315, 1148, 568, 1047, 1071, 311, 76996, 922, 279, 6825, 13, 5810, 304, 420, 1633, 3838, 1051, 4648, 323, 198, 325, 36235, 26, 5340, 65670, 596, 6917, 2111, 2324, 26, 46993, 323, 42216, 7314, 264, 198, 4930, 323, 75699, 3938, 3871, 26, 5678, 11, 264, 8060, 11, 323, 459, 82509, 269, 13, 3277, 568, 9107, 279, 6134, 1578, 11, 76996, 574, 11509, 555, 1077, 6691, 596, 4950, 13, 10455, 11879, 291, 311, 279, 15197, 13, 1283, 4024, 311, 1077, 11, 323, 1364, 6656, 323, 2231, 1077, 2010, 2403, 813, 17308, 198, 4908, 264, 19781, 1716, 13, 330, 18293, 757, 3201, 11, 735, 13, 1359, 1364, 1071, 20428, 5092, 398, 13, 1628, 11, 449, 813, 6916, 2212, 1077, 11, 568, 6197, 1077, 704, 315, 279, 3130, 13, 42251, 315, 1077, 2678, 14247, 12960, 24101, 596, 4648, 574, 20781, 6612, 13, 5340, 65670, 11953, 1203, 311, 1077, 198, 27243, 264, 13710, 1918, 315, 9090, 430, 1903, 433, 5107, 311, 11984, 449, 198, 1820, 2678, 30118, 811, 315, 1077, 1938, 13, 19292, 24101, 596, 40968, 4107, 11, 902, 1047, 198, 33222, 57130, 1077, 11, 1047, 1027, 7106, 13, 3005, 2011, 617, 1047, 1077, 12544, 264, 198, 4930, 1712, 13, 3005, 27569, 1023, 3278, 315, 279, 6825, 889, 1047, 1922, 418, 1555, 198, 483, 544, 5499, 2919, 11, 323, 1047, 520, 1566, 17551, 1523, 872, 64074, 323, 8036, 198, 50828, 23900, 6548, 11, 311, 279, 29869, 37473, 16409, 315, 872, 8689, 13, 3639, 1550, 198, 20670, 1781, 922, 11, 1521, 3278, 11, 439, 814, 3419, 34259, 922, 30, 14910, 814, 47540, 198, 1820, 3242, 266, 1873, 430, 2322, 872, 22171, 3252, 19567, 11, 279, 85273, 198, 9210, 1053, 539, 1518, 1268, 814, 1051, 22109, 30, 8166, 24014, 11299, 389, 5340, 65670, 596, 198, 69, 8958, 34740, 439, 433, 11203, 389, 1077, 22095, 13, 2876, 1193, 369, 24101, 313, 2000, 24101, 596, 198, 4490, 22583, 17277, 13, 1952, 76996, 313, 438, 304, 2753, 6767, 11, 315, 3388, 11, 389, 735, 33501, 67643, 279, 1972, 1437, 3935, 315, 198, 1820, 21426, 13, 76996, 8774, 709, 1664, 3156, 1306, 279, 32079, 11, 719, 4024, 1523, 198, 1820, 1828, 1938, 449, 264, 3428, 34653, 13, 3279, 1071, 11, 323, 58644, 398, 55844, 1037, 279, 8952, 198, 33871, 3156, 10280, 13, 29084, 323, 11714, 735, 13, 10717, 520, 1077, 6134, 323, 198, 258, 3004, 369, 1077, 11, 323, 6693, 323, 11714, 3782, 76996, 596, 10052, 25, 27567, 1, 43182, 2731, 13, 2030, 279, 2919, 38247, 389, 323, 1364, 1550, 539, 636, 922, 13, 6419, 31234, 11, 46993, 323, 42216, 1047, 10862, 389, 279, 4883, 315, 5209, 84028, 198, 75593, 26640, 13, 42216, 596, 330, 52905, 67, 1, 574, 264, 49277, 832, 13, 2684, 1051, 74413, 198, 438, 73806, 11, 2046, 13368, 3521, 76339, 311, 3224, 2902, 20340, 13, 578, 6825, 14264, 198, 4575, 50786, 311, 9298, 79648, 3009, 1603, 279, 2697, 3838, 520, 682, 198, 31482, 315, 279, 3814, 13, 32980, 35120, 31202, 11, 10043, 42216, 596, 1841, 11, 3952, 311, 198, 13772, 287, 33173, 520, 279, 13587, 304, 7353, 53121, 11, 323, 52468, 813, 91222, 198, 998, 813, 6691, 13, 330, 2675, 2646, 1440, 1405, 499, 527, 449, 1124, 7752, 1359, 568, 1071, 27851, 13, 330, 1687, 198, 2527, 704, 369, 4376, 459, 6596, 596, 1629, 304, 279, 11714, 11, 323, 636, 2162, 449, 279, 198, 76, 34263, 2695, 351, 2439, 13, 1628, 279, 810, 1063, 315, 1124, 617, 1047, 311, 7172, 11, 279, 810, 814, 198, 53757, 311, 6678, 279, 5780, 13, 1442, 358, 636, 264, 6140, 11, 358, 2846, 2133, 311, 9567, 433, 1418, 198, 1820, 10160, 596, 856, 1648, 1210, 2030, 11, 3137, 439, 568, 2643, 11, 304, 32980, 35120, 31202, 596, 29947, 4851, 1070, 574, 912, 198, 61665, 315, 24521, 290, 13, 42216, 1047, 2728, 1461, 264, 893, 596, 2683, 11, 323, 568, 1053, 198, 30133, 555, 433, 11, 912, 5030, 1148, 3782, 13, 2684, 1051, 1063, 2574, 430, 32980, 35120, 31202, 1550, 539, 3371, 813, 6691, 13, 2684, 1051, 59938, 994, 279, 86631, 1841, 574, 10409, 11, 539, 449, 46993, 198, 438, 1077, 4885, 11, 719, 449, 3278, 315, 264, 2204, 1917, 26, 59938, 994, 279, 198, 18481, 574, 539, 264, 3224, 12675, 11, 719, 264, 5754, 37002, 26, 59938, 994, 198, 88960, 35120, 31202, 11, 98555, 505, 279, 5696, 596, 10954, 555, 1063, 63463, 12822, 345, 41450, 3412, 10508, 311, 279, 55060, 1841, 323, 2019, 1778, 35603, 315, 34296, 198, 300, 568, 1436, 6227, 13, 32980, 35120, 31202, 11, 889, 1047, 3940, 2324, 449, 2478, 198, 484, 36807, 11, 574, 304, 8137, 315, 13490, 1778, 439, 568, 1047, 13, 3861, 1778, 3814, 46993, 2231, 304, 11, 21078, 15508, 3725, 304, 1077, 4950, 11, 1418, 279, 198, 21321, 389, 279, 26976, 301, 39674, 839, 6596, 1306, 6596, 1139, 279, 3814, 13, 1283, 3288, 264, 5296, 505, 279, 5274, 304, 279, 6693, 1473, 7189, 3987, 499, 527, 539, 18290, 11, 76745, 13, 578, 1841, 14760, 1523, 3221, 279, 198, 16813, 10349, 1566, 3814, 11, 323, 1070, 574, 4400, 311, 656, 719, 311, 8493, 279, 198, 9471, 1070, 13, 358, 1053, 617, 3288, 499, 3492, 11, 719, 358, 1550, 539, 1390, 311, 436, 1559, 198, 9514, 13, 3639, 656, 499, 2019, 311, 279, 27803, 311, 45427, 323, 87117, 49043, 7673, 3005, 8122, 764, 20262, 279, 14438, 10349, 430, 6693, 345, 438, 1766, 430, 42216, 1047, 539, 1027, 1070, 13, 2030, 11, 8051, 1364, 7020, 1457, 198, 9210, 568, 574, 31952, 2299, 1077, 11, 439, 568, 2744, 1047, 98884, 1077, 11, 439, 4762, 198, 383, 2744, 1053, 11, 1364, 20365, 33337, 311, 17302, 1461, 449, 1148, 1364, 7020, 13, 3005, 198, 939, 13430, 11, 439, 1690, 264, 5333, 706, 14362, 3200, 1603, 11, 505, 75867, 1461, 449, 813, 198, 11828, 13, 2030, 279, 2132, 892, 433, 7077, 11, 1364, 574, 436, 37588, 13, 1102, 574, 4661, 10280, 198, 3473, 11, 323, 76996, 574, 1664, 389, 279, 1648, 311, 13654, 11, 65355, 323, 1633, 198, 5902, 11, 719, 2133, 14297, 709, 323, 1523, 279, 62862, 389, 735, 3238, 82, 6916, 11, 323, 198, 82, 15154, 449, 5340, 65670, 323, 735, 13, 520, 279, 14177, 2007, 13, 3005, 574, 60206, 311, 387, 198, 1445, 389, 14523, 369, 10280, 11, 323, 735, 13, 6612, 430, 568, 1053, 617, 311, 3041, 1077, 198, 455, 5246, 13, 2468, 2380, 297, 63510, 832, 6693, 76996, 436, 37588, 505, 264, 3177, 6212, 311, 6865, 264, 198, 81, 3713, 389, 1077, 6134, 13, 3005, 11953, 264, 198, 66, 3397, 11, 323, 1603, 1364, 12570, 1364, 7111, 520, 76996, 596, 3821, 389, 279, 96803, 198, 2048, 13, 330, 40, 26253, 856, 9042, 574, 5076, 1359, 1364, 1071, 13, 330, 40, 1097, 14931, 311, 289, 3448, 499, 345, 68753, 3520, 11, 719, 358, 1541, 956, 1440, 1148, 311, 656, 1210, 76996, 1047, 3177, 291, 279, 6962, 323, 574, 21939, 389, 1077, 32623, 2427, 785, 13, 330, 4599, 568, 4024, 704, 1550, 568, 2019, 313, 1875, 27423, 1071, 4400, 13, 76996, 11, 358, 1097, 2133, 2162, 304, 279, 198, 57902, 1251, 1210, 330, 2675, 1541, 956, 3152, 430, 11, 656, 499, 7673, 330, 8161, 956, 358, 1427, 439, 422, 358, 3152, 433, 30, 2650, 1790, 315, 420, 3460, 315, 3245, 374, 264, 5333, 198, 13066, 3950, 311, 46753, 7673, 4314, 2574, 2744, 2873, 17936, 304, 279, 198, 20231, 315, 279, 3814, 11, 719, 555, 6693, 313, 1875, 19875, 483, 421, 404, 839, 389, 1077, 13, 1472, 6227, 279, 6661, 358, 2751, 389, 856, 13306, 198, 1316, 7673, 330, 22404, 19704, 433, 477, 539, 1359, 1071, 46993, 5679, 3640, 398, 11, 330, 9210, 596, 7041, 1148, 706, 198, 71, 680, 6901, 13, 358, 2751, 2555, 704, 315, 430, 2697, 11494, 315, 264, 35120, 31202, 8334, 11, 323, 198, 1820, 2800, 358, 1440, 1606, 358, 1440, 42216, 13, 578, 8952, 1047, 15972, 76996, 832, 3245, 25, 430, 433, 3952, 1690, 1274, 311, 198, 7072, 264, 1917, 11, 323, 430, 704, 315, 1521, 1063, 1051, 40605, 43510, 13, 2030, 198, 1805, 1047, 14958, 369, 1077, 264, 2867, 59851, 13, 2684, 1051, 1778, 1274, 345, 438, 1606, 832, 574, 304, 279, 1917, 369, 2532, 832, 42777, 369, 1124, 13, 7570, 198, 1820, 328, 9073, 1047, 1027, 3169, 311, 279, 5333, 315, 279, 14708, 13, 2030, 1618, 60845, 76996, 1766, 279, 2294, 51215, 315, 279, 1917, 313, 9210, 198, 28753, 315, 420, 17192, 279, 1695, 7831, 810, 1109, 279, 45077, 13, 330, 2181, 3727, 757, 12491, 682, 279, 3026, 304, 279, 1917, 13, 42216, 34672, 369, 499, 11, 323, 3686, 568, 649, 656, 264, 3245, 1093, 420, 9135, 46993, 574, 83240, 20233, 7162, 709, 323, 1523, 279, 3130, 13, 386, 486, 41957, 2200, 198, 32345, 779, 8942, 291, 1077, 13, 3005, 574, 1457, 11, 389, 279, 7479, 520, 3325, 11, 2753, 12304, 1109, 198, 68753, 3520, 13, 330, 7009, 527, 539, 682, 1093, 42216, 11, 9901, 32450, 1359, 1364, 1071, 13, 3092, 7126, 374, 832, 11, 323, 701, 735, 2637, 1618, 304, 279, 3838, 11, 374, 198, 43063, 1210, 2468, 3116, 297, 63510, 304, 279, 6693, 42216, 86631, 3782, 2162, 13, 3005, 198, 444, 7096, 291, 1461, 304, 1077, 7833, 4251, 65510, 323, 30315, 369, 1461, 311, 6604, 13, 330, 40, 1097, 14931, 311, 387, 779, 3389, 11, 11517, 1359, 568, 1071, 13, 330, 791, 2144, 374, 11, 358, 1097, 682, 304, 13, 358, 198, 16514, 10043, 279, 1841, 704, 31048, 39697, 6588, 13, 1226, 42423, 704, 264, 28387, 323, 279, 3245, 198, 42286, 927, 1210, 46993, 14000, 1243, 430, 813, 1314, 6916, 574, 21363, 81073, 555, 813, 3185, 13, 96424, 94937, 1432, 41672, 86631, 1047, 1027, 32620, 20250, 311, 3041, 709, 682, 813, 49683, 26870, 198, 4291, 813, 13306, 1938, 13, 763, 813, 1280, 337, 306, 11, 4856, 45743, 1648, 11, 568, 574, 1790, 198, 258, 3021, 449, 813, 7555, 13, 2030, 449, 279, 31352, 90700, 826, 315, 279, 1176, 4038, 315, 198, 5730, 8950, 1047, 2586, 264, 12876, 311, 387, 26893, 3131, 1578, 520, 813, 3663, 198, 970, 13, 32171, 1047, 4529, 1461, 11, 539, 369, 1148, 568, 574, 11, 719, 369, 1148, 568, 9508, 198, 998, 387, 13, 3005, 7020, 1461, 1457, 313, 543, 813, 2678, 198, 485, 337, 2436, 11, 813, 7958, 811, 11, 813, 44667, 13, 25929, 389, 11, 1093, 1023, 198, 65762, 2533, 279, 1917, 6137, 11, 1364, 1053, 4048, 311, 834, 16167, 11, 311, 7958, 311, 198, 9978, 19704, 1461, 1148, 568, 574, 539, 13, 1628, 779, 11, 1203, 311, 32171, 4848, 5672, 1306, 813, 13306, 1938, 3782, 42216, 198, 4438, 68, 11, 539, 449, 264, 24710, 311, 14195, 279, 2362, 5133, 11, 719, 369, 198, 884, 60635, 2200, 13, 46993, 274, 22537, 291, 313, 383, 4934, 1695, 26085, 26, 46993, 574, 72729, 519, 313, 383, 198, 87692, 25065, 26, 1364, 834, 35012, 315, 1461, 323, 8710, 1077, 834, 55539, 313, 383, 198, 87692, 14765, 13, 1283, 4934, 2324, 311, 387, 10882, 323, 71414, 11, 2085, 198, 2827, 6417, 12922, 11, 264, 2697, 990, 323, 1790, 1514, 11, 264, 7172, 994, 832, 574, 198, 339, 874, 88, 13, 28704, 13909, 3582, 433, 574, 11, 323, 18538, 389, 264, 5076, 8197, 11, 8530, 345, 33980, 304, 813, 4851, 42216, 596, 1193, 79422, 574, 369, 23871, 26, 719, 420, 198, 71, 67391, 2011, 387, 315, 459, 4642, 3460, 313, 1962, 2262, 11, 902, 374, 28979, 11, 719, 198, 268, 4215, 479, 13, 2360, 33605, 3318, 1202, 2010, 198, 1885, 369, 603, 13, 4702, 264, 2697, 1629, 927, 279, 3224, 19795, 11, 36346, 7673, 1102, 574, 279, 13658, 315, 279, 1938, 1603, 46993, 596, 3814, 4034, 311, 198, 68753, 3520, 13, 578, 5274, 1047, 1027, 8036, 11, 56612, 311, 264, 4648, 11, 323, 42216, 574, 304, 198, 981, 6045, 315, 264, 13560, 13, 330, 1687, 3358, 733, 704, 311, 279, 62930, 7278, 16344, 323, 617, 198, 13066, 716, 1210, 330, 4897, 596, 539, 837, 11, 32171, 11, 323, 499, 1440, 433, 1210, 578, 19795, 527, 20268, 2653, 26, 459, 6596, 596, 1629, 198, 18614, 279, 3224, 690, 4546, 701, 1933, 1203, 1210, 6122, 323, 12141, 449, 701, 7555, 1359, 1071, 279, 3828, 345, 438, 1344, 2234, 3201, 505, 1461, 13, 578, 1566, 2478, 5672, 1047, 10409, 704, 1077, 15792, 7216, 11, 719, 1364, 2103, 39786, 198, 376, 2492, 315, 1077, 17563, 13, 3005, 198, 7349, 291, 2917, 13610, 8334, 819, 11, 4661, 1877, 1752, 13, 9393, 1364, 5602, 1461, 289, 1736, 994, 1364, 9932, 46993, 11, 1077, 5986, 6940, 198, 43842, 1503, 13, 330, 2675, 636, 704, 315, 1618, 1359, 1364, 1071, 15187, 13, 330, 40, 3287, 956, 2610, 499, 311, 2586, 198, 1445, 13, 1472, 2744, 7020, 358, 1053, 617, 311, 32918, 1063, 1938, 1210, 358, 3287, 956, 6865, 904, 6821, 315, 499, 21363, 198, 20019, 279, 8952, 311, 4048, 1268, 358, 574, 3794, 3235, 1210, 31909, 11, 832, 315, 313, 1, 1283, 20365, 33337, 927, 813, 7555, 596, 836, 13, 330, 32, 198, 29652, 358, 1440, 1633, 1664, 574, 304, 279, 4967, 35789, 13, 2684, 1053, 617, 1027, 198, 1820, 38736, 311, 2343, 422, 358, 4265, 439, 1790, 439, 2663, 709, 1210, 330, 2675, 2646, 3309, 757, 499, 1051, 2133, 311, 636, 12502, 1210, 39348, 291, 11, 568, 43578, 459, 6916, 2212, 1077, 13, 330, 40, 8967, 311, 3371, 499, 11, 26828, 26, 719, 499, 2751, 14363, 13, 5884, 5269, 11, 358, 313, 40, 38674, 311, 198, 73457, 499, 11, 26828, 1210, 2684, 574, 264, 10882, 8430, 315, 198, 5065, 2162, 922, 2133, 1070, 1578, 13, 1628, 11, 1457, 430, 279, 12047, 9568, 315, 198, 50828, 6574, 574, 927, 11, 568, 574, 74106, 44467, 13, 2030, 32171, 8738, 311, 198, 2752, 8071, 287, 1461, 1794, 655, 398, 13, 330, 40, 3077, 2751, 2555, 311, 3371, 499, 1359, 1364, 1071, 13, 330, 8161, 956, 617, 264, 5052, 11, 323, 1541, 956, 198, 4355, 7595, 13, 1442, 499, 656, 11, 358, 3358, 313, 40, 3358, 7940, 704, 315, 279, 3321, 13, 358, 3077, 2751, 264, 2035, 304, 198, 64, 3637, 13, 3005, 574, 264, 6555, 3828, 323, 568, 574, 21901, 315, 1077, 13, 1628, 568, 574, 539, 653, 726, 819, 922, 433, 13, 1283, 1550, 539, 1390, 1077, 311, 9352, 311, 904, 832, 775, 13, 330, 4054, 315, 279, 37465, 304, 279, 8952, 11, 264, 9083, 5874, 11, 706, 2751, 757, 2555, 311, 198, 3055, 520, 445, 11696, 263, 323, 473, 2925, 5673, 596, 13, 358, 1097, 2133, 389, 369, 279, 6186, 4251, 6412, 13, 1442, 198, 40, 1304, 1695, 814, 690, 2567, 757, 1210, 1283, 1047, 2231, 1077, 16038, 2085, 264, 5965, 76, 26, 323, 1457, 568, 2322, 1077, 17480, 198, 4291, 14765, 13, 2435, 1053, 617, 264, 13560, 198, 998, 3522, 11, 323, 1243, 814, 1053, 2019, 1695, 1481, 9188, 13, 1283, 574, 3794, 1022, 1664, 11, 682, 2574, 6646, 13, 2030, 430, 4536, 956, 904, 198, 20489, 3249, 584, 13434, 956, 387, 4885, 11, 374, 433, 30, 358, 1053, 1093, 311, 2733, 430, 358, 649, 3009, 304, 1457, 323, 1243, 323, 2019, 1268, 656, 499, 198, 3055, 1210, 578, 6420, 315, 76996, 596, 836, 7263, 709, 304, 813, 4059, 46993, 439, 568, 1047, 198, 2414, 1077, 430, 6693, 13, 3005, 1511, 311, 387, 264, 1695, 10775, 345, 8248, 1364, 1047, 2646, 1027, 279, 1890, 2533, 279, 1938, 315, 279, 13306, 13, 1283, 3463, 198, 1964, 19451, 9017, 1461, 574, 832, 315, 38141, 13, 2030, 904, 4879, 389, 813, 961, 311, 282, 73555, 433, 1193, 2322, 449, 9439, 21847, 13, 330, 40, 3358, 3371, 499, 1148, 584, 3358, 656, 1359, 568, 1071, 13, 330, 1687, 2834, 956, 733, 311, 904, 315, 279, 2362, 198, 27170, 13, 358, 3077, 1766, 264, 502, 5754, 7830, 304, 279, 3224, 430, 596, 62486, 198, 268, 1409, 311, 7937, 21739, 13, 1226, 3358, 733, 704, 311, 5124, 3886, 596, 323, 636, 1063, 14177, 13, 1628, 389, 279, 1648, 704, 568, 12439, 709, 311, 279, 6661, 315, 198, 50828, 9306, 13, 578, 6671, 91826, 660, 1461, 25, 32171, 449, 1077, 502, 3805, 198, 1073, 35460, 11, 1077, 502, 54266, 1073, 2136, 26, 813, 10882, 1841, 26, 32980, 35120, 31202, 596, 198, 4338, 846, 295, 1203, 323, 5225, 25212, 13, 578, 18427, 1047, 682, 279, 54904, 315, 264, 502, 62366, 304, 433, 13, 1283, 12020, 279, 198, 29652, 449, 18440, 768, 11, 1550, 539, 32262, 994, 1364, 16436, 264, 36213, 11, 6612, 198, 6200, 363, 11559, 13053, 9373, 323, 506, 495, 519, 520, 279, 1890, 892, 13, 3277, 279, 1841, 24465, 709, 1603, 279, 5124, 3886, 2035, 11, 568, 43578, 264, 198, 53770, 54135, 4121, 1139, 32980, 35120, 31202, 596, 539, 927, 92900, 1450, 13, 330, 40, 1541, 956, 4059, 279, 25212, 1359, 568, 1071, 13, 1628, 198, 88960, 68941, 813, 4817, 304, 33638, 13051, 13, 330, 3947, 596, 1120, 3403, 315, 279, 8117, 304, 757, 1359, 1071, 32980, 11, 330, 998, 1440, 1268, 311, 198, 36341, 264, 2763, 323, 2019, 4400, 11, 4491, 13, 1283, 93270, 31161, 398, 704, 315, 279, 1841, 323, 10235, 311, 51358, 433, 13, 330, 40, 3358, 1120, 3041, 1077, 279, 364, 13486, 927, 6, 1457, 323, 1243, 1359, 568, 1071, 13, 330, 8100, 3358, 198, 52738, 6573, 422, 358, 1095, 1077, 2559, 1210, 32171, 1047, 8208, 709, 279, 15376, 1853, 311, 279, 3838, 13, 3005, 1047, 279, 8352, 315, 198, 20986, 1664, 304, 1077, 15447, 11, 323, 1077, 2678, 9072, 449, 1202, 1317, 934, 484, 198, 438, 1077, 9048, 23283, 266, 1051, 47994, 323, 10671, 13, 3005, 2646, 40552, 14715, 11, 439, 198, 19875, 483, 574, 43131, 311, 656, 13, 42536, 369, 42216, 11, 40788, 648, 1550, 539, 1518, 1461, 13, 362, 8987, 6063, 43165, 198, 11748, 291, 520, 279, 2007, 304, 279, 18397, 48840, 11, 1418, 40788, 648, 41778, 289, 81635, 304, 198, 1820, 9979, 13, 32980, 35120, 31202, 11, 2133, 2212, 279, 3185, 1853, 311, 279, 9979, 6134, 449, 198, 41115, 315, 4106, 11033, 323, 264, 3224, 87117, 369, 813, 20268, 23152, 11, 5602, 198, 1964, 1555, 279, 3321, 58218, 74820, 927, 279, 45115, 11, 323, 20365, 33337, 13, 5112, 11, 2085, 264, 3492, 11, 568, 9165, 59091, 291, 1203, 311, 279, 1841, 1578, 11, 323, 11, 72179, 198, 18614, 279, 8941, 83353, 11, 9960, 5678, 449, 73671, 13, 763, 813, 13365, 332, 3093, 4059, 1051, 198, 66, 7770, 2294, 29600, 11, 323, 32883, 311, 813, 19683, 574, 832, 13, 578, 4330, 198, 78289, 1590, 304, 813, 18301, 1047, 4400, 8996, 311, 656, 449, 433, 13, 2468, 63134, 568, 1047, 8040, 264, 19675, 315, 3116, 4339, 13, 1102, 3952, 279, 198, 2050, 315, 279, 18288, 18592, 11, 279, 18165, 7498, 1392, 11, 323, 279, 356, 349, 331, 2191, 13, 1102, 198, 16514, 25, 330, 70738, 701, 1866, 2626, 1210, 578, 18841, 315, 40788, 648, 596, 26092, 42761, 8173, 719, 1550, 539, 54904, 198, 40617, 13, 1442, 1364, 4934, 311, 656, 279, 3460, 315, 3245, 1364, 198, 16514, 3815, 11, 430, 574, 1077, 31017, 13, 40788, 648, 323, 1077, 6278, 57859, 31657, 11, 42216, 198, 4438, 68, 323, 32171, 313, 1820, 54050, 574, 539, 50383, 449, 1778, 12135, 13, 1102, 198, 1068, 291, 1124, 449, 25065, 3156, 814, 1051, 1766, 704, 11, 994, 433, 9408, 1202, 198, 89112, 13, 3082, 311, 813, 11471, 11, 42216, 289, 3448, 291, 279, 21811, 8334, 1603, 11888, 198, 78, 63510, 13, 32171, 1047, 35661, 2697, 323, 29850, 4400, 26, 719, 86631, 574, 10284, 198, 54754, 7913, 13, 330, 36227, 1077, 279, 364, 13486, 927, 49982, 568, 3309, 32980, 11, 330, 438, 1243, 733, 1203, 323, 46764, 198, 18614, 279, 73671, 1578, 13, 11205, 5208, 574, 6435, 323, 11413, 927, 279, 198, 11389, 19795, 11, 719, 994, 814, 8813, 279, 3314, 5754, 86631, 22982, 1825, 279, 198, 339, 28635, 13, 1283, 3952, 17393, 198, 438, 2751, 3201, 449, 1124, 11, 32681, 520, 279, 3828, 596, 6962, 1725, 315, 73082, 13, 330, 14524, 3156, 358, 636, 7953, 4567, 11966, 8078, 1359, 568, 1071, 11, 330, 438, 358, 3358, 1095, 1077, 704, 13, 578, 3828, 7731, 30488, 1461, 449, 1077, 6548, 8521, 8469, 13, 1283, 1047, 1027, 16558, 345, 438, 279, 41463, 315, 279, 45304, 574, 304, 813, 7899, 13, 3005, 574, 2133, 311, 1304, 1461, 3974, 709, 311, 279, 6661, 315, 813, 11471, 311, 198, 3427, 3201, 520, 279, 3838, 6134, 26, 323, 810, 323, 810, 1364, 15393, 430, 433, 1053, 198, 1395, 5107, 13, 12361, 315, 32681, 994, 198, 32158, 24465, 1203, 505, 264, 463, 3018, 291, 2211, 676, 11, 568, 6656, 1765, 398, 13, 95944, 3357, 5238, 198, 9210, 1364, 27569, 9922, 505, 813, 57426, 8839, 311, 279, 24359, 315, 813, 198, 33780, 13, 17830, 1364, 4295, 389, 264, 3197, 311, 1304, 1461, 3009, 15038, 304, 1077, 12818, 198, 438, 1095, 1077, 636, 704, 315, 279, 1841, 13, 4800, 433, 5946, 1124, 11, 323, 439, 198, 61917, 814, 5946, 433, 13, 42216, 596, 1841, 5675, 389, 198, 1820, 35231, 11, 719, 18661, 389, 279, 1317, 2237, 50699, 11, 902, 30103, 3690, 449, 264, 198, 1030, 1113, 315, 15792, 10054, 13, 330, 40, 6562, 499, 4265, 1095, 1124, 636, 8469, 11, 42216, 13, 330, 40, 3309, 499, 584, 4265, 5944, 311, 45427, 1210, 3639, 279, 409, 10743, 574, 279, 5030, 449, 198, 65762, 11, 90064, 30, 5810, 574, 32171, 439, 198, 708, 655, 439, 46993, 13, 5414, 3177, 1841, 1940, 1810, 291, 323, 17948, 279, 2466, 1841, 17345, 13, 1952, 264, 11113, 5754, 198, 66372, 4400, 810, 6129, 1109, 11102, 27275, 79838, 1053, 617, 1027, 279, 198, 1407, 13, 2030, 389, 279, 10054, 279, 2678, 1841, 60132, 291, 2212, 323, 61014, 927, 279, 198, 7334, 315, 279, 6201, 13, 2468, 279, 5740, 315, 279, 4051, 1968, 433, 6656, 927, 13, 86631, 28634, 5678, 323, 14980, 198, 261, 440, 11, 449, 832, 6916, 21363, 520, 813, 3185, 13, 2684, 574, 912, 5222, 520, 682, 505, 198, 1820, 8334, 1234, 279, 8941, 83353, 13, 6419, 279, 6201, 75803, 264, 8987, 11, 46298, 6374, 12970, 198, 18257, 11, 1317, 11977, 17919, 16038, 279, 20268, 23962, 315, 12690, 13, 3277, 568, 198, 265, 3939, 279, 1841, 11, 507, 68070, 5169, 1766, 32171, 11961, 31175, 5757, 389, 279, 5015, 13, 763, 279, 198, 86, 25662, 315, 279, 1841, 279, 50552, 1047, 539, 1027, 56807, 3384, 11, 323, 555, 872, 3177, 198, 383, 1903, 704, 86631, 11, 2064, 17718, 294, 8934, 1570, 13, 578, 1023, 3697, 315, 507, 68070, 5169, 596, 4717, 1047, 93270, 1523, 279, 6201, 555, 430, 198, 1712, 13, 3161, 279, 12576, 315, 264, 26128, 11, 814, 2751, 279, 1841, 709, 13, 32980, 35120, 31202, 11203, 198, 67, 5599, 839, 389, 813, 3663, 30456, 13, 3277, 568, 3782, 311, 323, 9107, 813, 6548, 345, 87643, 4661, 97946, 1247, 291, 449, 16337, 13, 330, 40, 2846, 682, 1314, 1359, 1071, 32980, 35120, 31202, 13, 1628, 11, 994, 814, 9076, 1461, 198, 1336, 47190, 25, 330, 79384, 449, 279, 4027, 55051, 13, 358, 313, 40, 313, 1, 362, 993, 10753, 315, 198, 92724, 38817, 813, 3663, 13, 3161, 813, 11977, 568, 30831, 198, 40617, 726, 311, 264, 11961, 2361, 11, 323, 11299, 1203, 1578, 13, 96424, 53694, 5660, 1432, 1383, 10280, 6187, 76996, 574, 1203, 304, 279, 8952, 11, 264, 2697, 78853, 11, 719, 198, 838, 5485, 398, 11075, 311, 2567, 1077, 2324, 311, 1202, 1906, 315, 2532, 13, 3005, 1047, 264, 198, 36341, 449, 735, 13, 279, 3814, 1603, 1364, 2163, 13, 45563, 574, 704, 11, 323, 76996, 1047, 2231, 279, 18397, 48840, 304, 2015, 13, 735, 13, 7731, 555, 198, 1820, 2007, 323, 15746, 1077, 439, 1364, 7882, 922, 279, 3130, 13, 578, 3347, 2478, 5672, 1047, 1027, 1633, 11364, 311, 1461, 25, 311, 1520, 1077, 709, 323, 198, 2996, 279, 32249, 11, 311, 1373, 311, 1077, 304, 279, 59938, 439, 1364, 11203, 389, 279, 27240, 198, 258, 279, 52319, 48840, 26, 3010, 11, 439, 1364, 13241, 11, 311, 4546, 2678, 294, 62124, 2162, 198, 2000, 1077, 35788, 11, 323, 11, 3515, 14980, 927, 45563, 1418, 1364, 30205, 1124, 11, 311, 198, 68760, 1124, 304, 38586, 311, 430, 8582, 3130, 313, 383, 1047, 539, 1027, 779, 6380, 304, 198, 42820, 13, 330, 40, 3987, 499, 1541, 956, 2733, 439, 422, 499, 2011, 4822, 389, 1359, 1364, 1071, 74884, 13610, 13, 330, 2688, 198, 9210, 584, 1541, 956, 1390, 499, 313, 9514, 1440, 2731, 1109, 430, 1210, 330, 3947, 374, 912, 2035, 775, 304, 279, 4459, 1917, 430, 358, 1390, 311, 733, 311, 1359, 568, 198, 84359, 5042, 13, 330, 40, 2873, 311, 387, 2744, 39661, 389, 18570, 596, 45972, 311, 313, 998, 2567, 2574, 198, 998, 3522, 13, 5629, 11, 369, 1667, 323, 1667, 11, 433, 574, 71322, 5340, 65670, 26, 1457, 433, 374, 198, 9514, 1210, 330, 8161, 956, 499, 13383, 430, 11, 4619, 315, 701, 1694, 26259, 311, 757, 11, 433, 374, 198, 40, 889, 1097, 2073, 34229, 2915, 26259, 311, 499, 30, 358, 617, 12439, 198, 20019, 313, 258, 2204, 7634, 323, 304, 2204, 5627, 13, 358, 1053, 4856, 387, 198, 6881, 1109, 12660, 775, 304, 279, 1917, 1210, 2684, 574, 779, 1790, 430, 574, 75323, 304, 813, 198, 58331, 430, 568, 1550, 539, 1390, 1077, 311, 1518, 13, 3005, 1053, 387, 5115, 13171, 11, 568, 198, 83, 820, 5678, 11427, 351, 989, 11, 315, 78849, 1461, 704, 315, 33638, 58043, 422, 1364, 3596, 198, 8890, 22456, 13, 1628, 568, 574, 16984, 313, 2642, 14152, 11, 2533, 568, 4934, 1077, 779, 1790, 313, 9210, 568, 198, 41450, 387, 24931, 323, 7621, 2785, 3403, 311, 1935, 1077, 1524, 389, 1884, 3878, 13, 20696, 574, 5644, 369, 1077, 471, 311, 279, 8952, 13, 3005, 1047, 1027, 704, 198, 9210, 1938, 311, 2231, 19837, 389, 279, 11594, 25165, 1405, 24101, 11203, 449, 48135, 198, 89112, 26, 1364, 1047, 1903, 1077, 4883, 315, 2697, 21728, 389, 279, 6825, 26, 323, 1457, 198, 1964, 7937, 39585, 11, 19937, 11, 574, 304, 279, 14321, 13, 330, 644, 832, 1648, 11, 433, 690, 387, 264, 2697, 2731, 369, 499, 1109, 422, 46993, 323, 198, 20577, 1195, 1051, 539, 304, 279, 3838, 13, 330, 8100, 13452, 499, 11, 735, 13, 3005, 14117, 389, 499, 11, 2288, 11, 5423, 2533, 430, 3814, 198, 9493, 499, 3952, 2512, 315, 42216, 596, 6916, 1603, 584, 2751, 2999, 13, 358, 3629, 1781, 345, 42, 2637, 1148, 264, 1695, 10896, 499, 1053, 617, 1027, 13, 1472, 7020, 779, 1664, 1148, 311, 656, 198, 2000, 6691, 1210, 3005, 2103, 1436, 539, 7095, 1077, 7899, 922, 1077, 6691, 13, 330, 20577, 1195, 596, 6916, 374, 2133, 311, 387, 5115, 7833, 13, 3279, 374, 779, 12691, 315, 7639, 198, 2017, 433, 13, 9843, 520, 3325, 11, 15716, 814, 1051, 198, 998, 3522, 11, 1364, 7263, 7639, 1139, 279, 10652, 13, 1283, 374, 198, 88657, 11, 1541, 956, 499, 1781, 7673, 330, 26840, 1359, 1071, 735, 382, 1271, 3665, 813, 2324, 11, 568, 1436, 539, 2231, 904, 41463, 1139, 813, 7899, 13, 1102, 574, 539, 304, 3823, 7138, 311, 1755, 810, 315, 1461, 13, 330, 23025, 1317, 13739, 499, 617, 11, 9495, 304, 701, 3130, 313, 12840, 304, 279, 1917, 656, 499, 198, 36341, 922, 30, 3005, 574, 264, 2697, 41690, 315, 1884, 59938, 11, 994, 1364, 7731, 7636, 11, 477, 198, 9493, 5340, 65670, 11, 11961, 449, 1077, 11, 1903, 68782, 1234, 279, 29062, 311, 279, 198, 582, 45349, 3904, 315, 264, 24981, 2854, 315, 61165, 24196, 505, 4028, 279, 14321, 13, 7639, 3782, 304, 2744, 11, 1603, 568, 4024, 345, 438, 11, 49448, 927, 279, 1203, 315, 264, 10716, 11, 1053, 6179, 1077, 315, 279, 10973, 198, 10399, 2136, 315, 2324, 304, 279, 8952, 2085, 1077, 13, 330, 40, 733, 1475, 1938, 1606, 358, 2011, 1359, 568, 1053, 36015, 1077, 8485, 398, 26, 330, 8248, 11, 358, 3371, 198, 9514, 11, 279, 10885, 374, 8208, 704, 315, 433, 13, 3277, 1070, 574, 264, 6140, 430, 1475, 2107, 198, 16514, 21592, 2107, 11, 279, 17983, 5208, 3235, 264, 46979, 6244, 57399, 1210, 1283, 198, 32345, 264, 7362, 14397, 315, 21939, 704, 813, 6206, 11, 449, 264, 2697, 14362, 773, 315, 198, 1820, 28004, 13, 568, 1071, 313, 8370, 11, 1694, 25548, 11, 3445, 512, 31437, 279, 38736, 596, 279, 1005, 9135, 1628, 735, 13, 1053, 2559, 304, 279, 70458, 11, 30666, 20149, 11, 477, 733, 1203, 311, 813, 198, 3039, 323, 5409, 3201, 304, 813, 38411, 279, 2294, 6063, 6603, 389, 15173, 449, 198, 8370, 568, 323, 7639, 1047, 1027, 3318, 704, 264, 1162, 13, 2100, 735, 13, 7731, 555, 279, 18397, 48840, 2007, 323, 34793, 311, 1077, 3137, 315, 7639, 430, 198, 4354, 11714, 3871, 13, 35120, 31202, 311, 11477, 539, 311, 387, 2288, 1790, 64770, 922, 198, 88960, 13, 4800, 430, 499, 527, 198, 21470, 4885, 1359, 313, 32158, 91772, 1461, 289, 380, 3725, 11, 313, 1, 66372, 1063, 1938, 499, 690, 2586, 198, 998, 832, 315, 813, 7677, 13, 7570, 422, 499, 3287, 956, 3619, 7041, 11, 358, 1440, 198, 275, 1053, 54904, 499, 13, 1628, 313, 40, 4265, 1093, 499, 311, 1518, 757, 304, 856, 14113, 11, 735, 13, 1472, 198, 37593, 617, 1210, 3005, 14264, 264, 2697, 12703, 439, 279, 11714, 4024, 389, 13, 3005, 574, 2133, 311, 3194, 735, 627, 1225, 1790, 13, 6104, 1364, 574, 5986, 1364, 1047, 15746, 279, 9042, 369, 279, 892, 311, 198, 37437, 369, 1461, 13, 3005, 7020, 279, 1648, 568, 50365, 279, 4156, 6134, 13, 3005, 7020, 2288, 430, 11, 1120, 1306, 264, 26518, 430, 21699, 198, 1820, 1633, 9168, 304, 279, 1380, 316, 11, 735, 13, 1053, 2586, 311, 279, 4579, 315, 279, 32249, 198, 438, 1650, 25, 27567, 1, 25797, 2303, 11, 1070, 9135, 330, 32, 9188, 11, 264, 9188, 1359, 1364, 1053, 4320, 313, 8370, 574, 11, 568, 28478, 1077, 11, 279, 6300, 198, 2376, 13, 13440, 568, 3782, 709, 279, 32249, 520, 3131, 477, 3952, 813, 1648, 1203, 311, 45563, 1047, 198, 15237, 2954, 389, 3508, 813, 35491, 369, 279, 1938, 574, 14098, 477, 10437, 65, 31458, 13, 1283, 1053, 3194, 1077, 345, 37227, 26, 719, 568, 1053, 617, 5340, 65670, 323, 46993, 323, 313, 6102, 13, 6984, 304, 264, 12960, 198, 998, 7639, 11, 315, 3388, 13, 3005, 29676, 11, 430, 1566, 11714, 11, 389, 11961, 709, 449, 1461, 3156, 33433, 198, 1136, 12616, 304, 10280, 6187, 13, 46993, 323, 42216, 1051, 704, 26, 5340, 65670, 11, 3515, 198, 29844, 291, 76996, 449, 264, 97045, 430, 1047, 1027, 2163, 927, 304, 279, 8221, 505, 198, 1820, 42774, 596, 2626, 11, 1047, 379, 6513, 291, 11937, 311, 4950, 13, 3277, 279, 61794, 7376, 33433, 11, 76996, 436, 37588, 449, 264, 1212, 13, 3005, 198, 8110, 1534, 430, 14188, 315, 1124, 1047, 22066, 11, 323, 430, 735, 13, 578, 2697, 9042, 389, 279, 28745, 3952, 709, 279, 23104, 315, 279, 198, 65294, 288, 11, 323, 17948, 279, 6596, 304, 4062, 357, 4575, 4428, 8554, 13, 76996, 16392, 323, 4024, 927, 311, 735, 2637, 1077, 3776, 8679, 304, 8579, 61607, 922, 198, 1964, 13, 76996, 2163, 279, 2697, 3838, 520, 198, 51464, 11, 449, 279, 8761, 3177, 2103, 20252, 1555, 264, 8770, 315, 16054, 12056, 13, 578, 8952, 78251, 323, 75752, 1051, 2103, 3177, 291, 994, 1364, 4024, 389, 198, 67, 20850, 520, 8254, 297, 63510, 13, 3005, 1047, 1027, 12893, 311, 279, 3026, 596, 34933, 26741, 345, 438, 4024, 1070, 520, 3131, 13, 3005, 1047, 539, 3970, 22770, 22983, 36627, 2533, 1077, 198, 27510, 596, 4648, 26, 719, 1364, 1766, 1077, 389, 14523, 304, 279, 34933, 26741, 13, 1789, 279, 198, 5686, 892, 304, 3116, 4038, 11, 279, 1403, 7724, 1051, 3318, 3185, 555, 3185, 13, 76996, 596, 1421, 1947, 315, 1077, 3766, 2532, 1234, 22770, 22983, 1903, 1077, 198, 77, 651, 788, 13, 330, 1687, 1051, 682, 14931, 311, 6865, 315, 701, 12544, 1359, 1364, 1071, 13, 330, 40, 3987, 584, 4985, 198, 456, 389, 29921, 1210, 2468, 279, 3117, 842, 1403, 272, 2469, 198, 32345, 1027, 9277, 13, 330, 791, 26741, 374, 8987, 11, 4536, 956, 433, 7673, 2684, 527, 2380, 315, 198, 355, 313, 9514, 11, 7182, 11, 323, 264, 47027, 261, 1210, 578, 1176, 3177, 315, 279, 10280, 6693, 574, 5108, 1555, 279, 11276, 13, 22770, 22983, 2231, 704, 279, 13001, 323, 6656, 304, 264, 2626, 12970, 1648, 311, 1077, 198, 27295, 13, 330, 791, 47027, 261, 596, 836, 374, 27738, 9336, 1359, 1364, 1071, 13, 330, 32576, 499, 4265, 2731, 198, 8823, 1077, 449, 279, 17954, 82, 13, 1442, 1070, 596, 904, 1648, 311, 1304, 264, 16930, 11, 1364, 198, 80785, 433, 1210, 1102, 574, 1306, 8223, 994, 76996, 1766, 32980, 35120, 31202, 13, 5414, 6453, 11, 17345, 1448, 57658, 6548, 198, 7349, 291, 520, 1077, 505, 264, 28639, 3663, 13, 330, 40, 574, 304, 264, 879, 3130, 26, 719, 433, 2853, 27219, 628, 15655, 264, 2046, 11, 779, 358, 7882, 13, 3005, 1047, 37287, 311, 733, 11, 719, 735, 627, 32345, 28932, 2403, 433, 13, 3005, 574, 539, 3831, 11, 323, 1364, 1047, 2736, 16654, 198, 59178, 13, 1628, 1457, 279, 990, 315, 279, 26741, 17770, 2653, 13, 3005, 14980, 30488, 1461, 323, 31511, 48259, 813, 1450, 13, 1283, 92840, 311, 1781, 430, 1077, 47084, 574, 369, 813, 4498, 505, 279, 12675, 198, 1073, 264, 879, 8893, 311, 279, 1949, 26741, 13, 330, 12174, 11, 358, 2846, 682, 1314, 11, 9083, 76996, 1359, 568, 1071, 13, 86631, 374, 12798, 4848, 198, 78289, 1590, 264, 2046, 369, 757, 13, 578, 6811, 1990, 757, 323, 279, 1023, 87819, 198, 20019, 1618, 374, 430, 358, 636, 264, 26944, 8148, 389, 856, 35788, 323, 814, 1541, 956, 1210, 330, 42560, 11441, 264, 2046, 369, 264, 26944, 8148, 374, 2133, 1063, 13, 358, 2846, 912, 14545, 660, 198, 277, 380, 73632, 26, 358, 1541, 956, 617, 311, 617, 264, 26944, 8148, 1210, 330, 12389, 814, 3309, 499, 1148, 279, 12544, 374, 7673, 7639, 17882, 374, 2133, 311, 198, 86299, 389, 757, 13, 3639, 264, 3245, 433, 574, 198, 998, 387, 3025, 311, 1935, 420, 2324, 3502, 97586, 315, 32980, 35120, 31202, 596, 323, 1304, 433, 198, 14789, 1578, 0, 2052, 21522, 315, 3026, 1903, 709, 76996, 596, 1917, 25, 279, 2761, 301, 31095, 889, 82294, 198, 20322, 279, 26741, 304, 1344, 3713, 1776, 32997, 11, 1160, 16117, 15691, 92825, 26, 279, 198, 359, 939, 5389, 3026, 304, 279, 28036, 11, 3411, 4741, 311, 2500, 1938, 315, 90926, 11, 422, 198, 1962, 315, 6784, 26, 42216, 86631, 449, 813, 11102, 6916, 26, 735, 2637, 28682, 323, 3831, 11, 719, 198, 7712, 287, 912, 33397, 2035, 304, 279, 1917, 13, 22703, 287, 927, 1124, 682, 574, 279, 198, 9514, 65167, 17882, 13, 1283, 8967, 369, 1077, 11, 430, 10280, 6693, 11, 682, 430, 279, 198, 1605, 3026, 1051, 539, 313, 998, 872, 23948, 8333, 11, 25775, 11, 59772, 11, 2410, 13, 32980, 35120, 31202, 11203, 1203, 389, 279, 67026, 323, 15746, 1077, 3663, 13, 330, 4599, 358, 574, 264, 10585, 1359, 568, 1071, 11, 330, 438, 10837, 3235, 279, 6825, 11, 8260, 2999, 13, 7639, 198, 64, 36157, 11, 358, 2646, 3463, 358, 4265, 10457, 1618, 10307, 430, 6134, 311, 1518, 1461, 2586, 198, 258, 13, 82251, 956, 433, 279, 15123, 315, 264, 1917, 11, 90064, 30, 1102, 198, 467, 956, 1790, 315, 264, 10280, 311, 499, 11, 3060, 1210, 76996, 23114, 1461, 813, 6693, 25309, 15600, 11, 323, 11, 1606, 1077, 6548, 10409, 709, 198, 4291, 24014, 1457, 323, 1243, 520, 813, 1520, 33028, 11, 1364, 574, 539, 779, 10151, 1285, 439, 198, 32158, 2643, 617, 1027, 13, 3277, 832, 46605, 1285, 1047, 8208, 1523, 813, 13272, 11, 568, 31645, 198, 455, 520, 1077, 70205, 2740, 13, 1666, 1790, 439, 574, 3284, 11, 279, 8952, 66529, 389, 430, 10280, 6187, 13, 578, 198, 56544, 288, 4024, 922, 304, 7878, 4251, 78878, 449, 8314, 954, 315, 8770, 273, 998, 68, 304, 198, 50828, 3215, 75790, 11, 3815, 2478, 8679, 826, 13, 6193, 279, 8582, 27035, 11, 1405, 279, 198, 74, 49721, 1051, 7559, 11, 9041, 9017, 38245, 279, 1672, 80227, 54097, 315, 938, 15067, 198, 66467, 10786, 13, 7357, 26741, 1047, 1202, 93484, 315, 305, 8788, 13, 763, 279, 13658, 11, 3600, 198, 52898, 5762, 304, 279, 83249, 63891, 13, 33442, 11843, 4825, 1903, 872, 6435, 5208, 3235, 75752, 323, 1523, 198, 68, 3532, 3046, 13, 30088, 3916, 39428, 889, 1051, 3025, 311, 4321, 1344, 5795, 3235, 304, 28954, 198, 3306, 32997, 13, 42251, 279, 7029, 14365, 315, 279, 46979, 198, 1820, 13587, 11843, 4825, 1051, 28902, 304, 264, 5347, 292, 7219, 13, 43474, 1124, 11, 26435, 369, 198, 1820, 13402, 11, 1051, 279, 39736, 89069, 11, 279, 2015, 49201, 11, 323, 6295, 3842, 11, 889, 198, 11513, 588, 279, 51632, 13, 1952, 832, 3185, 315, 279, 60779, 11, 3221, 279, 4156, 11, 7731, 279, 37465, 304, 7123, 11, 304, 198, 5192, 13391, 19079, 323, 7878, 45233, 13, 1952, 279, 1023, 3185, 1047, 1027, 4694, 264, 198, 2050, 369, 279, 5687, 13, 578, 2655, 288, 14980, 1203, 2403, 279, 7147, 11, 5644, 311, 198, 6236, 704, 1990, 48041, 292, 826, 11, 439, 433, 1051, 313, 2000, 264, 36213, 477, 459, 51632, 198, 6797, 11, 439, 279, 1162, 2643, 387, 13, 6193, 4395, 2967, 6879, 279, 1306, 1773, 4481, 9096, 315, 10280, 13658, 13, 578, 37465, 29340, 11, 323, 76996, 29340, 449, 1124, 11, 1077, 7878, 3995, 7899, 16448, 198, 49532, 279, 2800, 13, 26541, 12688, 40120, 3782, 1555, 279, 61152, 2427, 448, 198, 28176, 323, 559, 606, 389, 1077, 17104, 74820, 3663, 11, 1077, 11113, 34801, 62626, 11, 1077, 198, 11600, 11, 2744, 1120, 264, 2697, 1384, 894, 13, 7639, 11, 84244, 287, 2403, 279, 7147, 11, 4028, 279, 83249, 11, 1766, 813, 6548, 198, 496, 17718, 9017, 1077, 15320, 13, 3639, 198, 64, 81631, 369, 5496, 323, 369, 23871, 1364, 1047, 0, 578, 97054, 63584, 1543, 1373, 279, 11266, 29400, 1473, 1, 1016, 283, 34143, 10456, 71329, 11, 323, 38674, 304, 5118, 488, 26, 9093, 4359, 11, 1524, 198, 27520, 4359, 11, 52677, 459, 85079, 40344, 449, 279, 5707, 315, 16089, 2136, 3485, 26236, 87819, 1210, 3005, 574, 1695, 11, 323, 1364, 1047, 1027, 459, 85079, 449, 279, 5707, 315, 198, 6200, 329, 2136, 13, 1628, 568, 27567, 16366, 10868, 574, 26139, 13, 5414, 5655, 22253, 7899, 11, 539, 2744, 837, 11, 712, 25111, 198, 412, 3485, 279, 5222, 315, 279, 2678, 2942, 13, 3279, 1047, 1027, 264, 1695, 10868, 311, 198, 40617, 26, 568, 1047, 1027, 264, 1695, 4538, 13, 7639, 596, 17764, 35534, 4059, 82294, 3201, 505, 279, 2532, 311, 279, 6945, 315, 813, 198, 27510, 927, 813, 10868, 596, 39682, 291, 18496, 11, 311, 279, 6825, 11, 311, 735, 2637, 311, 279, 198, 29652, 889, 1047, 16436, 311, 32918, 1461, 1606, 1364, 1550, 539, 7095, 1461, 11, 311, 198, 77264, 22983, 1566, 315, 682, 13, 1283, 6656, 264, 2697, 323, 10837, 813, 6548, 3235, 279, 1584, 198, 1073, 37465, 13, 1666, 422, 1364, 1051, 17371, 315, 813, 36752, 11, 1364, 30831, 198, 1964, 2010, 323, 65657, 9017, 1461, 13, 578, 37465, 29340, 25, 27567, 257, 330, 46, 27823, 9576, 315, 96058, 0, 32285, 408, 311, 603, 11, 584, 24739, 280, 415, 11514, 704, 1057, 7589, 11, 323, 3810, 304, 345, 415, 2893, 9405, 304, 603, 311, 11477, 1210, 578, 13587, 11843, 4825, 323, 5804, 3916, 39428, 934, 7403, 291, 279, 11537, 4339, 13, 3279, 596, 198, 89945, 28691, 36014, 449, 55349, 2136, 13, 578, 11452, 11, 11961, 264, 2697, 10980, 449, 1077, 6206, 48135, 304, 1077, 24301, 323, 198, 906, 661, 449, 279, 16066, 315, 279, 1917, 11, 8036, 1077, 6548, 323, 34793, 13, 578, 10280, 6693, 1047, 7263, 76996, 4376, 264, 21030, 21258, 13, 735, 13, 3288, 1077, 198, 64, 15310, 93297, 1162, 449, 1077, 1647, 13255, 11, 46993, 264, 27306, 18327, 13, 2030, 279, 8352, 315, 21258, 11, 927, 902, 76996, 596, 6548, 1047, 2840, 13111, 11, 574, 264, 198, 47991, 3830, 315, 61741, 13160, 304, 2999, 13, 7639, 596, 24166, 12, 1787, 4477, 11, 330, 3915, 264, 198, 37569, 1210, 350, 40458, 304, 279, 8579, 61607, 315, 1077, 34801, 62626, 574, 832, 315, 279, 61741, 430, 198, 10924, 12684, 13, 7639, 574, 8748, 369, 76996, 304, 279, 198, 6133, 83536, 13, 1198, 32158, 65657, 1523, 311, 279, 16392, 198, 32158, 28670, 13, 330, 791, 3885, 1304, 279, 1455, 70960, 2766, 315, 1933, 304, 279, 26741, 1210, 330, 7009, 527, 539, 904, 279, 2753, 10705, 1606, 358, 1097, 20806, 1023, 1274, 617, 264, 198, 331, 685, 311, 4774, 1124, 1210, 9636, 682, 813, 8485, 2676, 568, 574, 2917, 13610, 3722, 1748, 449, 1077, 13, 2052, 279, 5128, 198, 89325, 288, 568, 1053, 617, 1903, 311, 22770, 22983, 1234, 279, 13463, 8636, 198, 15145, 1077, 26875, 34522, 13, 2684, 1051, 1690, 2574, 568, 4934, 311, 2019, 311, 1077, 13, 1283, 4934, 311, 3371, 1077, 198, 9210, 568, 574, 14931, 1077, 6691, 1047, 8636, 26, 430, 279, 6825, 574, 4384, 2085, 198, 1964, 26, 430, 568, 7111, 4741, 311, 1521, 7446, 16659, 449, 1077, 439, 264, 27823, 198, 1543, 311, 813, 6596, 1603, 813, 36959, 13, 3639, 568, 2216, 1071, 574, 311, 68270, 198, 91627, 989, 3508, 1364, 1047, 1047, 1077, 10280, 14177, 13, 76996, 91772, 1461, 11, 4376, 86324, 11, 4376, 13194, 13, 2209, 433, 3958, 369, 26434, 369, 603, 311, 387, 1695, 198, 30562, 7673, 25681, 304, 1077, 6548, 436, 37588, 198, 1820, 38736, 315, 95046, 430, 2744, 1776, 900, 291, 304, 1461, 13, 330, 5159, 1841, 596, 1027, 68941, 304, 264, 12056, 3696, 2130, 19441, 2533, 4216, 420, 6693, 345, 438, 358, 617, 3279, 596, 86652, 304, 264, 20328, 1108, 13, 10435, 389, 701, 2574, 323, 2586, 369, 264, 198, 1425, 1210, 1283, 26253, 22770, 22983, 1436, 6865, 1148, 568, 1071, 26, 311, 387, 3738, 315, 433, 11, 568, 198, 14991, 9824, 398, 9408, 813, 7899, 264, 490, 19636, 13, 15469, 4024, 1203, 311, 279, 15197, 13, 3005, 574, 311, 387, 1949, 430, 13658, 3156, 4848, 297, 63510, 280, 8248, 1364, 1047, 19487, 311, 733, 2162, 13, 18165, 311, 832, 11, 568, 596, 449, 1077, 1457, 1210, 578, 198, 89945, 54097, 315, 279, 8952, 11, 73259, 839, 449, 279, 41466, 315, 34697, 323, 3596, 13553, 198, 258, 279, 83249, 26, 1903, 1077, 85184, 13, 1628, 345, 9620, 3422, 11, 422, 735, 13, 1051, 449, 46993, 27567, 12348, 596, 37913, 11, 4536, 956, 433, 7673, 330, 3112, 3686, 11, 499, 3136, 311, 26032, 757, 323, 1755, 757, 311, 7692, 9135, 330, 4054, 315, 279, 1455, 50189, 2574, 922, 62461, 374, 78504, 1457, 323, 198, 3473, 1210, 5810, 574, 1077, 2362, 4333, 323, 198, 37569, 10371, 311, 1935, 1077, 704, 369, 264, 53121, 12141, 13, 578, 30462, 53848, 198, 1073, 12822, 2403, 11447, 85007, 709, 304, 76996, 13, 22770, 22983, 1047, 8208, 555, 430, 892, 313, 46696, 449, 12491, 304, 1077, 4851, 323, 3776, 198, 5919, 12959, 13, 3005, 7020, 1633, 1664, 1148, 279, 4360, 1053, 387, 13, 76996, 1053, 6678, 198, 4291, 1461, 11, 323, 568, 1053, 3371, 1077, 1268, 17104, 1364, 7111, 449, 279, 3805, 389, 198, 1964, 3663, 323, 279, 12056, 922, 1077, 13, 578, 21706, 8050, 11633, 315, 279, 2697, 20328, 1108, 198, 41450, 2571, 1124, 3345, 3871, 13, 1283, 1053, 198, 22794, 76996, 596, 1450, 59772, 398, 323, 15648, 304, 1077, 6548, 13, 3011, 574, 813, 1749, 512, 998, 1514, 520, 3021, 28846, 1093, 459, 6264, 19995, 8334, 11, 3156, 5115, 15187, 279, 198, 89851, 12504, 323, 279, 8137, 574, 1070, 13, 578, 10280, 28361, 1047, 539, 8636, 704, 304, 279, 26741, 994, 22770, 22983, 4024, 198, 1445, 311, 433, 13, 1952, 1855, 96803, 2007, 574, 459, 19087, 11, 323, 30488, 433, 264, 6857, 198, 1073, 39640, 268, 36516, 323, 264, 48135, 4251, 1450, 7197, 62626, 13, 2684, 1051, 8314, 954, 315, 198, 8619, 398, 38067, 922, 11, 2288, 11, 323, 279, 1306, 1773, 4481, 2262, 315, 58785, 45405, 198, 438, 10054, 12, 47100, 13, 578, 61983, 3828, 889, 6476, 279, 63137, 4671, 7018, 1523, 279, 46979, 1139, 279, 198, 1637, 13, 3005, 574, 44865, 449, 21847, 11, 430, 837, 267, 35491, 11, 323, 449, 279, 198, 86258, 76978, 315, 279, 89358, 26741, 311, 9096, 13, 3005, 574, 5128, 304, 264, 3995, 11, 67163, 1648, 11, 323, 1606, 311, 1077, 10280, 198, 16514, 264, 19309, 323, 8967, 3987, 323, 279, 11471, 315, 279, 3995, 10425, 11, 1364, 198, 51908, 71414, 2574, 13, 578, 26741, 7731, 709, 11, 27569, 430, 433, 574, 539, 279, 81276, 11, 31645, 4028, 198, 1527, 4950, 311, 4950, 13, 578, 47027, 261, 11, 6832, 836, 574, 27738, 9336, 11, 574, 264, 16615, 11, 16025, 3828, 449, 264, 198, 4930, 11, 14618, 19689, 13, 3005, 8774, 709, 264, 4401, 5844, 857, 3904, 315, 2678, 3137, 311, 198, 1820, 4731, 13, 330, 5966, 10280, 1359, 1364, 1071, 24261, 3210, 11, 330, 906, 4024, 704, 1139, 279, 3224, 198, 258, 264, 18137, 2695, 6241, 323, 1047, 264, 1972, 892, 13, 358, 1541, 956, 1440, 1148, 358, 1097, 1618, 369, 345, 3852, 5269, 13, 330, 73819, 323, 63237, 11, 1332, 346, 4447, 323, 50625, 4447, 11, 3116, 13124, 315, 19692, 26, 430, 596, 198, 1820, 3460, 315, 9041, 584, 617, 709, 304, 1057, 961, 315, 279, 1917, 13, 3277, 358, 1781, 315, 198, 12840, 358, 7731, 1523, 311, 311, 11477, 313, 9135, 3005, 1047, 264, 28254, 5201, 369, 22770, 22983, 11, 323, 1077, 63426, 304, 279, 8952, 198, 67, 14657, 291, 505, 76996, 596, 304, 430, 433, 574, 311, 29960, 349, 1077, 2307, 22625, 11, 1418, 198, 68753, 3520, 596, 1047, 1027, 311, 2512, 369, 1077, 6978, 13, 56124, 22770, 22983, 34386, 11, 1364, 99507, 264, 2697, 58587, 13, 3005, 1047, 887, 398, 198, 6200, 3340, 279, 2440, 315, 264, 16088, 16893, 389, 279, 1203, 315, 1077, 1450, 11, 323, 574, 198, 26121, 33024, 264, 35113, 323, 5425, 1481, 3233, 389, 433, 13, 330, 40, 5895, 422, 499, 617, 14000, 2555, 1359, 1364, 1071, 11, 6548, 389, 279, 2440, 13, 330, 40, 617, 14000, 430, 279, 2380, 16405, 63510, 39653, 527, 539, 2728, 1359, 1071, 198, 77264, 22983, 46473, 26, 323, 9083, 27738, 9336, 11, 2103, 30929, 323, 84273, 11, 1903, 279, 198, 1067, 82, 315, 279, 26741, 13, 330, 40, 2846, 912, 58587, 1359, 1364, 1071, 11, 10917, 279, 35788, 389, 279, 2007, 13, 330, 2746, 499, 2834, 956, 198, 4151, 11, 499, 2834, 956, 13, 1666, 433, 574, 539, 2631, 430, 24014, 387, 12715, 389, 279, 3335, 11, 22770, 22983, 198, 35778, 912, 6666, 311, 420, 13, 9083, 27738, 9336, 2064, 15556, 449, 12939, 198, 438, 1095, 1077, 16757, 2610, 1077, 11157, 13, 362, 1450, 9508, 311, 2339, 22770, 22983, 596, 4851, 323, 3412, 433, 13, 21347, 459, 2362, 4333, 3250, 956, 1304, 499, 1427, 520, 264, 3828, 439, 422, 499, 198, 87692, 311, 1935, 264, 23556, 704, 315, 1077, 13, 4488, 856, 3492, 11, 9083, 36627, 11, 1364, 3358, 198, 37593, 6381, 1077, 4967, 26, 1364, 3358, 32918, 1461, 13, 358, 6562, 1359, 20536, 279, 198, 20493, 367, 261, 24261, 3210, 11, 330, 9210, 1063, 1695, 31348, 12637, 1093, 430, 1053, 198, 23609, 264, 27555, 311, 757, 13, 358, 1097, 439, 28360, 439, 264, 27275, 25675, 11, 719, 198, 40, 3077, 2751, 1742, 1210, 3005, 574, 1317, 323, 7589, 9373, 11, 719, 1364, 28670, 1077, 198, 75, 92916, 11, 5986, 2269, 15154, 15447, 449, 264, 3738, 30296, 13, 5340, 65670, 24573, 198, 41450, 617, 26435, 1077, 304, 86115, 6307, 311, 2489, 1077, 6548, 11, 323, 449, 1317, 198, 73, 1037, 67301, 11, 323, 1903, 1077, 264, 11401, 13, 578, 63137, 380, 1047, 3970, 279, 24014, 389, 32980, 198, 49, 8477, 31202, 596, 4251, 54888, 11, 323, 1047, 33341, 1139, 18147, 1228, 287, 11, 16267, 788, 4731, 13, 330, 40, 2846, 17510, 19101, 323, 1364, 596, 63134, 1359, 2854, 2106, 279, 198, 1637, 1234, 1202, 11745, 13, 330, 52182, 11, 1268, 358, 4265, 1093, 311, 15612, 0, 1442, 358, 3596, 636, 704, 315, 420, 1181, 8301, 37002, 9135, 578, 16088, 10398, 352, 11203, 520, 22770, 22983, 596, 46811, 26, 30488, 433, 279, 3830, 315, 9382, 13, 22770, 22983, 7020, 433, 1523, 311, 279, 43957, 315, 198, 1964, 57034, 8271, 13, 1666, 40605, 439, 279, 3814, 8272, 279, 1938, 11, 1364, 574, 198, 18310, 1077, 1847, 13, 3005, 1047, 5675, 2736, 11, 7389, 27567, 2746, 1364, 1436, 636, 76996, 704, 315, 279, 8952, 11, 433, 1053, 40821, 2574, 13, 3005, 1765, 76, 4147, 559, 4361, 67, 398, 430, 389, 279, 6825, 872, 12034, 1051, 7029, 198, 391, 472, 13, 1102, 574, 1618, 430, 814, 2322, 389, 4279, 5015, 13, 578, 61983, 63137, 43467, 4671, 7018, 704, 315, 279, 26741, 26, 279, 35612, 315, 279, 198, 22928, 12688, 93722, 23183, 1523, 13, 2468, 4330, 297, 63510, 22770, 22983, 3288, 9083, 198, 54, 569, 9336, 311, 1176, 87117, 11, 311, 279, 13051, 315, 430, 56452, 14792, 198, 9164, 13, 578, 26741, 11203, 2103, 477, 75371, 671, 332, 30666, 13, 10280, 574, 927, 345, 438, 1070, 1051, 912, 11714, 16064, 311, 1427, 4741, 311, 13, 22770, 22983, 6688, 279, 4330, 16405, 63510, 39653, 13, 5112, 1364, 7731, 1523, 520, 279, 2007, 198, 52759, 279, 6134, 11, 449, 279, 35788, 304, 4156, 315, 1077, 13, 2684, 527, 3738, 11555, 198, 9210, 527, 520, 1176, 5865, 315, 279, 8271, 26, 1306, 264, 1317, 892, 279, 50112, 198, 32089, 5097, 1124, 709, 323, 33822, 1124, 1139, 14385, 4661, 9651, 13, 19292, 1606, 369, 279, 1566, 2305, 1364, 1047, 2884, 279, 3245, 779, 3629, 304, 198, 1964, 4059, 11, 1202, 5150, 5178, 574, 4661, 2085, 17371, 3463, 13, 22770, 22983, 3952, 264, 16893, 505, 1077, 16088, 87041, 11, 323, 11, 4477, 264, 502, 198, 1530, 369, 433, 11, 3347, 291, 433, 927, 279, 2362, 832, 13, 5112, 1364, 48501, 433, 369, 832, 198, 1073, 279, 1890, 1404, 389, 279, 16088, 35788, 13, 763, 279, 18397, 48840, 11, 520, 279, 47027, 388, 6, 2007, 11, 9083, 27738, 9336, 574, 198, 83, 30855, 13, 330, 22404, 19704, 757, 1359, 1364, 1071, 11, 330, 2727, 369, 279, 3224, 323, 279, 4382, 2324, 1306, 198, 576, 13, 2435, 1781, 358, 2846, 1193, 264, 47027, 261, 323, 1541, 956, 1518, 4205, 11, 719, 358, 3077, 198, 23196, 6548, 304, 856, 2010, 13, 17882, 11, 323, 1364, 198, 339, 15872, 358, 1541, 956, 1518, 433, 13, 2030, 2646, 4059, 26, 358, 7318, 11, 1077, 709, 311, 11477, 369, 264, 2478, 198, 1073, 279, 503, 337, 2641, 1364, 706, 2728, 757, 1210, 46982, 279, 18397, 48840, 13326, 323, 40883, 3995, 3278, 3782, 323, 30912, 345, 71, 561, 1570, 477, 41308, 398, 439, 872, 6776, 574, 11, 323, 4024, 389, 872, 1648, 198, 33871, 13, 763, 872, 6206, 814, 5762, 279, 7039, 11, 539, 2744, 315, 2324, 323, 4648, 198, 66372, 11, 719, 315, 14553, 505, 6784, 11, 315, 8541, 29668, 11, 315, 11113, 67026, 11, 323, 198, 66, 8772, 315, 3090, 311, 98208, 23726, 13, 763, 872, 6548, 11, 439, 304, 76996, 596, 11, 27724, 279, 198, 4238, 315, 2532, 13, 2030, 1618, 323, 1070, 832, 1766, 3278, 11, 1093, 22770, 22983, 323, 9083, 27738, 9336, 345, 14965, 1047, 37104, 872, 348, 2328, 11, 889, 436, 5805, 2403, 279, 78316, 3633, 315, 279, 198, 14789, 11, 1202, 9669, 11, 1202, 26762, 58023, 13, 76020, 477, 779, 2403, 1403, 313, 69, 39724, 889, 7111, 704, 389, 279, 1917, 449, 279, 198, 69, 686, 1752, 34522, 315, 1884, 889, 617, 3970, 2324, 311, 1202, 43957, 11, 323, 11, 449, 279, 198, 65, 8714, 8830, 315, 5150, 3729, 11, 2103, 1766, 433, 1695, 13, 76020, 889, 198, 52898, 6975, 477, 1047, 9687, 539, 311, 4128, 16038, 872, 4335, 70181, 291, 198, 68075, 505, 279, 1377, 370, 315, 279, 14708, 13, 1628, 279, 33517, 11, 889, 1766, 279, 1633, 198, 2445, 372, 315, 279, 18340, 5153, 539, 2288, 78442, 369, 8541, 29668, 323, 2512, 11, 1095, 22770, 22983, 198, 438, 11, 304, 32415, 6767, 11, 279, 502, 47027, 261, 7636, 13, 2435, 1436, 539, 617, 198, 3415, 7725, 872, 8125, 13, 578, 87117, 48840, 574, 10409, 449, 872, 8579, 24196, 11, 279, 23941, 273, 315, 872, 198, 68075, 11, 279, 30103, 309, 315, 872, 31161, 4251, 19079, 13, 3277, 22770, 22983, 3782, 304, 11, 1364, 44865, 7000, 315, 1124, 13, 2435, 1550, 539, 1093, 1077, 345, 438, 1364, 7020, 433, 13, 13538, 1077, 11, 4619, 315, 279, 58905, 87117, 17203, 11, 1364, 574, 9298, 279, 198, 83543, 10398, 352, 439, 1364, 1047, 2163, 433, 13, 330, 40, 8101, 358, 3077, 8521, 1077, 1359, 1364, 1071, 311, 11937, 13, 6385, 1633, 13836, 574, 14363, 449, 8850, 315, 1148, 1364, 1047, 2884, 13, 96424, 53694, 23440, 1432, 42, 13, 5602, 76996, 369, 1193, 264, 4545, 389, 10280, 6187, 13, 1115, 574, 994, 279, 8485, 198, 56492, 20328, 1108, 1047, 10717, 304, 4156, 315, 279, 3838, 13, 76996, 1047, 73973, 12164, 18007, 304, 369, 264, 4545, 13, 46993, 596, 1370, 9761, 574, 198, 75593, 449, 4027, 4238, 323, 50380, 449, 70457, 323, 449, 279, 1206, 1683, 315, 1077, 198, 82694, 1824, 8772, 13, 735, 2637, 84244, 287, 1280, 337, 4501, 304, 4156, 315, 279, 4027, 11, 1047, 6656, 311, 1518, 76996, 198, 258, 279, 70458, 11, 323, 514, 10395, 311, 813, 7693, 13, 330, 40, 649, 956, 2586, 304, 1359, 1364, 39169, 13, 358, 1097, 704, 198, 82, 64069, 12, 50780, 449, 2999, 13, 330, 27264, 1461, 304, 369, 264, 10747, 315, 15600, 1359, 46993, 2663, 704, 13, 330, 8586, 596, 71322, 198, 27588, 65670, 323, 6691, 323, 1524, 42216, 9135, 46993, 1047, 20330, 2391, 279, 1566, 5672, 11, 719, 1364, 574, 10917, 709, 264, 34300, 198, 7096, 13, 76996, 10837, 311, 279, 4156, 6134, 323, 2663, 25, 330, 10149, 499, 2586, 304, 369, 264, 10747, 315, 198, 82694, 7673, 1666, 76996, 6656, 1203, 1139, 279, 3838, 11, 1364, 2322, 42216, 13, 1283, 1047, 2586, 704, 198, 258, 279, 14321, 11, 323, 1047, 8036, 279, 6134, 1139, 279, 1370, 9761, 4920, 1461, 13, 5414, 6916, 198, 16514, 2103, 304, 12786, 21719, 11, 323, 70955, 22340, 304, 264, 8485, 41044, 88733, 13, 578, 5222, 315, 43214, 3782, 1555, 279, 6134, 38678, 398, 13, 578, 8334, 596, 3663, 574, 198, 33222, 449, 1461, 13, 330, 56351, 304, 1063, 5627, 11, 719, 315, 3388, 313, 1875, 39469, 527, 814, 2133, 311, 14816, 7673, 330, 1548, 3250, 956, 2873, 311, 19065, 499, 26, 568, 2795, 433, 596, 682, 304, 279, 1847, 1210, 330, 68753, 3520, 11, 1587, 46993, 1440, 430, 358, 574, 539, 7636, 430, 3814, 7673, 330, 2746, 1364, 61637, 11, 433, 374, 539, 1606, 315, 4205, 279, 8334, 706, 1071, 13, 4470, 315, 279, 4027, 4238, 11, 3201, 505, 279, 70457, 323, 279, 43214, 11, 42216, 596, 198, 1594, 8710, 24634, 323, 305, 16094, 569, 13, 1283, 2231, 813, 1949, 1450, 389, 76996, 596, 17308, 13, 330, 40, 574, 7422, 430, 8530, 422, 358, 4024, 3201, 313, 1875, 34728, 1053, 387, 66275, 398, 11, 8434, 956, 433, 7673, 330, 2746, 46993, 1053, 1193, 2019, 2555, 323, 636, 433, 927, 449, 0, 3005, 3250, 956, 198, 82, 22537, 26, 358, 1781, 1364, 596, 2216, 4560, 311, 387, 3169, 13, 3005, 10800, 28639, 1475, 892, 358, 5916, 1077, 1450, 1210, 2052, 279, 3177, 1047, 8636, 704, 315, 76996, 596, 3663, 13, 9601, 574, 17936, 11, 1306, 198, 543, 313, 2017, 94179, 13, 3861, 1550, 5076, 2574, 11, 323, 1023, 1274, 16654, 26, 477, 198, 606, 574, 1695, 11, 439, 1077, 6691, 1047, 1027, 11, 323, 574, 2163, 40666, 11, 264, 57896, 11, 477, 198, 4908, 71322, 5340, 65670, 13, 20695, 1051, 779, 2204, 505, 198, 12840, 814, 9508, 311, 387, 25, 46993, 7953, 279, 6134, 11, 51513, 15600, 323, 198, 4355, 7595, 287, 449, 1077, 4851, 304, 67101, 26, 42216, 30488, 1077, 11, 14867, 16117, 26435, 198, 438, 289, 62837, 13, 578, 1193, 832, 1364, 3463, 2216, 2262, 291, 574, 735, 13, 1283, 9508, 198, 998, 3351, 779, 73170, 304, 813, 2697, 27605, 13, 1283, 574, 2744, 779, 24981, 11, 779, 198, 59502, 13, 1442, 2324, 5762, 912, 36394, 369, 1461, 11, 520, 3325, 433, 5762, 912, 43957, 13, 330, 3947, 596, 1193, 832, 3245, 11, 42216, 1359, 1364, 1071, 29059, 989, 13, 330, 88960, 35120, 31202, 198, 285, 2133, 311, 617, 813, 6140, 13, 1442, 21739, 304, 279, 1917, 649, 3665, 1461, 11, 7639, 198, 92493, 649, 1210, 578, 3177, 315, 430, 8982, 574, 304, 1077, 6548, 994, 1364, 4024, 704, 311, 279, 20328, 1108, 198, 33871, 13, 735, 13, 8272, 1077, 704, 323, 61218, 279, 71268, 304, 15884, 922, 1077, 13, 2209, 1070, 904, 6140, 315, 3515, 499, 2162, 369, 87117, 7673, 358, 1097, 311, 733, 389, 14523, 520, 4848, 1578, 35277, 82, 6548, 11, 1364, 1550, 539, 1518, 433, 13, 1283, 65682, 1124, 198, 1885, 37163, 398, 505, 279, 65442, 11, 323, 4024, 4856, 17345, 1203, 1139, 279, 198, 7830, 13, 330, 10156, 1268, 1690, 3026, 527, 304, 3021, 449, 499, 11, 76996, 7673, 4691, 7639, 11, 439, 86652, 198, 47823, 709, 279, 6825, 13, 330, 2822, 832, 430, 358, 1440, 315, 11, 7389, 313, 1875, 1, 66457, 13, 11115, 313, 1875, 31437, 358, 8967, 1359, 1364, 1071, 449, 38929, 11, 330, 285, 430, 7389, 832, 14921, 1633, 198, 82279, 3026, 11, 323, 430, 4536, 956, 2216, 3021, 1210, 330, 1687, 3358, 5387, 704, 13142, 46506, 12669, 323, 7182, 313, 2000, 11, 315, 3388, 11, 358, 1097, 1633, 198, 82279, 13, 10699, 374, 304, 3021, 449, 499, 28858, 2009, 77638, 818, 30, 5884, 315, 279, 2655, 288, 520, 198, 1820, 8952, 7673, 2009, 77638, 818, 374, 539, 304, 3021, 449, 757, 1210, 2684, 574, 1778, 100226, 304, 1077, 7899, 430, 17882, 574, 51512, 13, 735, 2637, 9191, 1109, 5678, 323, 810, 25165, 11, 1047, 2744, 1047, 459, 10535, 33464, 198, 2000, 3278, 13, 1283, 1047, 1027, 42762, 34386, 555, 1124, 11, 719, 279, 2144, 1047, 14958, 13, 1628, 7639, 810, 1109, 24740, 430, 1457, 11, 520, 1566, 11, 568, 1047, 1027, 10791, 13, 330, 8161, 956, 499, 2216, 3152, 430, 499, 527, 304, 3021, 449, 2009, 77638, 818, 7673, 358, 1097, 539, 304, 3021, 449, 21739, 26, 358, 9167, 956, 892, 198, 998, 387, 304, 3021, 13, 2100, 8369, 1550, 279, 5811, 3719, 430, 198, 20670, 5946, 2085, 9298, 264, 6278, 57859, 46229, 11, 2875, 323, 4856, 198, 89945, 743, 11, 20558, 1555, 264, 12056, 3696, 2130, 389, 4579, 11, 323, 15691, 304, 813, 198, 10888, 264, 19371, 44221, 660, 18012, 9145, 13, 2030, 279, 62705, 43578, 555, 323, 2163, 1461, 22095, 6953, 752, 345, 20986, 56949, 3725, 1306, 1124, 13, 3279, 596, 4059, 11, 1193, 264, 40146, 323, 198, 258, 81987, 6468, 23023, 13, 4314, 2574, 430, 3782, 779, 6847, 311, 7639, 11, 279, 198, 2715, 12181, 315, 3278, 11, 8485, 2697, 35048, 2805, 13757, 1093, 279, 39098, 198, 1073, 86652, 323, 279, 20328, 1108, 11, 1047, 2646, 1027, 813, 13, 1442, 1070, 574, 904, 38678, 198, 2762, 479, 11, 433, 574, 520, 5678, 13, 1283, 1047, 9408, 279, 8334, 5076, 313, 383, 1047, 198, 2629, 3948, 1461, 311, 387, 45743, 13, 55777, 279, 9145, 1579, 704, 315, 279, 34738, 82, 11, 568, 198, 28010, 813, 6435, 5208, 709, 279, 6825, 13, 2468, 2555, 1306, 1403, 297, 63510, 430, 3814, 11, 735, 13, 2231, 1523, 813, 13961, 198, 438, 34793, 13, 1283, 1047, 539, 1027, 3025, 311, 6212, 2533, 33433, 13, 763, 813, 198, 67, 57881, 2427, 785, 568, 1047, 7731, 555, 279, 2678, 4027, 11, 7422, 13, 578, 2262, 315, 813, 198, 3983, 2478, 4038, 389, 279, 6825, 574, 19019, 7231, 1648, 311, 59322, 13, 1283, 198, 14965, 1047, 8967, 311, 4018, 5678, 1022, 505, 2324, 1766, 5678, 1578, 304, 3345, 198, 22794, 449, 433, 26, 813, 1608, 10470, 574, 5655, 449, 433, 13, 1789, 279, 1176, 892, 11, 568, 1047, 22088, 311, 3488, 279, 24278, 315, 1148, 568, 1047, 198, 10655, 13, 1102, 1047, 4529, 25775, 11, 4359, 7020, 345, 998, 3041, 709, 4395, 323, 2586, 3201, 13, 763, 264, 1648, 11, 433, 1053, 617, 4529, 810, 198, 66, 61140, 311, 617, 20186, 13, 1283, 1047, 3463, 11, 520, 1176, 11, 430, 568, 1436, 198, 21828, 1523, 420, 3021, 369, 76996, 13, 578, 198, 6258, 511, 306, 5916, 315, 1077, 1450, 389, 813, 6916, 11, 279, 4545, 994, 568, 1047, 5762, 1077, 198, 258, 813, 11977, 1306, 1077, 6691, 596, 4648, 11, 279, 16579, 2678, 19015, 315, 1077, 198, 4310, 311, 279, 2697, 3838, 313, 543, 1521, 743, 813, 6680, 389, 4027, 13, 9636, 813, 11594, 28362, 735, 13, 21095, 1690, 26885, 1884, 12688, 198, 14097, 313, 2017, 813, 18496, 323, 48033, 520, 279, 5274, 11, 304, 813, 3130, 7636, 345, 4291, 5340, 65670, 9293, 7878, 38586, 82, 7953, 279, 17071, 11, 1524, 555, 198, 19875, 483, 596, 4027, 11, 449, 46993, 1120, 4028, 11, 11961, 304, 21847, 323, 198, 14658, 287, 813, 25165, 5643, 323, 24981, 6548, 13, 1283, 1047, 264, 2697, 6945, 315, 76996, 313, 64, 10885, 64630, 430, 568, 1047, 4529, 198, 40617, 726, 13, 1102, 8710, 76996, 28382, 264, 1450, 11, 902, 1047, 1027, 704, 315, 2134, 994, 198, 1820, 6382, 1047, 1027, 48110, 11, 323, 11509, 389, 264, 32366, 4051, 1968, 198, 8370, 1053, 617, 1027, 5115, 264, 2237, 1047, 568, 5762, 279, 6382, 7833, 13, 35053, 433, 574, 76996, 11, 1077, 7013, 41442, 922, 1077, 11, 6548, 3411, 198, 412, 11, 28682, 23726, 37163, 13, 3277, 1364, 574, 539, 520, 2162, 11, 433, 7731, 389, 735, 3238, 82, 198, 69334, 261, 11, 463, 7069, 2403, 813, 37204, 12913, 13, 3277, 1364, 574, 304, 279, 3838, 11, 433, 198, 6789, 1234, 279, 9160, 1824, 1136, 290, 13, 9220, 297, 63510, 304, 279, 6693, 11, 1243, 11, 323, 735, 13, 304, 813, 32623, 2427, 785, 11, 449, 279, 198, 27581, 463, 7069, 11, 539, 2403, 279, 37204, 12913, 11, 719, 2403, 813, 29062, 11, 1405, 198, 383, 1436, 1518, 433, 13, 1283, 7731, 4741, 304, 813, 10716, 11, 813, 6206, 48135, 2212, 813, 22095, 11, 323, 198, 7349, 291, 520, 433, 13, 1283, 574, 4560, 311, 6945, 279, 76996, 315, 279, 10512, 198, 258, 813, 2362, 2324, 313, 85134, 311, 1505, 264, 2035, 369, 1077, 13, 2684, 1047, 1027, 2478, 3278, 304, 813, 2362, 2324, 13, 2684, 1047, 1027, 3278, 889, 1047, 42777, 369, 1461, 11, 719, 568, 2231, 1124, 198, 318, 23557, 398, 704, 315, 813, 4059, 13, 35403, 198, 15145, 568, 1047, 568, 4234, 813, 1317, 14535, 704, 315, 279, 10716, 11, 1364, 574, 53728, 520, 198, 26301, 6134, 4994, 13, 35120, 31202, 574, 11509, 304, 279, 4827, 14321, 345, 64, 559, 88548, 922, 1077, 28004, 13, 330, 40, 3077, 1047, 3492, 311, 733, 311, 279, 8952, 1359, 1364, 1071, 13, 330, 40, 3463, 7344, 499, 4265, 198, 3427, 449, 757, 13, 1102, 5084, 439, 422, 358, 649, 956, 2559, 433, 7636, 13, 330, 11787, 499, 16984, 311, 4822, 304, 279, 3838, 7636, 7673, 1283, 10837, 709, 279, 62862, 311, 813, 3130, 323, 1344, 2234, 389, 1063, 17895, 13, 35120, 31202, 596, 274, 5598, 1047, 3719, 3428, 4647, 598, 26, 46993, 14980, 198, 8823, 16117, 927, 1077, 13, 330, 40, 1097, 50136, 14931, 1359, 1364, 1071, 313, 1, 466, 37852, 14931, 0, 3277, 358, 1781, 6832, 198, 60672, 682, 420, 374, 9135, 35120, 31202, 2231, 704, 264, 990, 71845, 6901, 1450, 323, 10791, 46993, 596, 198, 69, 14437, 13, 358, 8101, 499, 323, 358, 198, 8154, 2752, 1855, 1023, 13, 735, 13, 2646, 29695, 279, 6237, 304, 279, 2678, 13147, 26741, 311, 902, 32980, 198, 32345, 1027, 4529, 13, 9636, 279, 4251, 13001, 813, 8334, 819, 7216, 7111, 198, 496, 526, 989, 1317, 13, 2684, 574, 264, 1912, 2212, 279, 4950, 313, 6102, 17882, 11, 1403, 477, 198, 28956, 2655, 288, 11, 279, 3814, 29198, 389, 14523, 11, 323, 279, 11452, 13, 84854, 1120, 4871, 279, 6134, 389, 264, 7833, 10716, 574, 76996, 313, 21470, 264, 198, 68753, 3520, 439, 568, 2646, 1047, 3970, 1603, 11, 1077, 3663, 1933, 1752, 11, 1077, 6548, 7029, 198, 438, 653, 66154, 11, 1077, 6206, 86034, 2454, 304, 1077, 24301, 13, 3277, 568, 14980, 30488, 1077, 345, 32158, 1550, 539, 3351, 477, 1427, 709, 13, 578, 1912, 2212, 279, 4950, 1047, 85156, 311, 198, 329, 1800, 18083, 13, 8442, 76996, 323, 735, 13, 14958, 555, 198, 1820, 6134, 11, 25181, 11, 7636, 13, 330, 2675, 2011, 539, 1935, 433, 1093, 430, 11, 25237, 13, 2030, 11, 1306, 198, 543, 11, 304, 430, 3044, 313, 1875, 2181, 574, 1077, 1176, 6677, 430, 568, 574, 1070, 13, 6385, 7899, 574, 33201, 661, 11, 4225, 1191, 1752, 13, 330, 7009, 2019, 358, 6688, 1461, 279, 5076, 16088, 26, 430, 568, 596, 23069, 26, 430, 358, 198, 66206, 1126, 291, 1461, 1210, 358, 3782, 389, 14523, 520, 4848, 297, 63510, 323, 6688, 279, 198, 2106, 292, 1572, 13, 3277, 279, 3814, 29198, 3782, 389, 520, 8254, 11, 4395, 574, 682, 198, 1315, 13, 578, 16088, 10398, 352, 574, 1120, 439, 433, 1288, 387, 13, 358, 198, 42700, 311, 2019, 1695, 45427, 311, 1461, 323, 568, 313, 383, 574, 33173, 13, 358, 3287, 956, 3041, 1461, 198, 73254, 719, 1148, 574, 389, 279, 35788, 1359, 1364, 8220, 281, 635, 7162, 13, 330, 40, 7111, 520, 198, 1820, 2440, 26, 358, 2744, 1427, 1210, 3296, 264, 32931, 315, 279, 1912, 2212, 279, 4950, 11, 735, 3238, 82, 6548, 7111, 369, 264, 4545, 198, 20384, 398, 1139, 22770, 22983, 596, 13, 4702, 369, 264, 4545, 26, 1243, 279, 13734, 8036, 709, 198, 33871, 13, 1102, 574, 1664, 369, 22770, 22983, 430, 433, 1550, 13, 3005, 7111, 439, 422, 1364, 1047, 198, 29412, 264, 20457, 313, 35187, 1077, 6548, 11, 1524, 312, 41189, 13, 330, 1991, 1063, 832, 311, 198, 23609, 1077, 2035, 1210, 4740, 682, 11, 279, 9546, 315, 420, 893, 304, 420, 3130, 198, 266, 1778, 264, 892, 8967, 4400, 13, 1283, 574, 76996, 596, 4333, 11, 430, 574, 682, 13, 1102, 574, 279, 8334, 596, 58764, 3044, 430, 574, 13353, 1077, 198, 265, 53305, 1139, 31926, 13, 330, 40, 1097, 682, 1314, 1359, 1364, 40001, 4028, 279, 4950, 311, 279, 11452, 13, 1283, 1047, 2884, 4395, 568, 7020, 2085, 198, 1407, 13, 578, 8334, 11, 436, 22485, 369, 459, 9888, 11, 1053, 90417, 1578, 1139, 91132, 269, 13, 3161, 264, 9498, 893, 814, 1436, 617, 6818, 810, 71920, 11193, 313, 29177, 198, 19553, 9770, 1461, 311, 813, 7693, 323, 15203, 1461, 922, 11, 1436, 617, 31394, 1461, 198, 4291, 1168, 15889, 53984, 73307, 304, 10054, 55051, 13, 2030, 279, 36850, 291, 2547, 389, 279, 4950, 198, 29177, 2559, 912, 1778, 56375, 6514, 13, 1102, 574, 2009, 77638, 818, 11, 1306, 682, 11, 889, 6924, 32980, 35120, 31202, 596, 2324, 13, 1789, 11, 994, 198, 28221, 323, 37465, 1047, 39019, 682, 872, 5070, 11, 568, 25319, 4741, 198, 4291, 264, 11594, 3492, 430, 7263, 279, 2655, 288, 311, 872, 7693, 100174, 13, 2684, 574, 264, 502, 6514, 369, 1778, 5157, 313, 275, 1047, 1027, 6818, 22917, 13, 330, 22170, 433, 11, 369, 32450, 596, 25491, 1359, 568, 1071, 13, 578, 41705, 574, 539, 304, 279, 3838, 313, 25849, 387, 1327, 53471, 1534, 11, 13118, 11, 520, 198, 4354, 11, 315, 21448, 323, 10548, 505, 279, 10565, 48840, 13, 735, 13, 1550, 279, 990, 11, 813, 198, 4930, 19779, 711, 83, 323, 10151, 1285, 313, 3556, 18083, 13, 35120, 31202, 1168, 3903, 555, 279, 4950, 198, 4291, 1077, 3663, 28016, 26, 1418, 76996, 7731, 11, 294, 28109, 323, 99774, 12616, 11, 389, 1077, 198, 56492, 10716, 4871, 279, 6134, 26, 1418, 3814, 37465, 9165, 59091, 291, 3235, 279, 198, 6133, 83536, 11, 323, 279, 3814, 3821, 1543, 45135, 9850, 13850, 505, 4994, 279, 198, 11020, 13, 3277, 279, 1403, 2294, 77292, 430, 1051, 279, 13147, 26741, 11276, 198, 32345, 6656, 505, 41585, 42852, 279, 3130, 311, 18004, 77292, 304, 279, 198, 57902, 1251, 3177, 26, 32980, 35120, 31202, 9107, 813, 6548, 323, 12570, 279, 1176, 198, 5880, 430, 13160, 813, 471, 505, 279, 6453, 34681, 13, 3277, 433, 574, 2867, 430, 279, 8334, 1053, 3974, 11, 735, 13, 16392, 31161, 398, 505, 279, 198, 2788, 3002, 323, 4024, 927, 311, 76996, 596, 10716, 13, 330, 1548, 596, 682, 1314, 1457, 1359, 568, 1071, 313, 1, 300, 682, 1314, 439, 568, 649, 387, 11, 8009, 58725, 9135, 2650, 15234, 430, 499, 1288, 1440, 1778, 264, 3245, 13, 578, 2655, 288, 11, 7556, 4315, 5694, 11, 1047, 82294, 1523, 311, 872, 198, 67, 5859, 48840, 369, 4216, 11033, 13, 17882, 574, 7231, 264, 2478, 1566, 11470, 198, 300, 311, 279, 8334, 596, 2512, 3238, 82, 1450, 323, 198, 28708, 433, 311, 1077, 23726, 13, 578, 11245, 72498, 315, 279, 3814, 11, 315, 4038, 13118, 345, 67643, 3201, 1603, 1077, 4382, 2211, 676, 13, 330, 5159, 25237, 11, 856, 25237, 1359, 568, 1071, 9222, 74, 1570, 13, 330, 78403, 430, 358, 649, 656, 313, 2000, 198, 9514, 313, 266, 904, 892, 313, 1875, 2181, 574, 1306, 76996, 1047, 1922, 418, 1093, 264, 11102, 3245, 311, 1077, 3130, 430, 198, 77264, 22983, 36627, 323, 735, 13, 3782, 3663, 311, 3663, 13, 32980, 574, 5115, 17371, 198, 1729, 430, 892, 11, 264, 2697, 6437, 2212, 279, 23726, 11, 719, 1062, 5485, 398, 71414, 13, 330, 7816, 2574, 649, 3621, 311, 264, 12637, 1109, 358, 3596, 7020, 1070, 574, 9135, 568, 198, 84359, 311, 813, 6691, 11, 323, 14976, 4856, 33012, 65923, 311, 1077, 24014, 323, 198, 936, 44357, 13, 330, 2675, 1051, 2744, 264, 1695, 8334, 11, 32980, 1359, 1364, 1071, 13, 330, 10156, 499, 636, 1664, 198, 268, 1409, 311, 2586, 2162, 13, 358, 3358, 1935, 2512, 315, 499, 279, 2800, 315, 856, 2324, 13, 1226, 690, 198, 456, 499, 264, 13587, 79781, 994, 499, 649, 387, 922, 11, 323, 358, 649, 1935, 499, 704, 304, 198, 1820, 6246, 994, 358, 2586, 505, 990, 1210, 330, 40, 3358, 387, 24023, 323, 499, 3358, 387, 74817, 12844, 11, 7643, 1210, 2009, 77638, 818, 374, 2133, 311, 636, 701, 7126, 3288, 709, 1578, 13, 3161, 53474, 36399, 198, 39428, 264, 1938, 323, 1148, 358, 1304, 11, 584, 3358, 636, 3235, 1210, 330, 12174, 11, 32980, 11, 422, 358, 1436, 1518, 499, 5108, 304, 279, 6134, 1578, 323, 53302, 198, 2846, 1605, 6, 323, 596, 13886, 6, 304, 832, 11745, 9135, 578, 6574, 1990, 22770, 22983, 323, 2009, 77638, 818, 574, 1633, 11594, 13, 3005, 1047, 1027, 198, 28936, 264, 3460, 315, 84951, 21455, 389, 279, 84827, 315, 813, 4059, 198, 82977, 682, 279, 3814, 13, 1102, 1053, 387, 5107, 311, 3371, 994, 568, 3604, 198, 74, 943, 1077, 13, 3277, 279, 47979, 369, 7366, 32980, 1203, 311, 279, 2466, 26741, 1047, 1027, 198, 28010, 11, 279, 1023, 37465, 2163, 279, 3130, 11, 323, 22770, 22983, 323, 279, 8334, 1051, 198, 998, 3522, 13, 735, 13, 10717, 1077, 389, 1077, 1648, 311, 279, 6134, 13, 22653, 288, 1618, 26, 856, 836, 374, 2009, 77638, 818, 1210, 330, 40, 617, 539, 3970, 499, 2533, 499, 2163, 800, 13, 330, 2822, 26, 358, 313, 40, 66529, 369, 264, 2478, 4038, 1210, 330, 40, 23289, 814, 656, 539, 1440, 430, 499, 1051, 313, 9210, 499, 617, 1047, 904, 198, 20281, 8952, 3217, 1210, 330, 40, 4985, 539, 3371, 1124, 11, 315, 3388, 1210, 1628, 8617, 11, 555, 4382, 27848, 14771, 11, 433, 574, 28902, 430, 1855, 1288, 198, 45734, 279, 1023, 596, 12410, 13, 2684, 1047, 1027, 264, 892, 11, 1120, 1603, 39493, 345, 9493, 1364, 1047, 1047, 832, 315, 1884, 30462, 50908, 430, 7170, 2586, 520, 198, 1820, 842, 315, 264, 1317, 3814, 13, 578, 8334, 574, 198, 1225, 3428, 11, 20781, 27027, 13, 6385, 3347, 41398, 4920, 1077, 11, 264, 4101, 315, 198, 9181, 5891, 833, 288, 323, 25429, 704, 11490, 37379, 11, 30462, 7692, 826, 11, 6435, 79157, 13, 3005, 1053, 617, 2728, 1475, 3987, 1364, 1047, 304, 279, 198, 14957, 11, 1120, 1243, 11, 369, 76996, 596, 25468, 3347, 13, 3005, 38674, 11937, 449, 430, 99469, 781, 44661, 430, 4131, 315, 4686, 198, 726, 5621, 899, 367, 13, 1628, 1364, 11953, 311, 1077, 3130, 279, 6677, 430, 279, 3814, 596, 14993, 1047, 198, 82850, 304, 46604, 313, 9210, 11, 8051, 32980, 35120, 31202, 1053, 3974, 11, 1364, 1047, 18661, 198, 42312, 555, 1148, 568, 1047, 16654, 13, 578, 4459, 3814, 1047, 6982, 1077, 279, 198, 61893, 33028, 315, 904, 44397, 15003, 311, 3243, 17882, 505, 813, 502, 68078, 13, 3005, 198, 32345, 14792, 1461, 304, 279, 51902, 11, 10307, 76996, 596, 73342, 7216, 198, 300, 1364, 1903, 1077, 1648, 709, 279, 32249, 311, 1077, 3130, 13, 15037, 11, 304, 682, 813, 3347, 198, 78, 1653, 1439, 311, 1077, 11, 1047, 1364, 3970, 430, 1427, 304, 813, 6548, 13, 96424, 1630, 5511, 1432, 1271, 5340, 65670, 24573, 11, 76996, 596, 11914, 315, 27219, 2919, 6, 25288, 3782, 198, 300, 264, 12523, 13, 735, 13, 14760, 279, 3754, 311, 1077, 430, 11714, 1603, 279, 892, 369, 198, 68753, 3520, 596, 19163, 13, 578, 2697, 14048, 574, 11821, 304, 5340, 65670, 596, 44202, 13, 45563, 1047, 198, 64, 13438, 1457, 11, 264, 2697, 58069, 3828, 7086, 386, 25877, 13, 1628, 5340, 65670, 1047, 198, 34500, 291, 389, 279, 6825, 279, 19297, 315, 1306, 1773, 4481, 11033, 13, 1102, 574, 198, 2017, 279, 1306, 1773, 4481, 11033, 430, 735, 13, 1903, 813, 17480, 13, 330, 3923, 656, 499, 3152, 555, 5605, 1364, 374, 5108, 2162, 369, 27219, 2919, 30, 330, 2688, 5986, 11, 8051, 1364, 374, 539, 5115, 1664, 13, 578, 2144, 374, 11, 5340, 65670, 1359, 313, 2000, 198, 275, 574, 330, 27588, 65670, 1, 323, 330, 42, 1210, 555, 420, 892, 11, 313, 1, 19041, 706, 1027, 264, 3460, 315, 198, 87698, 12, 4575, 1748, 709, 520, 279, 8952, 13, 1102, 12775, 956, 19543, 14243, 11, 719, 313, 1875, 27588, 65670, 2231, 1523, 279, 39571, 273, 1355, 33076, 304, 1077, 1450, 323, 45135, 4028, 520, 1461, 13, 330, 3947, 574, 264, 16930, 922, 279, 16088, 11, 323, 1364, 574, 36197, 26, 430, 596, 198, 543, 1210, 330, 8100, 4265, 2731, 2586, 2162, 323, 4822, 2162, 1359, 1071, 5340, 65670, 20193, 13, 330, 40, 3987, 433, 198, 73014, 956, 636, 304, 279, 16064, 13, 1115, 8679, 28936, 2626, 374, 264, 15526, 3460, 315, 198, 1626, 13, 3861, 3492, 2403, 499, 477, 904, 315, 701, 3070, 11, 323, 279, 13734, 596, 1022, 198, 57839, 61752, 775, 1210, 330, 3947, 596, 4400, 2403, 76996, 1359, 735, 13, 31256, 1077, 13, 1102, 5084, 433, 596, 264, 198, 61031, 5030, 315, 26434, 13, 94192, 1903, 264, 16930, 11, 323, 814, 4250, 1095, 198, 21470, 264, 3245, 733, 555, 13, 2030, 568, 13919, 11, 439, 358, 656, 11, 430, 433, 574, 539, 76996, 1210, 4452, 5340, 65670, 1047, 71836, 11937, 2403, 279, 3828, 596, 19163, 11, 682, 1364, 198, 32345, 8967, 311, 2019, 30010, 994, 1364, 5602, 76996, 596, 4319, 839, 6548, 323, 67163, 198, 33780, 13, 1628, 3952, 1077, 1867, 751, 291, 198, 79548, 316, 13, 1789, 279, 892, 520, 3325, 11, 76996, 596, 1917, 1047, 8208, 311, 9863, 922, 1077, 13, 2052, 198, 1964, 34300, 11412, 3935, 315, 2532, 54434, 1603, 1077, 64935, 13, 3277, 46993, 1053, 617, 3970, 1077, 11, 1364, 8774, 1077, 6134, 16447, 323, 4691, 198, 2000, 1120, 430, 832, 11714, 7636, 13, 2030, 1306, 5340, 65670, 1047, 22311, 11, 323, 198, 44, 25877, 11, 279, 58069, 11, 1047, 1922, 418, 704, 311, 279, 9309, 311, 8232, 264, 6661, 1203, 311, 198, 6600, 20786, 11, 76996, 42644, 6312, 291, 1077, 6134, 323, 34793, 304, 279, 2697, 8582, 14321, 13, 5340, 65670, 11, 1077, 2010, 304, 264, 43713, 11, 1077, 3663, 15884, 9439, 12, 846, 3690, 11, 1047, 8208, 198, 998, 4950, 26, 719, 735, 3238, 82, 3177, 11, 439, 13783, 11, 574, 49025, 927, 279, 1380, 316, 13, 76996, 198, 10462, 59091, 291, 311, 279, 6134, 13, 330, 11356, 358, 2586, 304, 323, 3137, 311, 499, 7673, 1283, 6656, 323, 3952, 264, 4062, 10795, 315, 279, 3130, 13, 578, 6945, 574, 2403, 198, 1820, 37204, 12913, 13, 2030, 568, 3952, 279, 5326, 323, 5762, 279, 6134, 7029, 13, 76996, 3782, 304, 323, 7731, 1523, 555, 279, 4027, 13, 3296, 1694, 1008, 69596, 568, 9152, 311, 198, 3306, 575, 279, 2697, 6945, 927, 323, 1234, 279, 3830, 1603, 1364, 5602, 433, 13, 1102, 374, 198, 67, 5599, 83, 1285, 422, 1364, 1053, 617, 15393, 1202, 26431, 11, 1047, 1364, 3970, 433, 13, 330, 40, 3077, 1027, 7422, 2574, 927, 1359, 1364, 1071, 13, 330, 2181, 5084, 311, 757, 358, 4265, 2731, 198, 1962, 733, 1203, 1210, 330, 4897, 1053, 387, 46337, 11, 8434, 956, 433, 11, 994, 499, 617, 2884, 779, 1664, 30, 1628, 345, 9620, 3422, 11, 2533, 499, 527, 539, 16390, 11, 76996, 313, 1875, 7189, 3287, 956, 656, 433, 9135, 358, 649, 956, 2567, 389, 26, 430, 596, 682, 1070, 374, 311, 433, 13, 2052, 198, 4354, 3814, 11, 304, 279, 13147, 26741, 11, 358, 6612, 433, 2133, 13, 358, 198, 99020, 5605, 311, 7182, 25, 364, 2675, 3287, 956, 656, 433, 11, 499, 3287, 956, 656, 433, 7112, 323, 682, 279, 198, 1712, 2555, 4871, 315, 757, 574, 5605, 11, 364, 2688, 1457, 11, 8530, 26, 719, 36113, 198, 9514, 1253, 30251, 45773, 735, 2637, 889, 1047, 93469, 682, 420, 704, 369, 5678, 323, 1047, 2586, 311, 279, 198, 25030, 3242, 13559, 0, 330, 1271, 733, 389, 1093, 420, 11, 8430, 430, 832, 706, 2324, 323, 4648, 304, 832, 596, 1450, 345, 438, 1243, 8530, 1063, 1938, 311, 1304, 264, 16930, 1093, 430, 9135, 3005, 7111, 709, 520, 198, 40617, 369, 75, 1540, 398, 13, 330, 40, 1097, 1120, 539, 34300, 3403, 11, 735, 1210, 330, 29089, 77, 956, 433, 387, 1437, 7403, 311, 2567, 389, 30, 6385, 1917, 574, 304, 9863, 922, 1077, 11, 323, 1364, 6612, 7636, 304, 264, 7029, 323, 198, 3274, 2035, 13, 1628, 11, 1606, 1077, 50581, 1051, 15107, 259, 2784, 3156, 814, 1051, 198, 2359, 311, 10885, 11, 76996, 6656, 389, 1461, 559, 4361, 65923, 13, 330, 40, 1781, 499, 527, 682, 16984, 358, 690, 2586, 1203, 311, 4822, 13, 37558, 2216, 198, 86, 1821, 757, 12660, 313, 258, 682, 279, 1917, 0, 2876, 520, 279, 8952, 11, 539, 1618, 11, 539, 198, 3852, 2050, 13, 330, 4599, 499, 2019, 430, 19093, 6944, 499, 1359, 1071, 735, 2637, 539, 1633, 43071, 11, 330, 40, 313, 40, 198, 27963, 499, 527, 3339, 264, 16930, 1210, 578, 1193, 1732, 198, 14965, 3596, 2216, 4934, 757, 574, 856, 6691, 11, 323, 358, 4024, 3201, 323, 2163, 1077, 9135, 3005, 48548, 813, 3663, 15499, 11, 323, 11, 5403, 1070, 2555, 1364, 1550, 539, 198, 8154, 2752, 11, 1364, 220, 15187, 13, 330, 2822, 26, 358, 656, 539, 3152, 13142, 46506, 12669, 1210, 1442, 568, 1047, 1766, 904, 51475, 304, 1077, 3663, 11, 568, 1053, 617, 8208, 389, 198, 25662, 16117, 26, 719, 1077, 10321, 6548, 19201, 1461, 13, 330, 2746, 499, 3152, 7639, 17882, 1359, 1071, 76996, 11, 330, 9514, 527, 11622, 5076, 13, 1283, 596, 539, 198, 258, 3021, 449, 757, 313, 1962, 11, 430, 374, 11, 904, 810, 1109, 568, 374, 304, 3021, 449, 264, 198, 3055, 5797, 7724, 13, 1283, 13452, 311, 387, 449, 757, 313, 2319, 11, 358, 1440, 430, 26, 719, 430, 3250, 956, 198, 14622, 313, 73254, 775, 13, 5884, 5269, 11, 1306, 420, 64935, 313, 1875, 25938, 374, 912, 64935, 11, 1716, 1210, 330, 1548, 3358, 1781, 757, 84834, 11, 520, 279, 3325, 13, 330, 2675, 2019, 568, 13452, 311, 387, 449, 499, 13, 76996, 1047, 1027, 11961, 304, 264, 3428, 10716, 555, 279, 4027, 13, 3005, 16392, 449, 264, 198, 82, 39095, 25429, 7351, 13, 763, 279, 6179, 2786, 315, 279, 14048, 11, 1364, 345, 32345, 12263, 735, 13, 304, 1077, 32623, 2427, 785, 323, 1776, 32997, 26, 323, 1457, 1364, 14980, 198, 15145, 1461, 11, 264, 35279, 3995, 7216, 11, 43789, 287, 279, 61607, 315, 1077, 65510, 198, 582, 2177, 1077, 17659, 13, 330, 40, 24863, 1461, 11, 735, 13, 1359, 1364, 1071, 72702, 2740, 13, 330, 4599, 358, 1518, 1461, 5108, 11, 358, 1390, 198, 998, 636, 1523, 323, 1095, 1461, 4321, 389, 757, 13, 3842, 7882, 311, 279, 14150, 13, 358, 198, 33134, 279, 1633, 1648, 568, 25562, 369, 279, 39736, 13, 3277, 358, 1518, 1461, 304, 279, 198, 3376, 1113, 48840, 11, 7155, 323, 19858, 1418, 1475, 832, 775, 374, 1344, 5100, 291, 323, 198, 40541, 1639, 11, 568, 313, 383, 5992, 1093, 264, 10087, 1210, 5112, 11, 4376, 51975, 315, 1077, 704, 58838, 11, 1364, 6656, 1077, 1203, 311, 1461, 323, 14980, 198, 70, 6795, 520, 279, 2678, 11756, 4027, 13, 1102, 574, 439, 1664, 369, 735, 13, 430, 1364, 1550, 539, 198, 4151, 813, 3663, 13, 1789, 430, 832, 4545, 279, 46996, 430, 574, 304, 1461, 559, 606, 304, 198, 26301, 6548, 13, 1283, 65657, 2212, 279, 559, 79736, 2697, 3130, 11, 520, 279, 30811, 3252, 4950, 345, 1820, 37204, 12913, 11, 279, 281, 2910, 1136, 290, 11, 279, 2362, 42390, 2442, 18033, 44402, 1234, 902, 198, 3561, 992, 67, 1047, 34833, 1903, 813, 23634, 11, 520, 813, 13365, 43686, 2007, 11, 39682, 291, 449, 198, 87820, 323, 6603, 11, 520, 279, 2217, 304, 279, 18327, 315, 813, 1866, 16615, 7216, 345, 34152, 16771, 323, 76656, 13, 330, 2675, 2351, 2771, 433, 596, 539, 198, 4345, 313, 6200, 309, 414, 11, 76996, 7673, 6385, 7899, 574, 296, 43435, 11, 323, 568, 7020, 1243, 430, 198, 32158, 574, 31299, 13, 92014, 11, 315, 3388, 11, 3734, 304, 279, 12625, 198, 1073, 832, 596, 33044, 11, 1051, 539, 31308, 389, 279, 6825, 13, 330, 36227, 757, 264, 1450, 7197, 62626, 1359, 1071, 76996, 304, 264, 296, 43435, 16630, 11, 323, 279, 2697, 198, 23296, 574, 11102, 1139, 1418, 735, 13, 27600, 1555, 264, 44402, 27580, 13, 5112, 1473, 12348, 596, 682, 927, 11, 90064, 11, 2533, 420, 13, 1442, 568, 4265, 2216, 42777, 568, 4265, 617, 2586, 198, 2017, 311, 45427, 13, 6984, 304, 264, 12960, 1364, 3782, 40605, 311, 1077, 25288, 13, 3005, 1053, 2646, 198, 3427, 1203, 11, 1364, 1071, 92925, 13, 3005, 574, 25226, 11, 1047, 1027, 61456, 198, 4575, 2656, 13, 1442, 814, 1436, 1781, 1778, 264, 3245, 922, 1077, 11, 1364, 3287, 956, 1390, 311, 198, 1395, 304, 872, 2362, 8952, 13, 735, 13, 29440, 1077, 11, 7064, 2718, 65464, 323, 84072, 13, 358, 617, 2728, 1461, 813, 39653, 22700, 315, 3115, 1210, 330, 15546, 775, 1047, 2680, 311, 279, 16088, 33044, 7673, 330, 77264, 22983, 36627, 11953, 279, 7039, 11, 315, 3388, 13, 358, 574, 1022, 14523, 505, 3116, 198, 998, 4848, 13, 3277, 22770, 22983, 2163, 279, 26741, 11, 279, 47027, 261, 1053, 617, 1124, 1210, 330, 12389, 499, 2944, 311, 1781, 430, 3060, 832, 315, 1521, 7724, 1053, 6562, 499, 198, 71, 2227, 7673, 330, 4155, 8996, 1359, 6137, 76996, 92468, 398, 26, 323, 1243, 11, 13598, 198, 71, 388, 491, 11, 313, 1, 39870, 313, 8248, 430, 596, 4856, 27873, 1210, 330, 40, 3077, 7170, 3463, 430, 22770, 22983, 313, 8248, 358, 1097, 2771, 1364, 374, 14268, 198, 42000, 449, 757, 13, 7570, 422, 1364, 313, 333, 1364, 313, 1875, 58841, 7673, 17882, 11, 358, 1541, 956, 4510, 313, 10445, 11, 735, 2637, 1364, 8434, 956, 0, 330, 60978, 1126, 11, 315, 3388, 1359, 1071, 735, 2637, 330, 258, 14944, 11, 90064, 13, 358, 2846, 1193, 4560, 311, 1505, 704, 6832, 16930, 433, 574, 1210, 32862, 1306, 430, 1364, 1071, 1695, 45427, 323, 4024, 704, 13, 3005, 6656, 304, 279, 198, 11020, 3195, 323, 31645, 18659, 38662, 1203, 520, 1461, 13, 330, 2675, 617, 2884, 757, 264, 2763, 315, 1695, 13, 3161, 264, 4062, 7351, 430, 574, 832, 315, 1077, 81750, 11, 76996, 15187, 8036, 198, 1820, 6134, 323, 43578, 1203, 1139, 279, 3130, 13, 735, 2637, 11011, 279, 6134, 3345, 345, 61665, 1364, 1047, 8208, 11, 323, 12504, 17345, 1139, 264, 10716, 13, 1071, 76996, 15187, 505, 4920, 1461, 345, 438, 11, 58218, 927, 11, 1364, 59334, 1461, 389, 279, 41803, 13, 578, 1828, 9888, 279, 6134, 1047, 8036, 4920, 1077, 11, 323, 735, 13, 574, 2163, 7636, 198, 998, 1778, 289, 62837, 2136, 323, 59045, 439, 279, 11714, 1047, 7263, 1461, 13, 1952, 9017, 6693, 11, 5340, 65670, 11, 889, 46498, 719, 2800, 16117, 304, 1077, 43713, 345, 86, 3448, 291, 311, 279, 81234, 315, 813, 3177, 927, 279, 1380, 316, 13, 330, 40, 6562, 499, 8434, 956, 733, 311, 198, 26894, 323, 1095, 701, 3177, 8395, 9135, 735, 2637, 1765, 76, 3876, 279, 43713, 323, 9439, 12932, 11, 1047, 279, 14413, 539, 311, 1825, 813, 198, 11020, 13, 330, 40, 1097, 539, 33173, 11, 5340, 65670, 11, 323, 358, 1097, 14931, 922, 279, 3177, 13, 13538, 568, 56807, 3384, 279, 3177, 11, 568, 15203, 927, 311, 279, 2362, 96637, 323, 198, 48554, 291, 5678, 304, 279, 9168, 13, 9220, 22178, 2085, 6212, 323, 1790, 18547, 198, 32345, 3309, 389, 1461, 13, 1283, 7111, 2362, 11, 305, 16094, 569, 26, 72600, 19781, 13, 49693, 750, 568, 198, 5807, 1636, 5678, 449, 17882, 11, 74820, 449, 2450, 11, 38732, 11, 92906, 28022, 345, 60301, 85943, 306, 13, 12334, 1047, 810, 7995, 3309, 1461, 279, 3987, 33028, 198, 1073, 813, 3021, 369, 76996, 1109, 1077, 1695, 45427, 21735, 13, 1283, 24465, 264, 1317, 11745, 323, 45374, 198, 998, 2073, 676, 304, 279, 6453, 13, 13142, 46506, 12669, 3782, 311, 1518, 76996, 279, 1828, 1938, 13, 3005, 1053, 617, 31890, 198, 40617, 422, 1364, 1436, 11, 719, 386, 25877, 1047, 64260, 291, 1461, 709, 311, 279, 52319, 48840, 293, 3023, 13603, 198, 15145, 1364, 1047, 892, 311, 12731, 13, 3005, 1047, 539, 3970, 279, 8334, 369, 1403, 4038, 345, 438, 279, 2349, 304, 1461, 89815, 1077, 13, 1283, 574, 65355, 11, 4856, 91425, 345, 26121, 455, 38662, 1664, 26435, 13, 1364, 1071, 11, 323, 1243, 25, 330, 76936, 956, 499, 2503, 1523, 7673, 1283, 13859, 266, 1534, 5678, 11, 439, 568, 1047, 430, 198, 9471, 279, 5651, 1603, 994, 568, 1047, 4691, 76996, 311, 32918, 1461, 13, 1283, 9076, 912, 21349, 43213, 8996, 280, 8248, 11, 1306, 10795, 287, 1077, 27851, 11, 1077, 3776, 65510, 11, 279, 5238, 2212, 1077, 198, 58331, 25, 27567, 22336, 2351, 539, 2133, 1203, 311, 430, 2035, 11, 315, 3388, 7673, 330, 12487, 18570, 596, 2751, 311, 10491, 369, 499, 13, 578, 3245, 369, 499, 311, 656, 374, 311, 198, 60021, 1314, 1618, 11, 76996, 13, 37558, 1618, 198, 41450, 3596, 62742, 499, 315, 4560, 311, 10102, 21739, 1210, 763, 34781, 315, 11937, 11, 76996, 31645, 264, 2697, 13, 1102, 574, 264, 16930, 922, 279, 198, 2106, 292, 1572, 13, 5414, 3021, 574, 32227, 45743, 11, 369, 568, 62639, 16038, 1077, 8835, 439, 422, 1364, 198, 32345, 539, 22066, 13, 330, 2675, 3041, 757, 279, 3492, 323, 358, 3358, 733, 323, 636, 701, 2574, 26, 358, 3077, 2751, 264, 1841, 315, 198, 2465, 1866, 1457, 1210, 330, 4071, 11, 13142, 11, 814, 617, 1193, 2884, 1148, 814, 3463, 574, 1314, 13, 91395, 1903, 198, 275, 11, 1070, 574, 264, 16930, 1210, 330, 2675, 1541, 956, 3152, 430, 499, 527, 2133, 311, 2559, 369, 420, 3460, 315, 3245, 30, 7357, 892, 1063, 24931, 3727, 264, 16930, 11, 527, 814, 2133, 311, 19065, 433, 389, 198, 9514, 7673, 358, 649, 956, 3137, 311, 499, 198, 333, 499, 16135, 1093, 264, 25294, 682, 279, 892, 1210, 6385, 5030, 8838, 2269, 533, 16630, 1047, 1202, 2515, 13, 1283, 11084, 1139, 279, 3130, 11, 719, 198, 383, 2103, 88106, 291, 264, 10716, 13, 330, 40, 8101, 499, 3077, 1027, 20910, 3249, 499, 9167, 956, 6755, 505, 757, 1359, 568, 1071, 13, 330, 40, 3077, 3970, 499, 810, 1109, 499, 3077, 3970, 757, 1210, 578, 4623, 315, 78739, 374, 2744, 2109, 773, 52985, 11, 323, 198, 998, 617, 264, 18010, 31657, 2744, 304, 279, 1022, 287, 11, 439, 433, 1051, 11, 574, 198, 4338, 89405, 287, 13, 330, 40, 6562, 499, 1053, 387, 1120, 264, 2697, 2766, 38761, 11, 13142, 13, 1102, 596, 779, 30571, 315, 198, 9514, 11, 2216, 13, 1102, 596, 539, 1606, 499, 2512, 369, 757, 26, 433, 596, 2216, 1606, 499, 198, 10727, 369, 6261, 1210, 330, 2675, 649, 956, 1427, 520, 757, 323, 2019, 430, 11, 34989, 1210, 1283, 10837, 813, 14654, 2212, 813, 37204, 313, 276, 2362, 31257, 26, 719, 279, 37204, 574, 198, 1225, 20784, 13, 330, 40, 2846, 1120, 12459, 856, 4851, 704, 369, 499, 11, 323, 430, 596, 279, 8206, 13, 98325, 358, 733, 11, 1274, 2019, 11, 364, 3947, 596, 279, 12637, 76996, 198, 2732, 6656, 1523, 994, 1364, 4024, 311, 279, 8952, 3238, 358, 3077, 2751, 779, 358, 2567, 1022, 198, 1820, 6825, 439, 1790, 439, 358, 649, 1210, 1115, 8545, 11, 12304, 8334, 574, 539, 198, 1820, 5679, 3640, 398, 37513, 12822, 1364, 1047, 2744, 3967, 13, 1102, 9508, 311, 1077, 198, 9210, 568, 574, 20781, 62200, 313, 9210, 30456, 813, 11594, 11827, 323, 15884, 198, 265, 14655, 7899, 1070, 47747, 48259, 2555, 61754, 11, 2555, 1364, 1436, 198, 1962, 37586, 449, 13, 330, 4071, 1148, 656, 499, 1390, 757, 311, 656, 30, 1442, 499, 4265, 198, 3323, 2503, 1523, 313, 1875, 7189, 1390, 499, 311, 2586, 2162, 13, 358, 1120, 1390, 198, 9514, 311, 2586, 1203, 11, 779, 430, 2574, 690, 387, 279, 1648, 814, 1511, 311, 387, 13, 4800, 198, 9210, 814, 617, 6656, 499, 704, 313, 1875, 32408, 3077, 2884, 4400, 315, 279, 3460, 13, 330, 18433, 499, 3021, 279, 8952, 11, 477, 1606, 499, 3021, 18570, 8599, 198, 4291, 279, 8952, 7673, 76996, 574, 27461, 19021, 555, 420, 892, 11, 19021, 323, 54317, 13, 3005, 1047, 198, 2063, 1555, 779, 1790, 430, 1475, 32015, 574, 31299, 304, 25429, 8835, 13, 330, 2746, 433, 690, 1304, 499, 3619, 2574, 904, 2731, 1359, 1364, 39169, 11, 330, 40, 1097, 198, 9738, 1203, 369, 2225, 8125, 9135, 2030, 1077, 4339, 9508, 11, 29392, 198, 268, 1409, 11, 311, 24981, 1461, 13, 330, 12487, 11, 439, 3117, 439, 358, 1097, 11920, 11, 433, 596, 682, 927, 11, 374, 433, 7673, 58801, 25, 27567, 22336, 1781, 46993, 706, 1077, 6206, 2539, 449, 42216, 11, 1541, 956, 499, 30, 8489, 345, 333, 499, 1935, 7639, 17882, 11, 499, 2351, 2133, 311, 617, 810, 12544, 1109, 46993, 198, 2099, 57636, 315, 13, 358, 649, 3371, 499, 1063, 2574, 922, 1461, 1457, 430, 690, 1304, 198, 9514, 1781, 11157, 1210, 330, 11769, 3492, 430, 499, 2019, 5039, 757, 1268, 1314, 358, 1097, 304, 539, 78849, 499, 345, 41444, 1359, 1364, 1071, 13, 330, 13058, 3026, 656, 539, 2019, 1884, 2574, 922, 1855, 1023, 1234, 198, 3852, 13463, 13, 358, 1541, 956, 1390, 499, 311, 198, 2063, 1203, 3156, 499, 617, 15042, 709, 1210, 1283, 574, 1633, 4251, 11, 719, 568, 13061, 709, 813, 9072, 323, 4024, 311, 279, 6134, 13, 330, 40, 8101, 358, 6912, 14599, 1359, 568, 1071, 13, 330, 40, 3077, 1027, 19762, 311, 733, 3201, 11, 719, 6691, 198, 33371, 1778, 264, 64864, 313, 40, 3358, 539, 38292, 499, 904, 810, 1210, 1283, 8813, 304, 813, 18301, 323, 11, 23062, 704, 264, 2678, 3830, 11, 5762, 433, 9017, 198, 1964, 13, 330, 3561, 992, 67, 1359, 568, 1071, 64702, 398, 13, 4427, 13305, 10791, 198, 40617, 304, 279, 6246, 11, 323, 358, 7263, 1461, 2162, 1210, 1283, 2163, 1077, 11509, 1070, 8982, 1752, 449, 13051, 11, 449, 279, 3830, 304, 1077, 198, 10888, 11, 323, 10837, 1523, 279, 32249, 323, 704, 1139, 279, 6825, 13, 2468, 279, 4579, 315, 198, 1820, 7504, 568, 4661, 79990, 449, 2999, 13, 358, 2846, 16089, 198, 9514, 3077, 1903, 433, 709, 1210, 96424, 30388, 1432, 67288, 31467, 1202, 43789, 14297, 430, 1060, 13, 5587, 574, 99945, 9439, 280, 17206, 5936, 1766, 279, 19795, 2103, 20268, 323, 279, 61316, 1414, 4336, 93858, 449, 198, 560, 13, 2030, 520, 5209, 11477, 1070, 574, 10683, 304, 279, 3805, 13, 763, 279, 66385, 315, 279, 198, 71127, 11, 5804, 3916, 39428, 7731, 389, 279, 79509, 323, 15746, 369, 10773, 1354, 13, 578, 198, 69, 35624, 11, 902, 1047, 20268, 704, 11, 574, 1694, 52834, 13, 5810, 323, 1070, 389, 198, 1637, 3321, 1355, 3385, 62036, 3153, 9107, 872, 342, 8039, 88, 96740, 311, 279, 7160, 13, 5340, 65670, 1047, 8208, 22917, 369, 264, 16706, 8577, 304, 5587, 323, 3782, 1203, 82787, 198, 4291, 502, 6848, 11, 1646, 342, 43456, 11, 323, 7878, 36232, 13, 3005, 11953, 704, 323, 198, 501, 7719, 19837, 389, 1077, 13219, 596, 25165, 11, 323, 4024, 1203, 311, 1077, 990, 449, 264, 198, 1897, 20659, 315, 14523, 2884, 13, 362, 10824, 315, 14425, 66, 4881, 323, 12056, 389, 279, 5015, 198, 32345, 2728, 1077, 459, 20343, 369, 264, 65510, 13, 3005, 24465, 433, 304, 47218, 389, 459, 198, 268, 19384, 389, 1077, 1648, 1203, 304, 279, 8761, 1841, 13, 32171, 56310, 11, 3515, 1903, 1695, 2391, 279, 4251, 6763, 11, 1047, 1027, 3288, 311, 198, 1820, 10683, 24428, 82, 13, 578, 1938, 1364, 198, 63578, 76996, 3769, 369, 264, 4382, 4251, 65510, 11, 1364, 574, 1633, 6380, 13, 9843, 198, 64, 6130, 7263, 1077, 264, 15860, 315, 9036, 3714, 288, 13, 2052, 1938, 1364, 8774, 1124, 1234, 198, 1820, 5663, 304, 264, 9168, 315, 3090, 11, 323, 520, 11714, 1364, 3952, 1124, 311, 32980, 198, 49, 8477, 31202, 11, 2103, 21078, 38097, 304, 279, 8952, 13, 1952, 76996, 11, 389, 735, 2637, 323, 389, 46993, 279, 12688, 1047, 2163, 1202, 1906, 17345, 13, 46993, 11, 1373, 4345, 287, 1077, 2324, 311, 502, 4787, 11, 574, 1099, 7403, 11, 810, 198, 61665, 1285, 3238, 82, 19351, 11, 1364, 198, 32345, 2728, 709, 279, 330, 35, 1412, 434, 1, 323, 574, 5403, 1972, 6603, 13, 3005, 574, 7422, 198, 8110, 11555, 11, 2288, 11, 369, 279, 1176, 892, 304, 1077, 2324, 13, 76996, 11, 439, 28682, 439, 3596, 11, 1047, 5675, 264, 2697, 315, 279, 12164, 685, 505, 1077, 198, 58331, 26, 1077, 7899, 1047, 5655, 6901, 13, 11208, 1364, 1047, 1027, 264, 5128, 3828, 11, 1364, 198, 16514, 1457, 17104, 13, 3005, 574, 1203, 304, 279, 8952, 1578, 11, 420, 892, 304, 279, 198, 5988, 596, 26741, 13, 735, 2637, 2133, 304, 832, 1938, 311, 1935, 32980, 35120, 31202, 264, 14351, 198, 1073, 14098, 11, 5602, 1077, 1070, 449, 264, 1716, 304, 1077, 11977, 11, 323, 264, 3177, 304, 1077, 198, 58331, 430, 568, 1047, 2646, 3970, 1603, 13, 1102, 13194, 1461, 11, 4856, 313, 58174, 1694, 439, 198, 20670, 1051, 449, 1461, 13, 3161, 279, 8736, 315, 10683, 279, 2697, 3838, 520, 8270, 5447, 3952, 389, 7878, 198, 33078, 13, 40788, 648, 574, 3838, 1824, 60051, 449, 2294, 17879, 2136, 13, 3005, 198, 26121, 392, 2788, 89341, 11, 3952, 1523, 279, 4335, 54499, 11, 323, 2231, 1124, 709, 1578, 198, 72408, 398, 70181, 291, 13, 1102, 574, 439, 422, 1364, 1766, 304, 33638, 5820, 323, 36709, 264, 198, 1864, 9497, 369, 1077, 6316, 300, 1918, 13, 578, 89646, 3752, 315, 279, 2697, 198, 7830, 1047, 1027, 2403, 433, 13, 5124, 3886, 1047, 264, 2697, 3703, 323, 198, 82, 2841, 279, 1888, 12046, 1105, 568, 1436, 3780, 26, 719, 568, 64770, 2872, 67, 1918, 313, 32345, 198, 82850, 3967, 311, 26122, 311, 4662, 311, 13305, 1234, 17510, 19101, 323, 311, 3026, 889, 1047, 198, 46584, 927, 485, 360, 3640, 13, 578, 3492, 4024, 922, 430, 5124, 3886, 596, 574, 912, 2035, 198, 2000, 264, 1695, 892, 13, 7570, 40788, 648, 596, 16553, 323, 289, 81635, 4745, 2403, 420, 198, 10888, 98450, 13, 3296, 279, 6278, 315, 5936, 279, 3838, 1824, 60051, 574, 2884, 13, 3861, 477, 1403, 9048, 198, 4581, 552, 1047, 2586, 704, 11, 294, 1619, 11163, 2718, 323, 289, 1619, 70351, 11, 323, 1047, 8208, 198, 1445, 311, 279, 3363, 1578, 13, 578, 198, 43791, 32720, 709, 11, 10773, 1354, 10409, 279, 12690, 449, 872, 50380, 10683, 11936, 345, 438, 2103, 2626, 8738, 41630, 13, 3296, 279, 1176, 1938, 315, 3297, 11, 40788, 648, 596, 6316, 300, 1918, 1047, 3719, 44743, 13, 5124, 3886, 11, 5108, 304, 505, 279, 4216, 7625, 10789, 11, 1766, 1077, 198, 82, 15154, 304, 279, 9979, 11, 1077, 3663, 28016, 304, 1077, 21726, 263, 13, 1283, 2231, 1523, 279, 198, 76, 34263, 2320, 6341, 323, 11, 2133, 927, 311, 1077, 11, 2231, 264, 1450, 389, 1077, 2010, 13, 330, 40, 8101, 1070, 596, 912, 16930, 11, 1243, 7673, 330, 3947, 596, 912, 16930, 1359, 1071, 8009, 40788, 648, 1139, 1077, 21726, 263, 13, 1283, 30280, 1523, 323, 59334, 279, 1203, 315, 1077, 13272, 13, 5112, 11, 994, 1364, 4745, 311, 198, 73216, 268, 11, 568, 9165, 59091, 291, 2212, 279, 9979, 11, 44033, 279, 14403, 1139, 64883, 345, 438, 97139, 291, 279, 43212, 11, 3318, 1749, 2740, 304, 813, 8987, 1648, 13, 578, 198, 82694, 12934, 48726, 1047, 66185, 9235, 13, 5112, 25, 27567, 91724, 499, 1390, 311, 1518, 264, 10896, 7673, 330, 40, 4265, 2731, 1518, 18570, 1359, 1364, 1071, 11, 2085, 3411, 709, 13, 330, 3112, 313, 15357, 956, 198, 27963, 358, 2846, 59771, 499, 13, 1666, 3117, 439, 198, 9210, 5900, 11, 358, 3077, 4934, 264, 1716, 1314, 3235, 13, 1102, 4536, 956, 279, 12544, 358, 1097, 198, 83360, 315, 3060, 1210, 1283, 1903, 1063, 15600, 198, 566, 6370, 1570, 323, 14198, 291, 1077, 264, 6710, 315, 23211, 13, 3277, 568, 1047, 2231, 1124, 389, 832, 198, 408, 315, 279, 9979, 2007, 11, 568, 4024, 927, 311, 1077, 1578, 13, 330, 40, 8101, 358, 4265, 22525, 311, 617, 3463, 315, 420, 1603, 11, 719, 682, 358, 3463, 315, 198, 16514, 4560, 311, 636, 264, 2697, 23871, 704, 315, 2324, 13, 1628, 1359, 313, 383, 31511, 48259, 198, 1964, 6916, 11, 313, 1, 300, 3117, 439, 358, 1097, 11920, 11, 433, 596, 1027, 5922, 1418, 11, 40788, 648, 13, 2360, 198, 60065, 1148, 358, 3077, 1047, 311, 656, 11, 358, 3077, 2744, 7111, 4741, 311, 5108, 1203, 198, 6881, 311, 499, 304, 279, 11714, 13, 10926, 358, 1541, 956, 2019, 433, 3403, 11, 719, 358, 8101, 499, 198, 33134, 358, 2733, 433, 682, 1314, 1210, 17586, 3411, 709, 11, 1364, 9277, 1077, 1450, 927, 813, 13, 330, 40, 8101, 584, 3940, 5076, 1359, 568, 4024, 389, 13, 330, 2675, 649, 956, 1977, 23871, 389, 198, 12840, 4536, 956, 1314, 13, 1472, 323, 358, 649, 10299, 1664, 3403, 26, 719, 1457, 430, 1070, 596, 198, 9738, 311, 387, 2500, 11, 433, 5992, 2204, 11, 17354, 1210, 4740, 430, 6693, 40788, 648, 3952, 709, 1077, 23104, 43132, 2740, 13, 578, 3987, 315, 198, 27510, 5812, 7064, 660, 449, 3776, 18809, 315, 18710, 13, 3005, 29340, 520, 1077, 198, 1816, 11, 311, 21165, 704, 1139, 11210, 24014, 13, 5124, 3886, 1047, 2728, 709, 813, 56226, 198, 27243, 26, 719, 279, 72998, 889, 3782, 311, 8270, 5447, 1550, 539, 2586, 1203, 13, 3277, 11, 520, 1566, 11, 568, 3952, 279, 15580, 323, 80411, 323, 23980, 922, 279, 3224, 198, 2000, 10373, 11, 568, 574, 2288, 3389, 13, 7089, 56226, 5794, 1047, 1027, 1603, 1461, 280, 939, 60530, 655, 552, 323, 41245, 2402, 1051, 2736, 1694, 743, 704, 13, 578, 2132, 8323, 198, 263, 813, 21515, 1053, 387, 4245, 304, 5887, 13, 3296, 279, 6278, 315, 3297, 814, 1051, 198, 1658, 1201, 398, 709, 2403, 433, 13, 5124, 3886, 520, 1566, 76035, 311, 2231, 279, 6671, 1139, 198, 5880, 13, 330, 1687, 2351, 539, 3339, 1695, 11, 58769, 1359, 568, 1071, 13, 1226, 527, 2288, 15326, 26, 430, 596, 1148, 596, 279, 5030, 449, 603, 1210, 3161, 682, 1077, 85536, 11, 40788, 648, 574, 53108, 48080, 315, 2324, 13, 330, 1687, 3358, 617, 311, 2567, 264, 3460, 315, 9689, 1359, 568, 1071, 32703, 989, 13, 330, 69633, 311, 16470, 198, 9210, 4131, 3235, 11, 323, 313, 333, 9875, 1390, 311, 4822, 927, 45427, 313, 1875, 51, 484, 648, 596, 4251, 3663, 6656, 97119, 13, 330, 2746, 433, 596, 3958, 9282, 11, 323, 814, 2351, 12502, 313, 1875, 72059, 527, 584, 311, 1440, 422, 814, 527, 12502, 477, 539, 7673, 2030, 279, 198, 47965, 4090, 574, 539, 2753, 30883, 13, 2684, 1051, 1403, 477, 2380, 9662, 84539, 12295, 198, 263, 279, 2132, 6558, 13, 1283, 6137, 311, 1304, 76220, 18726, 439, 311, 872, 198, 69, 399, 11218, 13, 9843, 568, 2751, 264, 49639, 505, 459, 47644, 3838, 11, 323, 6818, 198, 998, 10477, 433, 505, 1077, 13, 3005, 27724, 433, 304, 279, 9979, 198, 267, 1009, 13, 5124, 3886, 5678, 574, 51975, 26, 719, 279, 4623, 44898, 1461, 13, 7089, 1274, 198, 69, 1617, 6901, 389, 279, 91223, 29646, 315, 3823, 7138, 13, 9220, 8931, 3201, 11, 389, 279, 1023, 198, 8714, 11, 574, 264, 586, 3838, 430, 1047, 4272, 6702, 279, 6506, 5899, 16579, 11441, 198, 26466, 279, 1060, 1603, 13, 1283, 574, 539, 439, 3995, 439, 568, 1047, 1027, 26, 1070, 574, 279, 20900, 315, 10494, 198, 26301, 7555, 313, 383, 1047, 2646, 5535, 1077, 311, 733, 1139, 279, 23693, 26741, 520, 279, 198, 300, 28433, 13, 4800, 430, 1070, 574, 2133, 311, 387, 264, 1716, 11, 1070, 1053, 387, 2380, 198, 16455, 18222, 5304, 1461, 13, 3861, 3814, 11, 1306, 40788, 648, 574, 33173, 11, 568, 43578, 12248, 16117, 1139, 813, 198, 33678, 6509, 323, 704, 311, 279, 33419, 11, 1405, 568, 59207, 291, 709, 279, 15580, 449, 23418, 198, 69, 14437, 13, 40788, 648, 2646, 9687, 315, 430, 33433, 95440, 311, 279, 330, 34, 4763, 7278, 16344, 43352, 20375, 8931, 3201, 13, 35270, 1529, 28109, 304, 1475, 3321, 26, 264, 21030, 79648, 1051, 198, 29836, 291, 1603, 279, 33419, 13, 5659, 279, 3703, 3782, 198, 1820, 503, 2222, 315, 29247, 323, 17813, 11, 71414, 10652, 13, 3277, 5124, 3886, 6656, 279, 15580, 596, 2010, 1203, 9017, 8270, 5447, 11, 813, 198, 38824, 574, 1903, 709, 13, 1283, 1053, 52259, 279, 8582, 12295, 26, 568, 1053, 4546, 264, 198, 2308, 19393, 505, 6424, 313, 45010, 1274, 4934, 9709, 21662, 26, 568, 1436, 636, 264, 198, 5686, 25417, 27374, 15038, 13, 3277, 1364, 1766, 1461, 198, 67, 47737, 11, 1364, 1903, 279, 30485, 430, 1077, 3044, 4541, 33337, 13, 3005, 198, 29177, 539, 5387, 1461, 11, 719, 1364, 1053, 539, 4822, 304, 279, 89266, 660, 2697, 198, 7830, 13, 3277, 11, 264, 2046, 1306, 5124, 3886, 596, 4034, 311, 279, 330, 34, 4763, 7278, 16344, 1359, 459, 198, 12527, 479, 5355, 11721, 505, 6424, 449, 279, 502, 14891, 11, 40788, 648, 198, 95918, 704, 311, 1148, 1047, 1027, 279, 33508, 48840, 315, 279, 2362, 33419, 323, 1070, 198, 34500, 291, 11937, 13, 330, 40, 1097, 539, 9564, 499, 1359, 1364, 3309, 1461, 13, 330, 40, 1541, 956, 1524, 1440, 430, 358, 1097, 198, 2067, 6605, 499, 13, 2030, 358, 1097, 539, 2133, 311, 617, 4205, 311, 656, 449, 433, 11, 323, 198, 9210, 596, 10269, 1210, 2100, 433, 7077, 430, 735, 2637, 3339, 264, 10683, 90413, 311, 1518, 40788, 648, 345, 90963, 12025, 13382, 304, 279, 5754, 13, 578, 9282, 574, 8369, 11, 323, 568, 11953, 198, 26301, 59792, 23724, 927, 813, 6916, 13, 578, 2697, 3838, 574, 90256, 26, 264, 21030, 198, 28172, 3454, 82, 1051, 43335, 304, 279, 33419, 17884, 13, 578, 3703, 574, 39313, 11, 323, 264, 198, 2308, 19393, 304, 264, 4251, 23724, 574, 27890, 21662, 449, 279, 16736, 85273, 198, 1073, 813, 3169, 13, 2684, 1051, 12920, 1234, 279, 12690, 389, 279, 37125, 11, 323, 264, 502, 198, 7908, 389, 279, 18618, 13, 7570, 5124, 3886, 39786, 264, 502, 1427, 315, 44202, 13, 6193, 813, 78140, 27674, 315, 13179, 198, 42, 13, 20802, 2555, 315, 279, 3446, 13, 330, 40, 2846, 539, 12691, 315, 433, 11, 4491, 13, 358, 3077, 2586, 311, 656, 264, 1695, 1690, 2574, 198, 1820, 1566, 1060, 477, 779, 430, 358, 2646, 3463, 358, 1053, 656, 13, 5629, 358, 3952, 40788, 648, 3201, 505, 1077, 1695, 2361, 11, 323, 1306, 198, 9210, 4400, 4024, 1314, 13, 5112, 1070, 1051, 2574, 5108, 389, 75857, 383, 7111, 520, 198, 42, 13, 74884, 13610, 313, 1, 9210, 8967, 810, 20900, 13, 358, 1053, 387, 16089, 422, 499, 8434, 956, 198, 37890, 4205, 922, 433, 520, 18083, 13, 330, 40, 3358, 539, 6604, 315, 433, 11, 315, 3388, 1210, 1102, 574, 1243, 11, 994, 735, 13, 4691, 369, 40788, 648, 11, 430, 4491, 13, 5124, 3886, 596, 31175, 67391, 198, 17106, 373, 810, 10186, 13, 330, 8100, 8434, 956, 2559, 369, 433, 1359, 568, 1071, 13, 330, 8100, 7882, 704, 279, 1938, 358, 24330, 198, 1820, 12295, 50007, 323, 2751, 279, 27374, 1210, 358, 313, 40, 3358, 1935, 499, 198, 412, 1070, 11, 422, 499, 1053, 1093, 311, 1518, 1077, 1210, 735, 13, 559, 4361, 67, 398, 1765, 76, 4147, 430, 40788, 648, 1053, 10932, 311, 1518, 1461, 7636, 11, 1234, 198, 1820, 13463, 13, 330, 40, 8101, 358, 649, 1505, 1077, 1359, 568, 1071, 11, 323, 16392, 505, 279, 2697, 2007, 13, 330, 2746, 499, 313, 333, 499, 649, 2019, 4205, 311, 1520, 757, 704, 11, 28146, 11, 358, 4265, 15763, 433, 13, 5046, 3388, 11, 1364, 31869, 1268, 358, 1097, 16625, 13, 2030, 313, 36046, 422, 499, 1053, 198, 73457, 1077, 430, 279, 6825, 3250, 956, 1440, 313, 1875, 7189, 3358, 656, 682, 358, 649, 1359, 735, 13, 19487, 11, 323, 8272, 279, 1853, 311, 279, 33419, 13, 578, 2697, 33508, 48840, 198, 16514, 1633, 10882, 13, 362, 4251, 11245, 4950, 304, 264, 9309, 11, 264, 10269, 2007, 449, 198, 64, 18327, 3485, 433, 11, 264, 67724, 79781, 11, 323, 264, 52319, 84315, 24330, 279, 198, 3039, 13, 330, 40, 8434, 956, 2559, 369, 433, 1359, 1364, 1071, 5042, 26, 330, 708, 1618, 358, 1097, 13, 2684, 1694, 719, 832, 198, 524, 2196, 1363, 14924, 25, 11208, 374, 10455, 30, 22559, 449, 264, 3254, 3492, 13, 128009, 128006, 78191, 128007, 271]

In [55]:
x1 == x2

True

In [19]:
input_text

[{'role': 'user',
  'content': '<context>\nPresently, indeed, there appeared an able _extrait_ of\nGrampus\'s article in the valuable _Rapporteur scientifique et\nhistorique_, and Merman\'s mistakes were thus brought under the notice of\ncertain Frenchmen who are among the masters of those who know on\noriental subjects. In a word, Merman, though not extensively read, was\nextensively read about. Perhaps nobody, except his wife, for a\nmoment reflected on that. An amused society considered that he was\nseverely punished, but did not take the trouble to imagine his\nsensations; indeed this would have been a difficulty for persons less\nsensitive and excitable than Merman himself. Perhaps that popular\ncomparison of the Walrus had truth enough to bite and blister on\nthorough application, even if exultant ignorance had not applauded it. But it is well known that the walrus, though not in the least a\nmalignant animal, if allowed to display its remarkably plain person and\nblundering perf

### Interpret

In [190]:
use_chat_template = True
use_instruction = False
use_post_prompt = False
use_examples = False

cpt_path = checkpoints[-1]
task = 'qa1'

split_name = '4k'

model.rmt_config['segment_alignment'] = 'right'
eval_model_name = eval_model_template.format(cpt_path.split('adamw_wd1e-03_')[1].split('_')[0])
print('Evaluating ', eval_model_name)

with open(cpt_path, 'rb') as cpt:
    weights = torch.load(cpt)

model.load_state_dict(weights)
model.cuda()
model = model.eval()

device = 'cuda:0'

# configure the prompt
prompt_cfg = {
    'instruction': DEFAULT_PROMPTS[task]['instruction'] if use_instruction else '',
    'examples': DEFAULT_PROMPTS[task]['examples'] if use_examples else '',
    'post_prompt': DEFAULT_PROMPTS[task]['post_prompt'] if use_post_prompt else '',
    'template': DEFAULT_TEMPLATE,
    'chat_template': use_chat_template,
}
prompt_name = [f'{k}_yes' if prompt_cfg[k] else f'{k}_no' for k in prompt_cfg if k != 'template']
prompt_name = '_'.join(prompt_name)

# load dataset
data = datasets.load_dataset(dataset_name, split_name)
task_data = data[task]

# Prepare files with predictions, prompt, and generation configurations
outfile = Path(f'{results_folder}/{eval_model_name}/{task}_{split_name}_{prompt_name}.csv')
outfile.parent.mkdir(parents=True, exist_ok=True)
cfg_file = f'{results_folder}/{eval_model_name}/{task}_{split_name}_{prompt_name}.json'
# json.dump({'prompt': prompt_cfg, 'generate_kwargs': generate_kwargs}, open(cfg_file, 'w'), indent=4)

df = pd.DataFrame({'target': [], 'output': [], 'question': [], 'input': [], 
                   'n_segments': [], 'segments': []})

inputs = []
for sample in tqdm(task_data.select(range(20)), desc=f'task: {task} length: {split_name}'):
    target = sample['target']
    context = sample['input']
    question = sample['question']

    # format input text
    input_text = get_formatted_input(context, question, prompt_cfg['examples'],
                                        prompt_cfg['instruction'], prompt_cfg['post_prompt'],
                                        template=prompt_cfg['template'])

    if use_chat_template:

        template = "{} {} Answer with a single word."
        # context = tokenizer.decode(sample['input_tokens'])
        input_text = [
            {"role": "user", "content": template.format(context, question)},
        ]
        # input_text = [{'role': 'user', 'content': input_text}]

        model_inputs = tokenizer.apply_chat_template(input_text, add_generation_prompt=True,
                                                        return_tensors='pt').to(device)
        model_inputs = {'input_ids': model_inputs}
    else:
        model_inputs = tokenizer(input_text, return_tensors='pt',
                                    add_special_tokens=True).to(device)

    sample_length = model_inputs['input_ids'].shape[1]
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            attn_mask = torch.ones_like(model_inputs['input_ids'].bool().to(device))
            output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)
            # we need to reset memory states between samples for activation-beacon models
            # if 'activation-beacon' in model.name_or_path and hasattr(model, 'memory'):
            #     model.memory.reset()

    output = output[0]#[sample_length:]
    output = tokenizer.decode(output, skip_special_tokens=True).strip()
    # 1/0
    inp = tokenizer.decode(model_inputs['input_ids'][0].cpu(), add_special_tokens=True)

    segments = model.segment(input_ids=model_inputs['input_ids'])
    seg_texts = [tokenizer.decode(s['input_ids'][0], add_special_tokens=True) for s in segments]

    df.loc[len(df)] = [target, output, question, inp, len(segments), '\n------------\n'.join(seg_texts)]
# write results to csv file

# df.to_csv(outfile)

Evaluating  rmt-llama3.2-1b-8x1024-ct-v3-retrain-align_right


Using the latest cached version of the dataset since RMT-team/babilong couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '4k' at /home/jovyan/.cache/huggingface/datasets/RMT-team___babilong/4k/0.0.0/ee0d588794c7ac098062ee0d247c733d62e94fe2 (last modified on Thu Aug  1 17:59:53 2024).


task: qa1 length: 4k:   0%|          | 0/20 [00:00<?, ?it/s]

In [181]:
df

,target,output,question,input,n_segments,segments
0,bathroom,!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
1,kitchen,kitchen,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,3,<|begin_of_text|><|start_header_id|>system<|en...
2,kitchen,kitchen,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
3,kitchen,!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
4,bedroom,bedroom,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
5,office,office,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
6,garden,!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
7,bathroom,bathroom,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
8,kitchen,kitchen,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
9,bedroom,bedroom,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...


In [191]:
df

,target,output,question,input,n_segments,segments
0,bathroom,bathroom,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
1,kitchen,kitchen,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,3,<|begin_of_text|><|start_header_id|>system<|en...
2,kitchen,kitchen,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
3,kitchen,kitchen,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
4,bedroom,bedroom,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
5,office,office,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
6,garden,garden,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
7,bathroom,bathroom,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
8,kitchen,kitchen,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
9,bedroom,bedroom,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...


In [246]:
sample = task_data[16]

target = sample['target']
context = sample['input']
question = sample['question']

# format input text
input_text = get_formatted_input(context, question, prompt_cfg['examples'],
                                    prompt_cfg['instruction'], prompt_cfg['post_prompt'],
                                    template=prompt_cfg['template'])

if use_chat_template:

    template = "{} {} Answer with a single word."
    # context = tokenizer.decode(sample['input_tokens'])
    input_text = [
        {"role": "user", "content": template.format(context, question)},
    ]
    # input_text = [{'role': 'user', 'content': input_text}]

    model_inputs = tokenizer.apply_chat_template(input_text, add_generation_prompt=True,
                                                    return_tensors='pt').to(device)
    model_inputs = {'input_ids': model_inputs}
else:
    model_inputs = tokenizer(input_text, return_tensors='pt',
                                add_special_tokens=True).to(device)

sample_length = model_inputs['input_ids'].shape[1]

In [247]:
with torch.cuda.amp.autocast():
    with torch.no_grad():
        attn_mask = torch.ones_like(model_inputs['input_ids'].bool().to(device))
        output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)
        # we need to reset memory states between samples for activation-beacon models
        # if 'activation-beacon' in model.name_or_path and hasattr(model, 'memory'):
        #     model.memory.reset()

output = output[0]#[sample_length:]
tokenizer.decode(output, skip_special_tokens=False)

'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

In [250]:
target

'kitchen'

In [248]:
with torch.cuda.amp.autocast():
    with torch.no_grad():
        attn_mask = torch.ones_like(model_inputs['input_ids'].bool().to(device))
        output = model(**model_inputs)#, attention_mask=attn_mask)
        # output = model.generate(**model_inputs, **generate_kwargs, attention_mask=attn_mask)5            output = model(**model_inputs, attention_mask=attn_mask)

In [249]:
tokenizer.decode(output.logits.argmax(dim=-1)[0][:])

"Tags\n\n\n\n\n\nIting back Space\n\n  2022\nI's: December0 of 2023\nLet<|start_header_id|>system<|start_header_id|>\n\nLet the timeI best of the two, the two of the the, theest, for twokustering of the most of of the two of of of the the of the beginningthe of of of of the world of to of to of of or of most ofof the, variety of of the two world of of the only the ofof word of the case of<|eot_id|>'s to the lab<|eot_id|><|eot_id|> and, you same of the is is and of thetheasesuriousbers of of is the the, or is same of of ofof the thousand of of of of from of the of the many of the,the of<|eot_id|> room of the which of the, not in in thethe of of the is a that to of of the first way ofof of the of of one of the one of to be a thousand tothere of1,<|eot_id|> if of the two to of the to ofof to are to the and the to of the the, the the first are and firstthe number to of the to to to make the on ininside the the first of of of to of to to, to are no to toto of1,<|eot_id|> is to the other<|eo

In [251]:
tokenizer.decode(output.logits.argmax(dim=-1)[0][-1024*3 -1:])

",Tags k havek this such kbedimateskkbedkkkkkbedkbedkkkbedilitykbed kbedbedkbedbed bedbedbedbedbedbedbed Kbedbedbedbedkkkbedbed arebedbedbedbed kbedbedbedk k k of k kbed k and to to the city, \n\nbedODbed K k k kkbededbedbed kbedbedkbedbed the bedbedbed and I arebedbed kbed of K bedk k and onbedbedbed thebeded ofbed K Kbedbed k thek k k kkbedbedbedbedbedbedbed k K kbed a k bed ofbedbed kbedbed k bed world ofbed k bedk ofbed the bedbed ofbedbedbedbed kbed the bedbedbedbedbed is a k of be bebedbed k of bedkbed of the Kbedbed the k of thebedbed k isbed ak of k I is notbed k k are thebed k ofbedbedbed k the k to the we bed k ofbedbed kbedbed of k on k K not a about the kbedkbed I thek K I bed k arebed k with to the bed of k the kk k of I bed k ofbed in of sidebed of andbed, thebedbed kbedbed and I arekplacementedbed arebed sincebed kbed kbed the of kbedbedbed of k K kbed k of the bed kkARD K k and k thebed arebedbed k ink K k of the kbedkk the k, bed of by k ofkbed bed ofbed k k of to k k 

In [239]:
output.loss

0

In [237]:
tokenizer.decode(output.logits.argmax(dim=-1)[0][-2049:])

"bedimportk k kk kbedbed bed bedbed bed the bedbedbedbed bybedbedbedbedbedbedbedbedbedbedbedbednowledbedbedbedbed ofbedbedbedbedbedbed of ofbedbedbedbedbed ed edk kbedkbedbed though bed bedbedbed bedbedbed kbedbedbedbedkbed bed of thebededbedbed k intobed k ofbed bedbedbedbed ofbed bedbedarden bed of thebedbedbedbed a k of kbedbed we bedbed bedbedbed bedbedbedbedbed bed kbedbedbedbedbedbedbedbedbedbedbedbedbedbedbedbedbedbedbed ofbedbedbedbedbed k bedbedbedbedbed kbedbedbedbed bybed kbedbedbed bedbed k ofbed bedbedbedbedkbed least kbed bedk k kbed bedbedbedbedbed kbedbed ofbedbed kbedbedbedbedbedbedbedbed and \n\n much bed k of arebedbedbedbedbedbedbedbedbed ofbedbed thebedbedbedbed bed k frombedbedbed bedbed of k k isbedbedbed edbedbed bedbedbedbedbeded of bedbedbed ofdbedbedbedbedbedbedbedbed bedbed bedbedbedbedbedbedbedbedbedbed to bedbedbed ofbedbedbedbedbedbedbedbedbedbedbedkbed the k kbedbed ofbed bed k frombedbed I kbed bedbed bedbedbedbed kbed k bed to tokbed ofbedbedbedbedbed 

In [238]:
tokenizer.decode(model_inputs['input_ids'][0][-2049:])

'\nwhich remind us of days far earlier. Edward the Fourth and Richard the\nThird were chosen Kings, or at least had their claims to the Crown\nacknowledged, by gatherings of the citizens of London which remind us\nof the wars of Stephen and Matilda(59). Still even in this age, the\npower of Parliament was advancing(60); the anxiety of every pretender\nto get a parliamentary sanction for his claims was a sign of the\ngrowing importance of Parliament, and we get incidental notices which\nshow that a seat in the House of Commons, and that not as a knight of a\nshire, but as a burgess of a borough, was now an object of ambition for\nmen of the class from which knights of the shire were chosen, and even\nfor the sons of members of the Upper House(61). At last came the sixteenth century, the time of trial for parliamentary\ninstitutions in so many countries of Europe. Daniel travelled to the kitchen. Not a few assemblies which\nhad once been as free as our own Parliament were, during that ag

In [179]:
context

'If all this effort and\nexpenditure had resulted in success, it would be possible to keep\nsilent and shrug one\'s shoulders; but when the mode of undertaking\nthis expedition can be clearly shown to have been the direct cause of\nits failure, silence would be a crime. When Lord Wolseley told the\nsoldiers at Korti on their return from Metemmah, "It was not _your_\nfault that Gordon has perished and Khartoum fallen," the positiveness\nof his assurance may have been derived from the inner conviction of\nhis own stupendous error. The expedition was finally sanctioned in August, and the news of its\ncoming was known to General Gordon in September, before, indeed, his\nown despatches of 31st July were received in London, and broke the\nsuspense of nearly half a year. He thought that only a small force was\ncoming, under the command of Major-General Earle, and he at once, as\nalready described, sent his steamers back to Shendy, there to await\nthe troops and convey them to Khartoum. He see

In [51]:
df

,target,output,question,input,n_segments,segments
0,bathroom,bathroom,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
1,kitchen,kitchen,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,3,<|begin_of_text|><|start_header_id|>system<|en...
2,kitchen,kitchen,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
3,kitchen,!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
4,bedroom,bedroom,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
5,office,!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
6,garden,garden,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
7,bathroom,bathroom,Where is Sandra?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
8,kitchen,kitchen,Where is Mary?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...
9,bedroom,bedroom,Where is John?,<|begin_of_text|><|start_header_id|>system<|en...,4,<|begin_of_text|><|start_header_id|>system<|en...


In [ ]:
for i, row in df.iterrows():
    print(i, row.question, row.output, row.segments)
    print('\n\n\n\n')

In [31]:
segments = model.segment(input_ids=model_inputs['input_ids'])

In [32]:
[s['input_ids'].shape for s in segments]

[torch.Size([1, 594]),
 torch.Size([1, 1024]),
 torch.Size([1, 1024]),
 torch.Size([1, 1024])]